# Example usage notebook
Objects and functions in this notebook are listed with all paramaters to ilustrate their capabillities. Most of the paramaters have default values in the implementation

## Imports

In [1]:
import os
import sys

sys.path.append(os.getcwd())
os.chdir("../..")

import pandas as pd
import mlflow
pd.set_option("display.max_columns", 200)
pd.set_option("display.max_rows", 300)
os.environ["AWS_ACCESS_KEY_ID"] = "minioadmin"
os.environ["AWS_SECRET_ACCESS_KEY"] = "minioadmin"
os.environ["MLFLOW_S3_ENDPOINT_URL"] = f"http://10.152.183.156:9000"

In [2]:
import os
import json
from typing import Literal
import pandas as pd

from inference_model.preprocessing.preprocess import PreprocessData
from inference_model.training.trainer import Trainer
from inference_model.training.optuna_optimizer import LGBOptunaOptimizer

import dill
import numpy as np
from inference_model.utils import dill_dump, dill_load
from inference_model.training.utils import flatten_dict, get_or_create_experiment
from sklearn.model_selection import train_test_split

from pprint import pprint
import ast

## Train a testing model 

In [3]:
# 1. get/create some example data
with open("data/log_tiguan_27_mar_dac.txt") as f:
    data = ast.literal_eval(f.read())

df_pd = pd.DataFrame()
for data_value in data: 
    temp_df = pd.DataFrame(data_value[list(data_value)[0]]).sort_values(by="ts_millis:", ascending=True)["value"]
    temp_df.rename(list(data_value)[0], inplace=True)
    df_pd = pd.concat([df_pd, temp_df], axis = 1) 

df_pd.dropna(inplace=True)
df_pd["class"] = 0
df_pd["car_id"] = "123abc"
df_pd.loc[:100, ["class"]] = 1
df_pd.loc[:100, ["engine_load"]] = 100

df_pd.head()

,engine_load,engine_coolant_temp,engine_speed,vehicle_speed,intake_air_temp,maf,throttle_position,fuel_rg_pressure,barometric_pressure,control_voltage,class,car_id
0,100.0,17.0,904.5,0.0,10.0,12.55,83.14,37270.0,101.0,0.06,1,123abc
1,100.0,17.0,906.0,0.0,11.0,12.36,83.14,37800.0,101.0,14.56,1,123abc
2,100.0,17.0,905.0,0.0,10.0,12.36,83.53,37800.0,101.0,14.68,1,123abc
3,100.0,18.0,905.5,0.0,11.0,12.30,83.53,37800.0,101.0,14.72,1,123abc
4,100.0,18.0,907.0,0.0,11.0,12.47,83.14,36740.0,101.0,14.72,1,123abc


In [4]:
target_col = "class"
id_cols = ["car_id"]
cat_cols = []
cont_cols = df_pd.drop(columns=id_cols + cat_cols + [target_col]).columns.values.tolist()
df_pd[cat_cols] = df_pd[cat_cols].astype(str)

In [6]:
valid_size = 0.2
test_size = 0.5
random_state = 1
df_train, df_valid = train_test_split(df_pd, test_size=valid_size, stratify=df_pd[target_col], random_state=random_state)
df_valid, df_test = train_test_split(df_valid, test_size=test_size, stratify=df_valid[target_col], random_state=random_state)

In [7]:
prepare_data = PreprocessData(
        id_cols=id_cols,
        target_col=target_col,
        cat_cols=cat_cols,
        cont_cols=cont_cols,
    )
# this should be fitted only on training data
_ = prepare_data.fit(df=df_pd)

In [8]:
optimizer = LGBOptunaOptimizer(
        objective="binary",
        n_class=2,
    )

trainer = Trainer(
        cat_cols=prepare_data.cat_cols,
        target_col=prepare_data.target_col,
        id_cols=id_cols,
        objective="binary",
        n_class=2,
        optimizer=optimizer,
        preprocessors=[prepare_data],
)

metrics_dict = trainer.fit(
    df_train=df_train,
    df_valid=df_valid,
    df_test=df_test,
    )

[I 2024-05-22 13:01:18,589] A new study created in memory with name: LightGBMTuner
feature_fraction, val_score: inf:   0%|          | 0/7 [00:00<?, ?it/s]

Training until validation scores don't improve for 50 rounds


feature_fraction, val_score: 0.000002:  14%|#4        | 1/7 [00:03<00:21,  3.62s/it]

Early stopping, best iteration is:
[588]	valid_0's binary_logloss: 1.67891e-06
Training until validation scores don't improve for 50 rounds


feature_fraction, val_score: 0.000002:  29%|##8       | 2/7 [00:05<00:13,  2.72s/it]

...

Early stopping, best iteration is:
[176]	valid_0's binary_logloss: 3.62911e-06
Training until validation scores don't improve for 50 rounds


min_child_samples, val_score: 0.000000:  80%|########  | 4/5 [00:01<00:00,  3.78it/s]

Early stopping, best iteration is:
[204]	valid_0's binary_logloss: 3.89204e-07
Training until validation scores don't improve for 50 rounds


min_child_samples, val_score: 0.000000: 100%|##########| 5/5 [00:01<00:00,  3.56it/s]

Early stopping, best iteration is:
[252]	valid_0's binary_logloss: 2.16037e-07


In [12]:
# save/load trainer
dill_dump("lgbm_trainer.dill", trainer)
trainer = dill_load("lgbm_trainer.dill")

In [13]:
pprint(metrics_dict)

{'cls_report': {'0': {'f1-score': 1.0,
                      'precision': 1.0,
                      'recall': 1.0,
                      'support': 94},
                '1': {'f1-score': 1.0,
                      'precision': 1.0,
                      'recall': 1.0,
                      'support': 10},
                'accuracy': 1.0,
                'macro avg': {'f1-score': 1.0,
                              'precision': 1.0,
                              'recall': 1.0,
                              'support': 104},
                'weighted avg': {'f1-score': 1.0,
                                 'precision': 1.0,
                                 'recall': 1.0,
                                 'support': 104}},
 'cm': [[94, 0], [0, 10]],
 'prec_rec_curve': [[0.09615384615384616, 1.0, 1.0],
                    [1.0, 1.0, 0.0],
                    [0.0, 1.0]]}


In [14]:
pprint(trainer.compute_metrics(df_test, with_dynamic_binary_threshold=True))

{'cls_report': {'0': {'f1-score': 1.0,
                      'precision': 1.0,
                      'recall': 1.0,
                      'support': 94},
                '1': {'f1-score': 1.0,
                      'precision': 1.0,
                      'recall': 1.0,
                      'support': 10},
                'accuracy': 1.0,
                'macro avg': {'f1-score': 1.0,
                              'precision': 1.0,
                              'recall': 1.0,
                              'support': 104},
                'weighted avg': {'f1-score': 1.0,
                                 'precision': 1.0,
                                 'recall': 1.0,
                                 'support': 104}},
 'cm': [[94, 0], [0, 10]],
 'prec_rec_curve': [[0.09615384615384616, 1.0, 1.0],
                    [1.0, 1.0, 0.0],
                    [0.0, 1.0]]}


# Mlflow
* tracking
* model registration in Minio
* [permanent experiment deletion](https://mlflow.org/docs/2.11.3/tracking/artifacts-stores.html?highlight=delete%20experiment#deletion-behavior)
```
# in python
mlflow.get_experiment_by_name("success6g_lgbm").experiment_id
# in cmd
mlflow experiments delete -x 1
```

In [26]:
mlflow_host = "10.152.183.54"
mlflow_host_url = "mlflow.mlflow.svc.cluster.local"
mlflow_port = "5000"
os.environ["AWS_ACCESS_KEY_ID"] = "minioadmin"
os.environ["AWS_SECRET_ACCESS_KEY"] = "minioadmin"
os.environ["MLFLOW_S3_ENDPOINT_URL"] = f"http://10.152.183.156:9000"

mlflow.set_tracking_uri("http://" + mlflow_host + ":" + mlflow_port)
experiment_id = get_or_create_experiment("success6g")
mlflow.set_experiment(experiment_id=experiment_id)

<Experiment: artifact_location='s3://mlflow/3', creation_time=1716388845676, experiment_id='3', last_update_time=1716388845676, lifecycle_stage='active', name='success6g', tags={}>

In [27]:
metrics_dict_flattened = flatten_dict(metrics_dict)
# mlflow metrics can be only int, float not list
del metrics_dict_flattened["cm"]
del metrics_dict_flattened["prec_rec_curve"]

In [29]:
run_name = "test"
with mlflow.start_run(experiment_id=experiment_id, run_name=run_name, nested=True) as run:
    mlflow.log_params(trainer.optimizer.best)
    mlflow.log_metrics(metrics_dict_flattened)

    # Log tags
    mlflow.set_tags(
        tags={
            "project": "SUCCESS6G",
            "optimizer_engine": "optuna",
            "model_family": "ligtgbm",
            "feature_set_version": 1,
        }
    )
    # Log figure - for future fun
    # mlflow.log_figure(figure=correlation_plot, artifact_file="correlation_plot.png")

    artifact_path = "success6g_model"
    registered_model_name = "success6g_model"
    mlflow.pyfunc.log_model(python_model=trainer, 
                            artifact_path=artifact_path, 
                            registered_model_name=registered_model_name)  
    model_uri = mlflow.get_artifact_uri(artifact_path)
    print(f"Run ID:\n{run.info.run_id}\nModel uri:\n{model_uri}")

2024/05/22 14:44:19 INFO mlflow.types.utils: Unsupported type hint: typing.Union[pandas.core.frame.DataFrame, dict], skipping schema inference
2024/05/22 14:44:19 INFO mlflow.types.utils: Unsupported type hint: <class 'pandas.core.frame.DataFrame'>, skipping schema inference
Successfully registered model 'success6g_model'.
2024/05/22 14:44:22 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: success6g_model, version 1


Run ID:
24fedb96c5004767800fa32a3220c8dd
Model uri:
s3://mlflow/3/24fedb96c5004767800fa32a3220c8dd/artifacts/success6g_model


Created version '1' of model 'success6g_model'.


# Testing of predictions

## Example data

In [30]:
df_pd.iloc[:2]

,engine_load,engine_coolant_temp,engine_speed,vehicle_speed,intake_air_temp,maf,throttle_position,fuel_rg_pressure,barometric_pressure,control_voltage,class,car_id
0,100.0,17.0,904.5,0.0,10.0,12.55,83.14,37270.0,101.0,0.06,1,123abc
1,100.0,17.0,906.0,0.0,11.0,12.36,83.14,37800.0,101.0,14.56,1,123abc


In [31]:
# for testing using mlflow inference service
request_dict = {
    'engine_load': [100.0 ,100.0],
    'engine_coolant_temp': [17.0, 17.0],
    'engine_speed': [904.5, 906.0],
    'vehicle_speed': [0.0, 0.0],
    'intake_air_temp': [10.0, 11.0],
    'maf': [12.55, 12.36],
    'throttle_position': [83.14, 83.14],
    'fuel_rg_pressure': [37270.0, 37800.0],
    'barometric_pressure': [101.0, 101.0],
    'control_voltage': [0.06, 14.56],
    'class': [1, 1],
    'car_id': ["123abc", "123abc"],
}

## Trainer

In [32]:
trainer.predict(df=df_pd.iloc[:2].drop(columns=["class"]), context={})

,class
0,14.269467
1,14.269467


## Downloaded Trainer

In [33]:
model_uri

's3://mlflow/3/24fedb96c5004767800fa32a3220c8dd/artifacts/success6g_model'

In [34]:
loaded_trainer = mlflow.pyfunc.load_model(model_uri)

In [35]:
loaded_trainer.predict(df_pd.iloc[:2].drop(columns=["class"]))

,class
0,14.269467
1,14.269467


## Downloaded/Served Trainer
* i.e. testing [model locally](https://mlflow.org/docs/latest/deployment/deploy-model-to-kubernetes/tutorial.html?highlight=kserve#step-6-testing-model-serving-locally)

In [36]:
model_uri

's3://mlflow/3/24fedb96c5004767800fa32a3220c8dd/artifacts/success6g_model'

In [ ]:
! mlflow models serve -m "s3://mlflow/3/24fedb96c5004767800fa32a3220c8dd/artifacts/success6g_model" --env-manager local -p 5000

2024/05/22 14:45:07 INFO mlflow.models.flavor_backend_registry: Selected backend for flavor 'python_function'
2024/05/22 14:45:07 INFO mlflow.pyfunc.backend: === Running command 'exec gunicorn --timeout=60 -b 127.0.0.1:5000 -w 1 ${GUNICORN_CMD_ARGS} -- mlflow.pyfunc.scoring_server.wsgi:app'
[2024-05-22 14:45:07 +0000] [17971] [INFO] Starting gunicorn 22.0.0
[2024-05-22 14:45:07 +0000] [17971] [INFO] Listening at: http://127.0.0.1:5000 (17971)
[2024-05-22 14:45:07 +0000] [17971] [INFO] Using worker: sync
[2024-05-22 14:45:07 +0000] [17972] [INFO] Booting worker with pid: 17972


```
root@jupyter-5uperpalo:~/assignment# curl -X POST -H "Content-Type:application/json" --data '{"inputs": {"engine_load": [100.0 ,100.0], "engine_coolant_temp": [17.0, 17.0], "engine_speed": [904.5, 906.0], "vehicle_speed": [0.0, 0.0], "intake_air_temp": [10.0, 11.0], "maf": [12.55, 12.36], "throttle_position": [83.14, 83.14], "fuel_rg_pressure": [37270.0, 37800.0], "barometric_pressure": [101.0, 101.0], "control_voltage": [0.06, 14.56], "car_id": ["123abc", "123abc"]}}' http://127.0.0.1:5000/invocations
{"predictions": [{"class": 14.26946728558632}, {"class": 14.26946728558632}]}
```

# Model deployment using Kserve
* https://mlflow.org/docs/latest/deployment/deploy-model-to-kubernetes/tutorial.html?highlight=kserve#step-7-deploying-the-model-to-kserve

## Using `model_uri` 

* since I configured minio as artifact store I can use `model_uri` - I was **WRONG!**

procedure:
<details><summary>make sure the kserve is able to reach minio</summary>

*  by [defining s3-secret and attaching it to service account](https://kserve.github.io/website/0.7/modelserving/storage/s3/s3/#create-s3-secret)
* created ServiceAccount must be in a same namespace as the InferenceService we are going to create, [e.g. stackoverflow here](https://stackoverflow.com/a/70977798)
* `If an application does not specify a serviceAccountName, it uses the "default" service account.`[e.g. here](https://kubernetes.io/docs/reference/access-authn-authz/rbac/#role-binding-examples)

```
apiVersion: v1
kind: Secret
metadata:
  name: s3creds
  namespace: mlflow-kserve-success6g
  annotations:
     serving.kserve.io/s3-endpoint: 10.152.183.156:9000 # replace with your s3 endpoint e.g minio-service.kubeflow:9000
     serving.kserve.io/s3-usehttps: "0" # by default 1, if testing with minio you can set to 0
     # serving.kserve.io/s3-region: "us-east-2"
     serving.kserve.io/s3-useanoncredential: "false" # omitting this is the same as false, if true will ignore provided credential and use anonymous credentials
type: Opaque
stringData: # use `stringData` for raw credential string or `data` for base64 encoded string
  AWS_ACCESS_KEY_ID: "minioadmin"
  AWS_SECRET_ACCESS_KEY: "minioadmin"
```

`kubectl apply -f create-s3-secret.yaml`

</details>
<details><summary>create namespace</summary>

```
kubectl create namespace mlflow-kserve-success6g
namespace/mlflow-kserve-success6g created
```

</details>
<details><summary>specify configuration of the deployment in the file `mlflow-kserve-success6g.yaml`</summary>

```
apiVersion: "serving.kserve.io/v1beta1"
kind: "InferenceService"
metadata:
  name: "mlflow-success6g"
  namespace: "mlflow-kserve-success6g"
spec:
  predictor:
    model:
      modelFormat:
        name: mlflow
      protocolVersion: v2
      storageUri: "s3://mlflow/1/777cf64c922149a4b77c85987865deb0/artifacts/success6g_model"
```

</details>
<details><summary>deploy</summary>

```
kubectl apply -f mlflow-kserve-success6g.yaml
inferenceservice.serving.kserve.io/mlflow-success6g created
```
</details>
<details><summary>check the status of the deployment</summary>

`kubectl get inferenceservice mlflow-success6g`

</details>
<details><summary>specify testing json in `test-input.json`</summary>

```
{
    "inputs": [
      {
        "name": "input",
        "shape": [13],
        "datatype": "FP32",
        "data": {
			"engine_load": [100.0 ,100.0],
			"engine_coolant_temp": [17.0, 17.0],
			"engine_speed": [904.5, 906.0],
			"vehicle_speed": [0.0, 0.0],
			"intake_air_temp": [10.0, 11.0],
			"maf": [12.55, 12.36],
			"throttle_position": [83.14, 83.14],
			"fuel_rg_pressure": [37270.0, 37800.0],
			"barometric_pressure": [101.0, 101.0],
			"control_voltage": [0.06, 14.56],
			"car_id": ["123abc", "123abc"],
			}
      }
    ]
}
```

</details>
<details><summary>test json</summary>

```
SERVICE_HOSTNAME=$(kubectl get inferenceservice mlflow-success6g -n mlflow-kserve-success6g -o jsonpath='{.status.url}' | cut -d "/" -f 3)
curl -v \
  -H "Host: ${SERVICE_HOSTNAME}" \
  -H "Content-Type: application/json" \
  -d @./test-input.json \
  http://${INGRESS_HOST}:${INGRESS_PORT}/v2/models/mlflow-success6g/infer
```

</details>
<details><summary>but after all the trouble this can still fail if you developed the model in different python version than kserve is using</summary>

```
pmulinka@saiacheron:~/kubernetes/kserve$ kubectl logs -p mlflow-success6g-predictor-00001-deployment-74ccf4fb5d-lktz9 -n mlflow-kserve-success6g
Defaulted container "kserve-container" out of: kserve-container, queue-proxy, storage-initializer (init)
Environment tarball not found at '/mnt/models/environment.tar.gz'
Environment not found at './envs/environment'
2024-05-16 15:32:55,293 [mlserver.parallel] DEBUG - Starting response processing loop...
2024-05-16 15:32:55,298 [mlserver.rest] INFO - HTTP server running on http://0.0.0.0:8080
INFO:     Started server process [1]
INFO:     Waiting for application startup.
2024-05-16 15:32:55,411 [mlserver.metrics] INFO - Metrics server running on http://0.0.0.0:8082
2024-05-16 15:32:55,412 [mlserver.metrics] INFO - Prometheus scraping endpoint can be accessed on http://0.0.0.0:8082/metrics
INFO:     Started server process [1]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
2024-05-16 15:32:57,613 [mlserver.grpc] INFO - gRPC server running on http://0.0.0.0:9000
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8080 (Press CTRL+C to quit)
INFO:     Uvicorn running on http://0.0.0.0:8082 (Press CTRL+C to quit)
2024/05/16 15:32:58 WARNING mlflow.pyfunc: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - mlflow (current: 2.3.1, required: mlflow==2.12.2)
 - cryptography (current: 38.0.4, required: cryptography==40.0.2)
 - cytoolz (current: uninstalled, required: cytoolz==0.12.0)
 - defusedxml (current: uninstalled, required: defusedxml==0.7.1)
 - distributed (current: uninstalled, required: distributed==2023.5.1)
 - lightgbm (current: 3.3.5, required: lightgbm==4.3.0)
 - lz4 (current: uninstalled, required: lz4==4.3.2)
 - numpy (current: 1.23.5, required: numpy==1.24.3)
 - optuna-integration (current: uninstalled, required: optuna-integration==3.6.0)
 - pandas (current: 2.0.1, required: pandas==2.2.2)
 - pyarrow (current: 11.0.0, required: pyarrow==12.0.0)
 - redis (current: uninstalled, required: redis==5.0.4)
 - scipy (current: 1.9.1, required: scipy==1.10.1)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.
2024/05/16 15:32:58 WARNING mlflow.pyfunc: The version of Python that the model was saved in, `Python 3.10.11`, differs from the version of Python that is currently running, `Python 3.8.16`, and may be incompatible
2024-05-16 15:32:58,967 [mlserver] INFO - Couldn't load model 'mlflow-success6g'. Model will be removed from registry.
2024-05-16 15:32:58,967 [mlserver.parallel] ERROR - An error occurred processing a model update of type 'Load'.
Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/site-packages/mlserver/parallel/worker.py", line 158, in _process_model_update
    await self._model_registry.load(model_settings)
  File "/opt/conda/lib/python3.8/site-packages/mlserver/registry.py", line 293, in load
    return await self._models[model_settings.name].load(model_settings)
  File "/opt/conda/lib/python3.8/site-packages/mlserver/registry.py", line 148, in load
    await self._load_model(new_model)
  File "/opt/conda/lib/python3.8/site-packages/mlserver/registry.py", line 165, in _load_model
    model.ready = await model.load()
  File "/opt/conda/lib/python3.8/site-packages/mlserver_mlflow/runtime.py", line 155, in load
    self._model = mlflow.pyfunc.load_model(model_uri)
  File "/opt/conda/lib/python3.8/site-packages/mlflow/pyfunc/__init__.py", line 596, in load_model
    model_impl = importlib.import_module(conf[MAIN])._load_pyfunc(data_path)
  File "/opt/conda/lib/python3.8/site-packages/mlflow/pyfunc/model.py", line 291, in _load_pyfunc
    python_model = cloudpickle.load(f)
ModuleNotFoundError: No module named 'inference_model'
2024-05-16 15:32:58,970 [mlserver] INFO - Couldn't load model 'mlflow-success6g'. Model will be removed from registry.
2024-05-16 15:32:58,975 [mlserver.parallel] ERROR - An error occurred processing a model update of type 'Unload'.
Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/site-packages/mlserver/parallel/worker.py", line 160, in _process_model_update
    await self._model_registry.unload_version(
  File "/opt/conda/lib/python3.8/site-packages/mlserver/registry.py", line 302, in unload_version
    await model_registry.unload_version(version)
  File "/opt/conda/lib/python3.8/site-packages/mlserver/registry.py", line 201, in unload_version
    model = await self.get_model(version)
  File "/opt/conda/lib/python3.8/site-packages/mlserver/registry.py", line 237, in get_model
    raise ModelNotFound(self._name, version)
mlserver.errors.ModelNotFound: Model mlflow-success6g not found
2024-05-16 15:32:58,976 [mlserver] ERROR - Some of the models failed to load during startup!
Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/site-packages/mlserver/server.py", line 125, in start
    await asyncio.gather(
  File "/opt/conda/lib/python3.8/site-packages/mlserver/registry.py", line 293, in load
    return await self._models[model_settings.name].load(model_settings)
  File "/opt/conda/lib/python3.8/site-packages/mlserver/registry.py", line 148, in load
    await self._load_model(new_model)
  File "/opt/conda/lib/python3.8/site-packages/mlserver/registry.py", line 161, in _load_model
    model = await callback(model)
  File "/opt/conda/lib/python3.8/site-packages/mlserver/parallel/registry.py", line 152, in load_model
    loaded = await pool.load_model(model)
  File "/opt/conda/lib/python3.8/site-packages/mlserver/parallel/pool.py", line 74, in load_model
    await self._dispatcher.dispatch_update(load_message)
  File "/opt/conda/lib/python3.8/site-packages/mlserver/parallel/dispatcher.py", line 123, in dispatch_update
    return await asyncio.gather(
  File "/opt/conda/lib/python3.8/site-packages/mlserver/parallel/dispatcher.py", line 138, in _dispatch_update
    return await self._dispatch(worker_update)
  File "/opt/conda/lib/python3.8/site-packages/mlserver/parallel/dispatcher.py", line 146, in _dispatch
    return await self._wait_response(internal_id)
  File "/opt/conda/lib/python3.8/site-packages/mlserver/parallel/dispatcher.py", line 152, in _wait_response
    inference_response = await async_response
mlserver.parallel.errors.WorkerError: builtins.ModuleNotFoundError: No module named 'inference_model'
2024-05-16 15:32:58,978 [mlserver.parallel] INFO - Waiting for shutdown of default inference pool...
2024-05-16 15:32:59,163 [mlserver.parallel] INFO - Shutdown of default inference pool complete
2024-05-16 15:32:59,163 [mlserver.grpc] INFO - Waiting for gRPC server shutdown
2024-05-16 15:32:59,169 [mlserver.grpc] INFO - gRPC server shutdown complete
INFO:     Shutting down
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [1]
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [1]
```

</details>
<details><summary>and some possible readiness probe issues</summary>

* apparently I am not alone in this, e.g. [here](https://stackoverflow.com/questions/62569747/ml-model-pod-keeps-restarting-in-seldon-deployment)
```
pmulinka@saiacheron:~/kubernetes/kserve$ kubectl describe pod mlflow-success6g-predictor-00001-deployment-74ccf4fb5d-lktz9 -n mlflow-kserve-success6g
Events:
  Type     Reason     Age                    From     Message
  ----     ------     ----                   ----     -------
  Warning  Unhealthy  4m11s (x268 over 34m)  kubelet  Readiness probe failed: Get "http://10.1.4.204:8012/": context deadline exceeded (Client.Timeout exceeded while awaiting headers)
```
</details>

## Using docker image
<details><summary>create docker</summary>

```
sudo apt-get install python3.10-venv
python3.10 -m venv python310venv
source python310venv/bin/activate
pip install mlflow
pip install boto3
export AWS_ACCESS_KEY_ID=minioadmin
export AWS_SECRET_ACCESS_KEY=minioadmin
export MLFLOW_S3_ENDPOINT_URL=http://10.152.183.156:9000

mlflow models build-docker -m s3://mlflow/1/777cf64c922149a4b77c85987865deb0/artifacts/success6g_model -n 5uperpalo/mlflow-success6g --enable-mlserver
```
</details>
<details><summary>generate token and login to DockerHub, </summary>

[generate DockerHub token](https://hub.docker.com/settings/security?generateToken=true) and login
```
(python310venv) pmulinka@iesc-gpu3:~/scripts$ docker login  -u 5uperpalo
Password:
WARNING! Your password will be stored unencrypted in /home/pmulinka/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
```
</details>
<details><summary>push the image to dockerhub</summary>

```
(python310venv) pmulinka@iesc-gpu3:~/scripts$ docker push 5uperpalo/mlflow-success6g
Using default tag: latest
The push refers to repository [docker.io/5uperpalo/mlflow-success6g]
e5f76b582f6c: Pushed
624821c51019: Pushed
93256a85bb86: Pushed
2f62dcdb5ae7: Pushed
1146ddf03b42: Pushed
639ddc87d13f: Pushed
4f18e59d57c4: Pushed
5761e01a334a: Pushed
f31b6bde159a: Pushed
c25ef1a18d31: Pushed
40a774d6610c: Pushed
a2b215b4258b: Pushed
b56a9203d2a0: Pushed
2536dbfb2294: Pushed
67ea405a7d49: Pushed
a81c3606ee5c: Pushed
4a1518ebc26e: Mounted from library/ubuntu
latest: digest: sha256:57f653a3c8d63519c07dccffc6eb12dba5006db07ebcd869467ce752fb1b0bda size: 3899
```
</details>

### Appendix
<details><summary>output from making docker</summary>
    
```
(python310venv) pmulinka@iesc-gpu3:~/scripts$ mlflow models build-docker -m s3://mlflow/1/777cf64c922149a4b77c85987865deb0/artifacts/success6g_model -n 5uperpalo/mlflow-success6g --enable-mlserver
2024/05/16 22:20:25 INFO mlflow.models.flavor_backend_registry: Selected backend for flavor 'python_function'
2024/05/16 22:20:25 INFO mlflow.pyfunc.backend: Building docker image with name 5uperpalo/mlflow-success6g
#0 building with "default" instance using docker driver

#1 [internal] load build definition from Dockerfile
#1 transferring dockerfile: 2.53kB done
#1 DONE 0.1s

#2 [internal] load metadata for docker.io/library/ubuntu:20.04
#2 DONE 1.6s

#3 [internal] load .dockerignore
#3 transferring context: 2B done
#3 DONE 0.0s

#4 [ 1/17] FROM docker.io/library/ubuntu:20.04@sha256:874aca52f79ae5f8258faff03e10ce99ae836f6e7d2df6ecd3da5c1cad3a912b
#4 resolve docker.io/library/ubuntu:20.04@sha256:874aca52f79ae5f8258faff03e10ce99ae836f6e7d2df6ecd3da5c1cad3a912b 0.0s done
#4 ...

#5 [internal] load build context
#5 transferring context: 197.37kB done
#5 DONE 0.1s

#4 [ 1/17] FROM docker.io/library/ubuntu:20.04@sha256:874aca52f79ae5f8258faff03e10ce99ae836f6e7d2df6ecd3da5c1cad3a912b
#4 sha256:2abc4dfd83182546da40dfae3e391db0810ad4a130fb5a887c6ceb3df8e1d310 2.29kB / 2.29kB done
#4 sha256:d4c3c94e5e10ed15503bda7e145a3652ee935c0b2e9de9b5c98df7ec0a0cd925 0B / 27.51MB 0.2s
#4 sha256:874aca52f79ae5f8258faff03e10ce99ae836f6e7d2df6ecd3da5c1cad3a912b 1.13kB / 1.13kB done
#4 sha256:cc9cc8169c9517ae035cf293b15f06922cb8c6c864d625a72b7b18667f264b70 424B / 424B done
#4 sha256:d4c3c94e5e10ed15503bda7e145a3652ee935c0b2e9de9b5c98df7ec0a0cd925 3.15MB / 27.51MB 0.4s
#4 sha256:d4c3c94e5e10ed15503bda7e145a3652ee935c0b2e9de9b5c98df7ec0a0cd925 8.39MB / 27.51MB 0.7s
#4 sha256:d4c3c94e5e10ed15503bda7e145a3652ee935c0b2e9de9b5c98df7ec0a0cd925 11.53MB / 27.51MB 0.9s
#4 sha256:d4c3c94e5e10ed15503bda7e145a3652ee935c0b2e9de9b5c98df7ec0a0cd925 16.78MB / 27.51MB 1.2s
#4 sha256:d4c3c94e5e10ed15503bda7e145a3652ee935c0b2e9de9b5c98df7ec0a0cd925 22.02MB / 27.51MB 1.5s
#4 sha256:d4c3c94e5e10ed15503bda7e145a3652ee935c0b2e9de9b5c98df7ec0a0cd925 25.17MB / 27.51MB 1.7s
#4 sha256:d4c3c94e5e10ed15503bda7e145a3652ee935c0b2e9de9b5c98df7ec0a0cd925 27.26MB / 27.51MB 1.8s
#4 sha256:d4c3c94e5e10ed15503bda7e145a3652ee935c0b2e9de9b5c98df7ec0a0cd925 27.51MB / 27.51MB 1.9s done
#4 extracting sha256:d4c3c94e5e10ed15503bda7e145a3652ee935c0b2e9de9b5c98df7ec0a0cd925
#4 extracting sha256:d4c3c94e5e10ed15503bda7e145a3652ee935c0b2e9de9b5c98df7ec0a0cd925 1.8s done
#4 DONE 4.0s

#6 [ 2/17] RUN apt-get -y update && DEBIAN_FRONTEND=noninteractive TZ=Etc/UTC apt-get install -y --no-install-recommends wget curl nginx ca-certificates bzip2 build-essential cmake git-core
#6 0.681 Get:1 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
#6 0.682 Get:2 http://archive.ubuntu.com/ubuntu focal InRelease [265 kB]
#6 1.324 Get:3 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
#6 1.432 Get:4 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [1205 kB]
#6 1.476 Get:5 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
#6 1.690 Get:6 http://archive.ubuntu.com/ubuntu focal/restricted amd64 Packages [33.4 kB]
#6 1.838 Get:7 http://archive.ubuntu.com/ubuntu focal/multiverse amd64 Packages [177 kB]
#6 1.897 Get:8 http://archive.ubuntu.com/ubuntu focal/main amd64 Packages [1275 kB]
#6 2.177 Get:9 http://security.ubuntu.com/ubuntu focal-security/restricted amd64 Packages [3608 kB]
#6 2.537 Get:10 http://archive.ubuntu.com/ubuntu focal/universe amd64 Packages [11.3 MB]
#6 3.807 Get:11 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [3669 kB]
#6 5.468 Get:12 http://security.ubuntu.com/ubuntu focal-security/multiverse amd64 Packages [29.8 kB]
#6 7.649 Get:13 http://archive.ubuntu.com/ubuntu focal-updates/restricted amd64 Packages [3758 kB]
#6 9.304 Get:14 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 Packages [1502 kB]
#6 9.973 Get:15 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [4143 kB]
#6 12.14 Get:16 http://archive.ubuntu.com/ubuntu focal-updates/multiverse amd64 Packages [32.5 kB]
#6 12.15 Get:17 http://archive.ubuntu.com/ubuntu focal-backports/main amd64 Packages [55.2 kB]
#6 12.18 Get:18 http://archive.ubuntu.com/ubuntu focal-backports/universe amd64 Packages [28.6 kB]
#6 12.44 Fetched 31.5 MB in 12s (2621 kB/s)
#6 12.44 Reading package lists...
#6 13.60 Reading package lists...
#6 14.67 Building dependency tree...
#6 14.83 Reading state information...
#6 15.04 bzip2 is already the newest version (1.0.8-2).
#6 15.04 The following additional packages will be installed:
#6 15.04   binutils binutils-common binutils-x86-64-linux-gnu cmake-data cpp cpp-9
#6 15.04   dpkg-dev fontconfig-config fonts-dejavu-core g++ g++-9 gcc gcc-9 gcc-9-base
#6 15.04   git-man iproute2 libarchive13 libasan5 libasn1-8-heimdal libatomic1
#6 15.04   libbinutils libbrotli1 libbsd0 libc-dev-bin libc6-dev libcap2 libcap2-bin
#6 15.04   libcc1-0 libcrypt-dev libctf-nobfd0 libctf0 libcurl3-gnutls libcurl4
#6 15.04   libdpkg-perl libelf1 liberror-perl libexpat1 libfontconfig1 libfreetype6
#6 15.04   libgcc-9-dev libgd3 libgdbm-compat4 libgdbm6 libgomp1 libgssapi-krb5-2
#6 15.04   libgssapi3-heimdal libhcrypto4-heimdal libheimbase1-heimdal
#6 15.04   libheimntlm0-heimdal libhx509-5-heimdal libicu66 libisl22 libitm1 libjbig0
#6 15.04   libjpeg-turbo8 libjpeg8 libjsoncpp1 libk5crypto3 libkeyutils1
#6 15.04   libkrb5-26-heimdal libkrb5-3 libkrb5support0 libldap-2.4-2 libldap-common
#6 15.04   liblsan0 libmnl0 libmpc3 libmpfr6 libnghttp2-14
#6 15.04   libnginx-mod-http-image-filter libnginx-mod-http-xslt-filter
#6 15.04   libnginx-mod-mail libnginx-mod-stream libperl5.30 libpng16-16 libpsl5
#6 15.04   libquadmath0 librhash0 libroken18-heimdal librtmp1 libsasl2-2
#6 15.04   libsasl2-modules-db libsqlite3-0 libssh-4 libssl1.1 libstdc++-9-dev libtiff5
#6 15.04   libtsan0 libubsan1 libuv1 libwebp6 libwind0-heimdal libx11-6 libx11-data
#6 15.04   libxau6 libxcb1 libxdmcp6 libxml2 libxpm4 libxslt1.1 libxtables12
#6 15.04   linux-libc-dev make nginx-common nginx-core openssl patch perl
#6 15.04   perl-modules-5.30 tzdata ucf xz-utils
#6 15.04 Suggested packages:
#6 15.04   binutils-doc cmake-doc ninja-build cpp-doc gcc-9-locales debian-keyring
#6 15.04   g++-multilib g++-9-multilib gcc-9-doc gcc-multilib manpages-dev autoconf
#6 15.04   automake libtool flex bison gdb gcc-doc gcc-9-multilib gettext-base
#6 15.04   git-daemon-run | git-daemon-sysvinit git-doc git-el git-email git-gui gitk
#6 15.04   gitweb git-cvs git-mediawiki git-svn iproute2-doc lrzip glibc-doc gnupg
#6 15.04   | gnupg2 bzr libgd-tools gdbm-l10n krb5-doc krb5-user libstdc++-9-doc
#6 15.04   make-doc fcgiwrap nginx-doc ssl-cert ed diffutils-doc perl-doc
#6 15.04   libterm-readline-gnu-perl | libterm-readline-perl-perl libb-debug-perl
#6 15.04   liblocale-codes-perl
#6 15.04 Recommended packages:
#6 15.04   fakeroot gnupg | gnupg2 libalgorithm-merge-perl less ssh-client libatm1
#6 15.04   manpages manpages-dev libpam-cap libfile-fcntllock-perl
#6 15.04   liblocale-gettext-perl krb5-locales publicsuffix libsasl2-modules netbase
#6 15.30 The following NEW packages will be installed:
#6 15.30   binutils binutils-common binutils-x86-64-linux-gnu build-essential
#6 15.30   ca-certificates cmake cmake-data cpp cpp-9 curl dpkg-dev fontconfig-config
#6 15.30   fonts-dejavu-core g++ g++-9 gcc gcc-9 gcc-9-base git git-man iproute2
#6 15.30   libarchive13 libasan5 libasn1-8-heimdal libatomic1 libbinutils libbrotli1
#6 15.30   libbsd0 libc-dev-bin libc6-dev libcap2 libcap2-bin libcc1-0 libcrypt-dev
#6 15.30   libctf-nobfd0 libctf0 libcurl3-gnutls libcurl4 libdpkg-perl libelf1
#6 15.30   liberror-perl libexpat1 libfontconfig1 libfreetype6 libgcc-9-dev libgd3
#6 15.30   libgdbm-compat4 libgdbm6 libgomp1 libgssapi-krb5-2 libgssapi3-heimdal
#6 15.30   libhcrypto4-heimdal libheimbase1-heimdal libheimntlm0-heimdal
#6 15.30   libhx509-5-heimdal libicu66 libisl22 libitm1 libjbig0 libjpeg-turbo8
#6 15.30   libjpeg8 libjsoncpp1 libk5crypto3 libkeyutils1 libkrb5-26-heimdal libkrb5-3
#6 15.30   libkrb5support0 libldap-2.4-2 libldap-common liblsan0 libmnl0 libmpc3
#6 15.30   libmpfr6 libnghttp2-14 libnginx-mod-http-image-filter
#6 15.30   libnginx-mod-http-xslt-filter libnginx-mod-mail libnginx-mod-stream
#6 15.30   libperl5.30 libpng16-16 libpsl5 libquadmath0 librhash0 libroken18-heimdal
#6 15.30   librtmp1 libsasl2-2 libsasl2-modules-db libsqlite3-0 libssh-4 libssl1.1
#6 15.30   libstdc++-9-dev libtiff5 libtsan0 libubsan1 libuv1 libwebp6 libwind0-heimdal
#6 15.30   libx11-6 libx11-data libxau6 libxcb1 libxdmcp6 libxml2 libxpm4 libxslt1.1
#6 15.30   libxtables12 linux-libc-dev make nginx nginx-common nginx-core openssl patch
#6 15.30   perl perl-modules-5.30 tzdata ucf wget xz-utils
#6 15.41 0 upgraded, 119 newly installed, 0 to remove and 0 not upgraded.
#6 15.41 Need to get 82.1 MB of archives.
#6 15.41 After this operation, 389 MB of additional disk space will be used.
#6 15.41 Get:1 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 perl-modules-5.30 all 5.30.0-9ubuntu0.5 [2739 kB]
#6 16.07 Get:2 http://archive.ubuntu.com/ubuntu focal/main amd64 libgdbm6 amd64 1.18.1-5 [27.4 kB]
#6 16.07 Get:3 http://archive.ubuntu.com/ubuntu focal/main amd64 libgdbm-compat4 amd64 1.18.1-5 [6244 B]
#6 16.07 Get:4 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libperl5.30 amd64 5.30.0-9ubuntu0.5 [3941 kB]
#6 16.76 Get:5 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 perl amd64 5.30.0-9ubuntu0.5 [224 kB]
#6 16.79 Get:6 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libssl1.1 amd64 1.1.1f-1ubuntu2.22 [1323 kB]
#6 17.04 Get:7 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 openssl amd64 1.1.1f-1ubuntu2.22 [622 kB]
#6 17.15 Get:8 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 ca-certificates all 20230311ubuntu0.20.04.1 [152 kB]
#6 17.18 Get:9 http://archive.ubuntu.com/ubuntu focal/main amd64 libbsd0 amd64 0.10.0-1 [45.4 kB]
#6 17.20 Get:10 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libcap2 amd64 1:2.32-1ubuntu0.1 [15.8 kB]
#6 17.20 Get:11 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libelf1 amd64 0.176-1.1ubuntu0.1 [44.2 kB]
#6 17.33 Get:12 http://archive.ubuntu.com/ubuntu focal/main amd64 libmnl0 amd64 1.0.4-2 [12.3 kB]
#6 17.38 Get:13 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libxtables12 amd64 1.8.4-3ubuntu2.1 [28.7 kB]
#6 17.42 Get:14 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libcap2-bin amd64 1:2.32-1ubuntu0.1 [26.2 kB]
#6 17.46 Get:15 http://archive.ubuntu.com/ubuntu focal/main amd64 iproute2 amd64 5.5.0-1ubuntu1 [858 kB]
#6 20.61 Get:16 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libexpat1 amd64 2.2.9-1ubuntu0.6 [74.6 kB]
#6 21.00 Get:17 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 tzdata all 2024a-0ubuntu0.20.04 [301 kB]
#6 22.38 Get:18 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libicu66 amd64 66.1-2ubuntu2.1 [8515 kB]
#6 51.24 Get:19 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libsqlite3-0 amd64 3.31.1-4ubuntu0.6 [549 kB]
#6 53.10 Get:20 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libxml2 amd64 2.9.10+dfsg-5ubuntu0.20.04.7 [640 kB]
#6 55.32 Get:21 http://archive.ubuntu.com/ubuntu focal/main amd64 ucf all 3.0038+nmu1 [51.6 kB]
#6 55.50 Get:21 http://archive.ubuntu.com/ubuntu focal/main amd64 ucf all 3.0038+nmu1 [51.6 kB]
#6 55.60 Get:22 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 xz-utils amd64 5.2.4-1ubuntu1.1 [82.6 kB]
#6 55.66 Get:23 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libkrb5support0 amd64 1.17-6ubuntu4.4 [31.0 kB]
#6 55.70 Get:24 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libk5crypto3 amd64 1.17-6ubuntu4.4 [79.9 kB]
#6 55.72 Get:25 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libkeyutils1 amd64 1.6-6ubuntu1.1 [10.3 kB]
#6 55.73 Get:26 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libkrb5-3 amd64 1.17-6ubuntu4.4 [330 kB]
#6 55.80 Get:27 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libgssapi-krb5-2 amd64 1.17-6ubuntu4.4 [121 kB]
#6 55.82 Get:28 http://archive.ubuntu.com/ubuntu focal/main amd64 libpng16-16 amd64 1.6.37-2 [179 kB]
#6 55.86 Get:29 http://archive.ubuntu.com/ubuntu focal/main amd64 libpsl5 amd64 0.21.0-1ubuntu1 [51.5 kB]
#6 55.86 Get:30 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libuv1 amd64 1.34.2-1ubuntu1.5 [80.9 kB]
#6 55.88 Get:31 http://archive.ubuntu.com/ubuntu focal/main amd64 libxau6 amd64 1:1.0.9-0ubuntu1 [7488 B]
#6 55.96 Get:32 http://archive.ubuntu.com/ubuntu focal/main amd64 libxdmcp6 amd64 1:1.1.3-0ubuntu1 [10.6 kB]
#6 56.01 Get:33 http://archive.ubuntu.com/ubuntu focal/main amd64 libxcb1 amd64 1.14-2 [44.7 kB]
#6 56.08 Get:34 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libx11-data all 2:1.6.9-2ubuntu1.6 [114 kB]
#6 56.15 Get:35 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libx11-6 amd64 2:1.6.9-2ubuntu1.6 [577 kB]
#6 56.26 Get:36 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 wget amd64 1.20.3-1ubuntu2 [348 kB]
#6 56.32 Get:37 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 binutils-common amd64 2.34-6ubuntu1.9 [208 kB]
#6 56.37 Get:38 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libbinutils amd64 2.34-6ubuntu1.9 [475 kB]
#6 56.46 Get:39 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libctf-nobfd0 amd64 2.34-6ubuntu1.9 [48.2 kB]
#6 56.47 Get:40 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libctf0 amd64 2.34-6ubuntu1.9 [46.6 kB]
#6 56.48 Get:41 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 binutils-x86-64-linux-gnu amd64 2.34-6ubuntu1.9 [1614 kB]
#6 56.76 Get:42 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 binutils amd64 2.34-6ubuntu1.9 [3380 B]
#6 56.85 Get:43 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libc-dev-bin amd64 2.31-0ubuntu9.15 [71.8 kB]
#6 57.00 Get:44 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 linux-libc-dev amd64 5.4.0-182.202 [1134 kB]
#6 57.28 Get:45 http://archive.ubuntu.com/ubuntu focal/main amd64 libcrypt-dev amd64 1:4.4.10-10ubuntu4 [104 kB]
#6 57.30 Get:46 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libc6-dev amd64 2.31-0ubuntu9.15 [2519 kB]
#6 57.75 Get:47 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 gcc-9-base amd64 9.4.0-1ubuntu1~20.04.2 [18.9 kB]
#6 57.75 Get:48 http://archive.ubuntu.com/ubuntu focal/main amd64 libisl22 amd64 0.22.1-1 [592 kB]
#6 57.87 Get:49 http://archive.ubuntu.com/ubuntu focal/main amd64 libmpfr6 amd64 4.0.2-1 [240 kB]
#6 57.91 Get:50 http://archive.ubuntu.com/ubuntu focal/main amd64 libmpc3 amd64 1.1.0-1 [40.8 kB]
#6 57.92 Get:51 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 cpp-9 amd64 9.4.0-1ubuntu1~20.04.2 [7502 kB]
#6 59.27 Get:52 http://archive.ubuntu.com/ubuntu focal/main amd64 cpp amd64 4:9.3.0-1ubuntu2 [27.6 kB]
#6 59.27 Get:53 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libcc1-0 amd64 10.5.0-1ubuntu1~20.04 [48.8 kB]
#6 59.36 Get:54 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libgomp1 amd64 10.5.0-1ubuntu1~20.04 [102 kB]
#6 59.54 Get:55 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libitm1 amd64 10.5.0-1ubuntu1~20.04 [26.2 kB]
#6 59.55 Get:56 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libatomic1 amd64 10.5.0-1ubuntu1~20.04 [9284 B]
#6 59.55 Get:57 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libasan5 amd64 9.4.0-1ubuntu1~20.04.2 [2752 kB]
#6 60.09 Get:58 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 liblsan0 amd64 10.5.0-1ubuntu1~20.04 [835 kB]
#6 60.24 Get:59 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libtsan0 amd64 10.5.0-1ubuntu1~20.04 [2016 kB]
#6 60.61 Get:60 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libubsan1 amd64 10.5.0-1ubuntu1~20.04 [785 kB]
#6 60.76 Get:61 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libquadmath0 amd64 10.5.0-1ubuntu1~20.04 [146 kB]
#6 60.78 Get:62 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libgcc-9-dev amd64 9.4.0-1ubuntu1~20.04.2 [2359 kB]
#6 61.28 Get:63 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 gcc-9 amd64 9.4.0-1ubuntu1~20.04.2 [8276 kB]
#6 62.82 Get:64 http://archive.ubuntu.com/ubuntu focal/main amd64 gcc amd64 4:9.3.0-1ubuntu2 [5208 B]
#6 62.91 Get:65 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libstdc++-9-dev amd64 9.4.0-1ubuntu1~20.04.2 [1722 kB]
#6 63.39 Get:66 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 g++-9 amd64 9.4.0-1ubuntu1~20.04.2 [8421 kB]
#6 65.04 Get:67 http://archive.ubuntu.com/ubuntu focal/main amd64 g++ amd64 4:9.3.0-1ubuntu2 [1604 B]
#6 65.06 Get:68 http://archive.ubuntu.com/ubuntu focal/main amd64 make amd64 4.2.1-1.2 [162 kB]
#6 65.07 Get:69 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libdpkg-perl all 1.19.7ubuntu3.2 [231 kB]
#6 65.16 Get:70 http://archive.ubuntu.com/ubuntu focal/main amd64 patch amd64 2.7.6-6 [105 kB]
#6 65.20 Get:71 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 dpkg-dev all 1.19.7ubuntu3.2 [679 kB]
#6 65.32 Get:72 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 build-essential amd64 12.8ubuntu1.1 [4664 B]
#6 65.32 Get:73 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 cmake-data all 3.16.3-1ubuntu1.20.04.1 [1613 kB]
#6 65.61 Get:74 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libarchive13 amd64 3.4.0-2ubuntu1.2 [327 kB]
#6 65.66 Get:75 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libbrotli1 amd64 1.0.7-6ubuntu0.1 [267 kB]
#6 65.80 Get:76 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libroken18-heimdal amd64 7.7.0+dfsg-1ubuntu1.4 [42.5 kB]
#6 65.94 Get:77 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libasn1-8-heimdal amd64 7.7.0+dfsg-1ubuntu1.4 [181 kB]
#6 66.25 Get:78 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libheimbase1-heimdal amd64 7.7.0+dfsg-1ubuntu1.4 [30.4 kB]
#6 66.29 Get:79 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libhcrypto4-heimdal amd64 7.7.0+dfsg-1ubuntu1.4 [88.1 kB]
#6 66.55 Get:80 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libwind0-heimdal amd64 7.7.0+dfsg-1ubuntu1.4 [47.7 kB]
#6 66.71 Get:81 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libhx509-5-heimdal amd64 7.7.0+dfsg-1ubuntu1.4 [107 kB]
#6 67.13 Get:82 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libkrb5-26-heimdal amd64 7.7.0+dfsg-1ubuntu1.4 [207 kB]
#6 67.89 Get:83 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libheimntlm0-heimdal amd64 7.7.0+dfsg-1ubuntu1.4 [15.1 kB]
#6 67.92 Get:84 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libgssapi3-heimdal amd64 7.7.0+dfsg-1ubuntu1.4 [96.5 kB]
#6 68.05 Get:85 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libsasl2-modules-db amd64 2.1.27+dfsg-2ubuntu0.1 [14.7 kB]
#6 68.06 Get:86 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libsasl2-2 amd64 2.1.27+dfsg-2ubuntu0.1 [49.3 kB]
#6 68.29 Get:87 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libldap-common all 2.4.49+dfsg-2ubuntu1.10 [16.5 kB]
#6 68.38 Get:88 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libldap-2.4-2 amd64 2.4.49+dfsg-2ubuntu1.10 [155 kB]
#6 68.50 Get:89 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libnghttp2-14 amd64 1.40.0-1ubuntu0.3 [79.9 kB]
#6 68.53 Get:90 http://archive.ubuntu.com/ubuntu focal/main amd64 librtmp1 amd64 2.4+20151223.gitfa8646d.1-2build1 [54.9 kB]
#6 68.54 Get:91 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libssh-4 amd64 0.9.3-2ubuntu2.5 [171 kB]
#6 68.57 Get:92 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libcurl4 amd64 7.68.0-1ubuntu2.22 [235 kB]
#6 68.62 Get:93 http://archive.ubuntu.com/ubuntu focal/main amd64 libjsoncpp1 amd64 1.7.4-3.1ubuntu2 [75.6 kB]
#6 68.63 Get:94 http://archive.ubuntu.com/ubuntu focal/main amd64 librhash0 amd64 1.3.9-1 [113 kB]
#6 68.65 Get:95 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 cmake amd64 3.16.3-1ubuntu1.20.04.1 [3668 kB]
#6 69.31 Get:96 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 curl amd64 7.68.0-1ubuntu2.22 [161 kB]
#6 69.33 Get:97 http://archive.ubuntu.com/ubuntu focal/main amd64 fonts-dejavu-core all 2.37-1 [1041 kB]
#6 69.60 Get:98 http://archive.ubuntu.com/ubuntu focal/main amd64 fontconfig-config all 2.13.1-2ubuntu3 [28.8 kB]
#6 69.70 Get:99 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libcurl3-gnutls amd64 7.68.0-1ubuntu2.22 [233 kB]
#6 69.83 Get:100 http://archive.ubuntu.com/ubuntu focal/main amd64 liberror-perl all 0.17029-1 [26.5 kB]
#6 69.84 Get:101 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 git-man all 1:2.25.1-1ubuntu3.11 [887 kB]
#6 70.00 Get:102 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 git amd64 1:2.25.1-1ubuntu3.11 [4605 kB]
#6 70.90 Get:103 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libfreetype6 amd64 2.10.1-2ubuntu0.3 [341 kB]
#6 70.96 Get:104 http://archive.ubuntu.com/ubuntu focal/main amd64 libfontconfig1 amd64 2.13.1-2ubuntu3 [114 kB]
#6 70.97 Get:105 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libjpeg-turbo8 amd64 2.0.3-0ubuntu1.20.04.3 [118 kB]
#6 71.00 Get:106 http://archive.ubuntu.com/ubuntu focal/main amd64 libjpeg8 amd64 8c-2ubuntu8 [2194 B]
#6 71.00 Get:107 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libjbig0 amd64 2.1-3.1ubuntu0.20.04.1 [27.3 kB]
#6 71.00 Get:108 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libwebp6 amd64 0.6.1-2ubuntu0.20.04.3 [185 kB]
#6 71.14 Get:109 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libtiff5 amd64 4.1.0+git191117-2ubuntu0.20.04.12 [164 kB]
#6 71.36 Get:110 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libxpm4 amd64 1:3.5.12-1ubuntu0.20.04.2 [34.9 kB]
#6 71.36 Get:111 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libgd3 amd64 2.2.5-5.2ubuntu2.1 [118 kB]
#6 71.39 Get:112 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 nginx-common all 1.18.0-0ubuntu1.4 [37.7 kB]
#6 71.40 Get:113 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libnginx-mod-http-image-filter amd64 1.18.0-0ubuntu1.4 [14.8 kB]
#6 71.41 Get:114 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libxslt1.1 amd64 1.1.34-4ubuntu0.20.04.1 [151 kB]
#6 71.43 Get:115 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libnginx-mod-http-xslt-filter amd64 1.18.0-0ubuntu1.4 [13.0 kB]
#6 71.44 Get:116 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libnginx-mod-mail amd64 1.18.0-0ubuntu1.4 [42.9 kB]
#6 71.47 Get:117 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libnginx-mod-stream amd64 1.18.0-0ubuntu1.4 [67.4 kB]
#6 71.49 Get:118 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 nginx-core amd64 1.18.0-0ubuntu1.4 [425 kB]
#6 71.58 Get:119 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 nginx all 1.18.0-0ubuntu1.4 [3620 B]
#6 71.74 debconf: delaying package configuration, since apt-utils is not installed
#6 71.80 Fetched 82.1 MB in 56s (1459 kB/s)
#6 71.85 Selecting previously unselected package perl-modules-5.30.
#6 71.85 (Reading database ...
(Reading database ... 5%
(Reading database ... 10%
(Reading database ... 15%
(Reading database ... 20%
(Reading database ... 25%
(Reading database ... 30%
(Reading database ... 35%
(Reading database ... 40%
(Reading database ... 45%
(Reading database ... 50%
(Reading database ... 55%
(Reading database ... 60%
(Reading database ... 65%
(Reading database ... 70%
(Reading database ... 75%
(Reading database ... 80%
(Reading database ... 85%
(Reading database ... 90%
(Reading database ... 95%
(Reading database ... 100%
(Reading database ... 4124 files and directories currently installed.)
#6 71.86 Preparing to unpack .../000-perl-modules-5.30_5.30.0-9ubuntu0.5_all.deb ...
#6 71.87 Unpacking perl-modules-5.30 (5.30.0-9ubuntu0.5) ...
#6 72.25 Selecting previously unselected package libgdbm6:amd64.
#6 72.25 Preparing to unpack .../001-libgdbm6_1.18.1-5_amd64.deb ...
#6 72.26 Unpacking libgdbm6:amd64 (1.18.1-5) ...
#6 72.33 Selecting previously unselected package libgdbm-compat4:amd64.
#6 72.33 Preparing to unpack .../002-libgdbm-compat4_1.18.1-5_amd64.deb ...
#6 72.34 Unpacking libgdbm-compat4:amd64 (1.18.1-5) ...
#6 72.41 Selecting previously unselected package libperl5.30:amd64.
#6 72.41 Preparing to unpack .../003-libperl5.30_5.30.0-9ubuntu0.5_amd64.deb ...
#6 72.42 Unpacking libperl5.30:amd64 (5.30.0-9ubuntu0.5) ...
#6 72.92 Selecting previously unselected package perl.
#6 72.93 Preparing to unpack .../004-perl_5.30.0-9ubuntu0.5_amd64.deb ...
#6 72.95 Unpacking perl (5.30.0-9ubuntu0.5) ...
#6 73.07 Selecting previously unselected package libssl1.1:amd64.
#6 73.08 Preparing to unpack .../005-libssl1.1_1.1.1f-1ubuntu2.22_amd64.deb ...
#6 73.08 Unpacking libssl1.1:amd64 (1.1.1f-1ubuntu2.22) ...
#6 73.27 Selecting previously unselected package openssl.
#6 73.28 Preparing to unpack .../006-openssl_1.1.1f-1ubuntu2.22_amd64.deb ...
#6 73.28 Unpacking openssl (1.1.1f-1ubuntu2.22) ...
#6 73.43 Selecting previously unselected package ca-certificates.
#6 73.43 Preparing to unpack .../007-ca-certificates_20230311ubuntu0.20.04.1_all.deb ...
#6 73.44 Unpacking ca-certificates (20230311ubuntu0.20.04.1) ...
#6 73.55 Selecting previously unselected package libbsd0:amd64.
#6 73.56 Preparing to unpack .../008-libbsd0_0.10.0-1_amd64.deb ...
#6 73.56 Unpacking libbsd0:amd64 (0.10.0-1) ...
#6 73.64 Selecting previously unselected package libcap2:amd64.
#6 73.64 Preparing to unpack .../009-libcap2_1%3a2.32-1ubuntu0.1_amd64.deb ...
#6 73.65 Unpacking libcap2:amd64 (1:2.32-1ubuntu0.1) ...
#6 73.72 Selecting previously unselected package libelf1:amd64.
#6 73.72 Preparing to unpack .../010-libelf1_0.176-1.1ubuntu0.1_amd64.deb ...
#6 73.73 Unpacking libelf1:amd64 (0.176-1.1ubuntu0.1) ...
#6 73.81 Selecting previously unselected package libmnl0:amd64.
#6 73.81 Preparing to unpack .../011-libmnl0_1.0.4-2_amd64.deb ...
#6 73.82 Unpacking libmnl0:amd64 (1.0.4-2) ...
#6 73.89 Selecting previously unselected package libxtables12:amd64.
#6 73.89 Preparing to unpack .../012-libxtables12_1.8.4-3ubuntu2.1_amd64.deb ...
#6 73.90 Unpacking libxtables12:amd64 (1.8.4-3ubuntu2.1) ...
#6 73.96 Selecting previously unselected package libcap2-bin.
#6 73.96 Preparing to unpack .../013-libcap2-bin_1%3a2.32-1ubuntu0.1_amd64.deb ...
#6 73.96 Unpacking libcap2-bin (1:2.32-1ubuntu0.1) ...
#6 74.04 Selecting previously unselected package iproute2.
#6 74.04 Preparing to unpack .../014-iproute2_5.5.0-1ubuntu1_amd64.deb ...
#6 74.05 Unpacking iproute2 (5.5.0-1ubuntu1) ...
#6 74.22 Selecting previously unselected package libexpat1:amd64.
#6 74.22 Preparing to unpack .../015-libexpat1_2.2.9-1ubuntu0.6_amd64.deb ...
#6 74.22 Unpacking libexpat1:amd64 (2.2.9-1ubuntu0.6) ...
#6 74.33 Selecting previously unselected package tzdata.
#6 74.33 Preparing to unpack .../016-tzdata_2024a-0ubuntu0.20.04_all.deb ...
#6 74.34 Unpacking tzdata (2024a-0ubuntu0.20.04) ...
#6 74.60 Selecting previously unselected package libicu66:amd64.
#6 74.60 Preparing to unpack .../017-libicu66_66.1-2ubuntu2.1_amd64.deb ...
#6 74.61 Unpacking libicu66:amd64 (66.1-2ubuntu2.1) ...
#6 75.46 Selecting previously unselected package libsqlite3-0:amd64.
#6 75.46 Preparing to unpack .../018-libsqlite3-0_3.31.1-4ubuntu0.6_amd64.deb ...
#6 75.46 Unpacking libsqlite3-0:amd64 (3.31.1-4ubuntu0.6) ...
#6 75.59 Selecting previously unselected package libxml2:amd64.
#6 75.60 Preparing to unpack .../019-libxml2_2.9.10+dfsg-5ubuntu0.20.04.7_amd64.deb ...
#6 75.60 Unpacking libxml2:amd64 (2.9.10+dfsg-5ubuntu0.20.04.7) ...
#6 75.75 Selecting previously unselected package ucf.
#6 75.75 Preparing to unpack .../020-ucf_3.0038+nmu1_all.deb ...
#6 75.76 Moving old data out of the way
#6 75.76 Unpacking ucf (3.0038+nmu1) ...
#6 75.83 Selecting previously unselected package xz-utils.
#6 75.83 Preparing to unpack .../021-xz-utils_5.2.4-1ubuntu1.1_amd64.deb ...
#6 75.84 Unpacking xz-utils (5.2.4-1ubuntu1.1) ...
#6 75.92 Selecting previously unselected package libkrb5support0:amd64.
#6 75.93 Preparing to unpack .../022-libkrb5support0_1.17-6ubuntu4.4_amd64.deb ...
#6 75.93 Unpacking libkrb5support0:amd64 (1.17-6ubuntu4.4) ...
#6 76.00 Selecting previously unselected package libk5crypto3:amd64.
#6 76.01 Preparing to unpack .../023-libk5crypto3_1.17-6ubuntu4.4_amd64.deb ...
#6 76.01 Unpacking libk5crypto3:amd64 (1.17-6ubuntu4.4) ...
#6 76.10 Selecting previously unselected package libkeyutils1:amd64.
#6 76.10 Preparing to unpack .../024-libkeyutils1_1.6-6ubuntu1.1_amd64.deb ...
#6 76.11 Unpacking libkeyutils1:amd64 (1.6-6ubuntu1.1) ...
#6 76.17 Selecting previously unselected package libkrb5-3:amd64.
#6 76.18 Preparing to unpack .../025-libkrb5-3_1.17-6ubuntu4.4_amd64.deb ...
#6 76.18 Unpacking libkrb5-3:amd64 (1.17-6ubuntu4.4) ...
#6 76.31 Selecting previously unselected package libgssapi-krb5-2:amd64.
#6 76.31 Preparing to unpack .../026-libgssapi-krb5-2_1.17-6ubuntu4.4_amd64.deb ...
#6 76.32 Unpacking libgssapi-krb5-2:amd64 (1.17-6ubuntu4.4) ...
#6 76.41 Selecting previously unselected package libpng16-16:amd64.
#6 76.41 Preparing to unpack .../027-libpng16-16_1.6.37-2_amd64.deb ...
#6 76.42 Unpacking libpng16-16:amd64 (1.6.37-2) ...
#6 76.52 Selecting previously unselected package libpsl5:amd64.
#6 76.52 Preparing to unpack .../028-libpsl5_0.21.0-1ubuntu1_amd64.deb ...
#6 76.53 Unpacking libpsl5:amd64 (0.21.0-1ubuntu1) ...
#6 76.61 Selecting previously unselected package libuv1:amd64.
#6 76.61 Preparing to unpack .../029-libuv1_1.34.2-1ubuntu1.5_amd64.deb ...
#6 76.62 Unpacking libuv1:amd64 (1.34.2-1ubuntu1.5) ...
#6 76.70 Selecting previously unselected package libxau6:amd64.
#6 76.71 Preparing to unpack .../030-libxau6_1%3a1.0.9-0ubuntu1_amd64.deb ...
#6 76.71 Unpacking libxau6:amd64 (1:1.0.9-0ubuntu1) ...
#6 76.78 Selecting previously unselected package libxdmcp6:amd64.
#6 76.79 Preparing to unpack .../031-libxdmcp6_1%3a1.1.3-0ubuntu1_amd64.deb ...
#6 76.79 Unpacking libxdmcp6:amd64 (1:1.1.3-0ubuntu1) ...
#6 76.86 Selecting previously unselected package libxcb1:amd64.
#6 76.87 Preparing to unpack .../032-libxcb1_1.14-2_amd64.deb ...
#6 76.87 Unpacking libxcb1:amd64 (1.14-2) ...
#6 76.95 Selecting previously unselected package libx11-data.
#6 76.95 Preparing to unpack .../033-libx11-data_2%3a1.6.9-2ubuntu1.6_all.deb ...
#6 76.96 Unpacking libx11-data (2:1.6.9-2ubuntu1.6) ...
#6 77.08 Selecting previously unselected package libx11-6:amd64.
#6 77.08 Preparing to unpack .../034-libx11-6_2%3a1.6.9-2ubuntu1.6_amd64.deb ...
#6 77.09 Unpacking libx11-6:amd64 (2:1.6.9-2ubuntu1.6) ...
#6 77.22 Selecting previously unselected package wget.
#6 77.22 Preparing to unpack .../035-wget_1.20.3-1ubuntu2_amd64.deb ...
#6 77.23 Unpacking wget (1.20.3-1ubuntu2) ...
#6 77.33 Selecting previously unselected package binutils-common:amd64.
#6 77.33 Preparing to unpack .../036-binutils-common_2.34-6ubuntu1.9_amd64.deb ...
#6 77.34 Unpacking binutils-common:amd64 (2.34-6ubuntu1.9) ...
#6 77.44 Selecting previously unselected package libbinutils:amd64.
#6 77.45 Preparing to unpack .../037-libbinutils_2.34-6ubuntu1.9_amd64.deb ...
#6 77.45 Unpacking libbinutils:amd64 (2.34-6ubuntu1.9) ...
#6 77.59 Selecting previously unselected package libctf-nobfd0:amd64.
#6 77.59 Preparing to unpack .../038-libctf-nobfd0_2.34-6ubuntu1.9_amd64.deb ...
#6 77.60 Unpacking libctf-nobfd0:amd64 (2.34-6ubuntu1.9) ...
#6 77.68 Selecting previously unselected package libctf0:amd64.
#6 77.68 Preparing to unpack .../039-libctf0_2.34-6ubuntu1.9_amd64.deb ...
#6 77.69 Unpacking libctf0:amd64 (2.34-6ubuntu1.9) ...
#6 77.76 Selecting previously unselected package binutils-x86-64-linux-gnu.
#6 77.76 Preparing to unpack .../040-binutils-x86-64-linux-gnu_2.34-6ubuntu1.9_amd64.deb ...
#6 77.77 Unpacking binutils-x86-64-linux-gnu (2.34-6ubuntu1.9) ...
#6 78.04 Selecting previously unselected package binutils.
#6 78.05 Preparing to unpack .../041-binutils_2.34-6ubuntu1.9_amd64.deb ...
#6 78.05 Unpacking binutils (2.34-6ubuntu1.9) ...
#6 78.12 Selecting previously unselected package libc-dev-bin.
#6 78.12 Preparing to unpack .../042-libc-dev-bin_2.31-0ubuntu9.15_amd64.deb ...
#6 78.13 Unpacking libc-dev-bin (2.31-0ubuntu9.15) ...
#6 78.20 Selecting previously unselected package linux-libc-dev:amd64.
#6 78.21 Preparing to unpack .../043-linux-libc-dev_5.4.0-182.202_amd64.deb ...
#6 78.21 Unpacking linux-libc-dev:amd64 (5.4.0-182.202) ...
#6 78.41 Selecting previously unselected package libcrypt-dev:amd64.
#6 78.42 Preparing to unpack .../044-libcrypt-dev_1%3a4.4.10-10ubuntu4_amd64.deb ...
#6 78.42 Unpacking libcrypt-dev:amd64 (1:4.4.10-10ubuntu4) ...
#6 78.50 Selecting previously unselected package libc6-dev:amd64.
#6 78.51 Preparing to unpack .../045-libc6-dev_2.31-0ubuntu9.15_amd64.deb ...
#6 78.51 Unpacking libc6-dev:amd64 (2.31-0ubuntu9.15) ...
#6 78.88 Selecting previously unselected package gcc-9-base:amd64.
#6 78.88 Preparing to unpack .../046-gcc-9-base_9.4.0-1ubuntu1~20.04.2_amd64.deb ...
#6 78.89 Unpacking gcc-9-base:amd64 (9.4.0-1ubuntu1~20.04.2) ...
#6 78.97 Selecting previously unselected package libisl22:amd64.
#6 78.97 Preparing to unpack .../047-libisl22_0.22.1-1_amd64.deb ...
#6 78.98 Unpacking libisl22:amd64 (0.22.1-1) ...
#6 79.12 Selecting previously unselected package libmpfr6:amd64.
#6 79.12 Preparing to unpack .../048-libmpfr6_4.0.2-1_amd64.deb ...
#6 79.13 Unpacking libmpfr6:amd64 (4.0.2-1) ...
#6 79.24 Selecting previously unselected package libmpc3:amd64.
#6 79.24 Preparing to unpack .../049-libmpc3_1.1.0-1_amd64.deb ...
#6 79.25 Unpacking libmpc3:amd64 (1.1.0-1) ...
#6 79.32 Selecting previously unselected package cpp-9.
#6 79.32 Preparing to unpack .../050-cpp-9_9.4.0-1ubuntu1~20.04.2_amd64.deb ...
#6 79.33 Unpacking cpp-9 (9.4.0-1ubuntu1~20.04.2) ...
#6 80.06 Selecting previously unselected package cpp.
#6 80.06 Preparing to unpack .../051-cpp_4%3a9.3.0-1ubuntu2_amd64.deb ...
#6 80.07 Unpacking cpp (4:9.3.0-1ubuntu2) ...
#6 80.14 Selecting previously unselected package libcc1-0:amd64.
#6 80.14 Preparing to unpack .../052-libcc1-0_10.5.0-1ubuntu1~20.04_amd64.deb ...
#6 80.15 Unpacking libcc1-0:amd64 (10.5.0-1ubuntu1~20.04) ...
#6 80.23 Selecting previously unselected package libgomp1:amd64.
#6 80.23 Preparing to unpack .../053-libgomp1_10.5.0-1ubuntu1~20.04_amd64.deb ...
#6 80.24 Unpacking libgomp1:amd64 (10.5.0-1ubuntu1~20.04) ...
#6 80.34 Selecting previously unselected package libitm1:amd64.
#6 80.34 Preparing to unpack .../054-libitm1_10.5.0-1ubuntu1~20.04_amd64.deb ...
#6 80.35 Unpacking libitm1:amd64 (10.5.0-1ubuntu1~20.04) ...
#6 80.43 Selecting previously unselected package libatomic1:amd64.
#6 80.43 Preparing to unpack .../055-libatomic1_10.5.0-1ubuntu1~20.04_amd64.deb ...
#6 80.44 Unpacking libatomic1:amd64 (10.5.0-1ubuntu1~20.04) ...
#6 80.51 Selecting previously unselected package libasan5:amd64.
#6 80.51 Preparing to unpack .../056-libasan5_9.4.0-1ubuntu1~20.04.2_amd64.deb ...
#6 80.52 Unpacking libasan5:amd64 (9.4.0-1ubuntu1~20.04.2) ...
#6 80.87 Selecting previously unselected package liblsan0:amd64.
#6 80.88 Preparing to unpack .../057-liblsan0_10.5.0-1ubuntu1~20.04_amd64.deb ...
#6 80.88 Unpacking liblsan0:amd64 (10.5.0-1ubuntu1~20.04) ...
#6 81.05 Selecting previously unselected package libtsan0:amd64.
#6 81.05 Preparing to unpack .../058-libtsan0_10.5.0-1ubuntu1~20.04_amd64.deb ...
#6 81.06 Unpacking libtsan0:amd64 (10.5.0-1ubuntu1~20.04) ...
#6 81.35 Selecting previously unselected package libubsan1:amd64.
#6 81.35 Preparing to unpack .../059-libubsan1_10.5.0-1ubuntu1~20.04_amd64.deb ...
#6 81.36 Unpacking libubsan1:amd64 (10.5.0-1ubuntu1~20.04) ...
#6 81.52 Selecting previously unselected package libquadmath0:amd64.
#6 81.52 Preparing to unpack .../060-libquadmath0_10.5.0-1ubuntu1~20.04_amd64.deb ...
#6 81.53 Unpacking libquadmath0:amd64 (10.5.0-1ubuntu1~20.04) ...
#6 81.62 Selecting previously unselected package libgcc-9-dev:amd64.
#6 81.62 Preparing to unpack .../061-libgcc-9-dev_9.4.0-1ubuntu1~20.04.2_amd64.deb ...
#6 81.63 Unpacking libgcc-9-dev:amd64 (9.4.0-1ubuntu1~20.04.2) ...
#6 81.92 Selecting previously unselected package gcc-9.
#6 81.92 Preparing to unpack .../062-gcc-9_9.4.0-1ubuntu1~20.04.2_amd64.deb ...
#6 81.93 Unpacking gcc-9 (9.4.0-1ubuntu1~20.04.2) ...
#6 82.71 Selecting previously unselected package gcc.
#6 82.71 Preparing to unpack .../063-gcc_4%3a9.3.0-1ubuntu2_amd64.deb ...
#6 82.72 Unpacking gcc (4:9.3.0-1ubuntu2) ...
#6 82.79 Selecting previously unselected package libstdc++-9-dev:amd64.
#6 82.79 Preparing to unpack .../064-libstdc++-9-dev_9.4.0-1ubuntu1~20.04.2_amd64.deb ...
#6 82.80 Unpacking libstdc++-9-dev:amd64 (9.4.0-1ubuntu1~20.04.2) ...
#6 83.09 Selecting previously unselected package g++-9.
#6 83.09 Preparing to unpack .../065-g++-9_9.4.0-1ubuntu1~20.04.2_amd64.deb ...
#6 83.10 Unpacking g++-9 (9.4.0-1ubuntu1~20.04.2) ...
#6 83.88 Selecting previously unselected package g++.
#6 83.89 Preparing to unpack .../066-g++_4%3a9.3.0-1ubuntu2_amd64.deb ...
#6 83.89 Unpacking g++ (4:9.3.0-1ubuntu2) ...
#6 83.95 Selecting previously unselected package make.
#6 83.96 Preparing to unpack .../067-make_4.2.1-1.2_amd64.deb ...
#6 83.96 Unpacking make (4.2.1-1.2) ...
#6 84.06 Selecting previously unselected package libdpkg-perl.
#6 84.06 Preparing to unpack .../068-libdpkg-perl_1.19.7ubuntu3.2_all.deb ...
#6 84.07 Unpacking libdpkg-perl (1.19.7ubuntu3.2) ...
#6 84.17 Selecting previously unselected package patch.
#6 84.18 Preparing to unpack .../069-patch_2.7.6-6_amd64.deb ...
#6 84.18 Unpacking patch (2.7.6-6) ...
#6 84.27 Selecting previously unselected package dpkg-dev.
#6 84.28 Preparing to unpack .../070-dpkg-dev_1.19.7ubuntu3.2_all.deb ...
#6 84.28 Unpacking dpkg-dev (1.19.7ubuntu3.2) ...
#6 84.42 Selecting previously unselected package build-essential.
#6 84.42 Preparing to unpack .../071-build-essential_12.8ubuntu1.1_amd64.deb ...
#6 84.43 Unpacking build-essential (12.8ubuntu1.1) ...
#6 84.52 Selecting previously unselected package cmake-data.
#6 84.52 Preparing to unpack .../072-cmake-data_3.16.3-1ubuntu1.20.04.1_all.deb ...
#6 84.54 Unpacking cmake-data (3.16.3-1ubuntu1.20.04.1) ...
#6 84.99 Selecting previously unselected package libarchive13:amd64.
#6 84.99 Preparing to unpack .../073-libarchive13_3.4.0-2ubuntu1.2_amd64.deb ...
#6 85.00 Unpacking libarchive13:amd64 (3.4.0-2ubuntu1.2) ...
#6 85.12 Selecting previously unselected package libbrotli1:amd64.
#6 85.12 Preparing to unpack .../074-libbrotli1_1.0.7-6ubuntu0.1_amd64.deb ...
#6 85.13 Unpacking libbrotli1:amd64 (1.0.7-6ubuntu0.1) ...
#6 85.24 Selecting previously unselected package libroken18-heimdal:amd64.
#6 85.25 Preparing to unpack .../075-libroken18-heimdal_7.7.0+dfsg-1ubuntu1.4_amd64.deb ...
#6 85.25 Unpacking libroken18-heimdal:amd64 (7.7.0+dfsg-1ubuntu1.4) ...
#6 85.33 Selecting previously unselected package libasn1-8-heimdal:amd64.
#6 85.33 Preparing to unpack .../076-libasn1-8-heimdal_7.7.0+dfsg-1ubuntu1.4_amd64.deb ...
#6 85.34 Unpacking libasn1-8-heimdal:amd64 (7.7.0+dfsg-1ubuntu1.4) ...
#6 85.45 Selecting previously unselected package libheimbase1-heimdal:amd64.
#6 85.45 Preparing to unpack .../077-libheimbase1-heimdal_7.7.0+dfsg-1ubuntu1.4_amd64.deb ...
#6 85.46 Unpacking libheimbase1-heimdal:amd64 (7.7.0+dfsg-1ubuntu1.4) ...
#6 85.54 Selecting previously unselected package libhcrypto4-heimdal:amd64.
#6 85.54 Preparing to unpack .../078-libhcrypto4-heimdal_7.7.0+dfsg-1ubuntu1.4_amd64.deb ...
#6 85.55 Unpacking libhcrypto4-heimdal:amd64 (7.7.0+dfsg-1ubuntu1.4) ...
#6 85.64 Selecting previously unselected package libwind0-heimdal:amd64.
#6 85.64 Preparing to unpack .../079-libwind0-heimdal_7.7.0+dfsg-1ubuntu1.4_amd64.deb ...
#6 85.65 Unpacking libwind0-heimdal:amd64 (7.7.0+dfsg-1ubuntu1.4) ...
#6 85.74 Selecting previously unselected package libhx509-5-heimdal:amd64.
#6 85.74 Preparing to unpack .../080-libhx509-5-heimdal_7.7.0+dfsg-1ubuntu1.4_amd64.deb ...
#6 85.75 Unpacking libhx509-5-heimdal:amd64 (7.7.0+dfsg-1ubuntu1.4) ...
#6 85.84 Selecting previously unselected package libkrb5-26-heimdal:amd64.
#6 85.84 Preparing to unpack .../081-libkrb5-26-heimdal_7.7.0+dfsg-1ubuntu1.4_amd64.deb ...
#6 85.85 Unpacking libkrb5-26-heimdal:amd64 (7.7.0+dfsg-1ubuntu1.4) ...
#6 85.96 Selecting previously unselected package libheimntlm0-heimdal:amd64.
#6 85.96 Preparing to unpack .../082-libheimntlm0-heimdal_7.7.0+dfsg-1ubuntu1.4_amd64.deb ...
#6 85.97 Unpacking libheimntlm0-heimdal:amd64 (7.7.0+dfsg-1ubuntu1.4) ...
#6 86.04 Selecting previously unselected package libgssapi3-heimdal:amd64.
#6 86.04 Preparing to unpack .../083-libgssapi3-heimdal_7.7.0+dfsg-1ubuntu1.4_amd64.deb ...
#6 86.08 Unpacking libgssapi3-heimdal:amd64 (7.7.0+dfsg-1ubuntu1.4) ...
#6 86.16 Selecting previously unselected package libsasl2-modules-db:amd64.
#6 86.16 Preparing to unpack .../084-libsasl2-modules-db_2.1.27+dfsg-2ubuntu0.1_amd64.deb ...
#6 86.17 Unpacking libsasl2-modules-db:amd64 (2.1.27+dfsg-2ubuntu0.1) ...
#6 86.25 Selecting previously unselected package libsasl2-2:amd64.
#6 86.25 Preparing to unpack .../085-libsasl2-2_2.1.27+dfsg-2ubuntu0.1_amd64.deb ...
#6 86.26 Unpacking libsasl2-2:amd64 (2.1.27+dfsg-2ubuntu0.1) ...
#6 86.33 Selecting previously unselected package libldap-common.
#6 86.34 Preparing to unpack .../086-libldap-common_2.4.49+dfsg-2ubuntu1.10_all.deb ...
#6 86.34 Unpacking libldap-common (2.4.49+dfsg-2ubuntu1.10) ...
#6 86.42 Selecting previously unselected package libldap-2.4-2:amd64.
#6 86.42 Preparing to unpack .../087-libldap-2.4-2_2.4.49+dfsg-2ubuntu1.10_amd64.deb ...
#6 86.43 Unpacking libldap-2.4-2:amd64 (2.4.49+dfsg-2ubuntu1.10) ...
#6 86.53 Selecting previously unselected package libnghttp2-14:amd64.
#6 86.54 Preparing to unpack .../088-libnghttp2-14_1.40.0-1ubuntu0.3_amd64.deb ...
#6 86.54 Unpacking libnghttp2-14:amd64 (1.40.0-1ubuntu0.3) ...
#6 86.63 Selecting previously unselected package librtmp1:amd64.
#6 86.64 Preparing to unpack .../089-librtmp1_2.4+20151223.gitfa8646d.1-2build1_amd64.deb ...
#6 86.64 Unpacking librtmp1:amd64 (2.4+20151223.gitfa8646d.1-2build1) ...
#6 86.73 Selecting previously unselected package libssh-4:amd64.
#6 86.74 Preparing to unpack .../090-libssh-4_0.9.3-2ubuntu2.5_amd64.deb ...
#6 86.74 Unpacking libssh-4:amd64 (0.9.3-2ubuntu2.5) ...
#6 86.85 Selecting previously unselected package libcurl4:amd64.
#6 86.85 Preparing to unpack .../091-libcurl4_7.68.0-1ubuntu2.22_amd64.deb ...
#6 86.86 Unpacking libcurl4:amd64 (7.68.0-1ubuntu2.22) ...
#6 86.96 Selecting previously unselected package libjsoncpp1:amd64.
#6 86.97 Preparing to unpack .../092-libjsoncpp1_1.7.4-3.1ubuntu2_amd64.deb ...
#6 86.97 Unpacking libjsoncpp1:amd64 (1.7.4-3.1ubuntu2) ...
#6 87.07 Selecting previously unselected package librhash0:amd64.
#6 87.07 Preparing to unpack .../093-librhash0_1.3.9-1_amd64.deb ...
#6 87.08 Unpacking librhash0:amd64 (1.3.9-1) ...
#6 87.16 Selecting previously unselected package cmake.
#6 87.16 Preparing to unpack .../094-cmake_3.16.3-1ubuntu1.20.04.1_amd64.deb ...
#6 87.17 Unpacking cmake (3.16.3-1ubuntu1.20.04.1) ...
#6 87.60 Selecting previously unselected package curl.
#6 87.60 Preparing to unpack .../095-curl_7.68.0-1ubuntu2.22_amd64.deb ...
#6 87.61 Unpacking curl (7.68.0-1ubuntu2.22) ...
#6 87.71 Selecting previously unselected package fonts-dejavu-core.
#6 87.71 Preparing to unpack .../096-fonts-dejavu-core_2.37-1_all.deb ...
#6 87.72 Unpacking fonts-dejavu-core (2.37-1) ...
#6 87.90 Selecting previously unselected package fontconfig-config.
#6 87.91 Preparing to unpack .../097-fontconfig-config_2.13.1-2ubuntu3_all.deb ...
#6 87.92 Unpacking fontconfig-config (2.13.1-2ubuntu3) ...
#6 88.00 Selecting previously unselected package libcurl3-gnutls:amd64.
#6 88.00 Preparing to unpack .../098-libcurl3-gnutls_7.68.0-1ubuntu2.22_amd64.deb ...
#6 88.01 Unpacking libcurl3-gnutls:amd64 (7.68.0-1ubuntu2.22) ...
#6 88.11 Selecting previously unselected package liberror-perl.
#6 88.12 Preparing to unpack .../099-liberror-perl_0.17029-1_all.deb ...
#6 88.12 Unpacking liberror-perl (0.17029-1) ...
#6 88.19 Selecting previously unselected package git-man.
#6 88.19 Preparing to unpack .../100-git-man_1%3a2.25.1-1ubuntu3.11_all.deb ...
#6 88.20 Unpacking git-man (1:2.25.1-1ubuntu3.11) ...
#6 88.37 Selecting previously unselected package git.
#6 88.38 Preparing to unpack .../101-git_1%3a2.25.1-1ubuntu3.11_amd64.deb ...
#6 88.43 Unpacking git (1:2.25.1-1ubuntu3.11) ...
#6 88.98 Selecting previously unselected package libfreetype6:amd64.
#6 88.99 Preparing to unpack .../102-libfreetype6_2.10.1-2ubuntu0.3_amd64.deb ...
#6 89.00 Unpacking libfreetype6:amd64 (2.10.1-2ubuntu0.3) ...
#6 89.12 Selecting previously unselected package libfontconfig1:amd64.
#6 89.12 Preparing to unpack .../103-libfontconfig1_2.13.1-2ubuntu3_amd64.deb ...
#6 89.13 Unpacking libfontconfig1:amd64 (2.13.1-2ubuntu3) ...
#6 89.23 Selecting previously unselected package libjpeg-turbo8:amd64.
#6 89.23 Preparing to unpack .../104-libjpeg-turbo8_2.0.3-0ubuntu1.20.04.3_amd64.deb ...
#6 89.24 Unpacking libjpeg-turbo8:amd64 (2.0.3-0ubuntu1.20.04.3) ...
#6 89.32 Selecting previously unselected package libjpeg8:amd64.
#6 89.33 Preparing to unpack .../105-libjpeg8_8c-2ubuntu8_amd64.deb ...
#6 89.33 Unpacking libjpeg8:amd64 (8c-2ubuntu8) ...
#6 89.40 Selecting previously unselected package libjbig0:amd64.
#6 89.40 Preparing to unpack .../106-libjbig0_2.1-3.1ubuntu0.20.04.1_amd64.deb ...
#6 89.41 Unpacking libjbig0:amd64 (2.1-3.1ubuntu0.20.04.1) ...
#6 89.49 Selecting previously unselected package libwebp6:amd64.
#6 89.49 Preparing to unpack .../107-libwebp6_0.6.1-2ubuntu0.20.04.3_amd64.deb ...
#6 89.50 Unpacking libwebp6:amd64 (0.6.1-2ubuntu0.20.04.3) ...
#6 89.60 Selecting previously unselected package libtiff5:amd64.
#6 89.60 Preparing to unpack .../108-libtiff5_4.1.0+git191117-2ubuntu0.20.04.12_amd64.deb ...
#6 89.61 Unpacking libtiff5:amd64 (4.1.0+git191117-2ubuntu0.20.04.12) ...
#6 89.71 Selecting previously unselected package libxpm4:amd64.
#6 89.71 Preparing to unpack .../109-libxpm4_1%3a3.5.12-1ubuntu0.20.04.2_amd64.deb ...
#6 89.72 Unpacking libxpm4:amd64 (1:3.5.12-1ubuntu0.20.04.2) ...
#6 89.80 Selecting previously unselected package libgd3:amd64.
#6 89.80 Preparing to unpack .../110-libgd3_2.2.5-5.2ubuntu2.1_amd64.deb ...
#6 89.81 Unpacking libgd3:amd64 (2.2.5-5.2ubuntu2.1) ...
#6 89.92 Selecting previously unselected package nginx-common.
#6 89.92 Preparing to unpack .../111-nginx-common_1.18.0-0ubuntu1.4_all.deb ...
#6 89.99 Unpacking nginx-common (1.18.0-0ubuntu1.4) ...
#6 90.08 Selecting previously unselected package libnginx-mod-http-image-filter.
#6 90.08 Preparing to unpack .../112-libnginx-mod-http-image-filter_1.18.0-0ubuntu1.4_amd64.deb ...
#6 90.09 Unpacking libnginx-mod-http-image-filter (1.18.0-0ubuntu1.4) ...
#6 90.17 Selecting previously unselected package libxslt1.1:amd64.
#6 90.17 Preparing to unpack .../113-libxslt1.1_1.1.34-4ubuntu0.20.04.1_amd64.deb ...
#6 90.18 Unpacking libxslt1.1:amd64 (1.1.34-4ubuntu0.20.04.1) ...
#6 90.28 Selecting previously unselected package libnginx-mod-http-xslt-filter.
#6 90.28 Preparing to unpack .../114-libnginx-mod-http-xslt-filter_1.18.0-0ubuntu1.4_amd64.deb ...
#6 90.29 Unpacking libnginx-mod-http-xslt-filter (1.18.0-0ubuntu1.4) ...
#6 90.37 Selecting previously unselected package libnginx-mod-mail.
#6 90.37 Preparing to unpack .../115-libnginx-mod-mail_1.18.0-0ubuntu1.4_amd64.deb ...
#6 90.38 Unpacking libnginx-mod-mail (1.18.0-0ubuntu1.4) ...
#6 90.46 Selecting previously unselected package libnginx-mod-stream.
#6 90.46 Preparing to unpack .../116-libnginx-mod-stream_1.18.0-0ubuntu1.4_amd64.deb ...
#6 90.47 Unpacking libnginx-mod-stream (1.18.0-0ubuntu1.4) ...
#6 90.56 Selecting previously unselected package nginx-core.
#6 90.56 Preparing to unpack .../117-nginx-core_1.18.0-0ubuntu1.4_amd64.deb ...
#6 90.57 Unpacking nginx-core (1.18.0-0ubuntu1.4) ...
#6 90.69 Selecting previously unselected package nginx.
#6 90.70 Preparing to unpack .../118-nginx_1.18.0-0ubuntu1.4_all.deb ...
#6 90.71 Unpacking nginx (1.18.0-0ubuntu1.4) ...
#6 90.79 Setting up libexpat1:amd64 (2.2.9-1ubuntu0.6) ...
#6 90.81 Setting up libxau6:amd64 (1:1.0.9-0ubuntu1) ...
#6 90.84 Setting up libkeyutils1:amd64 (1.6-6ubuntu1.1) ...
#6 90.86 Setting up libpsl5:amd64 (0.21.0-1ubuntu1) ...
#6 90.88 Setting up perl-modules-5.30 (5.30.0-9ubuntu0.5) ...
#6 90.90 Setting up libssl1.1:amd64 (1.1.1f-1ubuntu2.22) ...
#6 91.02 Setting up libbrotli1:amd64 (1.0.7-6ubuntu0.1) ...
#6 91.04 Setting up libsqlite3-0:amd64 (3.31.1-4ubuntu0.6) ...
#6 91.06 Setting up binutils-common:amd64 (2.34-6ubuntu1.9) ...
#6 91.08 Setting up libnghttp2-14:amd64 (1.40.0-1ubuntu0.3) ...
#6 91.11 Setting up linux-libc-dev:amd64 (5.4.0-182.202) ...
#6 91.13 Setting up libctf-nobfd0:amd64 (2.34-6ubuntu1.9) ...
#6 91.15 Setting up nginx-common (1.18.0-0ubuntu1.4) ...
#6 91.69 Setting up libgomp1:amd64 (10.5.0-1ubuntu1~20.04) ...
#6 91.71 Setting up libldap-common (2.4.49+dfsg-2ubuntu1.10) ...
#6 91.74 Setting up libjbig0:amd64 (2.1-3.1ubuntu0.20.04.1) ...
#6 91.76 Setting up libcap2:amd64 (1:2.32-1ubuntu0.1) ...
#6 91.79 Setting up libkrb5support0:amd64 (1.17-6ubuntu4.4) ...
#6 91.81 Setting up libsasl2-modules-db:amd64 (2.1.27+dfsg-2ubuntu0.1) ...
#6 91.83 Setting up tzdata (2024a-0ubuntu0.20.04) ...
#6 91.96
#6 91.96 Current default time zone: 'Etc/UTC'
#6 91.96 Local time is now:      Thu May 16 20:22:03 UTC 2024.
#6 91.96 Universal Time is now:  Thu May 16 20:22:03 UTC 2024.
#6 91.96 Run 'dpkg-reconfigure tzdata' if you wish to change it.
#6 91.96
#6 91.99 Setting up libcap2-bin (1:2.32-1ubuntu0.1) ...
#6 92.01 Setting up libuv1:amd64 (1.34.2-1ubuntu1.5) ...
#6 92.03 Setting up libx11-data (2:1.6.9-2ubuntu1.6) ...
#6 92.05 Setting up make (4.2.1-1.2) ...
#6 92.08 Setting up libmpfr6:amd64 (4.0.2-1) ...
#6 92.10 Setting up librtmp1:amd64 (2.4+20151223.gitfa8646d.1-2build1) ...
#6 92.12 Setting up xz-utils (5.2.4-1ubuntu1.1) ...
#6 92.14 update-alternatives: using /usr/bin/xz to provide /usr/bin/lzma (lzma) in auto mode
#6 92.14 update-alternatives: warning: skip creation of /usr/share/man/man1/lzma.1.gz because associated file /usr/share/man/man1/xz.1.gz (of link group lzma) doesn't exist
#6 92.14 update-alternatives: warning: skip creation of /usr/share/man/man1/unlzma.1.gz because associated file /usr/share/man/man1/unxz.1.gz (of link group lzma) doesn't exist
#6 92.14 update-alternatives: warning: skip creation of /usr/share/man/man1/lzcat.1.gz because associated file /usr/share/man/man1/xzcat.1.gz (of link group lzma) doesn't exist
#6 92.14 update-alternatives: warning: skip creation of /usr/share/man/man1/lzmore.1.gz because associated file /usr/share/man/man1/xzmore.1.gz (of link group lzma) doesn't exist
#6 92.14 update-alternatives: warning: skip creation of /usr/share/man/man1/lzless.1.gz because associated file /usr/share/man/man1/xzless.1.gz (of link group lzma) doesn't exist
#6 92.14 update-alternatives: warning: skip creation of /usr/share/man/man1/lzdiff.1.gz because associated file /usr/share/man/man1/xzdiff.1.gz (of link group lzma) doesn't exist
#6 92.14 update-alternatives: warning: skip creation of /usr/share/man/man1/lzcmp.1.gz because associated file /usr/share/man/man1/xzcmp.1.gz (of link group lzma) doesn't exist
#6 92.14 update-alternatives: warning: skip creation of /usr/share/man/man1/lzgrep.1.gz because associated file /usr/share/man/man1/xzgrep.1.gz (of link group lzma) doesn't exist
#6 92.14 update-alternatives: warning: skip creation of /usr/share/man/man1/lzegrep.1.gz because associated file /usr/share/man/man1/xzegrep.1.gz (of link group lzma) doesn't exist
#6 92.14 update-alternatives: warning: skip creation of /usr/share/man/man1/lzfgrep.1.gz because associated file /usr/share/man/man1/xzfgrep.1.gz (of link group lzma) doesn't exist
#6 92.16 Setting up libquadmath0:amd64 (10.5.0-1ubuntu1~20.04) ...
#6 92.18 Setting up libpng16-16:amd64 (1.6.37-2) ...
#6 92.20 Setting up libmpc3:amd64 (1.1.0-1) ...
#6 92.22 Setting up libmnl0:amd64 (1.0.4-2) ...
#6 92.24 Setting up libatomic1:amd64 (10.5.0-1ubuntu1~20.04) ...
#6 92.26 Setting up patch (2.7.6-6) ...
#6 92.28 Setting up libwebp6:amd64 (0.6.1-2ubuntu0.20.04.3) ...
#6 92.30 Setting up fonts-dejavu-core (2.37-1) ...
#6 92.40 Setting up ucf (3.0038+nmu1) ...
#6 92.53 Setting up libk5crypto3:amd64 (1.17-6ubuntu4.4) ...
#6 92.55 Setting up libjpeg-turbo8:amd64 (2.0.3-0ubuntu1.20.04.3) ...
#6 92.57 Setting up libxtables12:amd64 (1.8.4-3ubuntu2.1) ...
#6 92.59 Setting up libsasl2-2:amd64 (2.1.27+dfsg-2ubuntu0.1) ...
#6 92.61 Setting up libroken18-heimdal:amd64 (7.7.0+dfsg-1ubuntu1.4) ...
#6 92.64 Setting up libubsan1:amd64 (10.5.0-1ubuntu1~20.04) ...
#6 92.66 Setting up librhash0:amd64 (1.3.9-1) ...
#6 92.68 Setting up libcrypt-dev:amd64 (1:4.4.10-10ubuntu4) ...
#6 92.70 Setting up git-man (1:2.25.1-1ubuntu3.11) ...
#6 92.72 Setting up libisl22:amd64 (0.22.1-1) ...
#6 92.74 Setting up cmake-data (3.16.3-1ubuntu1.20.04.1) ...
#6 92.77 Setting up libkrb5-3:amd64 (1.17-6ubuntu4.4) ...
#6 92.79 Setting up libbinutils:amd64 (2.34-6ubuntu1.9) ...
#6 92.81 Setting up libc-dev-bin (2.31-0ubuntu9.15) ...
#6 92.83 Setting up openssl (1.1.1f-1ubuntu2.22) ...
#6 92.86 Setting up libbsd0:amd64 (0.10.0-1) ...
#6 92.88 Setting up libelf1:amd64 (0.176-1.1ubuntu0.1) ...
#6 92.91 Setting up libcc1-0:amd64 (10.5.0-1ubuntu1~20.04) ...
#6 92.93 Setting up liblsan0:amd64 (10.5.0-1ubuntu1~20.04) ...
#6 92.95 Setting up libitm1:amd64 (10.5.0-1ubuntu1~20.04) ...
#6 92.97 Setting up libjsoncpp1:amd64 (1.7.4-3.1ubuntu2) ...
#6 92.99 Setting up libheimbase1-heimdal:amd64 (7.7.0+dfsg-1ubuntu1.4) ...
#6 93.01 Setting up libgdbm6:amd64 (1.18.1-5) ...
#6 93.03 Setting up gcc-9-base:amd64 (9.4.0-1ubuntu1~20.04.2) ...
#6 93.05 Setting up libtsan0:amd64 (10.5.0-1ubuntu1~20.04) ...
#6 93.07 Setting up libctf0:amd64 (2.34-6ubuntu1.9) ...
#6 93.09 Setting up libjpeg8:amd64 (8c-2ubuntu8) ...
#6 93.11 Setting up libnginx-mod-mail (1.18.0-0ubuntu1.4) ...
#6 93.16 Setting up libxdmcp6:amd64 (1:1.1.3-0ubuntu1) ...
#6 93.18 Setting up libxcb1:amd64 (1.14-2) ...
#6 93.20 Setting up wget (1.20.3-1ubuntu2) ...
#6 93.22 Setting up fontconfig-config (2.13.1-2ubuntu3) ...
#6 93.55 Setting up iproute2 (5.5.0-1ubuntu1) ...
#6 93.75 Setting up libicu66:amd64 (66.1-2ubuntu2.1) ...
#6 93.77 Setting up libasn1-8-heimdal:amd64 (7.7.0+dfsg-1ubuntu1.4) ...
#6 93.79 Setting up libnginx-mod-stream (1.18.0-0ubuntu1.4) ...
#6 93.84 Setting up libhcrypto4-heimdal:amd64 (7.7.0+dfsg-1ubuntu1.4) ...
#6 93.86 Setting up ca-certificates (20230311ubuntu0.20.04.1) ...
#6 95.13 Updating certificates in /etc/ssl/certs...
#6 96.88 137 added, 0 removed; done.
#6 96.95 Setting up libasan5:amd64 (9.4.0-1ubuntu1~20.04.2) ...
#6 96.98 Setting up libwind0-heimdal:amd64 (7.7.0+dfsg-1ubuntu1.4) ...
#6 97.00 Setting up libfreetype6:amd64 (2.10.1-2ubuntu0.3) ...
#6 97.02 Setting up libgssapi-krb5-2:amd64 (1.17-6ubuntu4.4) ...
#6 97.04 Setting up libgdbm-compat4:amd64 (1.18.1-5) ...
#6 97.06 Setting up libssh-4:amd64 (0.9.3-2ubuntu2.5) ...
#6 97.08 Setting up cpp-9 (9.4.0-1ubuntu1~20.04.2) ...
#6 97.10 Setting up libperl5.30:amd64 (5.30.0-9ubuntu0.5) ...
#6 97.12 Setting up libc6-dev:amd64 (2.31-0ubuntu9.15) ...
#6 97.14 Setting up libx11-6:amd64 (2:1.6.9-2ubuntu1.6) ...
#6 97.16 Setting up libtiff5:amd64 (4.1.0+git191117-2ubuntu0.20.04.12) ...
#6 97.18 Setting up libfontconfig1:amd64 (2.13.1-2ubuntu3) ...
#6 97.20 Setting up libxml2:amd64 (2.9.10+dfsg-5ubuntu0.20.04.7) ...
#6 97.22 Setting up binutils-x86-64-linux-gnu (2.34-6ubuntu1.9) ...
#6 97.24 Setting up libxpm4:amd64 (1:3.5.12-1ubuntu0.20.04.2) ...
#6 97.26 Setting up libarchive13:amd64 (3.4.0-2ubuntu1.2) ...
#6 97.28 Setting up libhx509-5-heimdal:amd64 (7.7.0+dfsg-1ubuntu1.4) ...
#6 97.30 Setting up binutils (2.34-6ubuntu1.9) ...
#6 97.32 Setting up libgcc-9-dev:amd64 (9.4.0-1ubuntu1~20.04.2) ...
#6 97.34 Setting up perl (5.30.0-9ubuntu0.5) ...
#6 97.39 Setting up libgd3:amd64 (2.2.5-5.2ubuntu2.1) ...
#6 97.41 Setting up libdpkg-perl (1.19.7ubuntu3.2) ...
#6 97.43 Setting up cpp (4:9.3.0-1ubuntu2) ...
#6 97.46 Setting up libxslt1.1:amd64 (1.1.34-4ubuntu0.20.04.1) ...
#6 97.48 Setting up libkrb5-26-heimdal:amd64 (7.7.0+dfsg-1ubuntu1.4) ...
#6 97.50 Setting up gcc-9 (9.4.0-1ubuntu1~20.04.2) ...
#6 97.52 Setting up libstdc++-9-dev:amd64 (9.4.0-1ubuntu1~20.04.2) ...
#6 97.54 Setting up libnginx-mod-http-image-filter (1.18.0-0ubuntu1.4) ...
#6 97.59 Setting up libheimntlm0-heimdal:amd64 (7.7.0+dfsg-1ubuntu1.4) ...
#6 97.61 Setting up gcc (4:9.3.0-1ubuntu2) ...
#6 97.65 Setting up dpkg-dev (1.19.7ubuntu3.2) ...
#6 97.69 Setting up liberror-perl (0.17029-1) ...
#6 97.71 Setting up libnginx-mod-http-xslt-filter (1.18.0-0ubuntu1.4) ...
#6 97.75 Setting up libgssapi3-heimdal:amd64 (7.7.0+dfsg-1ubuntu1.4) ...
#6 97.77 Setting up g++-9 (9.4.0-1ubuntu1~20.04.2) ...
#6 97.79 Setting up g++ (4:9.3.0-1ubuntu2) ...
#6 97.85 update-alternatives: using /usr/bin/g++ to provide /usr/bin/c++ (c++) in auto mode
#6 97.85 update-alternatives: warning: skip creation of /usr/share/man/man1/c++.1.gz because associated file /usr/share/man/man1/g++.1.gz (of link group c++) doesn't exist
#6 97.87 Setting up build-essential (12.8ubuntu1.1) ...
#6 97.89 Setting up nginx-core (1.18.0-0ubuntu1.4) ...
#6 97.98 invoke-rc.d: could not determine current runlevel
#6 97.99 invoke-rc.d: policy-rc.d denied execution of start.
#6 98.00 Setting up nginx (1.18.0-0ubuntu1.4) ...
#6 98.02 Setting up libldap-2.4-2:amd64 (2.4.49+dfsg-2ubuntu1.10) ...
#6 98.04 Setting up libcurl3-gnutls:amd64 (7.68.0-1ubuntu2.22) ...
#6 98.06 Setting up git (1:2.25.1-1ubuntu3.11) ...
#6 98.14 Setting up libcurl4:amd64 (7.68.0-1ubuntu2.22) ...
#6 98.16 Setting up curl (7.68.0-1ubuntu2.22) ...
#6 98.18 Setting up cmake (3.16.3-1ubuntu1.20.04.1) ...
#6 98.20 Processing triggers for libc-bin (2.31-0ubuntu9.15) ...
#6 98.25 Processing triggers for ca-certificates (20230311ubuntu0.20.04.1) ...
#6 98.26 Updating certificates in /etc/ssl/certs...
#6 99.63 0 added, 0 removed; done.
#6 99.63 Running hooks in /etc/ca-certificates/update.d...
#6 99.63 done.
#6 DONE 100.6s

#7 [ 3/17] RUN DEBIAN_FRONTEND=noninteractive TZ=Etc/UTC apt-get -y install tzdata     libssl-dev zlib1g-dev libbz2-dev libreadline-dev libsqlite3-dev wget curl llvm     libncursesw5-dev xz-utils tk-dev libxml2-dev libxmlsec1-dev libffi-dev liblzma-dev
#7 0.464 Reading package lists...
#7 1.538 Building dependency tree...
#7 1.711 Reading state information...
#7 1.915 curl is already the newest version (7.68.0-1ubuntu2.22).
#7 1.915 tzdata is already the newest version (2024a-0ubuntu0.20.04).
#7 1.915 tzdata set to manually installed.
#7 1.915 wget is already the newest version (1.20.3-1ubuntu2).
#7 1.915 xz-utils is already the newest version (5.2.4-1ubuntu1.1).
#7 1.915 xz-utils set to manually installed.
#7 1.915 The following additional packages will be installed:
#7 1.915   binfmt-support bzip2-doc file icu-devtools libclang-cpp10 libdrm-amdgpu1
#7 1.915   libdrm-common libdrm-intel1 libdrm-nouveau2 libdrm-radeon1 libdrm2 libedit2
#7 1.915   libevent-2.1-7 libexpat1-dev libfontconfig1-dev libfontenc1 libfreetype-dev
#7 1.915   libfreetype6-dev libgcrypt20-dev libgl1 libgl1-mesa-dri libglapi-mesa
#7 1.915   libglib2.0-0 libglib2.0-data libglvnd0 libglx-mesa0 libglx0 libgmp-dev
#7 1.915   libgmpxx4ldbl libgnutls-dane0 libgnutls-openssl27 libgnutls28-dev
#7 1.915   libgnutlsxx28 libgpg-error-dev libice-dev libice6 libicu-dev libidn2-dev
#7 1.915   libllvm10 libllvm12 libmagic-mgc libmagic1 libmpdec2 libncurses-dev libnspr4
#7 1.915   libnspr4-dev libnss3 libnss3-dev libp11-kit-dev libpciaccess0 libpfm4
#7 1.915   libpipeline1 libpng-dev libpng-tools libpthread-stubs0-dev libpython3-stdlib
#7 1.915   libpython3.8-minimal libpython3.8-stdlib libreadline8 libsensors-config
#7 1.915   libsensors5 libsm-dev libsm6 libtasn1-6-dev libtasn1-doc libtcl8.6 libtk8.6
#7 1.915   libunbound8 libutempter0 libvulkan1 libwayland-client0 libx11-dev
#7 1.915   libx11-xcb1 libxau-dev libxaw7 libxcb-dri2-0 libxcb-dri3-0 libxcb-glx0
#7 1.915   libxcb-present0 libxcb-randr0 libxcb-shape0 libxcb-shm0 libxcb-sync1
#7 1.915   libxcb-xfixes0 libxcb1-dev libxcomposite1 libxdmcp-dev libxext-dev libxext6
#7 1.915   libxfixes3 libxft-dev libxft2 libxi6 libxinerama1 libxkbfile1 libxmlsec1
#7 1.915   libxmlsec1-gcrypt libxmlsec1-gnutls libxmlsec1-nss libxmlsec1-openssl
#7 1.915   libxmu6 libxmuu1 libxrandr2 libxrender-dev libxrender1 libxshmfence1
#7 1.915   libxslt1-dev libxss-dev libxss1 libxt-dev libxt6 libxtst6 libxv1
#7 1.915   libxxf86dga1 libxxf86vm1 libyaml-0-2 libz3-4 libz3-dev llvm-10 llvm-10-dev
#7 1.915   llvm-10-runtime llvm-10-tools llvm-runtime mesa-vulkan-drivers mime-support
#7 1.915   nettle-dev pkg-config python3 python3-minimal python3-pkg-resources
#7 1.915   python3-pygments python3-yaml python3.8 python3.8-minimal readline-common
#7 1.915   shared-mime-info tcl tcl-dev tcl8.6 tcl8.6-dev tk tk8.6 tk8.6-dev uuid-dev
#7 1.915   x11-common x11-utils x11proto-core-dev x11proto-dev x11proto-scrnsaver-dev
#7 1.915   x11proto-xext-dev xbitmaps xdg-user-dirs xorg-sgml-doctools xterm xtrans-dev
#7 1.917 Suggested packages:
#7 1.917   freetype2-doc libgcrypt20-doc gmp-doc libgmp10-doc libmpfr-dev dns-root-data
#7 1.917   gnutls-bin gnutls-doc libice-doc icu-doc liblzma-doc ncurses-doc pciutils
#7 1.917   readline-doc lm-sensors libsm-doc sqlite3-doc libssl-doc libx11-doc
#7 1.917   libxcb-doc libxext-doc libxt-doc llvm-10-doc python3-doc python3-tk
#7 1.917   python3-venv python3-setuptools python-pygments-doc ttf-bitstream-vera
#7 1.917   python3.8-venv python3.8-doc tcl-doc tcl-tclreadline tcl8.6-doc tk-doc
#7 1.917   tk8.6-doc mesa-utils xfonts-cyrillic
#7 2.228 The following NEW packages will be installed:
#7 2.228   binfmt-support bzip2-doc file icu-devtools libbz2-dev libclang-cpp10
#7 2.228   libdrm-amdgpu1 libdrm-common libdrm-intel1 libdrm-nouveau2 libdrm-radeon1
#7 2.228   libdrm2 libedit2 libevent-2.1-7 libexpat1-dev libffi-dev libfontconfig1-dev
#7 2.228   libfontenc1 libfreetype-dev libfreetype6-dev libgcrypt20-dev libgl1
#7 2.228   libgl1-mesa-dri libglapi-mesa libglib2.0-0 libglib2.0-data libglvnd0
#7 2.228   libglx-mesa0 libglx0 libgmp-dev libgmpxx4ldbl libgnutls-dane0
#7 2.228   libgnutls-openssl27 libgnutls28-dev libgnutlsxx28 libgpg-error-dev
#7 2.228   libice-dev libice6 libicu-dev libidn2-dev libllvm10 libllvm12 liblzma-dev
#7 2.228   libmagic-mgc libmagic1 libmpdec2 libncurses-dev libncursesw5-dev libnspr4
#7 2.228   libnspr4-dev libnss3 libnss3-dev libp11-kit-dev libpciaccess0 libpfm4
#7 2.228   libpipeline1 libpng-dev libpng-tools libpthread-stubs0-dev libpython3-stdlib
#7 2.229   libpython3.8-minimal libpython3.8-stdlib libreadline-dev libreadline8
#7 2.229   libsensors-config libsensors5 libsm-dev libsm6 libsqlite3-dev libssl-dev
#7 2.229   libtasn1-6-dev libtasn1-doc libtcl8.6 libtk8.6 libunbound8 libutempter0
#7 2.229   libvulkan1 libwayland-client0 libx11-dev libx11-xcb1 libxau-dev libxaw7
#7 2.229   libxcb-dri2-0 libxcb-dri3-0 libxcb-glx0 libxcb-present0 libxcb-randr0
#7 2.229   libxcb-shape0 libxcb-shm0 libxcb-sync1 libxcb-xfixes0 libxcb1-dev
#7 2.229   libxcomposite1 libxdmcp-dev libxext-dev libxext6 libxfixes3 libxft-dev
#7 2.229   libxft2 libxi6 libxinerama1 libxkbfile1 libxml2-dev libxmlsec1
#7 2.229   libxmlsec1-dev libxmlsec1-gcrypt libxmlsec1-gnutls libxmlsec1-nss
#7 2.229   libxmlsec1-openssl libxmu6 libxmuu1 libxrandr2 libxrender-dev libxrender1
#7 2.229   libxshmfence1 libxslt1-dev libxss-dev libxss1 libxt-dev libxt6 libxtst6
#7 2.229   libxv1 libxxf86dga1 libxxf86vm1 libyaml-0-2 libz3-4 libz3-dev llvm llvm-10
#7 2.229   llvm-10-dev llvm-10-runtime llvm-10-tools llvm-runtime mesa-vulkan-drivers
#7 2.229   mime-support nettle-dev pkg-config python3 python3-minimal
#7 2.229   python3-pkg-resources python3-pygments python3-yaml python3.8
#7 2.229   python3.8-minimal readline-common shared-mime-info tcl tcl-dev tcl8.6
#7 2.229   tcl8.6-dev tk tk-dev tk8.6 tk8.6-dev uuid-dev x11-common x11-utils
#7 2.229   x11proto-core-dev x11proto-dev x11proto-scrnsaver-dev x11proto-xext-dev
#7 2.229   xbitmaps xdg-user-dirs xorg-sgml-doctools xterm xtrans-dev zlib1g-dev
#7 2.493 0 upgraded, 167 newly installed, 0 to remove and 0 not upgraded.
#7 2.493 Need to get 138 MB of archives.
#7 2.493 After this operation, 1062 MB of additional disk space will be used.
#7 2.493 Get:1 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libpython3.8-minimal amd64 3.8.10-0ubuntu1~20.04.9 [718 kB]
#7 3.260 Get:2 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 python3.8-minimal amd64 3.8.10-0ubuntu1~20.04.9 [1890 kB]
#7 4.013 Get:3 http://archive.ubuntu.com/ubuntu focal/main amd64 python3-minimal amd64 3.8.2-0ubuntu2 [23.6 kB]
#7 4.022 Get:4 http://archive.ubuntu.com/ubuntu focal/main amd64 mime-support all 3.64ubuntu1 [30.6 kB]
#7 4.038 Get:5 http://archive.ubuntu.com/ubuntu focal/main amd64 libmpdec2 amd64 2.4.2-3 [81.1 kB]
#7 4.067 Get:6 http://archive.ubuntu.com/ubuntu focal/main amd64 readline-common all 8.0-4 [53.5 kB]
#7 4.089 Get:7 http://archive.ubuntu.com/ubuntu focal/main amd64 libreadline8 amd64 8.0-4 [131 kB]
#7 4.143 Get:8 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libpython3.8-stdlib amd64 3.8.10-0ubuntu1~20.04.9 [1674 kB]
#7 4.808 Get:9 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 python3.8 amd64 3.8.10-0ubuntu1~20.04.9 [387 kB]
#7 4.961 Get:10 http://archive.ubuntu.com/ubuntu focal/main amd64 libpython3-stdlib amd64 3.8.2-0ubuntu2 [7068 B]
#7 4.963 Get:11 http://archive.ubuntu.com/ubuntu focal/main amd64 python3 amd64 3.8.2-0ubuntu2 [47.6 kB]
#7 4.983 Get:12 http://archive.ubuntu.com/ubuntu focal/main amd64 libmagic-mgc amd64 1:5.38-4 [218 kB]
#7 5.068 Get:13 http://archive.ubuntu.com/ubuntu focal/main amd64 libmagic1 amd64 1:5.38-4 [75.9 kB]
#7 5.100 Get:14 http://archive.ubuntu.com/ubuntu focal/main amd64 file amd64 1:5.38-4 [23.3 kB]
#7 5.108 Get:15 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libglib2.0-0 amd64 2.64.6-1~ubuntu20.04.7 [1289 kB]
#7 5.621 Get:16 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libglib2.0-data all 2.64.6-1~ubuntu20.04.7 [5944 B]
#7 5.623 Get:17 http://archive.ubuntu.com/ubuntu focal/main amd64 libyaml-0-2 amd64 0.2.2-1 [48.9 kB]
#7 5.640 Get:18 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 python3-pkg-resources all 45.2.0-1ubuntu0.1 [130 kB]
#7 5.693 Get:19 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 python3-yaml amd64 5.3.1-1ubuntu0.1 [136 kB]
#7 5.752 Get:20 http://archive.ubuntu.com/ubuntu focal/main amd64 shared-mime-info amd64 1.15-1 [430 kB]
#7 5.921 Get:21 http://archive.ubuntu.com/ubuntu focal/main amd64 xdg-user-dirs amd64 0.17-2ubuntu1 [48.3 kB]
#7 5.940 Get:22 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libdrm-common all 2.4.107-8ubuntu1~20.04.2 [5396 B]
#7 5.941 Get:23 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libdrm2 amd64 2.4.107-8ubuntu1~20.04.2 [34.1 kB]
#7 5.952 Get:24 http://archive.ubuntu.com/ubuntu focal/main amd64 libedit2 amd64 3.1-20191231-1 [87.0 kB]
#7 5.991 Get:25 http://archive.ubuntu.com/ubuntu focal/main amd64 libpipeline1 amd64 1.5.2-2build1 [27.7 kB]
#7 6.000 Get:26 http://archive.ubuntu.com/ubuntu focal/main amd64 libxext6 amd64 2:1.3.4-0ubuntu1 [29.1 kB]
#7 6.016 Get:27 http://archive.ubuntu.com/ubuntu focal/main amd64 libxmuu1 amd64 2:1.1.3-0ubuntu1 [9728 B]
#7 6.019 Get:28 http://archive.ubuntu.com/ubuntu focal/universe amd64 binfmt-support amd64 2.2.0-2 [58.2 kB]
#7 6.041 Get:29 http://archive.ubuntu.com/ubuntu focal/main amd64 bzip2-doc all 1.0.8-2 [501 kB]
#7 6.241 Get:30 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 icu-devtools amd64 66.1-2ubuntu2.1 [189 kB]
#7 6.320 Get:31 http://archive.ubuntu.com/ubuntu focal/main amd64 libbz2-dev amd64 1.0.8-2 [30.2 kB]
#7 6.329 Get:32 http://archive.ubuntu.com/ubuntu focal/main amd64 libllvm10 amd64 1:10.0.0-4ubuntu1 [15.3 MB]
#7 12.57 Get:33 http://archive.ubuntu.com/ubuntu focal/universe amd64 libclang-cpp10 amd64 1:10.0.0-4ubuntu1 [9944 kB]
#7 16.83 Get:34 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libdrm-amdgpu1 amd64 2.4.107-8ubuntu1~20.04.2 [18.6 kB]
#7 16.83 Get:35 http://archive.ubuntu.com/ubuntu focal/main amd64 libpciaccess0 amd64 0.16-0ubuntu1 [17.9 kB]
#7 16.84 Get:36 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libdrm-intel1 amd64 2.4.107-8ubuntu1~20.04.2 [60.3 kB]
#7 16.86 Get:37 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libdrm-nouveau2 amd64 2.4.107-8ubuntu1~20.04.2 [16.6 kB]
#7 16.87 Get:38 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libdrm-radeon1 amd64 2.4.107-8ubuntu1~20.04.2 [19.7 kB]
#7 16.87 Get:39 http://archive.ubuntu.com/ubuntu focal/main amd64 libevent-2.1-7 amd64 2.1.11-stable-1 [138 kB]
#7 16.93 Get:40 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libexpat1-dev amd64 2.2.9-1ubuntu0.6 [116 kB]
#7 16.98 Get:41 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 zlib1g-dev amd64 1:1.2.11.dfsg-2ubuntu1.5 [155 kB]
#7 17.04 Get:42 http://archive.ubuntu.com/ubuntu focal/main amd64 libpng-dev amd64 1.6.37-2 [175 kB]
#7 17.11 Get:43 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libfreetype-dev amd64 2.10.1-2ubuntu0.3 [493 kB]
#7 17.30 Get:44 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libfreetype6-dev amd64 2.10.1-2ubuntu0.3 [9808 B]
#7 17.31 Get:45 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 uuid-dev amd64 2.34-0.1ubuntu9.6 [33.6 kB]
#7 17.32 Get:46 http://archive.ubuntu.com/ubuntu focal/main amd64 pkg-config amd64 0.29.1-0ubuntu4 [45.5 kB]
#7 17.34 Get:47 http://archive.ubuntu.com/ubuntu focal/main amd64 libfontconfig1-dev amd64 2.13.1-2ubuntu3 [731 kB]
#7 17.63 Get:48 http://archive.ubuntu.com/ubuntu focal/main amd64 libfontenc1 amd64 1:1.1.4-0ubuntu1 [14.0 kB]
#7 17.64 Get:49 http://archive.ubuntu.com/ubuntu focal/main amd64 libgpg-error-dev amd64 1.37-1 [109 kB]
#7 17.68 Get:50 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libgcrypt20-dev amd64 1.8.5-5ubuntu1.1 [471 kB]
#7 17.87 Get:51 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libglapi-mesa amd64 21.2.6-0ubuntu0.1~20.04.2 [27.4 kB]
#7 17.88 Get:52 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libllvm12 amd64 1:12.0.0-3ubuntu1~20.04.5 [18.8 MB]
#7 25.65 Get:53 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libsensors-config all 1:3.6.0-2ubuntu1.1 [6052 B]
#7 25.67 Get:54 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libsensors5 amd64 1:3.6.0-2ubuntu1.1 [27.2 kB]
#7 25.67 Get:55 http://archive.ubuntu.com/ubuntu focal/main amd64 libvulkan1 amd64 1.2.131.2-1 [93.3 kB]
#7 25.68 Get:56 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libgl1-mesa-dri amd64 21.2.6-0ubuntu0.1~20.04.2 [11.0 MB]
#7 30.46 Get:57 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libx11-xcb1 amd64 2:1.6.9-2ubuntu1.6 [9448 B]
#7 30.46 Get:58 http://archive.ubuntu.com/ubuntu focal/main amd64 libxcb-dri2-0 amd64 1.14-2 [6920 B]
#7 30.47 Get:59 http://archive.ubuntu.com/ubuntu focal/main amd64 libxcb-dri3-0 amd64 1.14-2 [6552 B]
#7 30.47 Get:60 http://archive.ubuntu.com/ubuntu focal/main amd64 libxcb-glx0 amd64 1.14-2 [22.1 kB]
#7 30.57 Get:61 http://archive.ubuntu.com/ubuntu focal/main amd64 libxcb-present0 amd64 1.14-2 [5560 B]
#7 30.62 Get:62 http://archive.ubuntu.com/ubuntu focal/main amd64 libxcb-shm0 amd64 1.14-2 [5584 B]
#7 30.67 Get:63 http://archive.ubuntu.com/ubuntu focal/main amd64 libxcb-sync1 amd64 1.14-2 [8884 B]
#7 30.73 Get:64 http://archive.ubuntu.com/ubuntu focal/main amd64 libxcb-xfixes0 amd64 1.14-2 [9296 B]
#7 30.79 Get:65 http://archive.ubuntu.com/ubuntu focal/main amd64 libxfixes3 amd64 1:5.0.3-2 [10.9 kB]
#7 30.86 Get:66 http://archive.ubuntu.com/ubuntu focal/main amd64 libxshmfence1 amd64 1.3-1 [5028 B]
#7 30.90 Get:67 http://archive.ubuntu.com/ubuntu focal/main amd64 libxxf86vm1 amd64 1:1.1.4-1build1 [10.2 kB]
#7 30.96 Get:68 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libglx-mesa0 amd64 21.2.6-0ubuntu0.1~20.04.2 [137 kB]
#7 31.14 Get:69 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libgmpxx4ldbl amd64 2:6.2.0+dfsg-4ubuntu0.1 [9144 B]
#7 31.19 Get:70 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libgmp-dev amd64 2:6.2.0+dfsg-4ubuntu0.1 [320 kB]
#7 31.31 Get:71 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libgnutls-openssl27 amd64 3.6.13-2ubuntu1.11 [29.8 kB]
#7 31.40 Get:72 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libunbound8 amd64 1.9.4-2ubuntu1.5 [353 kB]
#7 31.66 Get:73 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libgnutls-dane0 amd64 3.6.13-2ubuntu1.11 [29.1 kB]
#7 31.71 Get:74 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libgnutlsxx28 amd64 3.6.13-2ubuntu1.11 [14.7 kB]
#7 31.76 Get:75 http://archive.ubuntu.com/ubuntu focal/main amd64 libidn2-dev amd64 2.2.0-2 [64.6 kB]
#7 31.81 Get:76 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libp11-kit-dev amd64 0.23.20-1ubuntu0.1 [65.2 kB]
#7 31.86 Get:77 http://archive.ubuntu.com/ubuntu focal/main amd64 libtasn1-6-dev amd64 4.16.0-2 [85.8 kB]
#7 31.92 Get:78 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 nettle-dev amd64 3.5.1+really3.5.1-2ubuntu0.2 [987 kB]
#7 32.16 Get:79 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libgnutls28-dev amd64 3.6.13-2ubuntu1.11 [873 kB]
#7 32.38 Get:80 http://archive.ubuntu.com/ubuntu focal/main amd64 x11-common all 1:7.7+19ubuntu14 [22.3 kB]
#7 32.43 Get:81 http://archive.ubuntu.com/ubuntu focal/main amd64 libice6 amd64 2:1.0.10-0ubuntu1 [41.0 kB]
#7 32.48 Get:82 http://archive.ubuntu.com/ubuntu focal/main amd64 xorg-sgml-doctools all 1:1.11-1 [12.9 kB]
#7 32.57 Get:83 http://archive.ubuntu.com/ubuntu focal/main amd64 x11proto-dev all 2019.2-1ubuntu1 [594 kB]
#7 32.88 Get:84 http://archive.ubuntu.com/ubuntu focal/main amd64 x11proto-core-dev all 2019.2-1ubuntu1 [2620 B]
#7 32.92 Get:85 http://archive.ubuntu.com/ubuntu focal/main amd64 libice-dev amd64 2:1.0.10-0ubuntu1 [47.8 kB]
#7 32.97 Get:86 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libicu-dev amd64 66.1-2ubuntu2.1 [9451 kB]
#7 34.76 Get:87 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libncurses-dev amd64 6.2-0ubuntu2.1 [340 kB]
#7 34.86 Get:88 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libncursesw5-dev amd64 6.2-0ubuntu2.1 [988 B]
#7 34.91 Get:89 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libnspr4 amd64 2:4.35-0ubuntu0.20.04.1 [108 kB]
#7 34.97 Get:90 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libnspr4-dev amd64 2:4.35-0ubuntu0.20.04.1 [207 kB]
#7 35.05 Get:91 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libnss3 amd64 2:3.98-0ubuntu0.20.04.2 [1391 kB]
#7 35.35 Get:92 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libnss3-dev amd64 2:3.98-0ubuntu0.20.04.2 [246 kB]
#7 35.44 Get:93 http://archive.ubuntu.com/ubuntu focal/main amd64 libpng-tools amd64 1.6.37-2 [26.1 kB]
#7 35.53 Get:94 http://archive.ubuntu.com/ubuntu focal/main amd64 libpthread-stubs0-dev amd64 0.4-1 [5384 B]
#7 35.57 Get:95 http://archive.ubuntu.com/ubuntu focal/main amd64 libreadline-dev amd64 8.0-4 [141 kB]
#7 35.75 Get:96 http://archive.ubuntu.com/ubuntu focal/main amd64 libsm6 amd64 2:1.2.3-1 [16.1 kB]
#7 35.80 Get:97 http://archive.ubuntu.com/ubuntu focal/main amd64 libsm-dev amd64 2:1.2.3-1 [17.0 kB]
#7 35.85 Get:98 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libsqlite3-dev amd64 3.31.1-4ubuntu0.6 [696 kB]
#7 36.05 Get:99 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libssl-dev amd64 1.1.1f-1ubuntu2.22 [1586 kB]
#7 36.41 Get:100 http://archive.ubuntu.com/ubuntu focal/main amd64 libtcl8.6 amd64 8.6.10+dfsg-1 [902 kB]
#7 36.61 Get:101 http://archive.ubuntu.com/ubuntu focal/main amd64 libxrender1 amd64 1:0.9.10-1 [18.7 kB]
#7 36.66 Get:102 http://archive.ubuntu.com/ubuntu focal/main amd64 libxft2 amd64 2.3.3-0ubuntu1 [39.2 kB]
#7 36.71 Get:103 http://archive.ubuntu.com/ubuntu focal/main amd64 libxss1 amd64 1:1.2.3-1 [8140 B]
#7 36.75 Get:104 http://archive.ubuntu.com/ubuntu focal/main amd64 libtk8.6 amd64 8.6.10-1 [714 kB]
#7 36.96 Get:105 http://archive.ubuntu.com/ubuntu focal/main amd64 libutempter0 amd64 1.1.6-4 [8256 B]
#7 37.01 Get:106 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libwayland-client0 amd64 1.18.0-1ubuntu0.1 [23.9 kB]
#7 37.10 Get:107 http://archive.ubuntu.com/ubuntu focal/main amd64 libxau-dev amd64 1:1.0.9-0ubuntu1 [9552 B]
#7 37.15 Get:108 http://archive.ubuntu.com/ubuntu focal/main amd64 libxdmcp-dev amd64 1:1.1.3-0ubuntu1 [25.3 kB]
#7 37.21 Get:109 http://archive.ubuntu.com/ubuntu focal/main amd64 xtrans-dev all 1.4.0-1 [68.9 kB]
#7 37.30 Get:110 http://archive.ubuntu.com/ubuntu focal/main amd64 libxcb1-dev amd64 1.14-2 [80.5 kB]
#7 37.36 Get:111 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libx11-dev amd64 2:1.6.9-2ubuntu1.6 [648 kB]
#7 37.54 Get:112 http://archive.ubuntu.com/ubuntu focal/main amd64 libxt6 amd64 1:1.1.5-1 [160 kB]
#7 37.61 Get:113 http://archive.ubuntu.com/ubuntu focal/main amd64 libxmu6 amd64 2:1.1.3-0ubuntu1 [45.8 kB]
#7 37.66 Get:114 http://archive.ubuntu.com/ubuntu focal/main amd64 libxaw7 amd64 2:1.0.13-1 [173 kB]
#7 37.73 Get:115 http://archive.ubuntu.com/ubuntu focal/main amd64 libxcb-randr0 amd64 1.14-2 [16.3 kB]
#7 37.82 Get:116 http://archive.ubuntu.com/ubuntu focal/main amd64 libxcb-shape0 amd64 1.14-2 [5928 B]
#7 37.86 Get:117 http://archive.ubuntu.com/ubuntu focal/main amd64 libxcomposite1 amd64 1:0.4.5-1 [6976 B]
#7 37.91 Get:118 http://archive.ubuntu.com/ubuntu focal/main amd64 x11proto-xext-dev all 2019.2-1ubuntu1 [2616 B]
#7 37.95 Get:119 http://archive.ubuntu.com/ubuntu focal/main amd64 libxext-dev amd64 2:1.3.4-0ubuntu1 [82.2 kB]
#7 38.11 Get:120 http://archive.ubuntu.com/ubuntu focal/main amd64 libxrender-dev amd64 1:0.9.10-1 [24.9 kB]
#7 38.17 Get:121 http://archive.ubuntu.com/ubuntu focal/main amd64 libxft-dev amd64 2.3.3-0ubuntu1 [49.1 kB]
#7 38.23 Get:122 http://archive.ubuntu.com/ubuntu focal/main amd64 libxi6 amd64 2:1.7.10-0ubuntu1 [29.9 kB]
#7 38.28 Get:123 http://archive.ubuntu.com/ubuntu focal/main amd64 libxinerama1 amd64 2:1.1.4-2 [6904 B]
#7 38.33 Get:124 http://archive.ubuntu.com/ubuntu focal/main amd64 libxkbfile1 amd64 1:1.1.0-1 [65.3 kB]
#7 38.40 Get:125 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libxml2-dev amd64 2.9.10+dfsg-5ubuntu0.20.04.7 [735 kB]
#7 38.62 Get:126 http://archive.ubuntu.com/ubuntu focal/main amd64 libxmlsec1 amd64 1.2.28-2 [127 kB]
#7 38.72 Get:127 http://archive.ubuntu.com/ubuntu focal/main amd64 libxmlsec1-gcrypt amd64 1.2.28-2 [41.5 kB]
#7 38.82 Get:128 http://archive.ubuntu.com/ubuntu focal/main amd64 libxmlsec1-gnutls amd64 1.2.28-2 [33.4 kB]
#7 38.88 Get:129 http://archive.ubuntu.com/ubuntu focal/main amd64 libxmlsec1-nss amd64 1.2.28-2 [61.8 kB]
#7 38.96 Get:130 http://archive.ubuntu.com/ubuntu focal/main amd64 libxmlsec1-openssl amd64 1.2.28-2 [75.9 kB]
#7 39.03 Get:131 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libxslt1-dev amd64 1.1.34-4ubuntu0.20.04.1 [219 kB]
#7 39.14 Get:132 http://archive.ubuntu.com/ubuntu focal/main amd64 libxmlsec1-dev amd64 1.2.28-2 [387 kB]
#7 39.25 Get:133 http://archive.ubuntu.com/ubuntu focal/main amd64 libxrandr2 amd64 2:1.5.2-0ubuntu1 [18.5 kB]
#7 39.29 Get:134 http://archive.ubuntu.com/ubuntu focal/main amd64 x11proto-scrnsaver-dev all 2019.2-1ubuntu1 [2624 B]
#7 39.33 Get:135 http://archive.ubuntu.com/ubuntu focal/main amd64 libxss-dev amd64 1:1.2.3-1 [11.9 kB]
#7 39.37 Get:136 http://archive.ubuntu.com/ubuntu focal/main amd64 libxt-dev amd64 1:1.1.5-1 [395 kB]
#7 39.50 Get:137 http://archive.ubuntu.com/ubuntu focal/main amd64 libxtst6 amd64 2:1.2.3-1 [12.8 kB]
#7 39.59 Get:138 http://archive.ubuntu.com/ubuntu focal/main amd64 libxv1 amd64 2:1.0.11-1 [10.7 kB]
#7 39.64 Get:139 http://archive.ubuntu.com/ubuntu focal/main amd64 libxxf86dga1 amd64 2:1.1.5-0ubuntu1 [12.0 kB]
#7 39.69 Get:140 http://archive.ubuntu.com/ubuntu focal/universe amd64 llvm-10-runtime amd64 1:10.0.0-4ubuntu1 [180 kB]
#7 39.88 Get:141 http://archive.ubuntu.com/ubuntu focal/universe amd64 llvm-runtime amd64 1:10.0-50~exp1 [2916 B]
#7 39.93 Get:142 http://archive.ubuntu.com/ubuntu focal/main amd64 libpfm4 amd64 4.10.1+git20-g7700f49-2 [266 kB]
#7 40.02 Get:143 http://archive.ubuntu.com/ubuntu focal/universe amd64 llvm-10 amd64 1:10.0.0-4ubuntu1 [5214 kB]
#7 41.04 Get:144 http://archive.ubuntu.com/ubuntu focal/universe amd64 llvm amd64 1:10.0-50~exp1 [3880 B]
#7 41.08 Get:145 http://archive.ubuntu.com/ubuntu focal/main amd64 libffi-dev amd64 3.3-4 [57.0 kB]
#7 41.13 Get:146 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 python3-pygments all 2.3.1+dfsg-1ubuntu2.2 [579 kB]
#7 41.29 Get:147 http://archive.ubuntu.com/ubuntu focal/universe amd64 llvm-10-tools amd64 1:10.0.0-4ubuntu1 [317 kB]
#7 41.38 Get:148 http://archive.ubuntu.com/ubuntu focal/universe amd64 libz3-4 amd64 4.8.7-4build1 [6792 kB]
#7 42.66 Get:149 http://archive.ubuntu.com/ubuntu focal/universe amd64 libz3-dev amd64 4.8.7-4build1 [67.5 kB]
#7 42.81 Get:150 http://archive.ubuntu.com/ubuntu focal/universe amd64 llvm-10-dev amd64 1:10.0.0-4ubuntu1 [26.0 MB]
#7 47.54 Get:151 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 mesa-vulkan-drivers amd64 21.2.6-0ubuntu0.1~20.04.2 [5788 kB]
#7 48.60 Get:152 http://archive.ubuntu.com/ubuntu focal/main amd64 tcl8.6 amd64 8.6.10+dfsg-1 [14.8 kB]
#7 48.65 Get:153 http://archive.ubuntu.com/ubuntu focal/universe amd64 tcl amd64 8.6.9+1 [5112 B]
#7 48.69 Get:154 http://archive.ubuntu.com/ubuntu focal/main amd64 tcl8.6-dev amd64 8.6.10+dfsg-1 [905 kB]
#7 48.89 Get:155 http://archive.ubuntu.com/ubuntu focal/universe amd64 tcl-dev amd64 8.6.9+1 [5760 B]
#7 48.94 Get:156 http://archive.ubuntu.com/ubuntu focal/main amd64 tk8.6 amd64 8.6.10-1 [12.5 kB]
#7 48.98 Get:157 http://archive.ubuntu.com/ubuntu focal/universe amd64 tk amd64 8.6.9+1 [3240 B]
#7 49.02 Get:158 http://archive.ubuntu.com/ubuntu focal/main amd64 tk8.6-dev amd64 8.6.10-1 [711 kB]
#7 49.20 Get:159 http://archive.ubuntu.com/ubuntu focal/universe amd64 tk-dev amd64 8.6.9+1 [3076 B]
#7 49.29 Get:160 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libglvnd0 amd64 1.3.2-1~ubuntu0.20.04.2 [48.1 kB]
#7 49.42 Get:161 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libglx0 amd64 1.3.2-1~ubuntu0.20.04.2 [32.5 kB]
#7 49.48 Get:162 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libgl1 amd64 1.3.2-1~ubuntu0.20.04.2 [85.8 kB]
#7 49.57 Get:163 http://archive.ubuntu.com/ubuntu focal/main amd64 x11-utils amd64 7.7+5 [199 kB]
#7 49.68 Get:164 http://archive.ubuntu.com/ubuntu focal/main amd64 xbitmaps all 1.1.1-2 [28.1 kB]
#7 49.72 Get:165 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 xterm amd64 353-1ubuntu1.20.04.2 [765 kB]
#7 49.91 Get:166 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 liblzma-dev amd64 5.2.4-1ubuntu1.1 [147 kB]
#7 49.98 Get:167 http://archive.ubuntu.com/ubuntu focal/main amd64 libtasn1-doc all 4.16.0-2 [303 kB]
#7 50.23 debconf: delaying package configuration, since apt-utils is not installed
#7 50.28 Fetched 138 MB in 48s (2890 kB/s)
#7 50.34 Selecting previously unselected package libpython3.8-minimal:amd64.
#7 50.34 (Reading database ...
(Reading database ... 5%
(Reading database ... 10%
(Reading database ... 15%
(Reading database ... 20%
(Reading database ... 25%
(Reading database ... 30%
(Reading database ... 35%
(Reading database ... 40%
(Reading database ... 45%
(Reading database ... 50%
(Reading database ... 55%
(Reading database ... 60%
(Reading database ... 65%
(Reading database ... 70%
(Reading database ... 75%
(Reading database ... 80%
(Reading database ... 85%
(Reading database ... 90%
(Reading database ... 95%
(Reading database ... 100%
(Reading database ... 16521 files and directories currently installed.)
#7 50.37 Preparing to unpack .../libpython3.8-minimal_3.8.10-0ubuntu1~20.04.9_amd64.deb ...
#7 50.37 Unpacking libpython3.8-minimal:amd64 (3.8.10-0ubuntu1~20.04.9) ...
#7 50.55 Selecting previously unselected package python3.8-minimal.
#7 50.56 Preparing to unpack .../python3.8-minimal_3.8.10-0ubuntu1~20.04.9_amd64.deb ...
#7 50.57 Unpacking python3.8-minimal (3.8.10-0ubuntu1~20.04.9) ...
#7 50.84 Setting up libpython3.8-minimal:amd64 (3.8.10-0ubuntu1~20.04.9) ...
#7 50.87 Setting up python3.8-minimal (3.8.10-0ubuntu1~20.04.9) ...
#7 51.59 Selecting previously unselected package python3-minimal.
#7 51.59 (Reading database ...
(Reading database ... 5%
(Reading database ... 10%
(Reading database ... 15%
(Reading database ... 20%
(Reading database ... 25%
(Reading database ... 30%
(Reading database ... 35%
(Reading database ... 40%
(Reading database ... 45%
(Reading database ... 50%
(Reading database ... 55%
(Reading database ... 60%
(Reading database ... 65%
(Reading database ... 70%
(Reading database ... 75%
(Reading database ... 80%
(Reading database ... 85%
(Reading database ... 90%
(Reading database ... 95%
(Reading database ... 100%
(Reading database ... 16804 files and directories currently installed.)
#7 51.61 Preparing to unpack .../0-python3-minimal_3.8.2-0ubuntu2_amd64.deb ...
#7 51.61 Unpacking python3-minimal (3.8.2-0ubuntu2) ...
#7 51.69 Selecting previously unselected package mime-support.
#7 51.70 Preparing to unpack .../1-mime-support_3.64ubuntu1_all.deb ...
#7 51.70 Unpacking mime-support (3.64ubuntu1) ...
#7 51.79 Selecting previously unselected package libmpdec2:amd64.
#7 51.79 Preparing to unpack .../2-libmpdec2_2.4.2-3_amd64.deb ...
#7 51.80 Unpacking libmpdec2:amd64 (2.4.2-3) ...
#7 51.89 Selecting previously unselected package readline-common.
#7 51.89 Preparing to unpack .../3-readline-common_8.0-4_all.deb ...
#7 51.90 Unpacking readline-common (8.0-4) ...
#7 51.98 Selecting previously unselected package libreadline8:amd64.
#7 51.99 Preparing to unpack .../4-libreadline8_8.0-4_amd64.deb ...
#7 51.99 Unpacking libreadline8:amd64 (8.0-4) ...
#7 52.08 Selecting previously unselected package libpython3.8-stdlib:amd64.
#7 52.08 Preparing to unpack .../5-libpython3.8-stdlib_3.8.10-0ubuntu1~20.04.9_amd64.deb ...
#7 52.09 Unpacking libpython3.8-stdlib:amd64 (3.8.10-0ubuntu1~20.04.9) ...
#7 52.32 Selecting previously unselected package python3.8.
#7 52.33 Preparing to unpack .../6-python3.8_3.8.10-0ubuntu1~20.04.9_amd64.deb ...
#7 52.33 Unpacking python3.8 (3.8.10-0ubuntu1~20.04.9) ...
#7 52.42 Selecting previously unselected package libpython3-stdlib:amd64.
#7 52.43 Preparing to unpack .../7-libpython3-stdlib_3.8.2-0ubuntu2_amd64.deb ...
#7 52.43 Unpacking libpython3-stdlib:amd64 (3.8.2-0ubuntu2) ...
#7 52.51 Setting up python3-minimal (3.8.2-0ubuntu2) ...
#7 52.85 Selecting previously unselected package python3.
#7 52.85 (Reading database ...
(Reading database ... 5%
(Reading database ... 10%
(Reading database ... 15%
(Reading database ... 20%
(Reading database ... 25%
(Reading database ... 30%
(Reading database ... 35%
(Reading database ... 40%
(Reading database ... 45%
(Reading database ... 50%
(Reading database ... 55%
(Reading database ... 60%
(Reading database ... 65%
(Reading database ... 70%
(Reading database ... 75%
(Reading database ... 80%
(Reading database ... 85%
(Reading database ... 90%
(Reading database ... 95%
(Reading database ... 100%
(Reading database ... 17228 files and directories currently installed.)
#7 52.88 Preparing to unpack .../000-python3_3.8.2-0ubuntu2_amd64.deb ...
#7 52.89 Unpacking python3 (3.8.2-0ubuntu2) ...
#7 52.96 Selecting previously unselected package libmagic-mgc.
#7 52.97 Preparing to unpack .../001-libmagic-mgc_1%3a5.38-4_amd64.deb ...
#7 52.97 Unpacking libmagic-mgc (1:5.38-4) ...
#7 53.09 Selecting previously unselected package libmagic1:amd64.
#7 53.09 Preparing to unpack .../002-libmagic1_1%3a5.38-4_amd64.deb ...
#7 53.10 Unpacking libmagic1:amd64 (1:5.38-4) ...
#7 53.17 Selecting previously unselected package file.
#7 53.18 Preparing to unpack .../003-file_1%3a5.38-4_amd64.deb ...
#7 53.18 Unpacking file (1:5.38-4) ...
#7 53.27 Selecting previously unselected package libglib2.0-0:amd64.
#7 53.27 Preparing to unpack .../004-libglib2.0-0_2.64.6-1~ubuntu20.04.7_amd64.deb ...
#7 53.28 Unpacking libglib2.0-0:amd64 (2.64.6-1~ubuntu20.04.7) ...
#7 53.46 Selecting previously unselected package libglib2.0-data.
#7 53.46 Preparing to unpack .../005-libglib2.0-data_2.64.6-1~ubuntu20.04.7_all.deb ...
#7 53.47 Unpacking libglib2.0-data (2.64.6-1~ubuntu20.04.7) ...
#7 53.53 Selecting previously unselected package libyaml-0-2:amd64.
#7 53.54 Preparing to unpack .../006-libyaml-0-2_0.2.2-1_amd64.deb ...
#7 53.54 Unpacking libyaml-0-2:amd64 (0.2.2-1) ...
#7 53.62 Selecting previously unselected package python3-pkg-resources.
#7 53.62 Preparing to unpack .../007-python3-pkg-resources_45.2.0-1ubuntu0.1_all.deb ...
#7 53.63 Unpacking python3-pkg-resources (45.2.0-1ubuntu0.1) ...
#7 53.71 Selecting previously unselected package python3-yaml.
#7 53.72 Preparing to unpack .../008-python3-yaml_5.3.1-1ubuntu0.1_amd64.deb ...
#7 53.72 Unpacking python3-yaml (5.3.1-1ubuntu0.1) ...
#7 53.82 Selecting previously unselected package shared-mime-info.
#7 53.82 Preparing to unpack .../009-shared-mime-info_1.15-1_amd64.deb ...
#7 53.83 Unpacking shared-mime-info (1.15-1) ...
#7 53.97 Selecting previously unselected package xdg-user-dirs.
#7 53.97 Preparing to unpack .../010-xdg-user-dirs_0.17-2ubuntu1_amd64.deb ...
#7 53.99 Unpacking xdg-user-dirs (0.17-2ubuntu1) ...
#7 54.08 Selecting previously unselected package libdrm-common.
#7 54.08 Preparing to unpack .../011-libdrm-common_2.4.107-8ubuntu1~20.04.2_all.deb ...
#7 54.09 Unpacking libdrm-common (2.4.107-8ubuntu1~20.04.2) ...
#7 54.15 Selecting previously unselected package libdrm2:amd64.
#7 54.16 Preparing to unpack .../012-libdrm2_2.4.107-8ubuntu1~20.04.2_amd64.deb ...
#7 54.16 Unpacking libdrm2:amd64 (2.4.107-8ubuntu1~20.04.2) ...
#7 54.23 Selecting previously unselected package libedit2:amd64.
#7 54.24 Preparing to unpack .../013-libedit2_3.1-20191231-1_amd64.deb ...
#7 54.24 Unpacking libedit2:amd64 (3.1-20191231-1) ...
#7 54.33 Selecting previously unselected package libpipeline1:amd64.
#7 54.34 Preparing to unpack .../014-libpipeline1_1.5.2-2build1_amd64.deb ...
#7 54.34 Unpacking libpipeline1:amd64 (1.5.2-2build1) ...
#7 54.42 Selecting previously unselected package libxext6:amd64.
#7 54.42 Preparing to unpack .../015-libxext6_2%3a1.3.4-0ubuntu1_amd64.deb ...
#7 54.42 Unpacking libxext6:amd64 (2:1.3.4-0ubuntu1) ...
#7 54.50 Selecting previously unselected package libxmuu1:amd64.
#7 54.50 Preparing to unpack .../016-libxmuu1_2%3a1.1.3-0ubuntu1_amd64.deb ...
#7 54.51 Unpacking libxmuu1:amd64 (2:1.1.3-0ubuntu1) ...
#7 54.58 Selecting previously unselected package binfmt-support.
#7 54.58 Preparing to unpack .../017-binfmt-support_2.2.0-2_amd64.deb ...
#7 54.60 Unpacking binfmt-support (2.2.0-2) ...
#7 54.67 Selecting previously unselected package bzip2-doc.
#7 54.67 Preparing to unpack .../018-bzip2-doc_1.0.8-2_all.deb ...
#7 54.68 Unpacking bzip2-doc (1.0.8-2) ...
#7 54.77 Selecting previously unselected package icu-devtools.
#7 54.77 Preparing to unpack .../019-icu-devtools_66.1-2ubuntu2.1_amd64.deb ...
#7 54.78 Unpacking icu-devtools (66.1-2ubuntu2.1) ...
#7 54.87 Selecting previously unselected package libbz2-dev:amd64.
#7 54.87 Preparing to unpack .../020-libbz2-dev_1.0.8-2_amd64.deb ...
#7 54.88 Unpacking libbz2-dev:amd64 (1.0.8-2) ...
#7 54.95 Selecting previously unselected package libllvm10:amd64.
#7 54.95 Preparing to unpack .../021-libllvm10_1%3a10.0.0-4ubuntu1_amd64.deb ...
#7 54.96 Unpacking libllvm10:amd64 (1:10.0.0-4ubuntu1) ...
#7 56.43 Selecting previously unselected package libclang-cpp10.
#7 56.43 Preparing to unpack .../022-libclang-cpp10_1%3a10.0.0-4ubuntu1_amd64.deb ...
#7 56.44 Unpacking libclang-cpp10 (1:10.0.0-4ubuntu1) ...
#7 57.39 Selecting previously unselected package libdrm-amdgpu1:amd64.
#7 57.40 Preparing to unpack .../023-libdrm-amdgpu1_2.4.107-8ubuntu1~20.04.2_amd64.deb ...
#7 57.40 Unpacking libdrm-amdgpu1:amd64 (2.4.107-8ubuntu1~20.04.2) ...
#7 57.47 Selecting previously unselected package libpciaccess0:amd64.
#7 57.48 Preparing to unpack .../024-libpciaccess0_0.16-0ubuntu1_amd64.deb ...
#7 57.48 Unpacking libpciaccess0:amd64 (0.16-0ubuntu1) ...
#7 57.56 Selecting previously unselected package libdrm-intel1:amd64.
#7 57.56 Preparing to unpack .../025-libdrm-intel1_2.4.107-8ubuntu1~20.04.2_amd64.deb ...
#7 57.57 Unpacking libdrm-intel1:amd64 (2.4.107-8ubuntu1~20.04.2) ...
#7 57.65 Selecting previously unselected package libdrm-nouveau2:amd64.
#7 57.65 Preparing to unpack .../026-libdrm-nouveau2_2.4.107-8ubuntu1~20.04.2_amd64.deb ...
#7 57.66 Unpacking libdrm-nouveau2:amd64 (2.4.107-8ubuntu1~20.04.2) ...
#7 57.73 Selecting previously unselected package libdrm-radeon1:amd64.
#7 57.73 Preparing to unpack .../027-libdrm-radeon1_2.4.107-8ubuntu1~20.04.2_amd64.deb ...
#7 57.74 Unpacking libdrm-radeon1:amd64 (2.4.107-8ubuntu1~20.04.2) ...
#7 57.81 Selecting previously unselected package libevent-2.1-7:amd64.
#7 57.81 Preparing to unpack .../028-libevent-2.1-7_2.1.11-stable-1_amd64.deb ...
#7 57.82 Unpacking libevent-2.1-7:amd64 (2.1.11-stable-1) ...
#7 57.90 Selecting previously unselected package libexpat1-dev:amd64.
#7 57.90 Preparing to unpack .../029-libexpat1-dev_2.2.9-1ubuntu0.6_amd64.deb ...
#7 57.91 Unpacking libexpat1-dev:amd64 (2.2.9-1ubuntu0.6) ...
#7 58.00 Selecting previously unselected package zlib1g-dev:amd64.
#7 58.00 Preparing to unpack .../030-zlib1g-dev_1%3a1.2.11.dfsg-2ubuntu1.5_amd64.deb ...
#7 58.00 Unpacking zlib1g-dev:amd64 (1:1.2.11.dfsg-2ubuntu1.5) ...
#7 58.09 Selecting previously unselected package libpng-dev:amd64.
#7 58.09 Preparing to unpack .../031-libpng-dev_1.6.37-2_amd64.deb ...
#7 58.10 Unpacking libpng-dev:amd64 (1.6.37-2) ...
#7 58.19 Selecting previously unselected package libfreetype-dev:amd64.
#7 58.19 Preparing to unpack .../032-libfreetype-dev_2.10.1-2ubuntu0.3_amd64.deb ...
#7 58.20 Unpacking libfreetype-dev:amd64 (2.10.1-2ubuntu0.3) ...
#7 58.34 Selecting previously unselected package libfreetype6-dev:amd64.
#7 58.34 Preparing to unpack .../033-libfreetype6-dev_2.10.1-2ubuntu0.3_amd64.deb ...
#7 58.36 Unpacking libfreetype6-dev:amd64 (2.10.1-2ubuntu0.3) ...
#7 58.42 Selecting previously unselected package uuid-dev:amd64.
#7 58.42 Preparing to unpack .../034-uuid-dev_2.34-0.1ubuntu9.6_amd64.deb ...
#7 58.43 Unpacking uuid-dev:amd64 (2.34-0.1ubuntu9.6) ...
#7 58.51 Selecting previously unselected package pkg-config.
#7 58.52 Preparing to unpack .../035-pkg-config_0.29.1-0ubuntu4_amd64.deb ...
#7 58.53 Unpacking pkg-config (0.29.1-0ubuntu4) ...
#7 58.60 Selecting previously unselected package libfontconfig1-dev:amd64.
#7 58.61 Preparing to unpack .../036-libfontconfig1-dev_2.13.1-2ubuntu3_amd64.deb ...
#7 58.61 Unpacking libfontconfig1-dev:amd64 (2.13.1-2ubuntu3) ...
#7 58.74 Selecting previously unselected package libfontenc1:amd64.
#7 58.74 Preparing to unpack .../037-libfontenc1_1%3a1.1.4-0ubuntu1_amd64.deb ...
#7 58.75 Unpacking libfontenc1:amd64 (1:1.1.4-0ubuntu1) ...
#7 58.82 Selecting previously unselected package libgpg-error-dev.
#7 58.82 Preparing to unpack .../038-libgpg-error-dev_1.37-1_amd64.deb ...
#7 58.83 Unpacking libgpg-error-dev (1.37-1) ...
#7 58.91 Selecting previously unselected package libgcrypt20-dev.
#7 58.91 Preparing to unpack .../039-libgcrypt20-dev_1.8.5-5ubuntu1.1_amd64.deb ...
#7 58.92 Unpacking libgcrypt20-dev (1.8.5-5ubuntu1.1) ...
#7 59.06 Selecting previously unselected package libglapi-mesa:amd64.
#7 59.06 Preparing to unpack .../040-libglapi-mesa_21.2.6-0ubuntu0.1~20.04.2_amd64.deb ...
#7 59.07 Unpacking libglapi-mesa:amd64 (21.2.6-0ubuntu0.1~20.04.2) ...
#7 59.14 Selecting previously unselected package libllvm12:amd64.
#7 59.14 Preparing to unpack .../041-libllvm12_1%3a12.0.0-3ubuntu1~20.04.5_amd64.deb ...
#7 59.15 Unpacking libllvm12:amd64 (1:12.0.0-3ubuntu1~20.04.5) ...
#7 60.92 Selecting previously unselected package libsensors-config.
#7 60.92 Preparing to unpack .../042-libsensors-config_1%3a3.6.0-2ubuntu1.1_all.deb ...
#7 60.93 Unpacking libsensors-config (1:3.6.0-2ubuntu1.1) ...
#7 61.01 Selecting previously unselected package libsensors5:amd64.
#7 61.01 Preparing to unpack .../043-libsensors5_1%3a3.6.0-2ubuntu1.1_amd64.deb ...
#7 61.11 Unpacking libsensors5:amd64 (1:3.6.0-2ubuntu1.1) ...
#7 61.18 Selecting previously unselected package libvulkan1:amd64.
#7 61.19 Preparing to unpack .../044-libvulkan1_1.2.131.2-1_amd64.deb ...
#7 61.20 Unpacking libvulkan1:amd64 (1.2.131.2-1) ...
#7 61.30 Selecting previously unselected package libgl1-mesa-dri:amd64.
#7 61.30 Preparing to unpack .../045-libgl1-mesa-dri_21.2.6-0ubuntu0.1~20.04.2_amd64.deb ...
#7 61.33 Unpacking libgl1-mesa-dri:amd64 (21.2.6-0ubuntu0.1~20.04.2) ...
#7 62.36 Selecting previously unselected package libx11-xcb1:amd64.
#7 62.36 Preparing to unpack .../046-libx11-xcb1_2%3a1.6.9-2ubuntu1.6_amd64.deb ...
#7 62.37 Unpacking libx11-xcb1:amd64 (2:1.6.9-2ubuntu1.6) ...
#7 62.44 Selecting previously unselected package libxcb-dri2-0:amd64.
#7 62.45 Preparing to unpack .../047-libxcb-dri2-0_1.14-2_amd64.deb ...
#7 62.46 Unpacking libxcb-dri2-0:amd64 (1.14-2) ...
#7 62.53 Selecting previously unselected package libxcb-dri3-0:amd64.
#7 62.53 Preparing to unpack .../048-libxcb-dri3-0_1.14-2_amd64.deb ...
#7 62.54 Unpacking libxcb-dri3-0:amd64 (1.14-2) ...
#7 62.62 Selecting previously unselected package libxcb-glx0:amd64.
#7 62.62 Preparing to unpack .../049-libxcb-glx0_1.14-2_amd64.deb ...
#7 62.63 Unpacking libxcb-glx0:amd64 (1.14-2) ...
#7 62.70 Selecting previously unselected package libxcb-present0:amd64.
#7 62.71 Preparing to unpack .../050-libxcb-present0_1.14-2_amd64.deb ...
#7 62.71 Unpacking libxcb-present0:amd64 (1.14-2) ...
#7 62.79 Selecting previously unselected package libxcb-shm0:amd64.
#7 62.79 Preparing to unpack .../051-libxcb-shm0_1.14-2_amd64.deb ...
#7 62.80 Unpacking libxcb-shm0:amd64 (1.14-2) ...
#7 62.87 Selecting previously unselected package libxcb-sync1:amd64.
#7 62.87 Preparing to unpack .../052-libxcb-sync1_1.14-2_amd64.deb ...
#7 62.88 Unpacking libxcb-sync1:amd64 (1.14-2) ...
#7 62.96 Selecting previously unselected package libxcb-xfixes0:amd64.
#7 62.96 Preparing to unpack .../053-libxcb-xfixes0_1.14-2_amd64.deb ...
#7 62.97 Unpacking libxcb-xfixes0:amd64 (1.14-2) ...
#7 63.04 Selecting previously unselected package libxfixes3:amd64.
#7 63.05 Preparing to unpack .../054-libxfixes3_1%3a5.0.3-2_amd64.deb ...
#7 63.05 Unpacking libxfixes3:amd64 (1:5.0.3-2) ...
#7 63.13 Selecting previously unselected package libxshmfence1:amd64.
#7 63.13 Preparing to unpack .../055-libxshmfence1_1.3-1_amd64.deb ...
#7 63.14 Unpacking libxshmfence1:amd64 (1.3-1) ...
#7 63.21 Selecting previously unselected package libxxf86vm1:amd64.
#7 63.21 Preparing to unpack .../056-libxxf86vm1_1%3a1.1.4-1build1_amd64.deb ...
#7 63.22 Unpacking libxxf86vm1:amd64 (1:1.1.4-1build1) ...
#7 63.30 Selecting previously unselected package libglx-mesa0:amd64.
#7 63.30 Preparing to unpack .../057-libglx-mesa0_21.2.6-0ubuntu0.1~20.04.2_amd64.deb ...
#7 63.31 Unpacking libglx-mesa0:amd64 (21.2.6-0ubuntu0.1~20.04.2) ...
#7 63.41 Selecting previously unselected package libgmpxx4ldbl:amd64.
#7 63.41 Preparing to unpack .../058-libgmpxx4ldbl_2%3a6.2.0+dfsg-4ubuntu0.1_amd64.deb ...
#7 63.42 Unpacking libgmpxx4ldbl:amd64 (2:6.2.0+dfsg-4ubuntu0.1) ...
#7 63.48 Selecting previously unselected package libgmp-dev:amd64.
#7 63.49 Preparing to unpack .../059-libgmp-dev_2%3a6.2.0+dfsg-4ubuntu0.1_amd64.deb ...
#7 63.50 Unpacking libgmp-dev:amd64 (2:6.2.0+dfsg-4ubuntu0.1) ...
#7 63.61 Selecting previously unselected package libgnutls-openssl27:amd64.
#7 63.61 Preparing to unpack .../060-libgnutls-openssl27_3.6.13-2ubuntu1.11_amd64.deb ...
#7 63.62 Unpacking libgnutls-openssl27:amd64 (3.6.13-2ubuntu1.11) ...
#7 63.69 Selecting previously unselected package libunbound8:amd64.
#7 63.69 Preparing to unpack .../061-libunbound8_1.9.4-2ubuntu1.5_amd64.deb ...
#7 63.70 Unpacking libunbound8:amd64 (1.9.4-2ubuntu1.5) ...
#7 63.82 Selecting previously unselected package libgnutls-dane0:amd64.
#7 63.83 Preparing to unpack .../062-libgnutls-dane0_3.6.13-2ubuntu1.11_amd64.deb ...
#7 63.83 Unpacking libgnutls-dane0:amd64 (3.6.13-2ubuntu1.11) ...
#7 63.91 Selecting previously unselected package libgnutlsxx28:amd64.
#7 63.91 Preparing to unpack .../063-libgnutlsxx28_3.6.13-2ubuntu1.11_amd64.deb ...
#7 63.92 Unpacking libgnutlsxx28:amd64 (3.6.13-2ubuntu1.11) ...
#7 63.98 Selecting previously unselected package libidn2-dev:amd64.
#7 63.99 Preparing to unpack .../064-libidn2-dev_2.2.0-2_amd64.deb ...
#7 63.99 Unpacking libidn2-dev:amd64 (2.2.0-2) ...
#7 64.07 Selecting previously unselected package libp11-kit-dev:amd64.
#7 64.08 Preparing to unpack .../065-libp11-kit-dev_0.23.20-1ubuntu0.1_amd64.deb ...
#7 64.08 Unpacking libp11-kit-dev:amd64 (0.23.20-1ubuntu0.1) ...
#7 64.16 Selecting previously unselected package libtasn1-6-dev:amd64.
#7 64.17 Preparing to unpack .../066-libtasn1-6-dev_4.16.0-2_amd64.deb ...
#7 64.18 Unpacking libtasn1-6-dev:amd64 (4.16.0-2) ...
#7 64.26 Selecting previously unselected package nettle-dev:amd64.
#7 64.26 Preparing to unpack .../067-nettle-dev_3.5.1+really3.5.1-2ubuntu0.2_amd64.deb ...
#7 64.27 Unpacking nettle-dev:amd64 (3.5.1+really3.5.1-2ubuntu0.2) ...
#7 64.44 Selecting previously unselected package libgnutls28-dev:amd64.
#7 64.44 Preparing to unpack .../068-libgnutls28-dev_3.6.13-2ubuntu1.11_amd64.deb ...
#7 64.45 Unpacking libgnutls28-dev:amd64 (3.6.13-2ubuntu1.11) ...
#7 64.63 Selecting previously unselected package x11-common.
#7 64.64 Preparing to unpack .../069-x11-common_1%3a7.7+19ubuntu14_all.deb ...
#7 64.67 dpkg-query: no packages found matching nux-tools
#7 64.69 Unpacking x11-common (1:7.7+19ubuntu14) ...
#7 64.77 Selecting previously unselected package libice6:amd64.
#7 64.78 Preparing to unpack .../070-libice6_2%3a1.0.10-0ubuntu1_amd64.deb ...
#7 64.78 Unpacking libice6:amd64 (2:1.0.10-0ubuntu1) ...
#7 64.85 Selecting previously unselected package xorg-sgml-doctools.
#7 64.86 Preparing to unpack .../071-xorg-sgml-doctools_1%3a1.11-1_all.deb ...
#7 64.87 Unpacking xorg-sgml-doctools (1:1.11-1) ...
#7 64.93 Selecting previously unselected package x11proto-dev.
#7 64.94 Preparing to unpack .../072-x11proto-dev_2019.2-1ubuntu1_all.deb ...
#7 64.94 Unpacking x11proto-dev (2019.2-1ubuntu1) ...
#7 65.08 Selecting previously unselected package x11proto-core-dev.
#7 65.09 Preparing to unpack .../073-x11proto-core-dev_2019.2-1ubuntu1_all.deb ...
#7 65.10 Unpacking x11proto-core-dev (2019.2-1ubuntu1) ...
#7 65.16 Selecting previously unselected package libice-dev:amd64.
#7 65.16 Preparing to unpack .../074-libice-dev_2%3a1.0.10-0ubuntu1_amd64.deb ...
#7 65.17 Unpacking libice-dev:amd64 (2:1.0.10-0ubuntu1) ...
#7 65.25 Selecting previously unselected package libicu-dev:amd64.
#7 65.25 Preparing to unpack .../075-libicu-dev_66.1-2ubuntu2.1_amd64.deb ...
#7 65.26 Unpacking libicu-dev:amd64 (66.1-2ubuntu2.1) ...
#7 66.22 Selecting previously unselected package libncurses-dev:amd64.
#7 66.23 Preparing to unpack .../076-libncurses-dev_6.2-0ubuntu2.1_amd64.deb ...
#7 66.24 Unpacking libncurses-dev:amd64 (6.2-0ubuntu2.1) ...
#7 66.36 Selecting previously unselected package libncursesw5-dev:amd64.
#7 66.36 Preparing to unpack .../077-libncursesw5-dev_6.2-0ubuntu2.1_amd64.deb ...
#7 66.37 Unpacking libncursesw5-dev:amd64 (6.2-0ubuntu2.1) ...
#7 66.44 Selecting previously unselected package libnspr4:amd64.
#7 66.44 Preparing to unpack .../078-libnspr4_2%3a4.35-0ubuntu0.20.04.1_amd64.deb ...
#7 66.45 Unpacking libnspr4:amd64 (2:4.35-0ubuntu0.20.04.1) ...
#7 66.54 Selecting previously unselected package libnspr4-dev.
#7 66.54 Preparing to unpack .../079-libnspr4-dev_2%3a4.35-0ubuntu0.20.04.1_amd64.deb ...
#7 66.55 Unpacking libnspr4-dev (2:4.35-0ubuntu0.20.04.1) ...
#7 66.66 Selecting previously unselected package libnss3:amd64.
#7 66.67 Preparing to unpack .../080-libnss3_2%3a3.98-0ubuntu0.20.04.2_amd64.deb ...
#7 66.67 Unpacking libnss3:amd64 (2:3.98-0ubuntu0.20.04.2) ...
#7 66.88 Selecting previously unselected package libnss3-dev:amd64.
#7 66.88 Preparing to unpack .../081-libnss3-dev_2%3a3.98-0ubuntu0.20.04.2_amd64.deb ...
#7 66.89 Unpacking libnss3-dev:amd64 (2:3.98-0ubuntu0.20.04.2) ...
#7 67.00 Selecting previously unselected package libpng-tools.
#7 67.00 Preparing to unpack .../082-libpng-tools_1.6.37-2_amd64.deb ...
#7 67.01 Unpacking libpng-tools (1.6.37-2) ...
#7 67.07 Selecting previously unselected package libpthread-stubs0-dev:amd64.
#7 67.07 Preparing to unpack .../083-libpthread-stubs0-dev_0.4-1_amd64.deb ...
#7 67.08 Unpacking libpthread-stubs0-dev:amd64 (0.4-1) ...
#7 67.17 Selecting previously unselected package libreadline-dev:amd64.
#7 67.18 Preparing to unpack .../084-libreadline-dev_8.0-4_amd64.deb ...
#7 67.19 Unpacking libreadline-dev:amd64 (8.0-4) ...
#7 67.29 Selecting previously unselected package libsm6:amd64.
#7 67.29 Preparing to unpack .../085-libsm6_2%3a1.2.3-1_amd64.deb ...
#7 67.30 Unpacking libsm6:amd64 (2:1.2.3-1) ...
#7 67.37 Selecting previously unselected package libsm-dev:amd64.
#7 67.37 Preparing to unpack .../086-libsm-dev_2%3a1.2.3-1_amd64.deb ...
#7 67.38 Unpacking libsm-dev:amd64 (2:1.2.3-1) ...
#7 67.45 Selecting previously unselected package libsqlite3-dev:amd64.
#7 67.45 Preparing to unpack .../087-libsqlite3-dev_3.31.1-4ubuntu0.6_amd64.deb ...
#7 67.46 Unpacking libsqlite3-dev:amd64 (3.31.1-4ubuntu0.6) ...
#7 67.60 Selecting previously unselected package libssl-dev:amd64.
#7 67.60 Preparing to unpack .../088-libssl-dev_1.1.1f-1ubuntu2.22_amd64.deb ...
#7 67.61 Unpacking libssl-dev:amd64 (1.1.1f-1ubuntu2.22) ...
#7 67.84 Selecting previously unselected package libtcl8.6:amd64.
#7 67.85 Preparing to unpack .../089-libtcl8.6_8.6.10+dfsg-1_amd64.deb ...
#7 67.85 Unpacking libtcl8.6:amd64 (8.6.10+dfsg-1) ...
#7 68.05 Selecting previously unselected package libxrender1:amd64.
#7 68.05 Preparing to unpack .../090-libxrender1_1%3a0.9.10-1_amd64.deb ...
#7 68.06 Unpacking libxrender1:amd64 (1:0.9.10-1) ...
#7 68.13 Selecting previously unselected package libxft2:amd64.
#7 68.14 Preparing to unpack .../091-libxft2_2.3.3-0ubuntu1_amd64.deb ...
#7 68.14 Unpacking libxft2:amd64 (2.3.3-0ubuntu1) ...
#7 68.23 Selecting previously unselected package libxss1:amd64.
#7 68.23 Preparing to unpack .../092-libxss1_1%3a1.2.3-1_amd64.deb ...
#7 68.24 Unpacking libxss1:amd64 (1:1.2.3-1) ...
#7 68.31 Selecting previously unselected package libtk8.6:amd64.
#7 68.31 Preparing to unpack .../093-libtk8.6_8.6.10-1_amd64.deb ...
#7 68.32 Unpacking libtk8.6:amd64 (8.6.10-1) ...
#7 68.47 Selecting previously unselected package libutempter0:amd64.
#7 68.47 Preparing to unpack .../094-libutempter0_1.1.6-4_amd64.deb ...
#7 68.48 Unpacking libutempter0:amd64 (1.1.6-4) ...
#7 68.55 Selecting previously unselected package libwayland-client0:amd64.
#7 68.56 Preparing to unpack .../095-libwayland-client0_1.18.0-1ubuntu0.1_amd64.deb ...
#7 68.56 Unpacking libwayland-client0:amd64 (1.18.0-1ubuntu0.1) ...
#7 68.63 Selecting previously unselected package libxau-dev:amd64.
#7 68.63 Preparing to unpack .../096-libxau-dev_1%3a1.0.9-0ubuntu1_amd64.deb ...
#7 68.64 Unpacking libxau-dev:amd64 (1:1.0.9-0ubuntu1) ...
#7 68.70 Selecting previously unselected package libxdmcp-dev:amd64.
#7 68.71 Preparing to unpack .../097-libxdmcp-dev_1%3a1.1.3-0ubuntu1_amd64.deb ...
#7 68.71 Unpacking libxdmcp-dev:amd64 (1:1.1.3-0ubuntu1) ...
#7 68.78 Selecting previously unselected package xtrans-dev.
#7 68.78 Preparing to unpack .../098-xtrans-dev_1.4.0-1_all.deb ...
#7 68.79 Unpacking xtrans-dev (1.4.0-1) ...
#7 68.87 Selecting previously unselected package libxcb1-dev:amd64.
#7 68.87 Preparing to unpack .../099-libxcb1-dev_1.14-2_amd64.deb ...
#7 68.88 Unpacking libxcb1-dev:amd64 (1.14-2) ...
#7 68.96 Selecting previously unselected package libx11-dev:amd64.
#7 68.96 Preparing to unpack .../100-libx11-dev_2%3a1.6.9-2ubuntu1.6_amd64.deb ...
#7 68.97 Unpacking libx11-dev:amd64 (2:1.6.9-2ubuntu1.6) ...
#7 69.12 Selecting previously unselected package libxt6:amd64.
#7 69.12 Preparing to unpack .../101-libxt6_1%3a1.1.5-1_amd64.deb ...
#7 69.12 Unpacking libxt6:amd64 (1:1.1.5-1) ...
#7 69.22 Selecting previously unselected package libxmu6:amd64.
#7 69.22 Preparing to unpack .../102-libxmu6_2%3a1.1.3-0ubuntu1_amd64.deb ...
#7 69.23 Unpacking libxmu6:amd64 (2:1.1.3-0ubuntu1) ...
#7 69.31 Selecting previously unselected package libxaw7:amd64.
#7 69.32 Preparing to unpack .../103-libxaw7_2%3a1.0.13-1_amd64.deb ...
#7 69.32 Unpacking libxaw7:amd64 (2:1.0.13-1) ...
#7 69.43 Selecting previously unselected package libxcb-randr0:amd64.
#7 69.43 Preparing to unpack .../104-libxcb-randr0_1.14-2_amd64.deb ...
#7 69.44 Unpacking libxcb-randr0:amd64 (1.14-2) ...
#7 69.51 Selecting previously unselected package libxcb-shape0:amd64.
#7 69.52 Preparing to unpack .../105-libxcb-shape0_1.14-2_amd64.deb ...
#7 69.52 Unpacking libxcb-shape0:amd64 (1.14-2) ...
#7 69.59 Selecting previously unselected package libxcomposite1:amd64.
#7 69.60 Preparing to unpack .../106-libxcomposite1_1%3a0.4.5-1_amd64.deb ...
#7 69.61 Unpacking libxcomposite1:amd64 (1:0.4.5-1) ...
#7 69.67 Selecting previously unselected package x11proto-xext-dev.
#7 69.67 Preparing to unpack .../107-x11proto-xext-dev_2019.2-1ubuntu1_all.deb ...
#7 69.68 Unpacking x11proto-xext-dev (2019.2-1ubuntu1) ...
#7 69.74 Selecting previously unselected package libxext-dev:amd64.
#7 69.75 Preparing to unpack .../108-libxext-dev_2%3a1.3.4-0ubuntu1_amd64.deb ...
#7 69.75 Unpacking libxext-dev:amd64 (2:1.3.4-0ubuntu1) ...
#7 69.84 Selecting previously unselected package libxrender-dev:amd64.
#7 69.84 Preparing to unpack .../109-libxrender-dev_1%3a0.9.10-1_amd64.deb ...
#7 69.84 Unpacking libxrender-dev:amd64 (1:0.9.10-1) ...
#7 69.90 Selecting previously unselected package libxft-dev:amd64.
#7 69.90 Preparing to unpack .../110-libxft-dev_2.3.3-0ubuntu1_amd64.deb ...
#7 69.91 Unpacking libxft-dev:amd64 (2.3.3-0ubuntu1) ...
#7 69.98 Selecting previously unselected package libxi6:amd64.
#7 69.99 Preparing to unpack .../111-libxi6_2%3a1.7.10-0ubuntu1_amd64.deb ...
#7 69.99 Unpacking libxi6:amd64 (2:1.7.10-0ubuntu1) ...
#7 70.07 Selecting previously unselected package libxinerama1:amd64.
#7 70.07 Preparing to unpack .../112-libxinerama1_2%3a1.1.4-2_amd64.deb ...
#7 70.08 Unpacking libxinerama1:amd64 (2:1.1.4-2) ...
#7 70.15 Selecting previously unselected package libxkbfile1:amd64.
#7 70.15 Preparing to unpack .../113-libxkbfile1_1%3a1.1.0-1_amd64.deb ...
#7 70.16 Unpacking libxkbfile1:amd64 (1:1.1.0-1) ...
#7 70.23 Selecting previously unselected package libxml2-dev:amd64.
#7 70.24 Preparing to unpack .../114-libxml2-dev_2.9.10+dfsg-5ubuntu0.20.04.7_amd64.deb ...
#7 70.25 Unpacking libxml2-dev:amd64 (2.9.10+dfsg-5ubuntu0.20.04.7) ...
#7 70.40 Selecting previously unselected package libxmlsec1:amd64.
#7 70.41 Preparing to unpack .../115-libxmlsec1_1.2.28-2_amd64.deb ...
#7 70.42 Unpacking libxmlsec1:amd64 (1.2.28-2) ...
#7 70.52 Selecting previously unselected package libxmlsec1-gcrypt:amd64.
#7 70.52 Preparing to unpack .../116-libxmlsec1-gcrypt_1.2.28-2_amd64.deb ...
#7 70.53 Unpacking libxmlsec1-gcrypt:amd64 (1.2.28-2) ...
#7 70.61 Selecting previously unselected package libxmlsec1-gnutls:amd64.
#7 70.62 Preparing to unpack .../117-libxmlsec1-gnutls_1.2.28-2_amd64.deb ...
#7 70.62 Unpacking libxmlsec1-gnutls:amd64 (1.2.28-2) ...
#7 70.69 Selecting previously unselected package libxmlsec1-nss:amd64.
#7 70.69 Preparing to unpack .../118-libxmlsec1-nss_1.2.28-2_amd64.deb ...
#7 70.70 Unpacking libxmlsec1-nss:amd64 (1.2.28-2) ...
#7 70.79 Selecting previously unselected package libxmlsec1-openssl:amd64.
#7 70.80 Preparing to unpack .../119-libxmlsec1-openssl_1.2.28-2_amd64.deb ...
#7 70.80 Unpacking libxmlsec1-openssl:amd64 (1.2.28-2) ...
#7 70.89 Selecting previously unselected package libxslt1-dev:amd64.
#7 70.89 Preparing to unpack .../120-libxslt1-dev_1.1.34-4ubuntu0.20.04.1_amd64.deb ...
#7 70.90 Unpacking libxslt1-dev:amd64 (1.1.34-4ubuntu0.20.04.1) ...
#7 71.00 Selecting previously unselected package libxmlsec1-dev.
#7 71.01 Preparing to unpack .../121-libxmlsec1-dev_1.2.28-2_amd64.deb ...
#7 71.01 Unpacking libxmlsec1-dev (1.2.28-2) ...
#7 71.15 Selecting previously unselected package libxrandr2:amd64.
#7 71.15 Preparing to unpack .../122-libxrandr2_2%3a1.5.2-0ubuntu1_amd64.deb ...
#7 71.15 Unpacking libxrandr2:amd64 (2:1.5.2-0ubuntu1) ...
#7 71.21 Selecting previously unselected package x11proto-scrnsaver-dev.
#7 71.22 Preparing to unpack .../123-x11proto-scrnsaver-dev_2019.2-1ubuntu1_all.deb ...
#7 71.22 Unpacking x11proto-scrnsaver-dev (2019.2-1ubuntu1) ...
#7 71.28 Selecting previously unselected package libxss-dev:amd64.
#7 71.28 Preparing to unpack .../124-libxss-dev_1%3a1.2.3-1_amd64.deb ...
#7 71.29 Unpacking libxss-dev:amd64 (1:1.2.3-1) ...
#7 71.36 Selecting previously unselected package libxt-dev:amd64.
#7 71.36 Preparing to unpack .../125-libxt-dev_1%3a1.1.5-1_amd64.deb ...
#7 71.37 Unpacking libxt-dev:amd64 (1:1.1.5-1) ...
#7 71.50 Selecting previously unselected package libxtst6:amd64.
#7 71.51 Preparing to unpack .../126-libxtst6_2%3a1.2.3-1_amd64.deb ...
#7 71.52 Unpacking libxtst6:amd64 (2:1.2.3-1) ...
#7 71.58 Selecting previously unselected package libxv1:amd64.
#7 71.59 Preparing to unpack .../127-libxv1_2%3a1.0.11-1_amd64.deb ...
#7 71.59 Unpacking libxv1:amd64 (2:1.0.11-1) ...
#7 71.66 Selecting previously unselected package libxxf86dga1:amd64.
#7 71.66 Preparing to unpack .../128-libxxf86dga1_2%3a1.1.5-0ubuntu1_amd64.deb ...
#7 71.66 Unpacking libxxf86dga1:amd64 (2:1.1.5-0ubuntu1) ...
#7 71.73 Selecting previously unselected package llvm-10-runtime.
#7 71.74 Preparing to unpack .../129-llvm-10-runtime_1%3a10.0.0-4ubuntu1_amd64.deb ...
#7 71.74 Unpacking llvm-10-runtime (1:10.0.0-4ubuntu1) ...
#7 71.84 Selecting previously unselected package llvm-runtime.
#7 71.84 Preparing to unpack .../130-llvm-runtime_1%3a10.0-50~exp1_amd64.deb ...
#7 71.85 Unpacking llvm-runtime (1:10.0-50~exp1) ...
#7 71.93 Selecting previously unselected package libpfm4:amd64.
#7 71.93 Preparing to unpack .../131-libpfm4_4.10.1+git20-g7700f49-2_amd64.deb ...
#7 71.94 Unpacking libpfm4:amd64 (4.10.1+git20-g7700f49-2) ...
#7 72.05 Selecting previously unselected package llvm-10.
#7 72.05 Preparing to unpack .../132-llvm-10_1%3a10.0.0-4ubuntu1_amd64.deb ...
#7 72.06 Unpacking llvm-10 (1:10.0.0-4ubuntu1) ...
#7 72.62 Selecting previously unselected package llvm.
#7 72.62 Preparing to unpack .../133-llvm_1%3a10.0-50~exp1_amd64.deb ...
#7 72.63 Unpacking llvm (1:10.0-50~exp1) ...
#7 72.71 Selecting previously unselected package libffi-dev:amd64.
#7 72.71 Preparing to unpack .../134-libffi-dev_3.3-4_amd64.deb ...
#7 72.72 Unpacking libffi-dev:amd64 (3.3-4) ...
#7 72.81 Selecting previously unselected package python3-pygments.
#7 72.81 Preparing to unpack .../135-python3-pygments_2.3.1+dfsg-1ubuntu2.2_all.deb ...
#7 72.82 Unpacking python3-pygments (2.3.1+dfsg-1ubuntu2.2) ...
#7 72.95 Selecting previously unselected package llvm-10-tools.
#7 72.95 Preparing to unpack .../136-llvm-10-tools_1%3a10.0.0-4ubuntu1_amd64.deb ...
#7 72.96 Unpacking llvm-10-tools (1:10.0.0-4ubuntu1) ...
#7 73.12 Selecting previously unselected package libz3-4:amd64.
#7 73.12 Preparing to unpack .../137-libz3-4_4.8.7-4build1_amd64.deb ...
#7 73.13 Unpacking libz3-4:amd64 (4.8.7-4build1) ...
#7 73.78 Selecting previously unselected package libz3-dev:amd64.
#7 73.78 Preparing to unpack .../138-libz3-dev_4.8.7-4build1_amd64.deb ...
#7 73.79 Unpacking libz3-dev:amd64 (4.8.7-4build1) ...
#7 73.88 Selecting previously unselected package llvm-10-dev.
#7 73.89 Preparing to unpack .../139-llvm-10-dev_1%3a10.0.0-4ubuntu1_amd64.deb ...
#7 73.90 Unpacking llvm-10-dev (1:10.0.0-4ubuntu1) ...
#7 76.60 Selecting previously unselected package mesa-vulkan-drivers:amd64.
#7 76.60 Preparing to unpack .../140-mesa-vulkan-drivers_21.2.6-0ubuntu0.1~20.04.2_amd64.deb ...
#7 76.61 Unpacking mesa-vulkan-drivers:amd64 (21.2.6-0ubuntu0.1~20.04.2) ...
#7 77.24 Selecting previously unselected package tcl8.6.
#7 77.24 Preparing to unpack .../141-tcl8.6_8.6.10+dfsg-1_amd64.deb ...
#7 77.25 Unpacking tcl8.6 (8.6.10+dfsg-1) ...
#7 77.32 Selecting previously unselected package tcl.
#7 77.32 Preparing to unpack .../142-tcl_8.6.9+1_amd64.deb ...
#7 77.34 Unpacking tcl (8.6.9+1) ...
#7 77.40 Selecting previously unselected package tcl8.6-dev:amd64.
#7 77.41 Preparing to unpack .../143-tcl8.6-dev_8.6.10+dfsg-1_amd64.deb ...
#7 77.42 Unpacking tcl8.6-dev:amd64 (8.6.10+dfsg-1) ...
#7 77.59 Selecting previously unselected package tcl-dev:amd64.
#7 77.59 Preparing to unpack .../144-tcl-dev_8.6.9+1_amd64.deb ...
#7 77.60 Unpacking tcl-dev:amd64 (8.6.9+1) ...
#7 77.68 Selecting previously unselected package tk8.6.
#7 77.69 Preparing to unpack .../145-tk8.6_8.6.10-1_amd64.deb ...
#7 77.69 Unpacking tk8.6 (8.6.10-1) ...
#7 77.76 Selecting previously unselected package tk.
#7 77.77 Preparing to unpack .../146-tk_8.6.9+1_amd64.deb ...
#7 77.78 Unpacking tk (8.6.9+1) ...
#7 77.85 Selecting previously unselected package tk8.6-dev:amd64.
#7 77.85 Preparing to unpack .../147-tk8.6-dev_8.6.10-1_amd64.deb ...
#7 77.86 Unpacking tk8.6-dev:amd64 (8.6.10-1) ...
#7 78.01 Selecting previously unselected package tk-dev:amd64.
#7 78.01 Preparing to unpack .../148-tk-dev_8.6.9+1_amd64.deb ...
#7 78.02 Unpacking tk-dev:amd64 (8.6.9+1) ...
#7 78.10 Selecting previously unselected package libglvnd0:amd64.
#7 78.10 Preparing to unpack .../149-libglvnd0_1.3.2-1~ubuntu0.20.04.2_amd64.deb ...
#7 78.11 Unpacking libglvnd0:amd64 (1.3.2-1~ubuntu0.20.04.2) ...
#7 78.21 Selecting previously unselected package libglx0:amd64.
#7 78.21 Preparing to unpack .../150-libglx0_1.3.2-1~ubuntu0.20.04.2_amd64.deb ...
#7 78.22 Unpacking libglx0:amd64 (1.3.2-1~ubuntu0.20.04.2) ...
#7 78.31 Selecting previously unselected package libgl1:amd64.
#7 78.31 Preparing to unpack .../151-libgl1_1.3.2-1~ubuntu0.20.04.2_amd64.deb ...
#7 78.32 Unpacking libgl1:amd64 (1.3.2-1~ubuntu0.20.04.2) ...
#7 78.41 Selecting previously unselected package x11-utils.
#7 78.42 Preparing to unpack .../152-x11-utils_7.7+5_amd64.deb ...
#7 78.42 Unpacking x11-utils (7.7+5) ...
#7 78.52 Selecting previously unselected package xbitmaps.
#7 78.53 Preparing to unpack .../153-xbitmaps_1.1.1-2_all.deb ...
#7 78.54 Unpacking xbitmaps (1.1.1-2) ...
#7 78.63 Selecting previously unselected package xterm.
#7 78.63 Preparing to unpack .../154-xterm_353-1ubuntu1.20.04.2_amd64.deb ...
#7 78.64 Unpacking xterm (353-1ubuntu1.20.04.2) ...
#7 78.78 Selecting previously unselected package liblzma-dev:amd64.
#7 78.78 Preparing to unpack .../155-liblzma-dev_5.2.4-1ubuntu1.1_amd64.deb ...
#7 78.79 Unpacking liblzma-dev:amd64 (5.2.4-1ubuntu1.1) ...
#7 78.88 Selecting previously unselected package libtasn1-doc.
#7 78.89 Preparing to unpack .../156-libtasn1-doc_4.16.0-2_all.deb ...
#7 78.89 Unpacking libtasn1-doc (4.16.0-2) ...
#7 79.03 Setting up libpipeline1:amd64 (1.5.2-2build1) ...
#7 79.05 Setting up libxcb-dri3-0:amd64 (1.14-2) ...
#7 79.08 Setting up bzip2-doc (1.0.8-2) ...
#7 79.10 Setting up libx11-xcb1:amd64 (2:1.6.9-2ubuntu1.6) ...
#7 79.12 Setting up libpciaccess0:amd64 (0.16-0ubuntu1) ...
#7 79.14 Setting up libncurses-dev:amd64 (6.2-0ubuntu2.1) ...
#7 79.16 Setting up libgnutls-openssl27:amd64 (3.6.13-2ubuntu1.11) ...
#7 79.18 Setting up libxcb-xfixes0:amd64 (1.14-2) ...
#7 79.20 Setting up mime-support (3.64ubuntu1) ...
#7 79.27 Setting up libncursesw5-dev:amd64 (6.2-0ubuntu2.1) ...
#7 79.29 Setting up libxrender1:amd64 (1:0.9.10-1) ...
#7 79.31 Setting up xdg-user-dirs (0.17-2ubuntu1) ...
#7 79.37 Setting up libpng-tools (1.6.37-2) ...
#7 79.39 Setting up libmagic-mgc (1:5.38-4) ...
#7 79.41 Setting up libyaml-0-2:amd64 (0.2.2-1) ...
#7 79.44 Setting up libglib2.0-0:amd64 (2.64.6-1~ubuntu20.04.7) ...
#7 79.46 No schema files found: doing nothing.
#7 79.47 Setting up libglvnd0:amd64 (1.3.2-1~ubuntu0.20.04.2) ...
#7 79.50 Setting up libxcb-glx0:amd64 (1.14-2) ...
#7 79.52 Setting up libtasn1-doc (4.16.0-2) ...
#7 79.54 Setting up libedit2:amd64 (3.1-20191231-1) ...
#7 79.56 Setting up libxcb-shape0:amd64 (1.14-2) ...
#7 79.59 Setting up x11-common (1:7.7+19ubuntu14) ...
#7 79.84 update-rc.d: warning: start and stop actions are no longer supported; falling back to defaults
#7 79.85 invoke-rc.d: could not determine current runlevel
#7 79.86 invoke-rc.d: policy-rc.d denied execution of start.
#7 79.87 Setting up libsensors-config (1:3.6.0-2ubuntu1.1) ...
#7 79.91 Setting up libmagic1:amd64 (1:5.38-4) ...
#7 79.95 Setting up libxext6:amd64 (2:1.3.4-0ubuntu1) ...
#7 79.97 Setting up libxcb-shm0:amd64 (1.14-2) ...
#7 79.99 Setting up file (1:5.38-4) ...
#7 80.01 Setting up libffi-dev:amd64 (3.3-4) ...
#7 80.04 Setting up libpthread-stubs0-dev:amd64 (0.4-1) ...
#7 80.06 Setting up libxxf86vm1:amd64 (1:1.1.4-1build1) ...
#7 80.08 Setting up libllvm10:amd64 (1:10.0.0-4ubuntu1) ...
#7 80.10 Setting up libxcb-present0:amd64 (1.14-2) ...
#7 80.12 Setting up xtrans-dev (1.4.0-1) ...
#7 80.15 Setting up libfontenc1:amd64 (1:1.1.4-0ubuntu1) ...
#7 80.17 Setting up libz3-4:amd64 (4.8.7-4build1) ...
#7 80.19 Setting up libglib2.0-data (2.64.6-1~ubuntu20.04.7) ...
#7 80.21 Setting up libgmpxx4ldbl:amd64 (2:6.2.0+dfsg-4ubuntu0.1) ...
#7 80.24 Setting up libpfm4:amd64 (4.10.1+git20-g7700f49-2) ...
#7 80.26 Setting up libexpat1-dev:amd64 (2.2.9-1ubuntu0.6) ...
#7 80.28 Setting up libsqlite3-dev:amd64 (3.31.1-4ubuntu0.6) ...
#7 80.30 Setting up libgpg-error-dev (1.37-1) ...
#7 80.32 Setting up libnspr4:amd64 (2:4.35-0ubuntu0.20.04.1) ...
#7 80.35 Setting up libxfixes3:amd64 (1:5.0.3-2) ...
#7 80.37 Setting up libxcb-sync1:amd64 (1.14-2) ...
#7 80.39 Setting up uuid-dev:amd64 (2.34-0.1ubuntu9.6) ...
#7 80.41 Setting up libllvm12:amd64 (1:12.0.0-3ubuntu1~20.04.5) ...
#7 80.44 Setting up shared-mime-info (1.15-1) ...
#7 83.97 Setting up libxinerama1:amd64 (2:1.1.4-2) ...
#7 83.99 Setting up libxv1:amd64 (2:1.0.11-1) ...
#7 84.01 Setting up libssl-dev:amd64 (1.1.1f-1ubuntu2.22) ...
#7 84.03 Setting up libevent-2.1-7:amd64 (2.1.11-stable-1) ...
#7 84.06 Setting up libxrandr2:amd64 (2:1.5.2-0ubuntu1) ...
#7 84.08 Setting up libtcl8.6:amd64 (8.6.10+dfsg-1) ...
#7 84.10 Setting up binfmt-support (2.2.0-2) ...
#7 84.34 invoke-rc.d: could not determine current runlevel
#7 84.35 invoke-rc.d: policy-rc.d denied execution of start.
#7 84.35 Setting up icu-devtools (66.1-2ubuntu2.1) ...
#7 84.37 Setting up pkg-config (0.29.1-0ubuntu4) ...
#7 84.45 Setting up libgnutlsxx28:amd64 (3.6.13-2ubuntu1.11) ...
#7 84.47 Setting up libsensors5:amd64 (1:3.6.0-2ubuntu1.1) ...
#7 84.49 Setting up libidn2-dev:amd64 (2.2.0-2) ...
#7 84.51 Setting up libglapi-mesa:amd64 (21.2.6-0ubuntu0.1~20.04.2) ...
#7 84.54 Setting up libvulkan1:amd64 (1.2.131.2-1) ...
#7 84.56 Setting up libutempter0:amd64 (1.1.6-4) ...
#7 84.58 Setting up liblzma-dev:amd64 (5.2.4-1ubuntu1.1) ...
#7 84.60 Setting up libxcb-dri2-0:amd64 (1.14-2) ...
#7 84.63 Setting up zlib1g-dev:amd64 (1:1.2.11.dfsg-2ubuntu1.5) ...
#7 84.65 Setting up libxshmfence1:amd64 (1.3-1) ...
#7 84.67 Setting up libxcb-randr0:amd64 (1.14-2) ...
#7 84.70 Setting up xorg-sgml-doctools (1:1.11-1) ...
#7 84.72 Setting up libxss1:amd64 (1:1.2.3-1) ...
#7 84.74 Setting up libxkbfile1:amd64 (1:1.1.0-1) ...
#7 84.76 Setting up libmpdec2:amd64 (2.4.2-3) ...
#7 84.78 Setting up libxmlsec1:amd64 (1.2.28-2) ...
#7 84.81 Setting up libtasn1-6-dev:amd64 (4.16.0-2) ...
#7 84.83 Setting up libdrm-common (2.4.107-8ubuntu1~20.04.2) ...
#7 84.85 Setting up libxcomposite1:amd64 (1:0.4.5-1) ...
#7 84.90 Setting up readline-common (8.0-4) ...
#7 84.93 Setting up libicu-dev:amd64 (66.1-2ubuntu2.1) ...
#7 84.95 Setting up libxmuu1:amd64 (2:1.1.3-0ubuntu1) ...
#7 84.98 Setting up xbitmaps (1.1.1-2) ...
#7 85.00 Setting up libp11-kit-dev:amd64 (0.23.20-1ubuntu0.1) ...
#7 85.02 Setting up libbz2-dev:amd64 (1.0.8-2) ...
#7 85.04 Setting up libwayland-client0:amd64 (1.18.0-1ubuntu0.1) ...
#7 85.06 Setting up x11proto-dev (2019.2-1ubuntu1) ...
#7 85.08 Setting up libnspr4-dev (2:4.35-0ubuntu0.20.04.1) ...
#7 85.11 Setting up libz3-dev:amd64 (4.8.7-4build1) ...
#7 85.13 Setting up libice6:amd64 (2:1.0.10-0ubuntu1) ...
#7 85.15 Setting up tcl8.6 (8.6.10+dfsg-1) ...
#7 85.18 Setting up libxft2:amd64 (2.3.3-0ubuntu1) ...
#7 85.20 Setting up libgmp-dev:amd64 (2:6.2.0+dfsg-4ubuntu0.1) ...
#7 85.22 Setting up libxau-dev:amd64 (1:1.0.9-0ubuntu1) ...
#7 85.24 Setting up libxi6:amd64 (2:1.7.10-0ubuntu1) ...
#7 85.27 Setting up nettle-dev:amd64 (3.5.1+really3.5.1-2ubuntu0.2) ...
#7 85.29 Setting up libice-dev:amd64 (2:1.0.10-0ubuntu1) ...
#7 85.31 Setting up libxmlsec1-openssl:amd64 (1.2.28-2) ...
#7 85.34 Setting up libxtst6:amd64 (2:1.2.3-1) ...
#7 85.36 Setting up tcl8.6-dev:amd64 (8.6.10+dfsg-1) ...
#7 85.38 Setting up libtk8.6:amd64 (8.6.10-1) ...
#7 85.40 Setting up libreadline8:amd64 (8.0-4) ...
#7 85.42 Setting up libpng-dev:amd64 (1.6.37-2) ...
#7 85.45 Setting up libxxf86dga1:amd64 (2:1.1.5-0ubuntu1) ...
#7 85.47 Setting up libnss3:amd64 (2:3.98-0ubuntu0.20.04.2) ...
#7 85.49 Setting up libreadline-dev:amd64 (8.0-4) ...
#7 85.51 Setting up libclang-cpp10 (1:10.0.0-4ubuntu1) ...
#7 85.54 Setting up llvm-10-runtime (1:10.0.0-4ubuntu1) ...
#7 85.56 Setting up libfreetype-dev:amd64 (2.10.1-2ubuntu0.3) ...
#7 85.59 Setting up libxdmcp-dev:amd64 (1:1.1.3-0ubuntu1) ...
#7 85.61 Setting up libxml2-dev:amd64 (2.9.10+dfsg-5ubuntu0.20.04.7) ...
#7 85.65 Setting up libunbound8:amd64 (1.9.4-2ubuntu1.5) ...
#7 85.67 Setting up libgcrypt20-dev (1.8.5-5ubuntu1.1) ...
#7 85.69 Setting up x11proto-core-dev (2019.2-1ubuntu1) ...
#7 85.71 Setting up llvm-runtime (1:10.0-50~exp1) ...
#7 85.74 Setting up libxmlsec1-gcrypt:amd64 (1.2.28-2) ...
#7 85.76 Setting up libxmlsec1-nss:amd64 (1.2.28-2) ...
#7 85.78 Setting up x11proto-xext-dev (2019.2-1ubuntu1) ...
#7 85.80 Setting up libdrm2:amd64 (2.4.107-8ubuntu1~20.04.2) ...
#7 85.82 Setting up libnss3-dev:amd64 (2:3.98-0ubuntu0.20.04.2) ...
#7 85.85 Setting up tcl (8.6.9+1) ...
#7 85.87 Setting up libpython3.8-stdlib:amd64 (3.8.10-0ubuntu1~20.04.9) ...
#7 85.89 Setting up python3.8 (3.8.10-0ubuntu1~20.04.9) ...
#7 86.58 Setting up libxmlsec1-gnutls:amd64 (1.2.28-2) ...
#7 86.61 Setting up x11proto-scrnsaver-dev (2019.2-1ubuntu1) ...
#7 86.63 Setting up libsm6:amd64 (2:1.2.3-1) ...
#7 86.65 Setting up libpython3-stdlib:amd64 (3.8.2-0ubuntu2) ...
#7 86.67 Setting up libdrm-amdgpu1:amd64 (2.4.107-8ubuntu1~20.04.2) ...
#7 86.69 Setting up tk8.6 (8.6.10-1) ...
#7 86.72 Setting up mesa-vulkan-drivers:amd64 (21.2.6-0ubuntu0.1~20.04.2) ...
#7 86.75 Setting up libgnutls-dane0:amd64 (3.6.13-2ubuntu1.11) ...
#7 86.77 Setting up libdrm-nouveau2:amd64 (2.4.107-8ubuntu1~20.04.2) ...
#7 86.79 Setting up libxcb1-dev:amd64 (1.14-2) ...
#7 86.81 Setting up libxslt1-dev:amd64 (1.1.34-4ubuntu0.20.04.1) ...
#7 86.84 Setting up libsm-dev:amd64 (2:1.2.3-1) ...
#7 86.86 Setting up libdrm-radeon1:amd64 (2.4.107-8ubuntu1~20.04.2) ...
#7 86.88 Setting up llvm-10 (1:10.0.0-4ubuntu1) ...
#7 86.90 Setting up libdrm-intel1:amd64 (2.4.107-8ubuntu1~20.04.2) ...
#7 86.93 Setting up libgl1-mesa-dri:amd64 (21.2.6-0ubuntu0.1~20.04.2) ...
#7 86.96 Setting up libx11-dev:amd64 (2:1.6.9-2ubuntu1.6) ...
#7 86.99 Setting up python3 (3.8.2-0ubuntu2) ...
#7 87.18 Setting up libfreetype6-dev:amd64 (2.10.1-2ubuntu0.3) ...
#7 87.21 Setting up python3-pygments (2.3.1+dfsg-1ubuntu2.2) ...
#7 87.83 Setting up llvm (1:10.0-50~exp1) ...
#7 87.85 Setting up libxt6:amd64 (1:1.1.5-1) ...
#7 87.87 Setting up tcl-dev:amd64 (8.6.9+1) ...
#7 87.90 Setting up libxext-dev:amd64 (2:1.3.4-0ubuntu1) ...
#7 87.92 Setting up tk (8.6.9+1) ...
#7 87.94 Setting up libxrender-dev:amd64 (1:0.9.10-1) ...
#7 87.96 Setting up python3-pkg-resources (45.2.0-1ubuntu0.1) ...
#7 88.28 Setting up libgnutls28-dev:amd64 (3.6.13-2ubuntu1.11) ...
#7 88.31 Setting up libxmu6:amd64 (2:1.1.3-0ubuntu1) ...
#7 88.33 Setting up libfontconfig1-dev:amd64 (2.13.1-2ubuntu3) ...
#7 88.35 Setting up libglx-mesa0:amd64 (21.2.6-0ubuntu0.1~20.04.2) ...
#7 88.37 Setting up libglx0:amd64 (1.3.2-1~ubuntu0.20.04.2) ...
#7 88.40 Setting up python3-yaml (5.3.1-1ubuntu0.1) ...
#7 88.69 Setting up libxaw7:amd64 (2:1.0.13-1) ...
#7 88.73 Setting up libxss-dev:amd64 (1:1.2.3-1) ...
#7 88.75 Setting up libgl1:amd64 (1.3.2-1~ubuntu0.20.04.2) ...
#7 88.77 Setting up libxt-dev:amd64 (1:1.1.5-1) ...
#7 88.79 Setting up libxmlsec1-dev (1.2.28-2) ...
#7 88.82 Setting up x11-utils (7.7+5) ...
#7 88.93 Setting up xterm (353-1ubuntu1.20.04.2) ...
#7 89.00 update-alternatives: using /usr/bin/xterm to provide /usr/bin/x-terminal-emulator (x-terminal-emulator) in auto mode
#7 89.00 update-alternatives: warning: skip creation of /usr/share/man/man1/x-terminal-emulator.1.gz because associated file /usr/share/man/man1/xterm.1.gz (of link group x-terminal-emulator) doesn't exist
#7 89.02 update-alternatives: using /usr/bin/lxterm to provide /usr/bin/x-terminal-emulator (x-terminal-emulator) in auto mode
#7 89.02 update-alternatives: warning: skip creation of /usr/share/man/man1/x-terminal-emulator.1.gz because associated file /usr/share/man/man1/lxterm.1.gz (of link group x-terminal-emulator) doesn't exist
#7 89.04 Setting up libxft-dev:amd64 (2.3.3-0ubuntu1) ...
#7 89.06 Setting up llvm-10-tools (1:10.0.0-4ubuntu1) ...
#7 89.08 Setting up llvm-10-dev (1:10.0.0-4ubuntu1) ...
#7 89.10 Setting up tk8.6-dev:amd64 (8.6.10-1) ...
#7 89.12 Setting up tk-dev:amd64 (8.6.9+1) ...
#7 89.14 Processing triggers for libc-bin (2.31-0ubuntu9.15) ...
#7 DONE 90.5s

#8 [ 4/17] RUN git clone     --depth 1     --branch $(git ls-remote --tags --sort=v:refname https://github.com/pyenv/pyenv.git | grep -o -E 'v[1-9]+(\.[1-9]+)+$' | tail -1)     https://github.com/pyenv/pyenv.git /root/.pyenv
#8 0.857 Cloning into '/root/.pyenv'...
#8 1.882 Note: switching to 'aabaa892438d05d96775042a129b1580fcd60105'.
#8 1.882
#8 1.882 You are in 'detached HEAD' state. You can look around, make experimental
#8 1.882 changes and commit them, and you can discard any commits you make in this
#8 1.882 state without impacting any branches by switching back to a branch.
#8 1.882
#8 1.882 If you want to create a new branch to retain commits you create, you may
#8 1.882 do so (now or later) by using -c with the switch command. Example:
#8 1.882
#8 1.882   git switch -c <new-branch-name>
#8 1.882
#8 1.882 Or undo this operation with:
#8 1.882
#8 1.882   git switch -
#8 1.882
#8 1.882 Turn off this advice by setting config variable advice.detachedHead to false
#8 1.882
#8 DONE 2.0s

#9 [ 5/17] RUN apt install -y python3.8 python3.8-distutils     && ln -s -f $(which python3.8) /usr/bin/python     && wget https://bootstrap.pypa.io/get-pip.py -O /tmp/get-pip.py     && python /tmp/get-pip.py
#9 0.502
#9 0.502 WARNING: apt does not have a stable CLI interface. Use with caution in scripts.
#9 0.502
#9 0.563 Reading package lists...
#9 1.644 Building dependency tree...
#9 1.808 Reading state information...
#9 2.060 python3.8 is already the newest version (3.8.10-0ubuntu1~20.04.9).
#9 2.060 python3.8 set to manually installed.
#9 2.060 The following NEW packages will be installed:
#9 2.060   python3-distutils python3-lib2to3
#9 2.331 0 upgraded, 2 newly installed, 0 to remove and 0 not upgraded.
#9 2.331 Need to get 218 kB of archives.
#9 2.331 After this operation, 2115 kB of additional disk space will be used.
#9 2.331 Get:1 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 python3-lib2to3 all 3.8.10-0ubuntu1~20.04 [76.3 kB]
#9 2.755 Get:2 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 python3-distutils all 3.8.10-0ubuntu1~20.04 [141 kB]
#9 3.105 debconf: delaying package configuration, since apt-utils is not installed
#9 3.173 Fetched 218 kB in 1s (254 kB/s)
#9 3.227 Selecting previously unselected package python3-lib2to3.
#9 3.227 (Reading database ...
(Reading database ... 5%
(Reading database ... 10%
(Reading database ... 15%
(Reading database ... 20%
(Reading database ... 25%
(Reading database ... 30%
(Reading database ... 35%
(Reading database ... 40%
(Reading database ... 45%
(Reading database ... 50%
(Reading database ... 55%
(Reading database ... 60%
(Reading database ... 65%
(Reading database ... 70%
(Reading database ... 75%
(Reading database ... 80%
(Reading database ... 85%
(Reading database ... 90%
(Reading database ... 95%
(Reading database ... 100%
(Reading database ... 24296 files and directories currently installed.)
#9 3.256 Preparing to unpack .../python3-lib2to3_3.8.10-0ubuntu1~20.04_all.deb ...
#9 3.263 Unpacking python3-lib2to3 (3.8.10-0ubuntu1~20.04) ...
#9 3.357 Selecting previously unselected package python3-distutils.
#9 3.358 Preparing to unpack .../python3-distutils_3.8.10-0ubuntu1~20.04_all.deb ...
#9 3.365 Unpacking python3-distutils (3.8.10-0ubuntu1~20.04) ...
#9 3.486 Setting up python3-lib2to3 (3.8.10-0ubuntu1~20.04) ...
#9 3.636 Setting up python3-distutils (3.8.10-0ubuntu1~20.04) ...
#9 3.862 --2024-05-16 20:23:48--  https://bootstrap.pypa.io/get-pip.py
#9 3.866 Resolving bootstrap.pypa.io (bootstrap.pypa.io)... 151.101.64.175, 151.101.128.175, 151.101.0.175, ...
#9 3.930 Connecting to bootstrap.pypa.io (bootstrap.pypa.io)|151.101.64.175|:443... connected.
#9 4.180 HTTP request sent, awaiting response... 200 OK
#9 4.192 Length: 2635835 (2.5M) [text/x-python]
#9 4.192 Saving to: '/tmp/get-pip.py'
#9 4.206
#9 4.206      0K .......... .......... .......... .......... ..........  1%  968K 3s
#9 4.244     50K .......... .......... .......... .......... ..........  3% 4.78M 2s
#9 4.254    100K .......... .......... .......... .......... ..........  5% 7.08M 1s
#9 4.261    150K .......... .......... .......... .......... ..........  7% 7.48M 1s
#9 4.267    200K .......... .......... .......... .......... ..........  9% 6.58M 1s
#9 4.275    250K .......... .......... .......... .......... .......... 11% 7.51M 1s
#9 4.281    300K .......... .......... .......... .......... .......... 13% 6.85M 1s
#9 4.288    350K .......... .......... .......... .......... .......... 15% 7.92M 1s
#9 4.295    400K .......... .......... .......... .......... .......... 17% 7.59M 1s
#9 4.301    450K .......... .......... .......... .......... .......... 19% 10.3M 0s
#9 4.306    500K .......... .......... .......... .......... .......... 21% 7.91M 0s
#9 4.312    550K .......... .......... .......... .......... .......... 23% 8.83M 0s
#9 4.317    600K .......... .......... .......... .......... .......... 25% 4.41M 0s
#9 4.329    650K .......... .......... .......... .......... .......... 27%  103M 0s
#9 4.329    700K .......... .......... .......... .......... .......... 29% 4.13M 0s
#9 4.341    750K .......... .......... .......... .......... .......... 31% 12.6M 0s
#9 4.345    800K .......... .......... .......... .......... .......... 33% 8.33M 0s
#9 4.350    850K .......... .......... .......... .......... .......... 34% 10.8M 0s
#9 4.355    900K .......... .......... .......... .......... .......... 36% 7.62M 0s
#9 4.362    950K .......... .......... .......... .......... .......... 38% 11.1M 0s
#9 4.366   1000K .......... .......... .......... .......... .......... 40% 14.0M 0s
#9 4.369   1050K .......... .......... .......... .......... .......... 42% 9.25M 0s
#9 4.375   1100K .......... .......... .......... .......... .......... 44% 12.5M 0s
#9 4.379   1150K .......... .......... .......... .......... .......... 46% 9.64M 0s
#9 4.383   1200K .......... .......... .......... .......... .......... 48% 9.55M 0s
#9 4.389   1250K .......... .......... .......... .......... .......... 50% 16.3M 0s
#9 4.392   1300K .......... .......... .......... .......... .......... 52% 8.00M 0s
#9 4.398   1350K .......... .......... .......... .......... .......... 54% 16.8M 0s
#9 4.401   1400K .......... .......... .......... .......... .......... 56% 10.0M 0s
#9 4.405   1450K .......... .......... .......... .......... .......... 58% 21.6M 0s
#9 4.408   1500K .......... .......... .......... .......... .......... 60% 11.4M 0s
#9 4.412   1550K .......... .......... .......... .......... .......... 62% 10.0M 0s
#9 4.417   1600K .......... .......... .......... .......... .......... 64% 11.1M 0s
#9 4.421   1650K .......... .......... .......... .......... .......... 66% 14.8M 0s
#9 4.425   1700K .......... .......... .......... .......... .......... 67% 15.5M 0s
#9 4.428   1750K .......... .......... .......... .......... .......... 69% 10.9M 0s
#9 4.432   1800K .......... .......... .......... .......... .......... 71% 15.7M 0s
#9 4.435   1850K .......... .......... .......... .......... .......... 73% 24.5M 0s
#9 4.437   1900K .......... .......... .......... .......... .......... 75% 8.37M 0s
#9 4.443   1950K .......... .......... .......... .......... .......... 77% 18.6M 0s
#9 4.446   2000K .......... .......... .......... .......... .......... 79% 20.2M 0s
#9 4.448   2050K .......... .......... .......... .......... .......... 81% 15.0M 0s
#9 4.451   2100K .......... .......... .......... .......... .......... 83% 11.1M 0s
#9 4.456   2150K .......... .......... .......... .......... .......... 85% 20.7M 0s
#9 4.458   2200K .......... .......... .......... .......... .......... 87% 18.9M 0s
#9 4.462   2250K .......... .......... .......... .......... .......... 89% 9.79M 0s
#9 4.466   2300K .......... .......... .......... .......... .......... 91% 14.9M 0s
#9 4.469   2350K .......... .......... .......... .......... .......... 93% 25.6M 0s
#9 4.471   2400K .......... .......... .......... .......... .......... 95% 10.0M 0s
#9 4.476   2450K .......... .......... .......... .......... .......... 97% 24.2M 0s
#9 4.478   2500K .......... .......... .......... .......... .......... 99% 17.4M 0s
#9 4.481   2550K .......... .......... ....                            100% 26.7M=0.3s
#9 4.481
#9 4.482 2024-05-16 20:23:49 (8.68 MB/s) - '/tmp/get-pip.py' saved [2635835/2635835]
#9 4.482
#9 6.548 Collecting pip
#9 6.614   Downloading pip-24.0-py3-none-any.whl.metadata (3.6 kB)
#9 6.895 Collecting setuptools
#9 6.909   Downloading setuptools-69.5.1-py3-none-any.whl.metadata (6.2 kB)
#9 6.992 Collecting wheel
#9 7.007   Downloading wheel-0.43.0-py3-none-any.whl.metadata (2.2 kB)
#9 7.077 Downloading pip-24.0-py3-none-any.whl (2.1 MB)
#9 7.224    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 15.0 MB/s eta 0:00:00
#9 7.243 Downloading setuptools-69.5.1-py3-none-any.whl (894 kB)
#9 7.297    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 894.6/894.6 kB 18.5 MB/s eta 0:00:00
#9 7.315 Downloading wheel-0.43.0-py3-none-any.whl (65 kB)
#9 7.331    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.8 MB/s eta 0:00:00
#9 7.407 Installing collected packages: wheel, setuptools, pip
#9 9.087 Successfully installed pip-24.0 setuptools-69.5.1 wheel-0.43.0
#9 9.088 WARNING: Running pip as the 'root' user can result in broken permissions and conflicting behaviour with the system package manager. It is recommended to use a virtual environment instead: https://pip.pypa.io/warnings/venv
#9 DONE 9.6s

#10 [ 6/17] RUN pip install virtualenv
#10 1.047 Collecting virtualenv
#10 1.095   Downloading virtualenv-20.26.2-py3-none-any.whl.metadata (4.4 kB)
#10 1.182 Collecting distlib<1,>=0.3.7 (from virtualenv)
#10 1.197   Downloading distlib-0.3.8-py2.py3-none-any.whl.metadata (5.1 kB)
#10 1.276 Collecting filelock<4,>=3.12.2 (from virtualenv)
#10 1.291   Downloading filelock-3.14.0-py3-none-any.whl.metadata (2.8 kB)
#10 1.368 Collecting platformdirs<5,>=3.9.1 (from virtualenv)
#10 1.383   Downloading platformdirs-4.2.2-py3-none-any.whl.metadata (11 kB)
#10 1.439 Downloading virtualenv-20.26.2-py3-none-any.whl (3.9 MB)
#10 1.656    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 18.3 MB/s eta 0:00:00
#10 1.673 Downloading distlib-0.3.8-py2.py3-none-any.whl (468 kB)
#10 1.706    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.9/468.9 kB 17.0 MB/s eta 0:00:00
#10 1.724 Downloading filelock-3.14.0-py3-none-any.whl (12 kB)
#10 1.752 Downloading platformdirs-4.2.2-py3-none-any.whl (18 kB)
#10 1.871 Installing collected packages: distlib, platformdirs, filelock, virtualenv
#10 2.123 Successfully installed distlib-0.3.8 filelock-3.14.0 platformdirs-4.2.2 virtualenv-20.26.2
#10 2.123 WARNING: Running pip as the 'root' user can result in broken permissions and conflicting behaviour with the system package manager. It is recommended to use a virtual environment instead: https://pip.pypa.io/warnings/venv
#10 DONE 2.3s

#11 [ 7/17] RUN apt-get install -y --no-install-recommends openjdk-11-jdk maven
#11 0.605 Reading package lists...
#11 1.699 Building dependency tree...
#11 1.870 Reading state information...
#11 2.070 The following additional packages will be installed:
#11 2.070   ca-certificates-java java-common libaopalliance-java libapache-pom-java
#11 2.070   libasound2 libasound2-data libatinject-jsr330-api-java libavahi-client3
#11 2.070   libavahi-common-data libavahi-common3 libcdi-api-java libcommons-cli-java
#11 2.070   libcommons-io-java libcommons-lang3-java libcommons-parent-java libcups2
#11 2.070   libdbus-1-3 libgeronimo-annotation-1.3-spec-java
#11 2.070   libgeronimo-interceptor-3.0-spec-java libgif7 libgraphite2-3 libguava-java
#11 2.070   libguice-java libharfbuzz0b libhawtjni-runtime-java libjansi-java
#11 2.070   libjansi-native-java libjsr305-java liblcms2-2 libmaven-parent-java
#11 2.070   libmaven-resolver-java libmaven-shared-utils-java libmaven3-core-java
#11 2.070   libpcsclite1 libplexus-cipher-java libplexus-classworlds-java
#11 2.070   libplexus-component-annotations-java libplexus-interpolation-java
#11 2.070   libplexus-sec-dispatcher-java libplexus-utils2-java libsisu-inject-java
#11 2.070   libsisu-plexus-java libslf4j-java libwagon-file-java
#11 2.071   libwagon-http-shaded-java libwagon-provider-api-java openjdk-11-jdk-headless
#11 2.071   openjdk-11-jre openjdk-11-jre-headless
#11 2.072 Suggested packages:
#11 2.072   default-jre libaopalliance-java-doc libasound2-plugins alsa-utils
#11 2.072   libatinject-jsr330-api-java-doc libservlet3.1-java libcommons-io-java-doc
#11 2.072   libcommons-lang3-java-doc cups-common libasm-java libcglib-java
#11 2.072   libjsr305-java-doc liblcms2-utils libmaven-shared-utils-java-doc
#11 2.072   liblogback-java pcscd libplexus-cipher-java-doc
#11 2.072   libplexus-classworlds-java-doc libplexus-sec-dispatcher-java-doc
#11 2.072   libplexus-utils2-java-doc junit4 testng libcommons-logging-java
#11 2.072   liblog4j1.2-java openjdk-11-demo openjdk-11-source visualvm libnss-mdns
#11 2.072   fonts-dejavu-extra fonts-ipafont-gothic fonts-ipafont-mincho
#11 2.072   fonts-wqy-microhei | fonts-wqy-zenhei fonts-indic
#11 2.072 Recommended packages:
#11 2.072   alsa-ucm-conf alsa-topology-conf dbus libatk-wrapper-java-jni
#11 2.072   fonts-dejavu-extra
#11 2.267 The following NEW packages will be installed:
#11 2.268   ca-certificates-java java-common libaopalliance-java libapache-pom-java
#11 2.268   libasound2 libasound2-data libatinject-jsr330-api-java libavahi-client3
#11 2.268   libavahi-common-data libavahi-common3 libcdi-api-java libcommons-cli-java
#11 2.268   libcommons-io-java libcommons-lang3-java libcommons-parent-java libcups2
#11 2.268   libdbus-1-3 libgeronimo-annotation-1.3-spec-java
#11 2.268   libgeronimo-interceptor-3.0-spec-java libgif7 libgraphite2-3 libguava-java
#11 2.268   libguice-java libharfbuzz0b libhawtjni-runtime-java libjansi-java
#11 2.268   libjansi-native-java libjsr305-java liblcms2-2 libmaven-parent-java
#11 2.268   libmaven-resolver-java libmaven-shared-utils-java libmaven3-core-java
#11 2.268   libpcsclite1 libplexus-cipher-java libplexus-classworlds-java
#11 2.268   libplexus-component-annotations-java libplexus-interpolation-java
#11 2.268   libplexus-sec-dispatcher-java libplexus-utils2-java libsisu-inject-java
#11 2.268   libsisu-plexus-java libslf4j-java libwagon-file-java
#11 2.269   libwagon-http-shaded-java libwagon-provider-api-java maven openjdk-11-jdk
#11 2.269   openjdk-11-jdk-headless openjdk-11-jre openjdk-11-jre-headless
#11 2.386 0 upgraded, 51 newly installed, 0 to remove and 0 not upgraded.
#11 2.386 Need to get 125 MB of archives.
#11 2.386 After this operation, 277 MB of additional disk space will be used.
#11 2.386 Get:1 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libdbus-1-3 amd64 1.12.16-2ubuntu2.3 [179 kB]
#11 2.602 Get:2 http://archive.ubuntu.com/ubuntu focal/main amd64 java-common all 0.72 [6816 B]
#11 2.603 Get:3 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libavahi-common-data amd64 0.7-4ubuntu7.3 [21.4 kB]
#11 2.607 Get:4 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libavahi-common3 amd64 0.7-4ubuntu7.3 [21.9 kB]
#11 2.610 Get:5 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libavahi-client3 amd64 0.7-4ubuntu7.3 [25.5 kB]
#11 2.615 Get:6 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libcups2 amd64 2.3.1-9ubuntu1.6 [233 kB]
#11 2.667 Get:7 http://archive.ubuntu.com/ubuntu focal/main amd64 liblcms2-2 amd64 2.9-4 [140 kB]
#11 2.690 Get:8 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libasound2-data all 1.2.2-2.1ubuntu2.5 [20.1 kB]
#11 2.694 Get:9 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libasound2 amd64 1.2.2-2.1ubuntu2.5 [335 kB]
#11 2.753 Get:10 http://archive.ubuntu.com/ubuntu focal/main amd64 libgraphite2-3 amd64 1.3.13-11build1 [73.5 kB]
#11 2.765 Get:11 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libharfbuzz0b amd64 2.6.4-1ubuntu4.2 [391 kB]
#11 2.928 Get:12 http://archive.ubuntu.com/ubuntu focal/main amd64 libpcsclite1 amd64 1.8.26-3 [22.0 kB]
#11 3.024 Get:13 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 openjdk-11-jre-headless amd64 11.0.22+7-0ubuntu2~20.04.1 [38.3 MB]
#11 10.09 Get:14 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 ca-certificates-java all 20190405ubuntu1.1 [12.4 kB]
#11 10.10 Get:15 http://archive.ubuntu.com/ubuntu focal/universe amd64 libapache-pom-java all 18-1 [4720 B]
#11 10.10 Get:16 http://archive.ubuntu.com/ubuntu focal/universe amd64 libatinject-jsr330-api-java all 1.0+ds1-5 [5348 B]
#11 10.10 Get:17 http://archive.ubuntu.com/ubuntu focal/universe amd64 libgeronimo-interceptor-3.0-spec-java all 1.0.1-4fakesync [8616 B]
#11 10.10 Get:18 http://archive.ubuntu.com/ubuntu focal/universe amd64 libcdi-api-java all 1.2-2 [54.5 kB]
#11 10.10 Get:19 http://archive.ubuntu.com/ubuntu focal/universe amd64 libcommons-cli-java all 1.4-1 [53.8 kB]
#11 10.10 Get:20 http://archive.ubuntu.com/ubuntu focal/universe amd64 libcommons-parent-java all 43-1 [10.8 kB]
#11 10.10 Get:21 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 libcommons-io-java all 2.6-2ubuntu0.20.04.1 [198 kB]
#11 10.11 Get:22 http://archive.ubuntu.com/ubuntu focal/universe amd64 libcommons-lang3-java all 3.8-2 [458 kB]
#11 10.28 Get:23 http://archive.ubuntu.com/ubuntu focal/universe amd64 libgeronimo-annotation-1.3-spec-java all 1.0-1 [10.7 kB]
#11 10.33 Get:24 http://archive.ubuntu.com/ubuntu focal/main amd64 libgif7 amd64 5.1.9-1 [32.2 kB]
#11 10.38 Get:25 http://archive.ubuntu.com/ubuntu focal/universe amd64 libjsr305-java all 0.1~+svn49-11 [27.0 kB]
#11 10.41 Get:26 http://archive.ubuntu.com/ubuntu focal/universe amd64 libguava-java all 19.0-1 [2028 kB]
#11 10.83 Get:27 http://archive.ubuntu.com/ubuntu focal/universe amd64 libaopalliance-java all 20070526-6 [9084 B]
#11 10.83 Get:28 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 libguice-java all 4.2.1-1ubuntu0.2 [1410 kB]
#11 11.08 Get:29 http://archive.ubuntu.com/ubuntu focal/universe amd64 libhawtjni-runtime-java all 1.17-1 [28.8 kB]
#11 11.08 Get:30 http://archive.ubuntu.com/ubuntu focal/universe amd64 libjansi-native-java all 1.8-1 [23.8 kB]
#11 11.08 Get:31 http://archive.ubuntu.com/ubuntu focal/universe amd64 libjansi-java all 1.18-1 [56.8 kB]
#11 11.09 Get:32 http://archive.ubuntu.com/ubuntu focal/universe amd64 libmaven-parent-java all 31-2 [5140 B]
#11 11.09 Get:33 http://archive.ubuntu.com/ubuntu focal/universe amd64 libplexus-utils2-java all 3.3.0-1 [250 kB]
#11 11.23 Get:34 http://archive.ubuntu.com/ubuntu focal/universe amd64 libwagon-provider-api-java all 3.3.4-1 [48.5 kB]
#11 11.36 Get:35 http://archive.ubuntu.com/ubuntu focal/universe amd64 libmaven-resolver-java all 1.4.1-1 [555 kB]
#11 11.52 Get:36 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 libmaven-shared-utils-java all 3.3.0-1ubuntu0.20.04.1 [149 kB]
#11 11.55 Get:37 http://archive.ubuntu.com/ubuntu focal/universe amd64 libplexus-cipher-java all 1.7-3 [15.1 kB]
#11 11.55 Get:38 http://archive.ubuntu.com/ubuntu focal/universe amd64 libplexus-classworlds-java all 2.6.0-1 [49.4 kB]
#11 11.56 Get:39 http://archive.ubuntu.com/ubuntu focal/universe amd64 libplexus-component-annotations-java all 2.1.0-1 [6564 B]
#11 11.56 Get:40 http://archive.ubuntu.com/ubuntu focal/universe amd64 libplexus-interpolation-java all 1.26-1 [76.8 kB]
#11 11.58 Get:41 http://archive.ubuntu.com/ubuntu focal/universe amd64 libplexus-sec-dispatcher-java all 1.4-4 [28.1 kB]
#11 11.58 Get:42 http://archive.ubuntu.com/ubuntu focal/universe amd64 libslf4j-java all 1.7.25-3 [141 kB]
#11 11.61 Get:43 http://archive.ubuntu.com/ubuntu focal/universe amd64 libsisu-inject-java all 0.3.3-1 [347 kB]
#11 11.67 Get:44 http://archive.ubuntu.com/ubuntu focal/universe amd64 libsisu-plexus-java all 0.3.3-3 [182 kB]
#11 11.79 Get:45 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 libmaven3-core-java all 3.6.3-1ubuntu0.1 [1535 kB]
#11 12.26 Get:46 http://archive.ubuntu.com/ubuntu focal/universe amd64 libwagon-file-java all 3.3.4-1 [7928 B]
#11 12.26 Get:47 http://archive.ubuntu.com/ubuntu focal/universe amd64 libwagon-http-shaded-java all 3.3.4-1 [1855 kB]
#11 12.59 Get:48 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 maven all 3.6.3-1ubuntu0.1 [17.8 kB]
#11 12.60 Get:49 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 openjdk-11-jre amd64 11.0.22+7-0ubuntu2~20.04.1 [194 kB]
#11 12.63 Get:50 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 openjdk-11-jdk-headless amd64 11.0.22+7-0ubuntu2~20.04.1 [73.7 MB]
#11 25.68 Get:51 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 openjdk-11-jdk amd64 11.0.22+7-0ubuntu2~20.04.1 [1338 kB]
#11 26.04 debconf: delaying package configuration, since apt-utils is not installed
#11 26.11 Fetched 125 MB in 24s (5286 kB/s)
#11 26.18 Selecting previously unselected package libdbus-1-3:amd64.
#11 26.18 (Reading database ...
(Reading database ... 5%
(Reading database ... 10%
(Reading database ... 15%
(Reading database ... 20%
(Reading database ... 25%
(Reading database ... 30%
(Reading database ... 35%
(Reading database ... 40%
(Reading database ... 45%
(Reading database ... 50%
(Reading database ... 55%
(Reading database ... 60%
(Reading database ... 65%
(Reading database ... 70%
(Reading database ... 75%
(Reading database ... 80%
(Reading database ... 85%
(Reading database ... 90%
(Reading database ... 95%
(Reading database ... 100%
(Reading database ... 24567 files and directories currently installed.)
#11 26.21 Preparing to unpack .../00-libdbus-1-3_1.12.16-2ubuntu2.3_amd64.deb ...
#11 26.22 Unpacking libdbus-1-3:amd64 (1.12.16-2ubuntu2.3) ...
#11 26.33 Selecting previously unselected package java-common.
#11 26.33 Preparing to unpack .../01-java-common_0.72_all.deb ...
#11 26.34 Unpacking java-common (0.72) ...
#11 26.40 Selecting previously unselected package libavahi-common-data:amd64.
#11 26.41 Preparing to unpack .../02-libavahi-common-data_0.7-4ubuntu7.3_amd64.deb ...
#11 26.42 Unpacking libavahi-common-data:amd64 (0.7-4ubuntu7.3) ...
#11 26.49 Selecting previously unselected package libavahi-common3:amd64.
#11 26.50 Preparing to unpack .../03-libavahi-common3_0.7-4ubuntu7.3_amd64.deb ...
#11 26.50 Unpacking libavahi-common3:amd64 (0.7-4ubuntu7.3) ...
#11 26.58 Selecting previously unselected package libavahi-client3:amd64.
#11 26.59 Preparing to unpack .../04-libavahi-client3_0.7-4ubuntu7.3_amd64.deb ...
#11 26.59 Unpacking libavahi-client3:amd64 (0.7-4ubuntu7.3) ...
#11 26.68 Selecting previously unselected package libcups2:amd64.
#11 26.68 Preparing to unpack .../05-libcups2_2.3.1-9ubuntu1.6_amd64.deb ...
#11 26.69 Unpacking libcups2:amd64 (2.3.1-9ubuntu1.6) ...
#11 26.80 Selecting previously unselected package liblcms2-2:amd64.
#11 26.80 Preparing to unpack .../06-liblcms2-2_2.9-4_amd64.deb ...
#11 26.81 Unpacking liblcms2-2:amd64 (2.9-4) ...
#11 26.90 Selecting previously unselected package libasound2-data.
#11 26.91 Preparing to unpack .../07-libasound2-data_1.2.2-2.1ubuntu2.5_all.deb ...
#11 26.91 Unpacking libasound2-data (1.2.2-2.1ubuntu2.5) ...
#11 27.01 Selecting previously unselected package libasound2:amd64.
#11 27.01 Preparing to unpack .../08-libasound2_1.2.2-2.1ubuntu2.5_amd64.deb ...
#11 27.02 Unpacking libasound2:amd64 (1.2.2-2.1ubuntu2.5) ...
#11 27.14 Selecting previously unselected package libgraphite2-3:amd64.
#11 27.14 Preparing to unpack .../09-libgraphite2-3_1.3.13-11build1_amd64.deb ...
#11 27.15 Unpacking libgraphite2-3:amd64 (1.3.13-11build1) ...
#11 27.24 Selecting previously unselected package libharfbuzz0b:amd64.
#11 27.24 Preparing to unpack .../10-libharfbuzz0b_2.6.4-1ubuntu4.2_amd64.deb ...
#11 27.25 Unpacking libharfbuzz0b:amd64 (2.6.4-1ubuntu4.2) ...
#11 27.37 Selecting previously unselected package libpcsclite1:amd64.
#11 27.38 Preparing to unpack .../11-libpcsclite1_1.8.26-3_amd64.deb ...
#11 27.38 Unpacking libpcsclite1:amd64 (1.8.26-3) ...
#11 27.47 Selecting previously unselected package openjdk-11-jre-headless:amd64.
#11 27.47 Preparing to unpack .../12-openjdk-11-jre-headless_11.0.22+7-0ubuntu2~20.04.1_amd64.deb ...
#11 27.48 Unpacking openjdk-11-jre-headless:amd64 (11.0.22+7-0ubuntu2~20.04.1) ...
#11 31.00 Selecting previously unselected package ca-certificates-java.
#11 31.00 Preparing to unpack .../13-ca-certificates-java_20190405ubuntu1.1_all.deb ...
#11 31.02 Unpacking ca-certificates-java (20190405ubuntu1.1) ...
#11 31.08 Selecting previously unselected package libapache-pom-java.
#11 31.09 Preparing to unpack .../14-libapache-pom-java_18-1_all.deb ...
#11 31.10 Unpacking libapache-pom-java (18-1) ...
#11 31.16 Selecting previously unselected package libatinject-jsr330-api-java.
#11 31.16 Preparing to unpack .../15-libatinject-jsr330-api-java_1.0+ds1-5_all.deb ...
#11 31.17 Unpacking libatinject-jsr330-api-java (1.0+ds1-5) ...
#11 31.23 Selecting previously unselected package libgeronimo-interceptor-3.0-spec-java.
#11 31.24 Preparing to unpack .../16-libgeronimo-interceptor-3.0-spec-java_1.0.1-4fakesync_all.deb ...
#11 31.25 Unpacking libgeronimo-interceptor-3.0-spec-java (1.0.1-4fakesync) ...
#11 31.31 Selecting previously unselected package libcdi-api-java.
#11 31.31 Preparing to unpack .../17-libcdi-api-java_1.2-2_all.deb ...
#11 31.32 Unpacking libcdi-api-java (1.2-2) ...
#11 31.39 Selecting previously unselected package libcommons-cli-java.
#11 31.39 Preparing to unpack .../18-libcommons-cli-java_1.4-1_all.deb ...
#11 31.40 Unpacking libcommons-cli-java (1.4-1) ...
#11 31.47 Selecting previously unselected package libcommons-parent-java.
#11 31.48 Preparing to unpack .../19-libcommons-parent-java_43-1_all.deb ...
#11 31.49 Unpacking libcommons-parent-java (43-1) ...
#11 31.55 Selecting previously unselected package libcommons-io-java.
#11 31.56 Preparing to unpack .../20-libcommons-io-java_2.6-2ubuntu0.20.04.1_all.deb ...
#11 31.56 Unpacking libcommons-io-java (2.6-2ubuntu0.20.04.1) ...
#11 31.66 Selecting previously unselected package libcommons-lang3-java.
#11 31.66 Preparing to unpack .../21-libcommons-lang3-java_3.8-2_all.deb ...
#11 31.67 Unpacking libcommons-lang3-java (3.8-2) ...
#11 31.78 Selecting previously unselected package libgeronimo-annotation-1.3-spec-java.
#11 31.78 Preparing to unpack .../22-libgeronimo-annotation-1.3-spec-java_1.0-1_all.deb ...
#11 31.79 Unpacking libgeronimo-annotation-1.3-spec-java (1.0-1) ...
#11 31.86 Selecting previously unselected package libgif7:amd64.
#11 31.87 Preparing to unpack .../23-libgif7_5.1.9-1_amd64.deb ...
#11 31.88 Unpacking libgif7:amd64 (5.1.9-1) ...
#11 31.95 Selecting previously unselected package libjsr305-java.
#11 31.95 Preparing to unpack .../24-libjsr305-java_0.1~+svn49-11_all.deb ...
#11 31.96 Unpacking libjsr305-java (0.1~+svn49-11) ...
#11 32.03 Selecting previously unselected package libguava-java.
#11 32.03 Preparing to unpack .../25-libguava-java_19.0-1_all.deb ...
#11 32.04 Unpacking libguava-java (19.0-1) ...
#11 32.29 Selecting previously unselected package libaopalliance-java.
#11 32.29 Preparing to unpack .../26-libaopalliance-java_20070526-6_all.deb ...
#11 32.30 Unpacking libaopalliance-java (20070526-6) ...
#11 32.36 Selecting previously unselected package libguice-java.
#11 32.37 Preparing to unpack .../27-libguice-java_4.2.1-1ubuntu0.2_all.deb ...
#11 32.37 Unpacking libguice-java (4.2.1-1ubuntu0.2) ...
#11 32.59 Selecting previously unselected package libhawtjni-runtime-java.
#11 32.60 Preparing to unpack .../28-libhawtjni-runtime-java_1.17-1_all.deb ...
#11 32.60 Unpacking libhawtjni-runtime-java (1.17-1) ...
#11 32.67 Selecting previously unselected package libjansi-native-java.
#11 32.68 Preparing to unpack .../29-libjansi-native-java_1.8-1_all.deb ...
#11 32.68 Unpacking libjansi-native-java (1.8-1) ...
#11 32.75 Selecting previously unselected package libjansi-java.
#11 32.75 Preparing to unpack .../30-libjansi-java_1.18-1_all.deb ...
#11 32.76 Unpacking libjansi-java (1.18-1) ...
#11 32.83 Selecting previously unselected package libmaven-parent-java.
#11 32.84 Preparing to unpack .../31-libmaven-parent-java_31-2_all.deb ...
#11 32.85 Unpacking libmaven-parent-java (31-2) ...
#11 32.91 Selecting previously unselected package libplexus-utils2-java.
#11 32.91 Preparing to unpack .../32-libplexus-utils2-java_3.3.0-1_all.deb ...
#11 32.92 Unpacking libplexus-utils2-java (3.3.0-1) ...
#11 33.02 Selecting previously unselected package libwagon-provider-api-java.
#11 33.02 Preparing to unpack .../33-libwagon-provider-api-java_3.3.4-1_all.deb ...
#11 33.03 Unpacking libwagon-provider-api-java (3.3.4-1) ...
#11 33.10 Selecting previously unselected package libmaven-resolver-java.
#11 33.11 Preparing to unpack .../34-libmaven-resolver-java_1.4.1-1_all.deb ...
#11 33.12 Unpacking libmaven-resolver-java (1.4.1-1) ...
#11 33.26 Selecting previously unselected package libmaven-shared-utils-java.
#11 33.27 Preparing to unpack .../35-libmaven-shared-utils-java_3.3.0-1ubuntu0.20.04.1_all.deb ...
#11 33.27 Unpacking libmaven-shared-utils-java (3.3.0-1ubuntu0.20.04.1) ...
#11 33.36 Selecting previously unselected package libplexus-cipher-java.
#11 33.37 Preparing to unpack .../36-libplexus-cipher-java_1.7-3_all.deb ...
#11 33.37 Unpacking libplexus-cipher-java (1.7-3) ...
#11 33.44 Selecting previously unselected package libplexus-classworlds-java.
#11 33.45 Preparing to unpack .../37-libplexus-classworlds-java_2.6.0-1_all.deb ...
#11 33.45 Unpacking libplexus-classworlds-java (2.6.0-1) ...
#11 33.53 Selecting previously unselected package libplexus-component-annotations-java.
#11 33.53 Preparing to unpack .../38-libplexus-component-annotations-java_2.1.0-1_all.deb ...
#11 33.54 Unpacking libplexus-component-annotations-java (2.1.0-1) ...
#11 33.60 Selecting previously unselected package libplexus-interpolation-java.
#11 33.60 Preparing to unpack .../39-libplexus-interpolation-java_1.26-1_all.deb ...
#11 33.61 Unpacking libplexus-interpolation-java (1.26-1) ...
#11 33.69 Selecting previously unselected package libplexus-sec-dispatcher-java.
#11 33.70 Preparing to unpack .../40-libplexus-sec-dispatcher-java_1.4-4_all.deb ...
#11 33.70 Unpacking libplexus-sec-dispatcher-java (1.4-4) ...
#11 33.77 Selecting previously unselected package libslf4j-java.
#11 33.77 Preparing to unpack .../41-libslf4j-java_1.7.25-3_all.deb ...
#11 33.78 Unpacking libslf4j-java (1.7.25-3) ...
#11 33.89 Selecting previously unselected package libsisu-inject-java.
#11 33.89 Preparing to unpack .../42-libsisu-inject-java_0.3.3-1_all.deb ...
#11 33.90 Unpacking libsisu-inject-java (0.3.3-1) ...
#11 34.01 Selecting previously unselected package libsisu-plexus-java.
#11 34.02 Preparing to unpack .../43-libsisu-plexus-java_0.3.3-3_all.deb ...
#11 34.02 Unpacking libsisu-plexus-java (0.3.3-3) ...
#11 34.12 Selecting previously unselected package libmaven3-core-java.
#11 34.12 Preparing to unpack .../44-libmaven3-core-java_3.6.3-1ubuntu0.1_all.deb ...
#11 34.13 Unpacking libmaven3-core-java (3.6.3-1ubuntu0.1) ...
#11 34.35 Selecting previously unselected package libwagon-file-java.
#11 34.36 Preparing to unpack .../45-libwagon-file-java_3.3.4-1_all.deb ...
#11 34.36 Unpacking libwagon-file-java (3.3.4-1) ...
#11 34.42 Selecting previously unselected package libwagon-http-shaded-java.
#11 34.43 Preparing to unpack .../46-libwagon-http-shaded-java_3.3.4-1_all.deb ...
#11 34.43 Unpacking libwagon-http-shaded-java (3.3.4-1) ...
#11 34.68 Selecting previously unselected package maven.
#11 34.68 Preparing to unpack .../47-maven_3.6.3-1ubuntu0.1_all.deb ...
#11 34.69 Unpacking maven (3.6.3-1ubuntu0.1) ...
#11 34.76 Selecting previously unselected package openjdk-11-jre:amd64.
#11 34.76 Preparing to unpack .../48-openjdk-11-jre_11.0.22+7-0ubuntu2~20.04.1_amd64.deb ...
#11 34.77 Unpacking openjdk-11-jre:amd64 (11.0.22+7-0ubuntu2~20.04.1) ...
#11 34.88 Selecting previously unselected package openjdk-11-jdk-headless:amd64.
#11 34.89 Preparing to unpack .../49-openjdk-11-jdk-headless_11.0.22+7-0ubuntu2~20.04.1_amd64.deb ...
#11 34.89 Unpacking openjdk-11-jdk-headless:amd64 (11.0.22+7-0ubuntu2~20.04.1) ...
#11 39.97 Selecting previously unselected package openjdk-11-jdk:amd64.
#11 39.97 Preparing to unpack .../50-openjdk-11-jdk_11.0.22+7-0ubuntu2~20.04.1_amd64.deb ...
#11 39.98 Unpacking openjdk-11-jdk:amd64 (11.0.22+7-0ubuntu2~20.04.1) ...
#11 40.14 Setting up libgraphite2-3:amd64 (1.3.13-11build1) ...
#11 40.16 Setting up liblcms2-2:amd64 (2.9-4) ...
#11 40.18 Setting up libslf4j-java (1.7.25-3) ...
#11 40.20 Setting up libplexus-utils2-java (3.3.0-1) ...
#11 40.22 Setting up libplexus-classworlds-java (2.6.0-1) ...
#11 40.24 Setting up libjsr305-java (0.1~+svn49-11) ...
#11 40.26 Setting up java-common (0.72) ...
#11 40.30 Setting up libaopalliance-java (20070526-6) ...
#11 40.32 Setting up libcommons-cli-java (1.4-1) ...
#11 40.34 Setting up libplexus-component-annotations-java (2.1.0-1) ...
#11 40.38 Setting up libplexus-cipher-java (1.7-3) ...
#11 40.40 Setting up libgeronimo-annotation-1.3-spec-java (1.0-1) ...
#11 40.42 Setting up libgeronimo-interceptor-3.0-spec-java (1.0.1-4fakesync) ...
#11 40.44 Setting up libasound2-data (1.2.2-2.1ubuntu2.5) ...
#11 40.46 Setting up libapache-pom-java (18-1) ...
#11 40.48 Setting up libavahi-common-data:amd64 (0.7-4ubuntu7.3) ...
#11 40.50 Setting up libatinject-jsr330-api-java (1.0+ds1-5) ...
#11 40.53 Setting up libdbus-1-3:amd64 (1.12.16-2ubuntu2.3) ...
#11 40.55 Setting up libplexus-interpolation-java (1.26-1) ...
#11 40.57 Setting up libpcsclite1:amd64 (1.8.26-3) ...
#11 40.59 Setting up libgif7:amd64 (5.1.9-1) ...
#11 40.61 Setting up libasound2:amd64 (1.2.2-2.1ubuntu2.5) ...
#11 40.63 Setting up libplexus-sec-dispatcher-java (1.4-4) ...
#11 40.65 Setting up libharfbuzz0b:amd64 (2.6.4-1ubuntu4.2) ...
#11 40.67 Setting up libwagon-http-shaded-java (3.3.4-1) ...
#11 40.69 Setting up libcdi-api-java (1.2-2) ...
#11 40.71 Setting up libhawtjni-runtime-java (1.17-1) ...
#11 40.73 Setting up libwagon-provider-api-java (3.3.4-1) ...
#11 40.75 Setting up libmaven-parent-java (31-2) ...
#11 40.77 Setting up libcommons-parent-java (43-1) ...
#11 40.79 Setting up libavahi-common3:amd64 (0.7-4ubuntu7.3) ...
#11 40.81 Setting up libmaven-resolver-java (1.4.1-1) ...
#11 40.84 Setting up libguava-java (19.0-1) ...
#11 40.86 Setting up libcommons-lang3-java (3.8-2) ...
#11 40.88 Setting up libjansi-native-java (1.8-1) ...
#11 40.90 Setting up libwagon-file-java (3.3.4-1) ...
#11 40.92 Setting up libavahi-client3:amd64 (0.7-4ubuntu7.3) ...
#11 40.94 Setting up libcommons-io-java (2.6-2ubuntu0.20.04.1) ...
#11 40.96 Setting up libguice-java (4.2.1-1ubuntu0.2) ...
#11 40.98 Setting up libjansi-java (1.18-1) ...
#11 41.00 Setting up libcups2:amd64 (2.3.1-9ubuntu1.6) ...
#11 41.02 Setting up libmaven-shared-utils-java (3.3.0-1ubuntu0.20.04.1) ...
#11 41.04 Setting up libsisu-inject-java (0.3.3-1) ...
#11 41.06 Setting up libsisu-plexus-java (0.3.3-3) ...
#11 41.08 Setting up libmaven3-core-java (3.6.3-1ubuntu0.1) ...
#11 41.11 Setting up openjdk-11-jre-headless:amd64 (11.0.22+7-0ubuntu2~20.04.1) ...
#11 41.31 update-alternatives: using /usr/lib/jvm/java-11-openjdk-amd64/bin/java to provide /usr/bin/java (java) in auto mode
#11 41.32 update-alternatives: using /usr/lib/jvm/java-11-openjdk-amd64/bin/jjs to provide /usr/bin/jjs (jjs) in auto mode
#11 41.33 update-alternatives: using /usr/lib/jvm/java-11-openjdk-amd64/bin/keytool to provide /usr/bin/keytool (keytool) in auto mode
#11 41.34 update-alternatives: using /usr/lib/jvm/java-11-openjdk-amd64/bin/rmid to provide /usr/bin/rmid (rmid) in auto mode
#11 41.34 update-alternatives: using /usr/lib/jvm/java-11-openjdk-amd64/bin/rmiregistry to provide /usr/bin/rmiregistry (rmiregistry) in auto mode
#11 41.35 update-alternatives: using /usr/lib/jvm/java-11-openjdk-amd64/bin/pack200 to provide /usr/bin/pack200 (pack200) in auto mode
#11 41.36 update-alternatives: using /usr/lib/jvm/java-11-openjdk-amd64/bin/unpack200 to provide /usr/bin/unpack200 (unpack200) in auto mode
#11 41.37 update-alternatives: using /usr/lib/jvm/java-11-openjdk-amd64/lib/jexec to provide /usr/bin/jexec (jexec) in auto mode
#11 41.92 Setting up openjdk-11-jre:amd64 (11.0.22+7-0ubuntu2~20.04.1) ...
#11 41.95 Setting up openjdk-11-jdk-headless:amd64 (11.0.22+7-0ubuntu2~20.04.1) ...
#11 41.98 update-alternatives: using /usr/lib/jvm/java-11-openjdk-amd64/bin/jar to provide /usr/bin/jar (jar) in auto mode
#11 41.99 update-alternatives: using /usr/lib/jvm/java-11-openjdk-amd64/bin/jarsigner to provide /usr/bin/jarsigner (jarsigner) in auto mode
#11 42.00 update-alternatives: using /usr/lib/jvm/java-11-openjdk-amd64/bin/javac to provide /usr/bin/javac (javac) in auto mode
#11 42.00 update-alternatives: using /usr/lib/jvm/java-11-openjdk-amd64/bin/javadoc to provide /usr/bin/javadoc (javadoc) in auto mode
#11 42.01 update-alternatives: using /usr/lib/jvm/java-11-openjdk-amd64/bin/javap to provide /usr/bin/javap (javap) in auto mode
#11 42.02 update-alternatives: using /usr/lib/jvm/java-11-openjdk-amd64/bin/jcmd to provide /usr/bin/jcmd (jcmd) in auto mode
#11 42.03 update-alternatives: using /usr/lib/jvm/java-11-openjdk-amd64/bin/jdb to provide /usr/bin/jdb (jdb) in auto mode
#11 42.03 update-alternatives: using /usr/lib/jvm/java-11-openjdk-amd64/bin/jdeprscan to provide /usr/bin/jdeprscan (jdeprscan) in auto mode
#11 42.04 update-alternatives: using /usr/lib/jvm/java-11-openjdk-amd64/bin/jdeps to provide /usr/bin/jdeps (jdeps) in auto mode
#11 42.05 update-alternatives: using /usr/lib/jvm/java-11-openjdk-amd64/bin/jfr to provide /usr/bin/jfr (jfr) in auto mode
#11 42.06 update-alternatives: using /usr/lib/jvm/java-11-openjdk-amd64/bin/jimage to provide /usr/bin/jimage (jimage) in auto mode
#11 42.07 update-alternatives: using /usr/lib/jvm/java-11-openjdk-amd64/bin/jinfo to provide /usr/bin/jinfo (jinfo) in auto mode
#11 42.07 update-alternatives: using /usr/lib/jvm/java-11-openjdk-amd64/bin/jlink to provide /usr/bin/jlink (jlink) in auto mode
#11 42.08 update-alternatives: using /usr/lib/jvm/java-11-openjdk-amd64/bin/jmap to provide /usr/bin/jmap (jmap) in auto mode
#11 42.09 update-alternatives: using /usr/lib/jvm/java-11-openjdk-amd64/bin/jmod to provide /usr/bin/jmod (jmod) in auto mode
#11 42.10 update-alternatives: using /usr/lib/jvm/java-11-openjdk-amd64/bin/jps to provide /usr/bin/jps (jps) in auto mode
#11 42.11 update-alternatives: using /usr/lib/jvm/java-11-openjdk-amd64/bin/jrunscript to provide /usr/bin/jrunscript (jrunscript) in auto mode
#11 42.11 update-alternatives: using /usr/lib/jvm/java-11-openjdk-amd64/bin/jshell to provide /usr/bin/jshell (jshell) in auto mode
#11 42.12 update-alternatives: using /usr/lib/jvm/java-11-openjdk-amd64/bin/jstack to provide /usr/bin/jstack (jstack) in auto mode
#11 42.13 update-alternatives: using /usr/lib/jvm/java-11-openjdk-amd64/bin/jstat to provide /usr/bin/jstat (jstat) in auto mode
#11 42.14 update-alternatives: using /usr/lib/jvm/java-11-openjdk-amd64/bin/jstatd to provide /usr/bin/jstatd (jstatd) in auto mode
#11 42.15 update-alternatives: using /usr/lib/jvm/java-11-openjdk-amd64/bin/rmic to provide /usr/bin/rmic (rmic) in auto mode
#11 42.16 update-alternatives: using /usr/lib/jvm/java-11-openjdk-amd64/bin/serialver to provide /usr/bin/serialver (serialver) in auto mode
#11 42.16 update-alternatives: using /usr/lib/jvm/java-11-openjdk-amd64/bin/jaotc to provide /usr/bin/jaotc (jaotc) in auto mode
#11 42.17 update-alternatives: using /usr/lib/jvm/java-11-openjdk-amd64/bin/jhsdb to provide /usr/bin/jhsdb (jhsdb) in auto mode
#11 42.19 Setting up maven (3.6.3-1ubuntu0.1) ...
#11 42.23 update-alternatives: using /usr/share/maven/bin/mvn to provide /usr/bin/mvn (mvn) in auto mode
#11 42.25 Setting up openjdk-11-jdk:amd64 (11.0.22+7-0ubuntu2~20.04.1) ...
#11 42.28 update-alternatives: using /usr/lib/jvm/java-11-openjdk-amd64/bin/jconsole to provide /usr/bin/jconsole (jconsole) in auto mode
#11 42.29 Setting up ca-certificates-java (20190405ubuntu1.1) ...
#11 42.35 head: cannot open '/etc/ssl/certs/java/cacerts' for reading: No such file or directory
#11 42.65 Adding debian:COMODO_RSA_Certification_Authority.pem
#11 42.66 Adding debian:Hongkong_Post_Root_CA_1.pem
#11 42.67 Adding debian:Entrust_Root_Certification_Authority_-_EC1.pem
#11 42.67 Adding debian:Certainly_Root_R1.pem
#11 42.68 Adding debian:Amazon_Root_CA_2.pem
#11 42.68 Adding debian:Certainly_Root_E1.pem
#11 42.68 Adding debian:AffirmTrust_Premium_ECC.pem
#11 42.68 Adding debian:ANF_Secure_Server_Root_CA.pem
#11 42.68 Adding debian:GlobalSign_Root_R46.pem
#11 42.69 Adding debian:Autoridad_de_Certificacion_Firmaprofesional_CIF_A62634068.pem
#11 42.69 Adding debian:Certum_Trusted_Root_CA.pem
#11 42.69 Adding debian:Go_Daddy_Root_Certificate_Authority_-_G2.pem
#11 42.69 Adding debian:SecureSign_RootCA11.pem
#11 42.69 Adding debian:Amazon_Root_CA_4.pem
#11 42.70 Adding debian:GDCA_TrustAUTH_R5_ROOT.pem
#11 42.70 Adding debian:T-TeleSec_GlobalRoot_Class_2.pem
#11 42.70 Adding debian:Trustwave_Global_ECC_P256_Certification_Authority.pem
#11 42.70 Adding debian:GTS_Root_R1.pem
#11 42.71 Adding debian:Izenpe.com.pem
#11 42.71 Adding debian:CFCA_EV_ROOT.pem
#11 42.71 Adding debian:DigiCert_Global_Root_G2.pem
#11 42.71 Adding debian:XRamp_Global_CA_Root.pem
#11 42.72 Adding debian:SSL.com_EV_Root_Certification_Authority_RSA_R2.pem
#11 42.72 Adding debian:Security_Communication_RootCA3.pem
#11 42.72 Adding debian:Starfield_Services_Root_Certificate_Authority_-_G2.pem
#11 42.72 Adding debian:Amazon_Root_CA_1.pem
#11 42.72 Adding debian:DigiCert_Assured_ID_Root_CA.pem
#11 42.72 Adding debian:SSL.com_Root_Certification_Authority_RSA.pem
#11 42.73 Adding debian:Telia_Root_CA_v2.pem
#11 42.73 Adding debian:SwissSign_Gold_CA_-_G2.pem
#11 42.73 Adding debian:GLOBALTRUST_2020.pem
#11 42.73 Adding debian:emSign_ECC_Root_CA_-_G3.pem
#11 42.73 Adding debian:Microsoft_RSA_Root_Certificate_Authority_2017.pem
#11 42.74 Adding debian:TunTrust_Root_CA.pem
#11 42.74 Adding debian:Security_Communication_ECC_RootCA1.pem
#11 42.74 Adding debian:DigiCert_TLS_ECC_P384_Root_G5.pem
#11 42.74 Adding debian:vTrus_Root_CA.pem
#11 42.75 Adding debian:Certum_EC-384_CA.pem
#11 42.76 Adding debian:CA_Disig_Root_R2.pem
#11 42.76 Adding debian:emSign_Root_CA_-_C1.pem
#11 42.77 Adding debian:AffirmTrust_Commercial.pem
#11 42.77 Adding debian:Entrust_Root_Certification_Authority_-_G4.pem
#11 42.78 Adding debian:Starfield_Class_2_CA.pem
#11 42.78 Adding debian:Baltimore_CyberTrust_Root.pem
#11 42.79 Adding debian:GTS_Root_R4.pem
#11 42.79 Adding debian:Starfield_Root_Certificate_Authority_-_G2.pem
#11 42.80 Adding debian:QuoVadis_Root_CA_1_G3.pem
#11 42.81 Adding debian:DigiCert_Trusted_Root_G4.pem
#11 42.81 Adding debian:DigiCert_High_Assurance_EV_Root_CA.pem
#11 42.82 Adding debian:AC_RAIZ_FNMT-RCM.pem
#11 42.82 Adding debian:Comodo_AAA_Services_root.pem
#11 42.82 Adding debian:emSign_ECC_Root_CA_-_C3.pem
#11 42.83 Adding debian:Entrust_Root_Certification_Authority.pem
#11 42.84 Adding debian:D-TRUST_EV_Root_CA_1_2020.pem
#11 42.85 Adding debian:Certigna_Root_CA.pem
#11 42.85 Adding debian:TeliaSonera_Root_CA_v1.pem
#11 42.86 Adding debian:Certum_Trusted_Network_CA.pem
#11 42.86 Adding debian:HiPKI_Root_CA_-_G1.pem
#11 42.87 Adding debian:SZAFIR_ROOT_CA2.pem
#11 42.87 Adding debian:USERTrust_RSA_Certification_Authority.pem
#11 42.88 Adding debian:OISTE_WISeKey_Global_Root_GC_CA.pem
#11 42.88 Adding debian:Buypass_Class_2_Root_CA.pem
#11 42.89 Adding debian:AC_RAIZ_FNMT-RCM_SERVIDORES_SEGUROS.pem
#11 42.89 Adding debian:e-Szigno_Root_CA_2017.pem
#11 42.90 Adding debian:Microsoft_ECC_Root_Certificate_Authority_2017.pem
#11 42.91 Adding debian:Security_Communication_Root_CA.pem
#11 42.91 Adding debian:GlobalSign_Root_CA.pem
#11 42.92 Adding debian:E-Tugra_Certification_Authority.pem
#11 42.93 Adding debian:Trustwave_Global_Certification_Authority.pem
#11 42.93 Adding debian:QuoVadis_Root_CA_2_G3.pem
#11 42.93 Adding debian:SSL.com_EV_Root_Certification_Authority_ECC.pem
#11 42.94 Adding debian:Autoridad_de_Certificacion_Firmaprofesional_CIF_A62634068_2.pem
#11 42.95 Adding debian:E-Tugra_Global_Root_CA_ECC_v3.pem
#11 42.95 Adding debian:Entrust.net_Premium_2048_Secure_Server_CA.pem
#11 42.96 Adding debian:AffirmTrust_Premium.pem
#11 42.96 Adding debian:Amazon_Root_CA_3.pem
#11 42.97 Adding debian:AffirmTrust_Networking.pem
#11 42.98 Adding debian:IdenTrust_Public_Sector_Root_CA_1.pem
#11 42.98 Adding debian:GlobalSign_ECC_Root_CA_-_R4.pem
#11 42.99 Adding debian:QuoVadis_Root_CA_3_G3.pem
#11 42.99 Adding debian:certSIGN_ROOT_CA.pem
#11 43.00 Adding debian:IdenTrust_Commercial_Root_CA_1.pem
#11 43.00 Adding debian:SecureTrust_CA.pem
#11 43.01 Adding debian:DigiCert_Global_Root_G3.pem
#11 43.01 Adding debian:GlobalSign_Root_E46.pem
#11 43.02 Adding debian:SwissSign_Silver_CA_-_G2.pem
#11 43.02 Adding debian:vTrus_ECC_Root_CA.pem
#11 43.03 Adding debian:GlobalSign_ECC_Root_CA_-_R5.pem
#11 43.04 Adding debian:OISTE_WISeKey_Global_Root_GB_CA.pem
#11 43.04 Adding debian:DigiCert_Assured_ID_Root_G3.pem
#11 43.05 Adding debian:emSign_Root_CA_-_G1.pem
#11 43.05 Adding debian:DigiCert_TLS_RSA4096_Root_G5.pem
#11 43.06 Adding debian:COMODO_Certification_Authority.pem
#11 43.06 Adding debian:DigiCert_Global_Root_CA.pem
#11 43.07 Adding debian:QuoVadis_Root_CA_2.pem
#11 43.08 Adding debian:TWCA_Global_Root_CA.pem
#11 43.09 Adding debian:ACCVRAIZ1.pem
#11 43.09 Adding debian:GTS_Root_R2.pem
#11 43.10 Adding debian:HARICA_TLS_RSA_Root_CA_2021.pem
#11 43.10 Adding debian:Actalis_Authentication_Root_CA.pem
#11 43.11 Adding debian:Certum_Trusted_Network_CA_2.pem
#11 43.11 Adding debian:Atos_TrustedRoot_2011.pem
#11 43.11 Adding debian:T-TeleSec_GlobalRoot_Class_3.pem
#11 43.13 Adding debian:NetLock_Arany_=Class_Gold=_Főtanúsítvány.pem
#11 43.13 Adding debian:Certigna.pem
#11 43.14 Adding debian:Microsec_e-Szigno_Root_CA_2009.pem
#11 43.14 Adding debian:UCA_Extended_Validation_Root.pem
#11 43.14 Adding debian:DigiCert_Assured_ID_Root_G2.pem
#11 43.15 Adding debian:QuoVadis_Root_CA_3.pem
#11 43.15 Adding debian:NAVER_Global_Root_Certification_Authority.pem
#11 43.16 Adding debian:GlobalSign_Root_CA_-_R6.pem
#11 43.17 Adding debian:TUBITAK_Kamu_SM_SSL_Kok_Sertifikasi_-_Surum_1.pem
#11 43.17 Adding debian:Trustwave_Global_ECC_P384_Certification_Authority.pem
#11 43.18 Adding debian:Go_Daddy_Class_2_CA.pem
#11 43.18 Adding debian:TWCA_Root_Certification_Authority.pem
#11 43.19 Adding debian:certSIGN_Root_CA_G2.pem
#11 43.19 Adding debian:COMODO_ECC_Certification_Authority.pem
#11 43.20 Adding debian:ISRG_Root_X2.pem
#11 43.21 Adding debian:D-TRUST_Root_Class_3_CA_2_EV_2009.pem
#11 43.21 Adding debian:D-TRUST_Root_Class_3_CA_2_2009.pem
#11 43.22 Adding debian:UCA_Global_G2_Root.pem
#11 43.22 Adding debian:D-TRUST_BR_Root_CA_1_2020.pem
#11 43.23 Adding debian:Secure_Global_CA.pem
#11 43.24 Adding debian:Hongkong_Post_Root_CA_3.pem
#11 43.24 Adding debian:Buypass_Class_3_Root_CA.pem
#11 43.25 Adding debian:Entrust_Root_Certification_Authority_-_G2.pem
#11 43.25 Adding debian:SSL.com_Root_Certification_Authority_ECC.pem
#11 43.26 Adding debian:Hellenic_Academic_and_Research_Institutions_RootCA_2015.pem
#11 43.27 Adding debian:E-Tugra_Global_Root_CA_RSA_v3.pem
#11 43.27 Adding debian:USERTrust_ECC_Certification_Authority.pem
#11 43.28 Adding debian:ePKI_Root_Certification_Authority.pem
#11 43.28 Adding debian:ISRG_Root_X1.pem
#11 43.29 Adding debian:HARICA_TLS_ECC_Root_CA_2021.pem
#11 43.29 Adding debian:GTS_Root_R3.pem
#11 43.30 Adding debian:Hellenic_Academic_and_Research_Institutions_ECC_RootCA_2015.pem
#11 43.30 Adding debian:Security_Communication_RootCA2.pem
#11 43.31 Adding debian:GlobalSign_Root_CA_-_R3.pem
#11 43.35 done.
#11 43.36 Processing triggers for mime-support (3.64ubuntu1) ...
#11 43.40 Processing triggers for libc-bin (2.31-0ubuntu9.15) ...
#11 43.45 Processing triggers for ca-certificates (20230311ubuntu0.20.04.1) ...
#11 43.47 Updating certificates in /etc/ssl/certs...
#11 44.82 0 added, 0 removed; done.
#11 44.82 Running hooks in /etc/ca-certificates/update.d...
#11 44.83
#11 45.16 done.
#11 45.16 done.
#11 DONE 45.9s

#12 [ 8/17] WORKDIR /opt/mlflow
#12 DONE 0.1s

#13 [ 9/17] RUN pip install mlflow==2.12.2
#13 1.009 Collecting mlflow==2.12.2
#13 1.075   Downloading mlflow-2.12.2-py3-none-any.whl.metadata (29 kB)
#13 1.207 Collecting Flask<4 (from mlflow==2.12.2)
#13 1.222   Downloading flask-3.0.3-py3-none-any.whl.metadata (3.2 kB)
#13 1.309 Collecting alembic!=1.10.0,<2 (from mlflow==2.12.2)
#13 1.324   Downloading alembic-1.13.1-py3-none-any.whl.metadata (7.4 kB)
#13 1.395 Collecting click<9,>=7.0 (from mlflow==2.12.2)
#13 1.409   Downloading click-8.1.7-py3-none-any.whl.metadata (3.0 kB)
#13 1.467 Collecting cloudpickle<4 (from mlflow==2.12.2)
#13 1.483   Downloading cloudpickle-3.0.0-py3-none-any.whl.metadata (7.0 kB)
#13 1.564 Collecting docker<8,>=4.0.0 (from mlflow==2.12.2)
#13 1.579   Downloading docker-7.0.0-py3-none-any.whl.metadata (3.5 kB)
#13 1.635 Collecting entrypoints<1 (from mlflow==2.12.2)
#13 1.651   Downloading entrypoints-0.4-py3-none-any.whl.metadata (2.6 kB)
#13 1.742 Collecting gitpython<4,>=3.1.9 (from mlflow==2.12.2)
#13 1.757   Downloading GitPython-3.1.43-py3-none-any.whl.metadata (13 kB)
#13 1.822 Collecting graphene<4 (from mlflow==2.12.2)
#13 1.837   Downloading graphene-3.3-py2.py3-none-any.whl.metadata (7.7 kB)
#13 1.946 Collecting importlib-metadata!=4.7.0,<8,>=3.7.0 (from mlflow==2.12.2)
#13 1.960   Downloading importlib_metadata-7.1.0-py3-none-any.whl.metadata (4.7 kB)
#13 2.027 Collecting markdown<4,>=3.3 (from mlflow==2.12.2)
#13 2.079   Downloading Markdown-3.6-py3-none-any.whl.metadata (7.0 kB)
#13 2.299 Collecting matplotlib<4 (from mlflow==2.12.2)
#13 2.314   Downloading matplotlib-3.7.5-cp38-cp38-manylinux_2_12_x86_64.manylinux2010_x86_64.whl.metadata (5.7 kB)
#13 2.593 Collecting numpy<2 (from mlflow==2.12.2)
#13 2.612   Downloading numpy-1.24.4-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.6 kB)
#13 2.685 Collecting packaging<25 (from mlflow==2.12.2)
#13 2.698   Downloading packaging-24.0-py3-none-any.whl.metadata (3.2 kB)
#13 2.902 Collecting pandas<3 (from mlflow==2.12.2)
#13 2.916   Downloading pandas-2.0.3-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (18 kB)
#13 3.172 Collecting protobuf<5,>=3.12.0 (from mlflow==2.12.2)
#13 3.187   Downloading protobuf-4.25.3-cp37-abi3-manylinux2014_x86_64.whl.metadata (541 bytes)
#13 3.323 Collecting pyarrow<16,>=4.0.0 (from mlflow==2.12.2)
#13 3.338   Downloading pyarrow-15.0.2-cp38-cp38-manylinux_2_28_x86_64.whl.metadata (3.0 kB)
#13 3.456 Collecting pytz<2025 (from mlflow==2.12.2)
#13 3.471   Downloading pytz-2024.1-py2.py3-none-any.whl.metadata (22 kB)
#13 3.482 Requirement already satisfied: pyyaml<7,>=5.1 in /usr/lib/python3/dist-packages (from mlflow==2.12.2) (5.3.1)
#13 3.672 Collecting querystring-parser<2 (from mlflow==2.12.2)
#13 3.689   Downloading querystring_parser-1.2.4-py2.py3-none-any.whl.metadata (559 bytes)
#13 3.780 Collecting requests<3,>=2.17.3 (from mlflow==2.12.2)
#13 3.794   Downloading requests-2.31.0-py3-none-any.whl.metadata (4.6 kB)
#13 3.969 Collecting scikit-learn<2 (from mlflow==2.12.2)
#13 3.985   Downloading scikit_learn-1.3.2-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
#13 4.159 Collecting scipy<2 (from mlflow==2.12.2)
#13 4.175   Downloading scipy-1.10.1-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (58 kB)
#13 4.199      ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 2.7 MB/s eta 0:00:00
#13 4.633 Collecting sqlalchemy<3,>=1.4.0 (from mlflow==2.12.2)
#13 4.648   Downloading SQLAlchemy-2.0.30-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (9.6 kB)
#13 4.709 Collecting sqlparse<1,>=0.4.0 (from mlflow==2.12.2)
#13 4.724   Downloading sqlparse-0.5.0-py3-none-any.whl.metadata (3.9 kB)
#13 4.795 Collecting Jinja2<4,>=2.11 (from mlflow==2.12.2)
#13 4.809   Downloading jinja2-3.1.4-py3-none-any.whl.metadata (2.6 kB)
#13 4.929 Collecting gunicorn<23 (from mlflow==2.12.2)
#13 4.944   Downloading gunicorn-22.0.0-py3-none-any.whl.metadata (4.4 kB)
#13 5.043 Collecting Mako (from alembic!=1.10.0,<2->mlflow==2.12.2)
#13 5.058   Downloading Mako-1.3.5-py3-none-any.whl.metadata (2.9 kB)
#13 5.122 Collecting typing-extensions>=4 (from alembic!=1.10.0,<2->mlflow==2.12.2)
#13 5.136   Downloading typing_extensions-4.11.0-py3-none-any.whl.metadata (3.0 kB)
#13 5.225 Collecting importlib-resources (from alembic!=1.10.0,<2->mlflow==2.12.2)
#13 5.241   Downloading importlib_resources-6.4.0-py3-none-any.whl.metadata (3.9 kB)
#13 5.348 Collecting urllib3>=1.26.0 (from docker<8,>=4.0.0->mlflow==2.12.2)
#13 5.362   Downloading urllib3-2.2.1-py3-none-any.whl.metadata (6.4 kB)
#13 5.445 Collecting Werkzeug>=3.0.0 (from Flask<4->mlflow==2.12.2)
#13 5.460   Downloading werkzeug-3.0.3-py3-none-any.whl.metadata (3.7 kB)
#13 5.519 Collecting itsdangerous>=2.1.2 (from Flask<4->mlflow==2.12.2)
#13 5.537   Downloading itsdangerous-2.2.0-py3-none-any.whl.metadata (1.9 kB)
#13 5.606 Collecting blinker>=1.6.2 (from Flask<4->mlflow==2.12.2)
#13 5.623   Downloading blinker-1.8.2-py3-none-any.whl.metadata (1.6 kB)
#13 5.716 Collecting gitdb<5,>=4.0.1 (from gitpython<4,>=3.1.9->mlflow==2.12.2)
#13 5.732   Downloading gitdb-4.0.11-py3-none-any.whl.metadata (1.2 kB)
#13 5.847 Collecting graphql-core<3.3,>=3.1 (from graphene<4->mlflow==2.12.2)
#13 5.862   Downloading graphql_core-3.2.3-py3-none-any.whl.metadata (10 kB)
#13 5.920 Collecting graphql-relay<3.3,>=3.1 (from graphene<4->mlflow==2.12.2)
#13 5.938   Downloading graphql_relay-3.2.0-py3-none-any.whl.metadata (12 kB)
#13 6.001 Collecting aniso8601<10,>=8 (from graphene<4->mlflow==2.12.2)
#13 6.017   Downloading aniso8601-9.0.1-py2.py3-none-any.whl.metadata (23 kB)
#13 6.122 Collecting zipp>=0.5 (from importlib-metadata!=4.7.0,<8,>=3.7.0->mlflow==2.12.2)
#13 6.136   Downloading zipp-3.18.2-py3-none-any.whl.metadata (3.5 kB)
#13 6.235 Collecting MarkupSafe>=2.0 (from Jinja2<4,>=2.11->mlflow==2.12.2)
#13 6.250   Downloading MarkupSafe-2.1.5-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.0 kB)
#13 6.406 Collecting contourpy>=1.0.1 (from matplotlib<4->mlflow==2.12.2)
#13 6.420   Downloading contourpy-1.1.1-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.9 kB)
#13 6.468 Collecting cycler>=0.10 (from matplotlib<4->mlflow==2.12.2)
#13 6.485   Downloading cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
#13 6.657 Collecting fonttools>=4.22.0 (from matplotlib<4->mlflow==2.12.2)
#13 6.672   Downloading fonttools-4.51.0-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (159 kB)
#13 6.694      ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.5/159.5 kB 7.7 MB/s eta 0:00:00
#13 6.787 Collecting kiwisolver>=1.0.1 (from matplotlib<4->mlflow==2.12.2)
#13 6.826   Downloading kiwisolver-1.4.5-cp38-cp38-manylinux_2_5_x86_64.manylinux1_x86_64.whl.metadata (6.4 kB)
#13 7.174 Collecting pillow>=6.2.0 (from matplotlib<4->mlflow==2.12.2)
#13 7.191   Downloading pillow-10.3.0-cp38-cp38-manylinux_2_28_x86_64.whl.metadata (9.2 kB)
#13 7.275 Collecting pyparsing>=2.3.1 (from matplotlib<4->mlflow==2.12.2)
#13 7.290   Downloading pyparsing-3.1.2-py3-none-any.whl.metadata (5.1 kB)
#13 7.351 Collecting python-dateutil>=2.7 (from matplotlib<4->mlflow==2.12.2)
#13 7.365   Downloading python_dateutil-2.9.0.post0-py2.py3-none-any.whl.metadata (8.4 kB)
#13 7.525 Collecting tzdata>=2022.1 (from pandas<3->mlflow==2.12.2)
#13 7.538   Downloading tzdata-2024.1-py2.py3-none-any.whl.metadata (1.4 kB)
#13 7.620 Collecting six (from querystring-parser<2->mlflow==2.12.2)
#13 7.633   Downloading six-1.16.0-py2.py3-none-any.whl.metadata (1.8 kB)
#13 7.758 Collecting charset-normalizer<4,>=2 (from requests<3,>=2.17.3->mlflow==2.12.2)
#13 7.772   Downloading charset_normalizer-3.3.2-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (33 kB)
#13 7.834 Collecting idna<4,>=2.5 (from requests<3,>=2.17.3->mlflow==2.12.2)
#13 7.848   Downloading idna-3.7-py3-none-any.whl.metadata (9.9 kB)
#13 7.919 Collecting certifi>=2017.4.17 (from requests<3,>=2.17.3->mlflow==2.12.2)
#13 7.933   Downloading certifi-2024.2.2-py3-none-any.whl.metadata (2.2 kB)
#13 8.043 Collecting joblib>=1.1.1 (from scikit-learn<2->mlflow==2.12.2)
#13 8.057   Downloading joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
#13 8.112 Collecting threadpoolctl>=2.0.0 (from scikit-learn<2->mlflow==2.12.2)
#13 8.130   Downloading threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
#13 8.419 Collecting greenlet!=0.4.17 (from sqlalchemy<3,>=1.4.0->mlflow==2.12.2)
#13 8.434   Downloading greenlet-3.0.3-cp38-cp38-manylinux_2_24_x86_64.manylinux_2_28_x86_64.whl.metadata (3.8 kB)
#13 8.606 Collecting smmap<6,>=3.0.1 (from gitdb<5,>=4.0.1->gitpython<4,>=3.1.9->mlflow==2.12.2)
#13 8.621   Downloading smmap-5.0.1-py3-none-any.whl.metadata (4.3 kB)
#13 8.891 Downloading mlflow-2.12.2-py3-none-any.whl (20.2 MB)
#13 9.854    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.2/20.2 MB 19.4 MB/s eta 0:00:00
#13 9.872 Downloading alembic-1.13.1-py3-none-any.whl (233 kB)
#13 9.889    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 17.0 MB/s eta 0:00:00
#13 9.904 Downloading click-8.1.7-py3-none-any.whl (97 kB)
#13 9.918    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 8.1 MB/s eta 0:00:00
#13 9.936 Downloading cloudpickle-3.0.0-py3-none-any.whl (20 kB)
#13 9.964 Downloading docker-7.0.0-py3-none-any.whl (147 kB)
#13 9.979    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.6/147.6 kB 11.3 MB/s eta 0:00:00
#13 9.996 Downloading entrypoints-0.4-py3-none-any.whl (5.3 kB)
#13 10.02 Downloading flask-3.0.3-py3-none-any.whl (101 kB)
#13 10.04    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 8.1 MB/s eta 0:00:00
#13 10.05 Downloading GitPython-3.1.43-py3-none-any.whl (207 kB)
#13 10.07    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 14.9 MB/s eta 0:00:00
#13 10.09 Downloading graphene-3.3-py2.py3-none-any.whl (128 kB)
#13 10.11    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 kB 9.6 MB/s eta 0:00:00
#13 10.13 Downloading gunicorn-22.0.0-py3-none-any.whl (84 kB)
#13 10.14    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.4/84.4 kB 6.8 MB/s eta 0:00:00
#13 10.16 Downloading importlib_metadata-7.1.0-py3-none-any.whl (24 kB)
#13 10.19 Downloading jinja2-3.1.4-py3-none-any.whl (133 kB)
#13 10.20    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.3/133.3 kB 10.5 MB/s eta 0:00:00
#13 10.22 Downloading Markdown-3.6-py3-none-any.whl (105 kB)
#13 10.23    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 8.5 MB/s eta 0:00:00
#13 10.24 Downloading matplotlib-3.7.5-cp38-cp38-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (9.2 MB)
#13 10.69    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 20.8 MB/s eta 0:00:00
#13 10.71 Downloading numpy-1.24.4-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 MB)
#13 11.54    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 19.7 MB/s eta 0:00:00
#13 11.56 Downloading packaging-24.0-py3-none-any.whl (53 kB)
#13 11.58    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.5/53.5 kB 4.1 MB/s eta 0:00:00
#13 11.59 Downloading pandas-2.0.3-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.4 MB)
#13 12.20    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 20.1 MB/s eta 0:00:00
#13 12.22 Downloading protobuf-4.25.3-cp37-abi3-manylinux2014_x86_64.whl (294 kB)
#13 12.25    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 12.9 MB/s eta 0:00:00
#13 12.27 Downloading pyarrow-15.0.2-cp38-cp38-manylinux_2_28_x86_64.whl (38.4 MB)
#13 14.11    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.4/38.4 MB 17.0 MB/s eta 0:00:00
#13 14.13 Downloading pytz-2024.1-py2.py3-none-any.whl (505 kB)
#13 14.17    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 505.5/505.5 kB 16.9 MB/s eta 0:00:00
#13 14.18 Downloading querystring_parser-1.2.4-py2.py3-none-any.whl (7.9 kB)
#13 14.21 Downloading requests-2.31.0-py3-none-any.whl (62 kB)
#13 14.22    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 5.0 MB/s eta 0:00:00
#13 14.24 Downloading scikit_learn-1.3.2-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (11.1 MB)
#13 14.78    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 20.6 MB/s eta 0:00:00
#13 14.80 Downloading scipy-1.10.1-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (34.5 MB)
#13 16.43    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 17.6 MB/s eta 0:00:00
#13 16.45 Downloading SQLAlchemy-2.0.30-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.1 MB)
#13 16.62    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 19.5 MB/s eta 0:00:00
#13 16.63 Downloading sqlparse-0.5.0-py3-none-any.whl (43 kB)
#13 16.65    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.3 MB/s eta 0:00:00
#13 16.66 Downloading aniso8601-9.0.1-py2.py3-none-any.whl (52 kB)
#13 16.68    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 4.0 MB/s eta 0:00:00
#13 16.70 Downloading blinker-1.8.2-py3-none-any.whl (9.5 kB)
#13 16.72 Downloading certifi-2024.2.2-py3-none-any.whl (163 kB)
#13 16.74    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 12.6 MB/s eta 0:00:00
#13 16.75 Downloading charset_normalizer-3.3.2-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (141 kB)
#13 16.77    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 10.9 MB/s eta 0:00:00
#13 16.79 Downloading contourpy-1.1.1-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (301 kB)
#13 16.81    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.1/301.1 kB 13.1 MB/s eta 0:00:00
#13 16.83 Downloading cycler-0.12.1-py3-none-any.whl (8.3 kB)
#13 16.85 Downloading fonttools-4.51.0-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (4.7 MB)
#13 17.09    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 20.2 MB/s eta 0:00:00
#13 17.11 Downloading gitdb-4.0.11-py3-none-any.whl (62 kB)
#13 17.12    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.9 MB/s eta 0:00:00
#13 17.14 Downloading graphql_core-3.2.3-py3-none-any.whl (202 kB)
#13 17.15    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.9/202.9 kB 15.1 MB/s eta 0:00:00
#13 17.17 Downloading graphql_relay-3.2.0-py3-none-any.whl (16 kB)
#13 17.20 Downloading greenlet-3.0.3-cp38-cp38-manylinux_2_24_x86_64.manylinux_2_28_x86_64.whl (622 kB)
#13 17.24    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 622.3/622.3 kB 17.2 MB/s eta 0:00:00
#13 17.25 Downloading idna-3.7-py3-none-any.whl (66 kB)
#13 17.26    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 5.3 MB/s eta 0:00:00
#13 17.28 Downloading importlib_resources-6.4.0-py3-none-any.whl (38 kB)
#13 17.31 Downloading itsdangerous-2.2.0-py3-none-any.whl (16 kB)
#13 17.33 Downloading joblib-1.4.2-py3-none-any.whl (301 kB)
#13 17.36    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 14.3 MB/s eta 0:00:00
#13 17.37 Downloading kiwisolver-1.4.5-cp38-cp38-manylinux_2_5_x86_64.manylinux1_x86_64.whl (1.2 MB)
#13 17.45    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 18.4 MB/s eta 0:00:00
#13 17.47 Downloading MarkupSafe-2.1.5-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (26 kB)
#13 17.49 Downloading pillow-10.3.0-cp38-cp38-manylinux_2_28_x86_64.whl (4.5 MB)
#13 17.72    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 20.2 MB/s eta 0:00:00
#13 17.74 Downloading pyparsing-3.1.2-py3-none-any.whl (103 kB)
#13 17.75    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 kB 8.1 MB/s eta 0:00:00
#13 17.77 Downloading python_dateutil-2.9.0.post0-py2.py3-none-any.whl (229 kB)
#13 17.79    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 kB 17.7 MB/s eta 0:00:00
#13 17.80 Downloading six-1.16.0-py2.py3-none-any.whl (11 kB)
#13 17.83 Downloading threadpoolctl-3.5.0-py3-none-any.whl (18 kB)
#13 17.85 Downloading typing_extensions-4.11.0-py3-none-any.whl (34 kB)
#13 17.88 Downloading tzdata-2024.1-py2.py3-none-any.whl (345 kB)
#13 17.91    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 14.9 MB/s eta 0:00:00
#13 17.92 Downloading urllib3-2.2.1-py3-none-any.whl (121 kB)
#13 17.94    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 9.4 MB/s eta 0:00:00
#13 17.95 Downloading werkzeug-3.0.3-py3-none-any.whl (227 kB)
#13 17.97    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.3/227.3 kB 16.6 MB/s eta 0:00:00
#13 17.98 Downloading zipp-3.18.2-py3-none-any.whl (8.3 kB)
#13 18.01 Downloading Mako-1.3.5-py3-none-any.whl (78 kB)
#13 18.03    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.3 MB/s eta 0:00:00
#13 18.04 Downloading smmap-5.0.1-py3-none-any.whl (24 kB)
#13 19.13 Installing collected packages: pytz, aniso8601, zipp, urllib3, tzdata, typing-extensions, threadpoolctl, sqlparse, smmap, six, pyparsing, protobuf, pillow, packaging, numpy, MarkupSafe, kiwisolver, joblib, itsdangerous, idna, greenlet, graphql-core, fonttools, entrypoints, cycler, cloudpickle, click, charset-normalizer, certifi, blinker, Werkzeug, sqlalchemy, scipy, requests, querystring-parser, python-dateutil, pyarrow, Mako, Jinja2, importlib-resources, importlib-metadata, gunicorn, graphql-relay, gitdb, contourpy, scikit-learn, pandas, matplotlib, markdown, graphene, gitpython, Flask, docker, alembic, mlflow
#13 34.72 Successfully installed Flask-3.0.3 Jinja2-3.1.4 Mako-1.3.5 MarkupSafe-2.1.5 Werkzeug-3.0.3 alembic-1.13.1 aniso8601-9.0.1 blinker-1.8.2 certifi-2024.2.2 charset-normalizer-3.3.2 click-8.1.7 cloudpickle-3.0.0 contourpy-1.1.1 cycler-0.12.1 docker-7.0.0 entrypoints-0.4 fonttools-4.51.0 gitdb-4.0.11 gitpython-3.1.43 graphene-3.3 graphql-core-3.2.3 graphql-relay-3.2.0 greenlet-3.0.3 gunicorn-22.0.0 idna-3.7 importlib-metadata-7.1.0 importlib-resources-6.4.0 itsdangerous-2.2.0 joblib-1.4.2 kiwisolver-1.4.5 markdown-3.6 matplotlib-3.7.5 mlflow-2.12.2 numpy-1.24.4 packaging-24.0 pandas-2.0.3 pillow-10.3.0 protobuf-4.25.3 pyarrow-15.0.2 pyparsing-3.1.2 python-dateutil-2.9.0.post0 pytz-2024.1 querystring-parser-1.2.4 requests-2.31.0 scikit-learn-1.3.2 scipy-1.10.1 six-1.16.0 smmap-5.0.1 sqlalchemy-2.0.30 sqlparse-0.5.0 threadpoolctl-3.5.0 typing-extensions-4.11.0 tzdata-2024.1 urllib3-2.2.1 zipp-3.18.2
#13 34.72 WARNING: Running pip as the 'root' user can result in broken permissions and conflicting behaviour with the system package manager. It is recommended to use a virtual environment instead: https://pip.pypa.io/warnings/venv
#13 DONE 36.4s

#14 [10/17] RUN mvn --batch-mode dependency:copy -Dartifact=org.mlflow:mlflow-scoring:2.12.2:pom -DoutputDirectory=/opt/java
#14 1.558 [INFO] Scanning for projects...
#14 1.845 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/plugins/maven-clean-plugin/2.5/maven-clean-plugin-2.5.pom
#14 2.423 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/plugins/maven-clean-plugin/2.5/maven-clean-plugin-2.5.pom (3.9 kB at 6.7 kB/s)
#14 2.448 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/plugins/maven-plugins/22/maven-plugins-22.pom
#14 2.499 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/plugins/maven-plugins/22/maven-plugins-22.pom (13 kB at 256 kB/s)
#14 2.511 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-parent/21/maven-parent-21.pom
#14 2.566 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-parent/21/maven-parent-21.pom (26 kB at 479 kB/s)
#14 2.583 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/apache/10/apache-10.pom
#14 2.624 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/apache/10/apache-10.pom (15 kB at 370 kB/s)
#14 2.647 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/plugins/maven-clean-plugin/2.5/maven-clean-plugin-2.5.jar
#14 2.693 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/plugins/maven-clean-plugin/2.5/maven-clean-plugin-2.5.jar (25 kB at 547 kB/s)
#14 2.711 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/plugins/maven-install-plugin/2.4/maven-install-plugin-2.4.pom
#14 2.745 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/plugins/maven-install-plugin/2.4/maven-install-plugin-2.4.pom (6.4 kB at 188 kB/s)
#14 2.753 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/plugins/maven-plugins/23/maven-plugins-23.pom
#14 2.792 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/plugins/maven-plugins/23/maven-plugins-23.pom (9.2 kB at 230 kB/s)
#14 2.800 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-parent/22/maven-parent-22.pom
#14 2.841 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-parent/22/maven-parent-22.pom (30 kB at 725 kB/s)
#14 2.852 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/apache/11/apache-11.pom
#14 2.888 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/apache/11/apache-11.pom (15 kB at 411 kB/s)
#14 2.902 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/plugins/maven-install-plugin/2.4/maven-install-plugin-2.4.jar
#14 2.948 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/plugins/maven-install-plugin/2.4/maven-install-plugin-2.4.jar (27 kB at 599 kB/s)
#14 2.962 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/plugins/maven-deploy-plugin/2.7/maven-deploy-plugin-2.7.pom
#14 2.999 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/plugins/maven-deploy-plugin/2.7/maven-deploy-plugin-2.7.pom (5.6 kB at 152 kB/s)
#14 3.015 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/plugins/maven-deploy-plugin/2.7/maven-deploy-plugin-2.7.jar
#14 3.052 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/plugins/maven-deploy-plugin/2.7/maven-deploy-plugin-2.7.jar (27 kB at 706 kB/s)
#14 3.113 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/plugins/maven-site-plugin/3.3/maven-site-plugin-3.3.pom
#14 3.150 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/plugins/maven-site-plugin/3.3/maven-site-plugin-3.3.pom (21 kB at 578 kB/s)
#14 3.174 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/plugins/maven-plugins/24/maven-plugins-24.pom
#14 3.212 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/plugins/maven-plugins/24/maven-plugins-24.pom (11 kB at 270 kB/s)
#14 3.222 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-parent/23/maven-parent-23.pom
#14 3.257 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-parent/23/maven-parent-23.pom (33 kB at 905 kB/s)
#14 3.268 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/apache/13/apache-13.pom
#14 3.305 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/apache/13/apache-13.pom (14 kB at 388 kB/s)
#14 3.323 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/plugins/maven-site-plugin/3.3/maven-site-plugin-3.3.jar
#14 3.373 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/plugins/maven-site-plugin/3.3/maven-site-plugin-3.3.jar (124 kB at 2.5 MB/s)
#14 3.397 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/plugins/maven-antrun-plugin/1.3/maven-antrun-plugin-1.3.pom
#14 3.430 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/plugins/maven-antrun-plugin/1.3/maven-antrun-plugin-1.3.pom (4.7 kB at 143 kB/s)
#14 3.437 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/plugins/maven-plugins/12/maven-plugins-12.pom
#14 3.472 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/plugins/maven-plugins/12/maven-plugins-12.pom (12 kB at 336 kB/s)
#14 3.479 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-parent/9/maven-parent-9.pom
#14 3.514 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-parent/9/maven-parent-9.pom (33 kB at 938 kB/s)
#14 3.568 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/apache/4/apache-4.pom
#14 3.625 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/apache/4/apache-4.pom (4.5 kB at 78 kB/s)
#14 3.636 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/plugins/maven-antrun-plugin/1.3/maven-antrun-plugin-1.3.jar
#14 3.670 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/plugins/maven-antrun-plugin/1.3/maven-antrun-plugin-1.3.jar (24 kB at 674 kB/s)
#14 3.678 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/plugins/maven-assembly-plugin/2.2-beta-5/maven-assembly-plugin-2.2-beta-5.pom
#14 3.713 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/plugins/maven-assembly-plugin/2.2-beta-5/maven-assembly-plugin-2.2-beta-5.pom (15 kB at 421 kB/s)
#14 3.721 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/plugins/maven-plugins/16/maven-plugins-16.pom
#14 3.756 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/plugins/maven-plugins/16/maven-plugins-16.pom (13 kB at 370 kB/s)
#14 3.763 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-parent/15/maven-parent-15.pom
#14 3.798 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-parent/15/maven-parent-15.pom (24 kB at 667 kB/s)
#14 3.806 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/apache/6/apache-6.pom
#14 3.841 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/apache/6/apache-6.pom (13 kB at 376 kB/s)
#14 3.853 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/plugins/maven-assembly-plugin/2.2-beta-5/maven-assembly-plugin-2.2-beta-5.jar
#14 3.904 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/plugins/maven-assembly-plugin/2.2-beta-5/maven-assembly-plugin-2.2-beta-5.jar (209 kB at 4.2 MB/s)
#14 3.925 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/plugins/maven-dependency-plugin/2.8/maven-dependency-plugin-2.8.pom
#14 3.959 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/plugins/maven-dependency-plugin/2.8/maven-dependency-plugin-2.8.pom (11 kB at 337 kB/s)
#14 3.975 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/plugins/maven-dependency-plugin/2.8/maven-dependency-plugin-2.8.jar
#14 4.016 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/plugins/maven-dependency-plugin/2.8/maven-dependency-plugin-2.8.jar (153 kB at 3.7 MB/s)
#14 4.053 [INFO]
#14 4.053 [INFO] ------------------< org.apache.maven:standalone-pom >-------------------
#14 4.053 [INFO] Building Maven Stub Project (No POM) 1
#14 4.053 [INFO] --------------------------------[ pom ]---------------------------------
#14 4.058 [INFO]
#14 4.058 [INFO] --- maven-dependency-plugin:2.8:copy (default-cli) @ standalone-pom ---
#14 4.071 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-artifact/2.0.9/maven-artifact-2.0.9.pom
#14 4.101 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-artifact/2.0.9/maven-artifact-2.0.9.pom (1.6 kB at 54 kB/s)
#14 4.104 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven/2.0.9/maven-2.0.9.pom
#14 4.140 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven/2.0.9/maven-2.0.9.pom (19 kB at 540 kB/s)
#14 4.147 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-parent/8/maven-parent-8.pom
#14 4.190 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-parent/8/maven-parent-8.pom (24 kB at 561 kB/s)
#14 4.210 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus-utils/1.5.1/plexus-utils-1.5.1.pom
#14 4.321 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus-utils/1.5.1/plexus-utils-1.5.1.pom (2.3 kB at 21 kB/s)
#14 4.327 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus/1.0.11/plexus-1.0.11.pom
#14 4.360 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus/1.0.11/plexus-1.0.11.pom (9.0 kB at 264 kB/s)
#14 4.369 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-plugin-api/2.0.9/maven-plugin-api-2.0.9.pom
#14 4.402 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-plugin-api/2.0.9/maven-plugin-api-2.0.9.pom (1.5 kB at 44 kB/s)
#14 4.412 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-project/2.0.9/maven-project-2.0.9.pom
#14 4.446 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-project/2.0.9/maven-project-2.0.9.pom (2.7 kB at 82 kB/s)
#14 4.457 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-settings/2.0.9/maven-settings-2.0.9.pom
#14 4.490 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-settings/2.0.9/maven-settings-2.0.9.pom (2.1 kB at 62 kB/s)
#14 4.502 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-model/2.0.9/maven-model-2.0.9.pom
#14 4.534 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-model/2.0.9/maven-model-2.0.9.pom (3.1 kB at 95 kB/s)
#14 4.547 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus-container-default/1.0-alpha-9-stable-1/plexus-container-default-1.0-alpha-9-stable-1.pom
#14 4.581 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus-container-default/1.0-alpha-9-stable-1/plexus-container-default-1.0-alpha-9-stable-1.pom (3.9 kB at 120 kB/s)
#14 4.587 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus-containers/1.0.3/plexus-containers-1.0.3.pom
#14 4.620 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus-containers/1.0.3/plexus-containers-1.0.3.pom (492 B at 15 kB/s)
#14 4.625 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus/1.0.4/plexus-1.0.4.pom
#14 4.658 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus/1.0.4/plexus-1.0.4.pom (5.7 kB at 174 kB/s)
#14 4.669 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/junit/junit/3.8.1/junit-3.8.1.pom
#14 4.705 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/junit/junit/3.8.1/junit-3.8.1.pom (998 B at 28 kB/s)
#14 4.712 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus-utils/1.0.4/plexus-utils-1.0.4.pom
#14 4.746 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus-utils/1.0.4/plexus-utils-1.0.4.pom (6.9 kB at 196 kB/s)
#14 4.753 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/classworlds/classworlds/1.1-alpha-2/classworlds-1.1-alpha-2.pom
#14 4.788 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/classworlds/classworlds/1.1-alpha-2/classworlds-1.1-alpha-2.pom (3.1 kB at 92 kB/s)
#14 4.794 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-profile/2.0.9/maven-profile-2.0.9.pom
#14 4.828 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-profile/2.0.9/maven-profile-2.0.9.pom (2.0 kB at 62 kB/s)
#14 4.838 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-artifact-manager/2.0.9/maven-artifact-manager-2.0.9.pom
#14 4.874 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-artifact-manager/2.0.9/maven-artifact-manager-2.0.9.pom (2.7 kB at 75 kB/s)
#14 4.883 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-repository-metadata/2.0.9/maven-repository-metadata-2.0.9.pom
#14 4.915 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-repository-metadata/2.0.9/maven-repository-metadata-2.0.9.pom (1.9 kB at 59 kB/s)
#14 4.926 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-plugin-registry/2.0.9/maven-plugin-registry-2.0.9.pom
#14 4.961 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-plugin-registry/2.0.9/maven-plugin-registry-2.0.9.pom (2.0 kB at 58 kB/s)
#14 4.973 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-core/2.0.9/maven-core-2.0.9.pom
#14 5.007 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-core/2.0.9/maven-core-2.0.9.pom (7.8 kB at 222 kB/s)
#14 5.018 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-plugin-parameter-documenter/2.0.9/maven-plugin-parameter-documenter-2.0.9.pom
#14 5.051 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-plugin-parameter-documenter/2.0.9/maven-plugin-parameter-documenter-2.0.9.pom (2.0 kB at 60 kB/s)
#14 5.060 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/reporting/maven-reporting-api/2.0.9/maven-reporting-api-2.0.9.pom
#14 5.093 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/reporting/maven-reporting-api/2.0.9/maven-reporting-api-2.0.9.pom (1.8 kB at 56 kB/s)
#14 5.099 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/reporting/maven-reporting/2.0.9/maven-reporting-2.0.9.pom
#14 5.131 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/reporting/maven-reporting/2.0.9/maven-reporting-2.0.9.pom (1.5 kB at 48 kB/s)
#14 5.141 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/doxia/doxia-sink-api/1.0-alpha-10/doxia-sink-api-1.0-alpha-10.pom
#14 5.174 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/doxia/doxia-sink-api/1.0-alpha-10/doxia-sink-api-1.0-alpha-10.pom (1.3 kB at 41 kB/s)
#14 5.179 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/doxia/doxia/1.0-alpha-10/doxia-1.0-alpha-10.pom
#14 5.213 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/doxia/doxia/1.0-alpha-10/doxia-1.0-alpha-10.pom (9.2 kB at 278 kB/s)
#14 5.219 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-parent/6/maven-parent-6.pom
#14 5.254 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-parent/6/maven-parent-6.pom (20 kB at 573 kB/s)
#14 5.264 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-error-diagnostics/2.0.9/maven-error-diagnostics-2.0.9.pom
#14 5.297 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-error-diagnostics/2.0.9/maven-error-diagnostics-2.0.9.pom (1.7 kB at 53 kB/s)
#14 5.307 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/commons-cli/commons-cli/1.0/commons-cli-1.0.pom
#14 5.341 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/commons-cli/commons-cli/1.0/commons-cli-1.0.pom (2.1 kB at 62 kB/s)
#14 5.351 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-plugin-descriptor/2.0.9/maven-plugin-descriptor-2.0.9.pom
#14 5.384 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-plugin-descriptor/2.0.9/maven-plugin-descriptor-2.0.9.pom (2.1 kB at 61 kB/s)
#14 5.394 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus-interactivity-api/1.0-alpha-4/plexus-interactivity-api-1.0-alpha-4.pom
#14 5.427 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus-interactivity-api/1.0-alpha-4/plexus-interactivity-api-1.0-alpha-4.pom (7.1 kB at 222 kB/s)
#14 5.436 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-monitor/2.0.9/maven-monitor-2.0.9.pom
#14 5.477 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-monitor/2.0.9/maven-monitor-2.0.9.pom (1.3 kB at 31 kB/s)
#14 5.486 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/classworlds/classworlds/1.1/classworlds-1.1.pom
#14 5.518 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/classworlds/classworlds/1.1/classworlds-1.1.pom (3.3 kB at 104 kB/s)
#14 5.525 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/reporting/maven-reporting-api/3.0/maven-reporting-api-3.0.pom
#14 5.558 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/reporting/maven-reporting-api/3.0/maven-reporting-api-3.0.pom (2.4 kB at 74 kB/s)
#14 5.563 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/shared/maven-shared-components/15/maven-shared-components-15.pom
#14 5.596 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/shared/maven-shared-components/15/maven-shared-components-15.pom (9.3 kB at 283 kB/s)
#14 5.603 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-parent/16/maven-parent-16.pom
#14 5.643 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-parent/16/maven-parent-16.pom (23 kB at 582 kB/s)
#14 5.651 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/apache/7/apache-7.pom
#14 5.684 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/apache/7/apache-7.pom (14 kB at 437 kB/s)
#14 5.692 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/doxia/doxia-sink-api/1.0/doxia-sink-api-1.0.pom
#14 5.724 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/doxia/doxia-sink-api/1.0/doxia-sink-api-1.0.pom (1.4 kB at 42 kB/s)
#14 5.729 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/doxia/doxia/1.0/doxia-1.0.pom
#14 5.762 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/doxia/doxia/1.0/doxia-1.0.pom (9.6 kB at 301 kB/s)
#14 5.768 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-parent/10/maven-parent-10.pom
#14 5.802 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-parent/10/maven-parent-10.pom (32 kB at 904 kB/s)
#14 5.815 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/reporting/maven-reporting-impl/2.0.5/maven-reporting-impl-2.0.5.pom
#14 5.847 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/reporting/maven-reporting-impl/2.0.5/maven-reporting-impl-2.0.5.pom (4.2 kB at 132 kB/s)
#14 5.857 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-project/2.0.6/maven-project-2.0.6.pom
#14 5.890 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-project/2.0.6/maven-project-2.0.6.pom (2.6 kB at 80 kB/s)
#14 5.895 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven/2.0.6/maven-2.0.6.pom
#14 5.928 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven/2.0.6/maven-2.0.6.pom (9.0 kB at 283 kB/s)
#14 5.933 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-parent/5/maven-parent-5.pom
#14 5.968 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-parent/5/maven-parent-5.pom (15 kB at 448 kB/s)
#14 5.974 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/apache/3/apache-3.pom
#14 6.006 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/apache/3/apache-3.pom (3.4 kB at 104 kB/s)
#14 6.014 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-settings/2.0.6/maven-settings-2.0.6.pom
#14 6.046 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-settings/2.0.6/maven-settings-2.0.6.pom (2.0 kB at 63 kB/s)
#14 6.055 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-model/2.0.6/maven-model-2.0.6.pom
#14 6.085 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-model/2.0.6/maven-model-2.0.6.pom (3.0 kB at 98 kB/s)
#14 6.094 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus-utils/1.4.1/plexus-utils-1.4.1.pom
#14 6.125 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus-utils/1.4.1/plexus-utils-1.4.1.pom (1.9 kB at 62 kB/s)
#14 6.132 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-profile/2.0.6/maven-profile-2.0.6.pom
#14 6.163 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-profile/2.0.6/maven-profile-2.0.6.pom (2.0 kB at 66 kB/s)
#14 6.172 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-artifact-manager/2.0.6/maven-artifact-manager-2.0.6.pom
#14 6.202 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-artifact-manager/2.0.6/maven-artifact-manager-2.0.6.pom (2.6 kB at 87 kB/s)
#14 6.210 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-repository-metadata/2.0.6/maven-repository-metadata-2.0.6.pom
#14 6.240 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-repository-metadata/2.0.6/maven-repository-metadata-2.0.6.pom (1.9 kB at 64 kB/s)
#14 6.248 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-artifact/2.0.6/maven-artifact-2.0.6.pom
#14 6.282 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-artifact/2.0.6/maven-artifact-2.0.6.pom (1.6 kB at 48 kB/s)
#14 6.290 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-plugin-registry/2.0.6/maven-plugin-registry-2.0.6.pom
#14 6.321 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-plugin-registry/2.0.6/maven-plugin-registry-2.0.6.pom (1.9 kB at 61 kB/s)
#14 6.331 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-plugin-api/2.0.6/maven-plugin-api-2.0.6.pom
#14 6.361 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-plugin-api/2.0.6/maven-plugin-api-2.0.6.pom (1.5 kB at 47 kB/s)
#14 6.369 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/doxia/doxia-core/1.0/doxia-core-1.0.pom
#14 6.402 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/doxia/doxia-core/1.0/doxia-core-1.0.pom (2.4 kB at 75 kB/s)
#14 6.411 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus-utils/1.5.7/plexus-utils-1.5.7.pom
#14 6.444 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus-utils/1.5.7/plexus-utils-1.5.7.pom (8.1 kB at 237 kB/s)
#14 6.451 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus/2.0.2/plexus-2.0.2.pom
#14 6.484 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus/2.0.2/plexus-2.0.2.pom (12 kB at 363 kB/s)
#14 6.493 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus-container-default/1.0-alpha-30/plexus-container-default-1.0-alpha-30.pom
#14 6.526 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus-container-default/1.0-alpha-30/plexus-container-default-1.0-alpha-30.pom (3.5 kB at 105 kB/s)
#14 6.531 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus-containers/1.0-alpha-30/plexus-containers-1.0-alpha-30.pom
#14 6.563 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus-containers/1.0-alpha-30/plexus-containers-1.0-alpha-30.pom (1.9 kB at 61 kB/s)
#14 6.569 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus-utils/1.4.5/plexus-utils-1.4.5.pom
#14 6.603 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus-utils/1.4.5/plexus-utils-1.4.5.pom (2.3 kB at 67 kB/s)
#14 6.610 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus-classworlds/1.2-alpha-9/plexus-classworlds-1.2-alpha-9.pom
#14 6.642 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus-classworlds/1.2-alpha-9/plexus-classworlds-1.2-alpha-9.pom (3.2 kB at 101 kB/s)
#14 6.647 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus/1.0.10/plexus-1.0.10.pom
#14 6.677 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus/1.0.10/plexus-1.0.10.pom (8.2 kB at 266 kB/s)
#14 6.684 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/doxia/doxia-site-renderer/1.0/doxia-site-renderer-1.0.pom
#14 6.716 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/doxia/doxia-site-renderer/1.0/doxia-site-renderer-1.0.pom (4.4 kB at 138 kB/s)
#14 6.721 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/doxia/doxia-sitetools/1.0/doxia-sitetools-1.0.pom
#14 6.752 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/doxia/doxia-sitetools/1.0/doxia-sitetools-1.0.pom (9.6 kB at 300 kB/s)
#14 6.758 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-parent/11/maven-parent-11.pom
#14 6.789 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-parent/11/maven-parent-11.pom (32 kB at 1.0 MB/s)
#14 6.796 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/apache/5/apache-5.pom
#14 6.828 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/apache/5/apache-5.pom (4.1 kB at 132 kB/s)
#14 6.835 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus-i18n/1.0-beta-7/plexus-i18n-1.0-beta-7.pom
#14 6.868 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus-i18n/1.0-beta-7/plexus-i18n-1.0-beta-7.pom (1.1 kB at 33 kB/s)
#14 6.873 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus-components/1.1.12/plexus-components-1.1.12.pom
#14 6.906 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus-components/1.1.12/plexus-components-1.1.12.pom (3.0 kB at 91 kB/s)
#14 6.913 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus-velocity/1.1.7/plexus-velocity-1.1.7.pom
#14 6.946 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus-velocity/1.1.7/plexus-velocity-1.1.7.pom (2.0 kB at 58 kB/s)
#14 6.954 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus-container-default/1.0-alpha-20/plexus-container-default-1.0-alpha-20.pom
#14 6.985 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus-container-default/1.0-alpha-20/plexus-container-default-1.0-alpha-20.pom (3.0 kB at 93 kB/s)
#14 6.990 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus-containers/1.0-alpha-20/plexus-containers-1.0-alpha-20.pom
#14 7.043 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus-containers/1.0-alpha-20/plexus-containers-1.0-alpha-20.pom (1.9 kB at 36 kB/s)
#14 7.050 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus-utils/1.3/plexus-utils-1.3.pom
#14 7.081 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus-utils/1.3/plexus-utils-1.3.pom (1.0 kB at 32 kB/s)
#14 7.086 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus/1.0.8/plexus-1.0.8.pom
#14 7.118 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus/1.0.8/plexus-1.0.8.pom (7.2 kB at 233 kB/s)
#14 7.125 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus-classworlds/1.2-alpha-7/plexus-classworlds-1.2-alpha-7.pom
#14 7.156 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus-classworlds/1.2-alpha-7/plexus-classworlds-1.2-alpha-7.pom (2.4 kB at 77 kB/s)
#14 7.161 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus/1.0.9/plexus-1.0.9.pom
#14 7.193 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus/1.0.9/plexus-1.0.9.pom (7.7 kB at 233 kB/s)
#14 7.200 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/velocity/velocity/1.5/velocity-1.5.pom
#14 7.233 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/velocity/velocity/1.5/velocity-1.5.pom (7.8 kB at 235 kB/s)
#14 7.240 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/commons-collections/commons-collections/3.1/commons-collections-3.1.pom
#14 7.271 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/commons-collections/commons-collections/3.1/commons-collections-3.1.pom (6.1 kB at 190 kB/s)
#14 7.277 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/commons-lang/commons-lang/2.1/commons-lang-2.1.pom
#14 7.307 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/commons-lang/commons-lang/2.1/commons-lang-2.1.pom (9.9 kB at 320 kB/s)
#14 7.320 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/oro/oro/2.0.8/oro-2.0.8.pom
#14 7.351 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/oro/oro/2.0.8/oro-2.0.8.pom (140 B at 4.4 kB/s)
#14 7.357 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/doxia/doxia-decoration-model/1.0/doxia-decoration-model-1.0.pom
#14 7.388 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/doxia/doxia-decoration-model/1.0/doxia-decoration-model-1.0.pom (3.2 kB at 102 kB/s)
#14 7.396 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/commons-collections/commons-collections/3.2/commons-collections-3.2.pom
#14 7.429 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/commons-collections/commons-collections/3.2/commons-collections-3.2.pom (11 kB at 334 kB/s)
#14 7.436 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/doxia/doxia-module-apt/1.0/doxia-module-apt-1.0.pom
#14 7.468 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/doxia/doxia-module-apt/1.0/doxia-module-apt-1.0.pom (2.3 kB at 70 kB/s)
#14 7.474 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/doxia/doxia-modules/1.0/doxia-modules-1.0.pom
#14 7.505 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/doxia/doxia-modules/1.0/doxia-modules-1.0.pom (2.4 kB at 75 kB/s)
#14 7.514 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/doxia/doxia-module-fml/1.0/doxia-module-fml-1.0.pom
#14 7.552 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/doxia/doxia-module-fml/1.0/doxia-module-fml-1.0.pom (2.7 kB at 71 kB/s)
#14 7.561 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/doxia/doxia-module-xdoc/1.0/doxia-module-xdoc-1.0.pom
#14 7.593 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/doxia/doxia-module-xdoc/1.0/doxia-module-xdoc-1.0.pom (2.2 kB at 70 kB/s)
#14 7.602 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/doxia/doxia-module-xhtml/1.0/doxia-module-xhtml-1.0.pom
#14 7.633 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/doxia/doxia-module-xhtml/1.0/doxia-module-xhtml-1.0.pom (1.6 kB at 50 kB/s)
#14 7.642 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/shared/maven-doxia-tools/1.0.2/maven-doxia-tools-1.0.2.pom
#14 7.676 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/shared/maven-doxia-tools/1.0.2/maven-doxia-tools-1.0.2.pom (5.9 kB at 168 kB/s)
#14 7.681 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/shared/maven-shared-components/11/maven-shared-components-11.pom
#14 7.714 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/shared/maven-shared-components/11/maven-shared-components-11.pom (8.3 kB at 261 kB/s)
#14 7.722 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/commons-io/commons-io/1.4/commons-io-1.4.pom
#14 7.755 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/commons-io/commons-io/1.4/commons-io-1.4.pom (13 kB at 387 kB/s)
#14 7.761 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/commons/commons-parent/7/commons-parent-7.pom
#14 7.795 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/commons/commons-parent/7/commons-parent-7.pom (17 kB at 515 kB/s)
#14 7.809 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/reporting/maven-reporting-api/2.0.6/maven-reporting-api-2.0.6.pom
#14 7.840 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/reporting/maven-reporting-api/2.0.6/maven-reporting-api-2.0.6.pom (1.8 kB at 58 kB/s)
#14 7.845 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/reporting/maven-reporting/2.0.6/maven-reporting-2.0.6.pom
#14 7.876 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/reporting/maven-reporting/2.0.6/maven-reporting-2.0.6.pom (1.4 kB at 46 kB/s)
#14 7.884 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/doxia/doxia-sink-api/1.0-alpha-7/doxia-sink-api-1.0-alpha-7.pom
#14 7.919 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/doxia/doxia-sink-api/1.0-alpha-7/doxia-sink-api-1.0-alpha-7.pom (424 B at 12 kB/s)
#14 7.925 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/doxia/doxia/1.0-alpha-7/doxia-1.0-alpha-7.pom
#14 7.956 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/doxia/doxia/1.0-alpha-7/doxia-1.0-alpha-7.pom (3.9 kB at 122 kB/s)
#14 7.963 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus-container-default/1.0-alpha-9/plexus-container-default-1.0-alpha-9.pom
#14 7.993 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus-container-default/1.0-alpha-9/plexus-container-default-1.0-alpha-9.pom (1.2 kB at 40 kB/s)
#14 8.002 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/commons-validator/commons-validator/1.2.0/commons-validator-1.2.0.pom
#14 8.035 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/commons-validator/commons-validator/1.2.0/commons-validator-1.2.0.pom (9.1 kB at 275 kB/s)
#14 8.043 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/commons-beanutils/commons-beanutils/1.7.0/commons-beanutils-1.7.0.pom
#14 8.074 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/commons-beanutils/commons-beanutils/1.7.0/commons-beanutils-1.7.0.pom (357 B at 12 kB/s)
#14 8.079 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/commons-logging/commons-logging/1.0.3/commons-logging-1.0.3.pom
#14 8.115 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/commons-logging/commons-logging/1.0.3/commons-logging-1.0.3.pom (866 B at 24 kB/s)
#14 8.122 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/commons-digester/commons-digester/1.6/commons-digester-1.6.pom
#14 8.153 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/commons-digester/commons-digester/1.6/commons-digester-1.6.pom (974 B at 31 kB/s)
#14 8.160 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/commons-beanutils/commons-beanutils/1.6/commons-beanutils-1.6.pom
#14 8.194 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/commons-beanutils/commons-beanutils/1.6/commons-beanutils-1.6.pom (2.3 kB at 68 kB/s)
#14 8.202 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/commons-logging/commons-logging/1.0/commons-logging-1.0.pom
#14 8.235 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/commons-logging/commons-logging/1.0/commons-logging-1.0.pom (163 B at 4.8 kB/s)
#14 8.241 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/commons-collections/commons-collections/2.0/commons-collections-2.0.pom
#14 8.272 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/commons-collections/commons-collections/2.0/commons-collections-2.0.pom (171 B at 5.3 kB/s)
#14 8.278 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/commons-collections/commons-collections/2.1/commons-collections-2.1.pom
#14 8.308 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/commons-collections/commons-collections/2.1/commons-collections-2.1.pom (3.3 kB at 111 kB/s)
#14 8.313 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/xml-apis/xml-apis/1.0.b2/xml-apis-1.0.b2.pom
#14 8.343 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/xml-apis/xml-apis/1.0.b2/xml-apis-1.0.b2.pom (2.2 kB at 73 kB/s)
#14 8.348 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/commons-logging/commons-logging/1.0.4/commons-logging-1.0.4.pom
#14 8.379 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/commons-logging/commons-logging/1.0.4/commons-logging-1.0.4.pom (5.3 kB at 175 kB/s)
#14 8.384 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/xml-apis/xml-apis/2.0.2/xml-apis-2.0.2.pom
#14 8.415 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/xml-apis/xml-apis/2.0.2/xml-apis-2.0.2.pom (346 B at 12 kB/s)
#14 8.427 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus-utils/1.5.8/plexus-utils-1.5.8.pom
#14 8.460 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus-utils/1.5.8/plexus-utils-1.5.8.pom (8.1 kB at 237 kB/s)
#14 8.468 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus-archiver/2.3/plexus-archiver-2.3.pom
#14 8.500 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus-archiver/2.3/plexus-archiver-2.3.pom (3.4 kB at 102 kB/s)
#14 8.505 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus-components/1.3/plexus-components-1.3.pom
#14 8.536 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus-components/1.3/plexus-components-1.3.pom (3.1 kB at 102 kB/s)
#14 8.540 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus/3.3/plexus-3.3.pom
#14 8.571 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus/3.3/plexus-3.3.pom (20 kB at 642 kB/s)
#14 8.577 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/sonatype/spice/spice-parent/17/spice-parent-17.pom
#14 8.609 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/sonatype/spice/spice-parent/17/spice-parent-17.pom (6.8 kB at 218 kB/s)
#14 8.613 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/sonatype/forge/forge-parent/10/forge-parent-10.pom
#14 8.644 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/sonatype/forge/forge-parent/10/forge-parent-10.pom (14 kB at 437 kB/s)
#14 8.653 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus-utils/3.0.10/plexus-utils-3.0.10.pom
#14 8.682 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus-utils/3.0.10/plexus-utils-3.0.10.pom (3.1 kB at 105 kB/s)
#14 8.689 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus-io/2.0.6/plexus-io-2.0.6.pom
#14 8.720 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus-io/2.0.6/plexus-io-2.0.6.pom (2.2 kB at 74 kB/s)
#14 8.724 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus-components/1.2/plexus-components-1.2.pom
#14 8.754 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus-components/1.2/plexus-components-1.2.pom (3.1 kB at 102 kB/s)
#14 8.758 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus/3.2/plexus-3.2.pom
#14 8.789 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus/3.2/plexus-3.2.pom (19 kB at 625 kB/s)
#14 8.798 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus-utils/3.0.8/plexus-utils-3.0.8.pom
#14 8.829 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus-utils/3.0.8/plexus-utils-3.0.8.pom (3.1 kB at 101 kB/s)
#14 8.836 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus-utils/3.0.9/plexus-utils-3.0.9.pom
#14 8.867 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus-utils/3.0.9/plexus-utils-3.0.9.pom (3.1 kB at 105 kB/s)
#14 8.874 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/shared/file-management/1.2.1/file-management-1.2.1.pom
#14 8.904 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/shared/file-management/1.2.1/file-management-1.2.1.pom (3.9 kB at 125 kB/s)
#14 8.909 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/shared/maven-shared-components/10/maven-shared-components-10.pom
#14 8.939 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/shared/maven-shared-components/10/maven-shared-components-10.pom (8.4 kB at 281 kB/s)
#14 8.947 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/shared/maven-shared-io/1.1/maven-shared-io-1.1.pom
#14 8.979 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/shared/maven-shared-io/1.1/maven-shared-io-1.1.pom (4.1 kB at 127 kB/s)
#14 8.983 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/shared/maven-shared-components/8/maven-shared-components-8.pom
#14 9.027 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/shared/maven-shared-components/8/maven-shared-components-8.pom (2.7 kB at 59 kB/s)
#14 9.031 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-parent/7/maven-parent-7.pom
#14 9.063 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-parent/7/maven-parent-7.pom (21 kB at 664 kB/s)
#14 9.070 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-artifact/2.0.2/maven-artifact-2.0.2.pom
#14 9.101 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-artifact/2.0.2/maven-artifact-2.0.2.pom (765 B at 25 kB/s)
#14 9.105 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven/2.0.2/maven-2.0.2.pom
#14 9.135 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven/2.0.2/maven-2.0.2.pom (13 kB at 425 kB/s)
#14 9.141 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus-utils/1.1/plexus-utils-1.1.pom
#14 9.172 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus-utils/1.1/plexus-utils-1.1.pom (767 B at 26 kB/s)
#14 9.177 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-artifact-manager/2.0.2/maven-artifact-manager-2.0.2.pom
#14 9.209 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-artifact-manager/2.0.2/maven-artifact-manager-2.0.2.pom (1.4 kB at 44 kB/s)
#14 9.214 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-repository-metadata/2.0.2/maven-repository-metadata-2.0.2.pom
#14 9.248 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-repository-metadata/2.0.2/maven-repository-metadata-2.0.2.pom (1.3 kB at 40 kB/s)
#14 9.254 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/wagon/wagon-provider-api/1.0-alpha-6/wagon-provider-api-1.0-alpha-6.pom
#14 9.288 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/wagon/wagon-provider-api/1.0-alpha-6/wagon-provider-api-1.0-alpha-6.pom (588 B at 18 kB/s)
#14 9.292 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/wagon/wagon/1.0-alpha-6/wagon-1.0-alpha-6.pom
#14 9.322 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/wagon/wagon/1.0-alpha-6/wagon-1.0-alpha-6.pom (6.4 kB at 213 kB/s)
#14 9.327 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus-utils/1.4.6/plexus-utils-1.4.6.pom
#14 9.360 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus-utils/1.4.6/plexus-utils-1.4.6.pom (2.3 kB at 69 kB/s)
#14 9.366 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus-utils/1.5.6/plexus-utils-1.5.6.pom
#14 9.397 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus-utils/1.5.6/plexus-utils-1.5.6.pom (5.3 kB at 166 kB/s)
#14 9.402 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus/1.0.12/plexus-1.0.12.pom
#14 9.432 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus/1.0.12/plexus-1.0.12.pom (9.8 kB at 316 kB/s)
#14 9.438 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/shared/maven-dependency-analyzer/1.4/maven-dependency-analyzer-1.4.pom
#14 9.473 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/shared/maven-dependency-analyzer/1.4/maven-dependency-analyzer-1.4.pom (5.2 kB at 147 kB/s)
#14 9.478 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/shared/maven-shared-components/19/maven-shared-components-19.pom
#14 9.508 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/shared/maven-shared-components/19/maven-shared-components-19.pom (6.4 kB at 205 kB/s)
#14 9.515 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/asm/asm/3.3.1/asm-3.3.1.pom
#14 9.545 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/asm/asm/3.3.1/asm-3.3.1.pom (266 B at 9.2 kB/s)
#14 9.548 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/asm/asm-parent/3.3.1/asm-parent-3.3.1.pom
#14 9.579 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/asm/asm-parent/3.3.1/asm-parent-3.3.1.pom (4.3 kB at 140 kB/s)
#14 9.586 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus-component-annotations/1.5.5/plexus-component-annotations-1.5.5.pom
#14 9.615 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus-component-annotations/1.5.5/plexus-component-annotations-1.5.5.pom (815 B at 29 kB/s)
#14 9.619 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus-containers/1.5.5/plexus-containers-1.5.5.pom
#14 9.649 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus-containers/1.5.5/plexus-containers-1.5.5.pom (4.2 kB at 141 kB/s)
#14 9.653 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus/2.0.7/plexus-2.0.7.pom
#14 9.697 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus/2.0.7/plexus-2.0.7.pom (17 kB at 393 kB/s)
#14 9.704 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-project/2.0.5/maven-project-2.0.5.pom
#14 9.735 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-project/2.0.5/maven-project-2.0.5.pom (1.8 kB at 59 kB/s)
#14 9.739 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven/2.0.5/maven-2.0.5.pom
#14 9.770 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven/2.0.5/maven-2.0.5.pom (5.7 kB at 190 kB/s)
#14 9.776 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-settings/2.0.5/maven-settings-2.0.5.pom
#14 9.806 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-settings/2.0.5/maven-settings-2.0.5.pom (1.7 kB at 54 kB/s)
#14 9.812 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-model/2.0.5/maven-model-2.0.5.pom
#14 9.843 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-model/2.0.5/maven-model-2.0.5.pom (2.7 kB at 88 kB/s)
#14 9.850 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-profile/2.0.5/maven-profile-2.0.5.pom
#14 9.880 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-profile/2.0.5/maven-profile-2.0.5.pom (1.7 kB at 53 kB/s)
#14 9.886 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-artifact-manager/2.0.5/maven-artifact-manager-2.0.5.pom
#14 9.916 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-artifact-manager/2.0.5/maven-artifact-manager-2.0.5.pom (1.8 kB at 61 kB/s)
#14 9.922 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-repository-metadata/2.0.5/maven-repository-metadata-2.0.5.pom
#14 9.953 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-repository-metadata/2.0.5/maven-repository-metadata-2.0.5.pom (1.5 kB at 50 kB/s)
#14 9.959 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-artifact/2.0.5/maven-artifact-2.0.5.pom
#14 9.990 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-artifact/2.0.5/maven-artifact-2.0.5.pom (727 B at 23 kB/s)
#14 9.997 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/shared/maven-dependency-tree/2.1/maven-dependency-tree-2.1.pom
#14 10.03 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/shared/maven-dependency-tree/2.1/maven-dependency-tree-2.1.pom (6.8 kB at 206 kB/s)
#14 10.04 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-project/2.2.0/maven-project-2.2.0.pom
#14 10.07 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-project/2.2.0/maven-project-2.2.0.pom (2.8 kB at 92 kB/s)
#14 10.07 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven/2.2.0/maven-2.2.0.pom
#14 10.10 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven/2.2.0/maven-2.2.0.pom (22 kB at 724 kB/s)
#14 10.11 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-settings/2.2.0/maven-settings-2.2.0.pom
#14 10.14 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-settings/2.2.0/maven-settings-2.2.0.pom (2.2 kB at 73 kB/s)
#14 10.15 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-model/2.2.0/maven-model-2.2.0.pom
#14 10.18 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-model/2.2.0/maven-model-2.2.0.pom (3.2 kB at 95 kB/s)
#14 10.19 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus-utils/1.5.15/plexus-utils-1.5.15.pom
#14 10.22 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus-utils/1.5.15/plexus-utils-1.5.15.pom (6.8 kB at 221 kB/s)
#14 10.22 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus-interpolation/1.11/plexus-interpolation-1.11.pom
#14 10.25 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus-interpolation/1.11/plexus-interpolation-1.11.pom (889 B at 29 kB/s)
#14 10.26 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus-components/1.1.14/plexus-components-1.1.14.pom
#14 10.29 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus-components/1.1.14/plexus-components-1.1.14.pom (5.8 kB at 188 kB/s)
#14 10.29 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-profile/2.2.0/maven-profile-2.2.0.pom
#14 10.33 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-profile/2.2.0/maven-profile-2.2.0.pom (2.2 kB at 66 kB/s)
#14 10.33 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-artifact-manager/2.2.0/maven-artifact-manager-2.2.0.pom
#14 10.37 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-artifact-manager/2.2.0/maven-artifact-manager-2.2.0.pom (3.1 kB at 97 kB/s)
#14 10.37 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-repository-metadata/2.2.0/maven-repository-metadata-2.2.0.pom
#14 10.40 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-repository-metadata/2.2.0/maven-repository-metadata-2.2.0.pom (1.9 kB at 60 kB/s)
#14 10.42 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-artifact/2.2.0/maven-artifact-2.2.0.pom
#14 10.45 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-artifact/2.2.0/maven-artifact-2.2.0.pom (1.6 kB at 51 kB/s)
#14 10.46 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/backport-util-concurrent/backport-util-concurrent/3.1/backport-util-concurrent-3.1.pom
#14 10.49 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/backport-util-concurrent/backport-util-concurrent/3.1/backport-util-concurrent-3.1.pom (880 B at 29 kB/s)
#14 10.50 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-plugin-registry/2.2.0/maven-plugin-registry-2.2.0.pom
#14 10.53 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-plugin-registry/2.2.0/maven-plugin-registry-2.2.0.pom (1.9 kB at 64 kB/s)
#14 10.53 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/eclipse/aether/aether-util/0.9.0.M2/aether-util-0.9.0.M2.pom
#14 10.56 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/eclipse/aether/aether-util/0.9.0.M2/aether-util-0.9.0.M2.pom (2.0 kB at 68 kB/s)
#14 10.57 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/eclipse/aether/aether/0.9.0.M2/aether-0.9.0.M2.pom
#14 10.60 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/eclipse/aether/aether/0.9.0.M2/aether-0.9.0.M2.pom (28 kB at 796 kB/s)
#14 10.61 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/shared/maven-common-artifact-filters/1.4/maven-common-artifact-filters-1.4.pom
#14 10.65 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/shared/maven-common-artifact-filters/1.4/maven-common-artifact-filters-1.4.pom (3.8 kB at 111 kB/s)
#14 10.65 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/shared/maven-shared-components/17/maven-shared-components-17.pom
#14 10.68 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/shared/maven-shared-components/17/maven-shared-components-17.pom (8.7 kB at 300 kB/s)
#14 10.69 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-artifact/2.0.8/maven-artifact-2.0.8.pom
#14 10.80 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-artifact/2.0.8/maven-artifact-2.0.8.pom (1.6 kB at 15 kB/s)
#14 10.80 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven/2.0.8/maven-2.0.8.pom
#14 10.87 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven/2.0.8/maven-2.0.8.pom (12 kB at 178 kB/s)
#14 10.88 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-model/2.0.8/maven-model-2.0.8.pom
#14 10.91 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-model/2.0.8/maven-model-2.0.8.pom (3.1 kB at 92 kB/s)
#14 10.92 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-project/2.0.8/maven-project-2.0.8.pom
#14 10.95 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-project/2.0.8/maven-project-2.0.8.pom (2.7 kB at 82 kB/s)
#14 10.96 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-settings/2.0.8/maven-settings-2.0.8.pom
#14 10.99 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-settings/2.0.8/maven-settings-2.0.8.pom (2.1 kB at 69 kB/s)
#14 10.99 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-profile/2.0.8/maven-profile-2.0.8.pom
#14 11.03 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-profile/2.0.8/maven-profile-2.0.8.pom (2.0 kB at 66 kB/s)
#14 11.03 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-artifact-manager/2.0.8/maven-artifact-manager-2.0.8.pom
#14 11.06 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-artifact-manager/2.0.8/maven-artifact-manager-2.0.8.pom (2.7 kB at 87 kB/s)
#14 11.07 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-repository-metadata/2.0.8/maven-repository-metadata-2.0.8.pom
#14 11.10 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-repository-metadata/2.0.8/maven-repository-metadata-2.0.8.pom (1.9 kB at 59 kB/s)
#14 11.11 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-plugin-registry/2.0.8/maven-plugin-registry-2.0.8.pom
#14 11.14 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-plugin-registry/2.0.8/maven-plugin-registry-2.0.8.pom (2.0 kB at 67 kB/s)
#14 11.15 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-plugin-api/2.0.8/maven-plugin-api-2.0.8.pom
#14 11.18 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-plugin-api/2.0.8/maven-plugin-api-2.0.8.pom (1.5 kB at 44 kB/s)
#14 11.19 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus-container-default/1.5.5/plexus-container-default-1.5.5.pom
#14 11.22 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus-container-default/1.5.5/plexus-container-default-1.5.5.pom (2.8 kB at 92 kB/s)
#14 11.22 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus-classworlds/2.2.2/plexus-classworlds-2.2.2.pom
#14 11.25 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus-classworlds/2.2.2/plexus-classworlds-2.2.2.pom (4.0 kB at 130 kB/s)
#14 11.26 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus/2.0.3/plexus-2.0.3.pom
#14 11.29 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus/2.0.3/plexus-2.0.3.pom (15 kB at 469 kB/s)
#14 11.30 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/xbean/xbean-reflect/3.4/xbean-reflect-3.4.pom
#14 11.33 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/xbean/xbean-reflect/3.4/xbean-reflect-3.4.pom (2.8 kB at 91 kB/s)
#14 11.33 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/xbean/xbean/3.4/xbean-3.4.pom
#14 11.37 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/xbean/xbean/3.4/xbean-3.4.pom (19 kB at 579 kB/s)
#14 11.37 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/log4j/log4j/1.2.12/log4j-1.2.12.pom
#14 11.40 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/log4j/log4j/1.2.12/log4j-1.2.12.pom (145 B at 4.8 kB/s)
#14 11.41 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/commons-logging/commons-logging-api/1.1/commons-logging-api-1.1.pom
#14 11.44 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/commons-logging/commons-logging-api/1.1/commons-logging-api-1.1.pom (5.3 kB at 178 kB/s)
#14 11.44 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/com/google/collections/google-collections/1.0/google-collections-1.0.pom
#14 11.47 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/com/google/collections/google-collections/1.0/google-collections-1.0.pom (2.5 kB at 83 kB/s)
#14 11.48 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/com/google/google/1/google-1.pom
#14 11.51 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/com/google/google/1/google-1.pom (1.6 kB at 52 kB/s)
#14 11.51 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/junit/junit/3.8.2/junit-3.8.2.pom
#14 11.54 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/junit/junit/3.8.2/junit-3.8.2.pom (747 B at 25 kB/s)
#14 11.54 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus-utils/2.1/plexus-utils-2.1.pom
#14 11.57 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus-utils/2.1/plexus-utils-2.1.pom (4.0 kB at 139 kB/s)
#14 11.58 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/sonatype/spice/spice-parent/16/spice-parent-16.pom
#14 11.61 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/sonatype/spice/spice-parent/16/spice-parent-16.pom (8.4 kB at 279 kB/s)
#14 11.61 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/sonatype/forge/forge-parent/5/forge-parent-5.pom
#14 11.64 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/sonatype/forge/forge-parent/5/forge-parent-5.pom (8.4 kB at 279 kB/s)
#14 11.65 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/shared/maven-invoker/2.0.11/maven-invoker-2.0.11.pom
#14 11.68 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/shared/maven-invoker/2.0.11/maven-invoker-2.0.11.pom (5.1 kB at 169 kB/s)
#14 11.68 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/shared/maven-shared-components/12/maven-shared-components-12.pom
#14 11.71 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/shared/maven-shared-components/12/maven-shared-components-12.pom (9.3 kB at 333 kB/s)
#14 11.71 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-parent/13/maven-parent-13.pom
#14 11.74 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-parent/13/maven-parent-13.pom (23 kB at 808 kB/s)
#14 11.75 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/commons-lang/commons-lang/2.6/commons-lang-2.6.pom
#14 11.78 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/commons-lang/commons-lang/2.6/commons-lang-2.6.pom (17 kB at 500 kB/s)
#14 11.79 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/commons/commons-parent/17/commons-parent-17.pom
#14 11.82 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/commons/commons-parent/17/commons-parent-17.pom (31 kB at 1.0 MB/s)
#14 11.83 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/commons-collections/commons-collections/3.2.1/commons-collections-3.2.1.pom
#14 11.86 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/commons-collections/commons-collections/3.2.1/commons-collections-3.2.1.pom (13 kB at 403 kB/s)
#14 11.87 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/commons/commons-parent/9/commons-parent-9.pom
#14 11.90 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/commons/commons-parent/9/commons-parent-9.pom (22 kB at 707 kB/s)
#14 11.94 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-artifact/2.0.9/maven-artifact-2.0.9.jar
#14 11.94 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-project/2.0.9/maven-project-2.0.9.jar
#14 11.94 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-plugin-api/2.0.9/maven-plugin-api-2.0.9.jar
#14 11.94 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-settings/2.0.9/maven-settings-2.0.9.jar
#14 11.95 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-profile/2.0.9/maven-profile-2.0.9.jar
#14 12.00 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-artifact/2.0.9/maven-artifact-2.0.9.jar (89 kB at 1.4 MB/s)
#14 12.00 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-plugin-registry/2.0.9/maven-plugin-registry-2.0.9.jar
#14 12.04 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-plugin-registry/2.0.9/maven-plugin-registry-2.0.9.jar (29 kB at 306 kB/s)
#14 12.04 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-model/2.0.9/maven-model-2.0.9.jar
#14 12.06 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-plugin-api/2.0.9/maven-plugin-api-2.0.9.jar (13 kB at 107 kB/s)
#14 12.06 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-core/2.0.9/maven-core-2.0.9.jar
#14 12.08 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-model/2.0.9/maven-model-2.0.9.jar (87 kB at 657 kB/s)
#14 12.08 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-plugin-parameter-documenter/2.0.9/maven-plugin-parameter-documenter-2.0.9.jar
#14 12.08 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-profile/2.0.9/maven-profile-2.0.9.jar (35 kB at 264 kB/s)
#14 12.08 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-error-diagnostics/2.0.9/maven-error-diagnostics-2.0.9.jar
#14 12.09 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-settings/2.0.9/maven-settings-2.0.9.jar (49 kB at 334 kB/s)
#14 12.09 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/commons-cli/commons-cli/1.0/commons-cli-1.0.jar
#14 12.10 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-project/2.0.9/maven-project-2.0.9.jar (122 kB at 766 kB/s)
#14 12.10 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-plugin-descriptor/2.0.9/maven-plugin-descriptor-2.0.9.jar
#14 12.11 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-plugin-parameter-documenter/2.0.9/maven-plugin-parameter-documenter-2.0.9.jar (21 kB at 124 kB/s)
#14 12.11 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus-interactivity-api/1.0-alpha-4/plexus-interactivity-api-1.0-alpha-4.jar
#14 12.12 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-error-diagnostics/2.0.9/maven-error-diagnostics-2.0.9.jar (14 kB at 79 kB/s)
#14 12.12 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-monitor/2.0.9/maven-monitor-2.0.9.jar
#14 12.12 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/commons-cli/commons-cli/1.0/commons-cli-1.0.jar (30 kB at 171 kB/s)
#14 12.12 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-artifact-manager/2.0.9/maven-artifact-manager-2.0.9.jar
#14 12.12 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-core/2.0.9/maven-core-2.0.9.jar (160 kB at 897 kB/s)
#14 12.12 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-repository-metadata/2.0.9/maven-repository-metadata-2.0.9.jar
#14 12.14 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus-interactivity-api/1.0-alpha-4/plexus-interactivity-api-1.0-alpha-4.jar (13 kB at 67 kB/s)
#14 12.14 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/reporting/maven-reporting-api/3.0/maven-reporting-api-3.0.jar
#14 12.15 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-monitor/2.0.9/maven-monitor-2.0.9.jar (10 kB at 50 kB/s)
#14 12.15 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/reporting/maven-reporting-impl/2.0.5/maven-reporting-impl-2.0.5.jar
#14 12.15 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-artifact-manager/2.0.9/maven-artifact-manager-2.0.9.jar (58 kB at 275 kB/s)
#14 12.15 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/doxia/doxia-core/1.0/doxia-core-1.0.jar
#14 12.16 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-repository-metadata/2.0.9/maven-repository-metadata-2.0.9.jar (25 kB at 116 kB/s)
#14 12.16 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/shared/maven-doxia-tools/1.0.2/maven-doxia-tools-1.0.2.jar
#14 12.18 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/reporting/maven-reporting-impl/2.0.5/maven-reporting-impl-2.0.5.jar (21 kB at 88 kB/s)
#14 12.18 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/commons-io/commons-io/1.4/commons-io-1.4.jar
#14 12.19 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/shared/maven-doxia-tools/1.0.2/maven-doxia-tools-1.0.2.jar (41 kB at 168 kB/s)
#14 12.19 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/commons-validator/commons-validator/1.2.0/commons-validator-1.2.0.jar
#14 12.19 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/reporting/maven-reporting-api/3.0/maven-reporting-api-3.0.jar (11 kB at 44 kB/s)
#14 12.19 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/commons-beanutils/commons-beanutils/1.7.0/commons-beanutils-1.7.0.jar
#14 12.20 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-plugin-descriptor/2.0.9/maven-plugin-descriptor-2.0.9.jar (37 kB at 142 kB/s)
#14 12.20 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/commons-digester/commons-digester/1.6/commons-digester-1.6.jar
#14 12.21 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/commons-io/commons-io/1.4/commons-io-1.4.jar (109 kB at 402 kB/s)
#14 12.21 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/commons-logging/commons-logging/1.0.4/commons-logging-1.0.4.jar
#14 12.22 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/doxia/doxia-core/1.0/doxia-core-1.0.jar (55 kB at 198 kB/s)
#14 12.22 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/oro/oro/2.0.8/oro-2.0.8.jar
#14 12.22 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/commons-validator/commons-validator/1.2.0/commons-validator-1.2.0.jar (91 kB at 327 kB/s)
#14 12.22 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/xml-apis/xml-apis/1.0.b2/xml-apis-1.0.b2.jar
#14 12.24 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/commons-beanutils/commons-beanutils/1.7.0/commons-beanutils-1.7.0.jar (189 kB at 640 kB/s)
#14 12.24 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/doxia/doxia-sink-api/1.0/doxia-sink-api-1.0.jar
#14 12.25 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/commons-logging/commons-logging/1.0.4/commons-logging-1.0.4.jar (38 kB at 125 kB/s)
#14 12.25 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/doxia/doxia-site-renderer/1.0/doxia-site-renderer-1.0.jar
#14 12.25 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/oro/oro/2.0.8/oro-2.0.8.jar (65 kB at 210 kB/s)
#14 12.25 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus-i18n/1.0-beta-7/plexus-i18n-1.0-beta-7.jar
#14 12.26 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/xml-apis/xml-apis/1.0.b2/xml-apis-1.0.b2.jar (109 kB at 347 kB/s)
#14 12.26 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus-velocity/1.1.7/plexus-velocity-1.1.7.jar
#14 12.26 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/commons-digester/commons-digester/1.6/commons-digester-1.6.jar (168 kB at 523 kB/s)
#14 12.27 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/velocity/velocity/1.5/velocity-1.5.jar
#14 12.27 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/doxia/doxia-sink-api/1.0/doxia-sink-api-1.0.jar (10 kB at 31 kB/s)
#14 12.27 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/doxia/doxia-decoration-model/1.0/doxia-decoration-model-1.0.jar
#14 12.28 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/doxia/doxia-site-renderer/1.0/doxia-site-renderer-1.0.jar (47 kB at 138 kB/s)
#14 12.28 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/doxia/doxia-module-apt/1.0/doxia-module-apt-1.0.jar
#14 12.28 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus-i18n/1.0-beta-7/plexus-i18n-1.0-beta-7.jar (11 kB at 31 kB/s)
#14 12.28 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/doxia/doxia-module-fml/1.0/doxia-module-fml-1.0.jar
#14 12.30 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus-velocity/1.1.7/plexus-velocity-1.1.7.jar (7.7 kB at 22 kB/s)
#14 12.30 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/doxia/doxia-module-xdoc/1.0/doxia-module-xdoc-1.0.jar
#14 12.30 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/doxia/doxia-decoration-model/1.0/doxia-decoration-model-1.0.jar (49 kB at 136 kB/s)
#14 12.30 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/doxia/doxia-module-xhtml/1.0/doxia-module-xhtml-1.0.jar
#14 12.32 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/doxia/doxia-module-apt/1.0/doxia-module-apt-1.0.jar (47 kB at 126 kB/s)
#14 12.32 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/doxia/doxia-module-fml/1.0/doxia-module-fml-1.0.jar (19 kB at 50 kB/s)
#14 12.32 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus-utils/3.0.9/plexus-utils-3.0.9.jar
#14 12.32 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus-archiver/2.3/plexus-archiver-2.3.jar
#14 12.32 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/velocity/velocity/1.5/velocity-1.5.jar (392 kB at 1.0 MB/s)
#14 12.32 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/shared/file-management/1.2.1/file-management-1.2.1.jar
#14 12.34 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/doxia/doxia-module-xdoc/1.0/doxia-module-xdoc-1.0.jar (28 kB at 71 kB/s)
#14 12.34 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/shared/maven-shared-io/1.1/maven-shared-io-1.1.jar
#14 12.34 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/doxia/doxia-module-xhtml/1.0/doxia-module-xhtml-1.0.jar (22 kB at 55 kB/s)
#14 12.34 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/wagon/wagon-provider-api/1.0-alpha-6/wagon-provider-api-1.0-alpha-6.jar
#14 12.35 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/shared/file-management/1.2.1/file-management-1.2.1.jar (38 kB at 92 kB/s)
#14 12.35 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus-container-default/1.0-alpha-9-stable-1/plexus-container-default-1.0-alpha-9-stable-1.jar
#14 12.36 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus-archiver/2.3/plexus-archiver-2.3.jar (186 kB at 451 kB/s)
#14 12.36 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/junit/junit/3.8.1/junit-3.8.1.jar
#14 12.36 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus-utils/3.0.9/plexus-utils-3.0.9.jar (232 kB at 554 kB/s)
#14 12.36 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus-io/2.0.6/plexus-io-2.0.6.jar
#14 12.37 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/shared/maven-shared-io/1.1/maven-shared-io-1.1.jar (39 kB at 93 kB/s)
#14 12.37 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/shared/maven-dependency-analyzer/1.4/maven-dependency-analyzer-1.4.jar
#14 12.37 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/wagon/wagon-provider-api/1.0-alpha-6/wagon-provider-api-1.0-alpha-6.jar (43 kB at 99 kB/s)
#14 12.37 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/asm/asm/3.3.1/asm-3.3.1.jar
#14 12.39 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/junit/junit/3.8.1/junit-3.8.1.jar (121 kB at 270 kB/s)
#14 12.39 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus-component-annotations/1.5.5/plexus-component-annotations-1.5.5.jar
#14 12.39 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus-container-default/1.0-alpha-9-stable-1/plexus-container-default-1.0-alpha-9-stable-1.jar (194 kB at 432 kB/s)
#14 12.39 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/shared/maven-dependency-tree/2.1/maven-dependency-tree-2.1.jar
#14 12.40 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus-io/2.0.6/plexus-io-2.0.6.jar (58 kB at 127 kB/s)
#14 12.40 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/eclipse/aether/aether-util/0.9.0.M2/aether-util-0.9.0.M2.jar
#14 12.40 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/shared/maven-dependency-analyzer/1.4/maven-dependency-analyzer-1.4.jar (27 kB at 60 kB/s)
#14 12.40 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/shared/maven-common-artifact-filters/1.4/maven-common-artifact-filters-1.4.jar
#14 12.40 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/asm/asm/3.3.1/asm-3.3.1.jar (44 kB at 95 kB/s)
#14 12.41 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/apache/maven/shared/maven-invoker/2.0.11/maven-invoker-2.0.11.jar
#14 12.42 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/codehaus/plexus/plexus-component-annotations/1.5.5/plexus-component-annotations-1.5.5.jar (4.2 kB at 8.8 kB/s)
#14 12.42 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/commons-lang/commons-lang/2.6/commons-lang-2.6.jar
#14 12.43 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/shared/maven-dependency-tree/2.1/maven-dependency-tree-2.1.jar (60 kB at 124 kB/s)
#14 12.43 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/commons-collections/commons-collections/3.2.1/commons-collections-3.2.1.jar
#14 12.43 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/shared/maven-common-artifact-filters/1.4/maven-common-artifact-filters-1.4.jar (32 kB at 65 kB/s)
#14 12.43 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/classworlds/classworlds/1.1/classworlds-1.1.jar
#14 12.43 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/eclipse/aether/aether-util/0.9.0.M2/aether-util-0.9.0.M2.jar (134 kB at 274 kB/s)
#14 12.44 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/shared/maven-invoker/2.0.11/maven-invoker-2.0.11.jar (29 kB at 59 kB/s)
#14 12.46 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/classworlds/classworlds/1.1/classworlds-1.1.jar (38 kB at 73 kB/s)
#14 12.47 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/commons-lang/commons-lang/2.6/commons-lang-2.6.jar (284 kB at 543 kB/s)
#14 12.55 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/commons-collections/commons-collections/3.2.1/commons-collections-3.2.1.jar (575 kB at 954 kB/s)
#14 12.81 [INFO] Configured Artifact: org.mlflow:mlflow-scoring:2.12.2:pom
#14 12.82 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/mlflow/mlflow-scoring/2.12.2/mlflow-scoring-2.12.2.pom
#14 12.85 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/mlflow/mlflow-scoring/2.12.2/mlflow-scoring-2.12.2.pom (4.6 kB at 148 kB/s)
#14 12.85 [INFO] Copying mlflow-scoring-2.12.2.pom to /opt/java/mlflow-scoring-2.12.2.pom
#14 12.86 [INFO] ------------------------------------------------------------------------
#14 12.86 [INFO] BUILD SUCCESS
#14 12.86 [INFO] ------------------------------------------------------------------------
#14 12.86 [INFO] Total time:  11.318 s
#14 12.86 [INFO] Finished at: 2024-05-16T20:25:32Z
#14 12.86 [INFO] ------------------------------------------------------------------------
#14 DONE 13.0s

#15 [11/17] RUN mvn --batch-mode dependency:copy -Dartifact=org.mlflow:mlflow-scoring:2.12.2:jar -DoutputDirectory=/opt/java/jars
#15 1.696 [INFO] Scanning for projects...
#15 1.897 [INFO]
#15 1.897 [INFO] ------------------< org.apache.maven:standalone-pom >-------------------
#15 1.898 [INFO] Building Maven Stub Project (No POM) 1
#15 1.898 [INFO] --------------------------------[ pom ]---------------------------------
#15 1.905 [INFO]
#15 1.905 [INFO] --- maven-dependency-plugin:2.8:copy (default-cli) @ standalone-pom ---
#15 2.594 [INFO] Configured Artifact: org.mlflow:mlflow-scoring:2.12.2:jar
#15 2.804 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/mlflow/mlflow-scoring/2.12.2/mlflow-scoring-2.12.2.jar
#15 3.207 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/mlflow/mlflow-scoring/2.12.2/mlflow-scoring-2.12.2.jar (29 kB at 72 kB/s)
#15 3.227 [INFO] Copying mlflow-scoring-2.12.2.jar to /opt/java/jars/mlflow-scoring-2.12.2.jar
#15 3.233 [INFO] ------------------------------------------------------------------------
#15 3.233 [INFO] BUILD SUCCESS
#15 3.233 [INFO] ------------------------------------------------------------------------
#15 3.235 [INFO] Total time:  1.556 s
#15 3.235 [INFO] Finished at: 2024-05-16T20:25:35Z
#15 3.235 [INFO] ------------------------------------------------------------------------
#15 DONE 3.3s

#16 [12/17] RUN cp /opt/java/mlflow-scoring-2.12.2.pom /opt/java/pom.xml
#16 DONE 0.6s

#17 [13/17] RUN cd /opt/java && mvn --batch-mode dependency:copy-dependencies -DoutputDirectory=/opt/java/jars
#17 1.727 [INFO] Scanning for projects...
#17 1.973 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/org/mlflow/mlflow-parent/2.12.2/mlflow-parent-2.12.2.pom
#17 2.254 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/org/mlflow/mlflow-parent/2.12.2/mlflow-parent-2.12.2.pom (11 kB at 38 kB/s)
#17 2.311 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/plugins/nexus-staging-maven-plugin/1.6.8/nexus-staging-maven-plugin-1.6.8.pom
#17 3.160 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/plugins/nexus-staging-maven-plugin/1.6.8/nexus-staging-maven-plugin-1.6.8.pom (12 kB at 14 kB/s)
#17 3.181 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/nexus/maven/nexus-staging/1.6.8/nexus-staging-1.6.8.pom
#17 3.580 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/nexus/maven/nexus-staging/1.6.8/nexus-staging-1.6.8.pom (2.8 kB at 6.9 kB/s)
#17 3.587 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/nexus/maven/nexus-maven-plugins/1.6.8/nexus-maven-plugins-1.6.8.pom
#17 4.039 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/nexus/maven/nexus-maven-plugins/1.6.8/nexus-maven-plugins-1.6.8.pom (18 kB at 39 kB/s)
#17 4.050 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/buildsupport/public-parent/6/public-parent-6.pom
#17 4.446 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/buildsupport/public-parent/6/public-parent-6.pom (760 B at 1.9 kB/s)
#17 4.452 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/buildsupport/buildsupport/6/buildsupport-6.pom
#17 4.881 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/buildsupport/buildsupport/6/buildsupport-6.pom (23 kB at 54 kB/s)
#17 5.020 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/nexus/maven/nexus-common/1.6.8/nexus-common-1.6.8.pom
#17 5.407 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/nexus/maven/nexus-common/1.6.8/nexus-common-1.6.8.pom (2.2 kB at 5.8 kB/s)
#17 5.425 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/plexus/plexus-sec-dispatcher/1.4/plexus-sec-dispatcher-1.4.pom
#17 5.816 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/plexus/plexus-sec-dispatcher/1.4/plexus-sec-dispatcher-1.4.pom (3.0 kB at 7.6 kB/s)
#17 5.832 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/spice/spice-parent/12/spice-parent-12.pom
#17 6.232 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/spice/spice-parent/12/spice-parent-12.pom (6.8 kB at 17 kB/s)
#17 6.240 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/forge/forge-parent/4/forge-parent-4.pom
#17 6.791 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/forge/forge-parent/4/forge-parent-4.pom (8.4 kB at 15 kB/s)
#17 6.808 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/plexus/plexus-cipher/1.7/plexus-cipher-1.7.pom
#17 7.249 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/plexus/plexus-cipher/1.7/plexus-cipher-1.7.pom (4.2 kB at 9.5 kB/s)
#17 7.257 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/spice/spice-parent/15/spice-parent-15.pom
#17 7.661 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/spice/spice-parent/15/spice-parent-15.pom (8.4 kB at 21 kB/s)
#17 7.677 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/google/guava/guava/14.0.1/guava-14.0.1.pom
#17 8.110 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/google/guava/guava/14.0.1/guava-14.0.1.pom (5.4 kB at 12 kB/s)
#17 8.118 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/google/guava/guava-parent/14.0.1/guava-parent-14.0.1.pom
#17 8.548 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/google/guava/guava-parent/14.0.1/guava-parent-14.0.1.pom (2.6 kB at 6.0 kB/s)
#17 8.555 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/oss/oss-parent/7/oss-parent-7.pom
#17 8.929 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/oss/oss-parent/7/oss-parent-7.pom (4.8 kB at 13 kB/s)
#17 8.941 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/nexus/nexus-client-core/2.14.3-02/nexus-client-core-2.14.3-02.pom
#17 9.328 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/nexus/nexus-client-core/2.14.3-02/nexus-client-core-2.14.3-02.pom (4.9 kB at 13 kB/s)
#17 9.335 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/nexus/nexus-components/2.14.3-02/nexus-components-2.14.3-02.pom
#17 9.726 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/nexus/nexus-components/2.14.3-02/nexus-components-2.14.3-02.pom (2.6 kB at 6.7 kB/s)
#17 9.732 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/nexus/nexus-parent/2.14.3-02/nexus-parent-2.14.3-02.pom
#17 10.21 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/nexus/nexus-parent/2.14.3-02/nexus-parent-2.14.3-02.pom (38 kB at 80 kB/s)
#17 10.22 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/buildsupport/public-parent/9/public-parent-9.pom
#17 10.63 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/buildsupport/public-parent/9/public-parent-9.pom (760 B at 1.9 kB/s)
#17 10.64 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/buildsupport/buildsupport/9/buildsupport-9.pom
#17 11.04 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/buildsupport/buildsupport/9/buildsupport-9.pom (24 kB at 59 kB/s)
#17 11.06 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/nexus/buildsupport/nexus-buildsupport-all/2.14.3-02/nexus-buildsupport-all-2.14.3-02.pom
#17 11.45 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/nexus/buildsupport/nexus-buildsupport-all/2.14.3-02/nexus-buildsupport-all-2.14.3-02.pom (6.5 kB at 17 kB/s)
#17 11.45 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/nexus/buildsupport/nexus-buildsupport/2.14.3-02/nexus-buildsupport-2.14.3-02.pom
#17 11.83 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/nexus/buildsupport/nexus-buildsupport/2.14.3-02/nexus-buildsupport-2.14.3-02.pom (2.2 kB at 5.8 kB/s)
#17 11.84 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/nexus/buildsupport/nexus-buildsupport-commons/2.14.3-02/nexus-buildsupport-commons-2.14.3-02.pom
#17 12.23 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/nexus/buildsupport/nexus-buildsupport-commons/2.14.3-02/nexus-buildsupport-commons-2.14.3-02.pom (3.1 kB at 8.0 kB/s)
#17 12.25 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/nexus/buildsupport/nexus-buildsupport-db/2.14.3-02/nexus-buildsupport-db-2.14.3-02.pom
#17 12.65 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/nexus/buildsupport/nexus-buildsupport-db/2.14.3-02/nexus-buildsupport-db-2.14.3-02.pom (3.2 kB at 8.0 kB/s)
#17 12.67 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/nexus/buildsupport/nexus-buildsupport-gwt/2.14.3-02/nexus-buildsupport-gwt-2.14.3-02.pom
#17 13.06 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/nexus/buildsupport/nexus-buildsupport-gwt/2.14.3-02/nexus-buildsupport-gwt-2.14.3-02.pom (2.2 kB at 5.6 kB/s)
#17 13.07 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/nexus/buildsupport/nexus-buildsupport-httpclient/2.14.3-02/nexus-buildsupport-httpclient-2.14.3-02.pom
#17 13.52 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/nexus/buildsupport/nexus-buildsupport-httpclient/2.14.3-02/nexus-buildsupport-httpclient-2.14.3-02.pom (2.1 kB at 4.7 kB/s)
#17 13.54 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/nexus/buildsupport/nexus-buildsupport-insight/2.14.3-02/nexus-buildsupport-insight-2.14.3-02.pom
#17 13.92 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/nexus/buildsupport/nexus-buildsupport-insight/2.14.3-02/nexus-buildsupport-insight-2.14.3-02.pom (4.4 kB at 12 kB/s)
#17 13.93 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/nexus/buildsupport/nexus-buildsupport-maven/2.14.3-02/nexus-buildsupport-maven-2.14.3-02.pom
#17 14.34 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/nexus/buildsupport/nexus-buildsupport-maven/2.14.3-02/nexus-buildsupport-maven-2.14.3-02.pom (7.1 kB at 17 kB/s)
#17 14.35 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/nexus/buildsupport/nexus-buildsupport-messaging/2.14.3-02/nexus-buildsupport-messaging-2.14.3-02.pom
#17 14.75 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/nexus/buildsupport/nexus-buildsupport-messaging/2.14.3-02/nexus-buildsupport-messaging-2.14.3-02.pom (5.0 kB at 12 kB/s)
#17 14.77 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/nexus/buildsupport/nexus-buildsupport-shiro/2.14.3-02/nexus-buildsupport-shiro-2.14.3-02.pom
#17 15.14 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/nexus/buildsupport/nexus-buildsupport-shiro/2.14.3-02/nexus-buildsupport-shiro-2.14.3-02.pom (3.4 kB at 9.1 kB/s)
#17 15.16 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/nexus/buildsupport/nexus-buildsupport-bouncycastle/2.14.3-02/nexus-buildsupport-bouncycastle-2.14.3-02.pom
#17 15.58 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/nexus/buildsupport/nexus-buildsupport-bouncycastle/2.14.3-02/nexus-buildsupport-bouncycastle-2.14.3-02.pom (2.1 kB at 5.0 kB/s)
#17 15.59 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/nexus/buildsupport/nexus-buildsupport-jetty/2.14.3-02/nexus-buildsupport-jetty-2.14.3-02.pom
#17 16.01 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/nexus/buildsupport/nexus-buildsupport-jetty/2.14.3-02/nexus-buildsupport-jetty-2.14.3-02.pom (4.5 kB at 11 kB/s)
#17 16.02 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/nexus/buildsupport/nexus-buildsupport-goodies/2.14.3-02/nexus-buildsupport-goodies-2.14.3-02.pom
#17 16.44 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/nexus/buildsupport/nexus-buildsupport-goodies/2.14.3-02/nexus-buildsupport-goodies-2.14.3-02.pom (3.6 kB at 8.6 kB/s)
#17 16.45 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/nexus/buildsupport/nexus-buildsupport-groovy/2.14.3-02/nexus-buildsupport-groovy-2.14.3-02.pom
#17 16.86 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/nexus/buildsupport/nexus-buildsupport-groovy/2.14.3-02/nexus-buildsupport-groovy-2.14.3-02.pom (3.1 kB at 7.5 kB/s)
#17 16.88 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/nexus/buildsupport/nexus-buildsupport-guice/2.14.3-02/nexus-buildsupport-guice-2.14.3-02.pom
#17 17.29 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/nexus/buildsupport/nexus-buildsupport-guice/2.14.3-02/nexus-buildsupport-guice-2.14.3-02.pom (3.7 kB at 8.8 kB/s)
#17 17.30 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/nexus/buildsupport/nexus-buildsupport-logging/2.14.3-02/nexus-buildsupport-logging-2.14.3-02.pom
#17 17.71 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/nexus/buildsupport/nexus-buildsupport-logging/2.14.3-02/nexus-buildsupport-logging-2.14.3-02.pom (3.3 kB at 8.1 kB/s)
#17 17.73 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/nexus/buildsupport/nexus-buildsupport-osgi/2.14.3-02/nexus-buildsupport-osgi-2.14.3-02.pom
#17 18.12 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/nexus/buildsupport/nexus-buildsupport-osgi/2.14.3-02/nexus-buildsupport-osgi-2.14.3-02.pom (2.0 kB at 5.2 kB/s)
#17 18.13 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/nexus/buildsupport/nexus-buildsupport-other/2.14.3-02/nexus-buildsupport-other-2.14.3-02.pom
#17 18.55 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/nexus/buildsupport/nexus-buildsupport-other/2.14.3-02/nexus-buildsupport-other-2.14.3-02.pom (7.0 kB at 17 kB/s)
#17 18.56 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/nexus/buildsupport/nexus-buildsupport-plexus/2.14.3-02/nexus-buildsupport-plexus-2.14.3-02.pom
#17 18.96 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/nexus/buildsupport/nexus-buildsupport-plexus/2.14.3-02/nexus-buildsupport-plexus-2.14.3-02.pom (4.5 kB at 11 kB/s)
#17 18.97 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/nexus/buildsupport/nexus-buildsupport-rest/2.14.3-02/nexus-buildsupport-rest-2.14.3-02.pom
#17 19.54 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/nexus/buildsupport/nexus-buildsupport-rest/2.14.3-02/nexus-buildsupport-rest-2.14.3-02.pom (5.1 kB at 9.0 kB/s)
#17 19.55 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/nexus/buildsupport/nexus-buildsupport-testing/2.14.3-02/nexus-buildsupport-testing-2.14.3-02.pom
#17 20.05 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/nexus/buildsupport/nexus-buildsupport-testing/2.14.3-02/nexus-buildsupport-testing-2.14.3-02.pom (5.0 kB at 9.9 kB/s)
#17 20.06 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/nexus/buildsupport/nexus-buildsupport-metrics/2.14.3-02/nexus-buildsupport-metrics-2.14.3-02.pom
#17 20.46 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/nexus/buildsupport/nexus-buildsupport-metrics/2.14.3-02/nexus-buildsupport-metrics-2.14.3-02.pom (3.0 kB at 7.6 kB/s)
#17 20.48 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/nexus/plugins/nexus-restlet1x-model/2.14.3-02/nexus-restlet1x-model-2.14.3-02.pom
#17 20.86 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/nexus/plugins/nexus-restlet1x-model/2.14.3-02/nexus-restlet1x-model-2.14.3-02.pom (3.5 kB at 9.3 kB/s)
#17 20.87 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/nexus/plugins/nexus-plugins-restlet1x/2.14.3-02/nexus-plugins-restlet1x-2.14.3-02.pom
#17 21.30 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/nexus/plugins/nexus-plugins-restlet1x/2.14.3-02/nexus-plugins-restlet1x-2.14.3-02.pom (6.0 kB at 14 kB/s)
#17 21.31 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/nexus/plugins/nexus-plugins/2.14.3-02/nexus-plugins-2.14.3-02.pom
#17 21.75 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/nexus/plugins/nexus-plugins/2.14.3-02/nexus-plugins-2.14.3-02.pom (32 kB at 73 kB/s)
#17 21.76 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/maven-model/3.0.4/maven-model-3.0.4.pom
#17 22.15 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/maven-model/3.0.4/maven-model-3.0.4.pom (3.8 kB at 9.9 kB/s)
#17 22.16 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/maven/3.0.4/maven-3.0.4.pom
#17 22.56 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/maven/3.0.4/maven-3.0.4.pom (22 kB at 54 kB/s)
#17 22.58 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/slf4j/slf4j-api/1.7.7/slf4j-api-1.7.7.pom
#17 22.99 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/slf4j/slf4j-api/1.7.7/slf4j-api-1.7.7.pom (2.7 kB at 6.7 kB/s)
#17 22.99 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/slf4j/slf4j-parent/1.7.7/slf4j-parent-1.7.7.pom
#17 23.44 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/slf4j/slf4j-parent/1.7.7/slf4j-parent-1.7.7.pom (12 kB at 27 kB/s)
#17 23.45 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/google/code/findbugs/jsr305/2.0.1/jsr305-2.0.1.pom
#17 23.86 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/google/code/findbugs/jsr305/2.0.1/jsr305-2.0.1.pom (965 B at 2.3 kB/s)
#17 23.87 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/intellij/annotations/9.0.4/annotations-9.0.4.pom
#17 24.34 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/intellij/annotations/9.0.4/annotations-9.0.4.pom (1.2 kB at 2.5 kB/s)
#17 24.35 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/commons-io/commons-io/2.4/commons-io-2.4.pom
#17 24.77 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/commons-io/commons-io/2.4/commons-io-2.4.pom (10 kB at 24 kB/s)
#17 24.78 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/commons/commons-parent/25/commons-parent-25.pom
#17 25.16 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/commons/commons-parent/25/commons-parent-25.pom (48 kB at 126 kB/s)
#17 25.18 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/apache/9/apache-9.pom
#17 25.59 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/apache/9/apache-9.pom (15 kB at 37 kB/s)
#17 25.60 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/thoughtworks/xstream/xstream/1.4.7/xstream-1.4.7.pom
#17 26.05 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/thoughtworks/xstream/xstream/1.4.7/xstream-1.4.7.pom (11 kB at 25 kB/s)
#17 26.06 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/thoughtworks/xstream/xstream-parent/1.4.7/xstream-parent-1.4.7.pom
#17 26.43 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/thoughtworks/xstream/xstream-parent/1.4.7/xstream-parent-1.4.7.pom (26 kB at 71 kB/s)
#17 26.44 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/codehaus/codehaus-parent/3/codehaus-parent-3.pom
#17 26.89 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/codehaus/codehaus-parent/3/codehaus-parent-3.pom (4.1 kB at 9.2 kB/s)
#17 26.90 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/xmlpull/xmlpull/1.1.3.1/xmlpull-1.1.3.1.pom
#17 27.31 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/xmlpull/xmlpull/1.1.3.1/xmlpull-1.1.3.1.pom (386 B at 927 B/s)
#17 27.32 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/xpp3/xpp3_min/1.1.4c/xpp3_min-1.1.4c.pom
#17 27.75 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/xpp3/xpp3_min/1.1.4c/xpp3_min-1.1.4c.pom (1.6 kB at 3.8 kB/s)
#17 27.75 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/joda-time/joda-time/2.2/joda-time-2.2.pom
#17 28.24 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/joda-time/joda-time/2.2/joda-time-2.2.pom (16 kB at 33 kB/s)
#17 28.26 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/commons-beanutils/commons-beanutils-core/1.8.3/commons-beanutils-core-1.8.3.pom
#17 28.68 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/commons-beanutils/commons-beanutils-core/1.8.3/commons-beanutils-core-1.8.3.pom (1.6 kB at 3.9 kB/s)
#17 28.69 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/sisu/siesta/siesta-client/1.7/siesta-client-1.7.pom
#17 29.07 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/sisu/siesta/siesta-client/1.7/siesta-client-1.7.pom (1.8 kB at 4.8 kB/s)
#17 29.08 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/sisu/siesta/siesta/1.7/siesta-1.7.pom
#17 29.47 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/sisu/siesta/siesta/1.7/siesta-1.7.pom (12 kB at 31 kB/s)
#17 29.48 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/buildsupport/public-parent/5/public-parent-5.pom
#17 29.87 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/buildsupport/public-parent/5/public-parent-5.pom (760 B at 2.0 kB/s)
#17 29.87 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/buildsupport/buildsupport/5/buildsupport-5.pom
#17 30.32 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/buildsupport/buildsupport/5/buildsupport-5.pom (22 kB at 50 kB/s)
#17 30.33 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/sisu/siesta/siesta-common/1.7/siesta-common-1.7.pom
#17 30.74 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/sisu/siesta/siesta-common/1.7/siesta-common-1.7.pom (2.1 kB at 5.3 kB/s)
#17 30.75 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/javax/ws/rs/jsr311-api/1.1.1/jsr311-api-1.1.1.pom
#17 31.16 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/javax/ws/rs/jsr311-api/1.1.1/jsr311-api-1.1.1.pom (5.5 kB at 13 kB/s)
#17 31.17 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/sun/jersey/jersey-core/1.17.1/jersey-core-1.17.1.pom
#17 31.54 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/sun/jersey/jersey-core/1.17.1/jersey-core-1.17.1.pom (10 kB at 28 kB/s)
#17 31.55 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/sun/jersey/jersey-project/1.17.1/jersey-project-1.17.1.pom
#17 31.99 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/sun/jersey/jersey-project/1.17.1/jersey-project-1.17.1.pom (21 kB at 48 kB/s)
#17 32.00 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/net/java/jvnet-parent/1/jvnet-parent-1.pom
#17 32.40 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/net/java/jvnet-parent/1/jvnet-parent-1.pom (4.7 kB at 12 kB/s)
#17 32.40 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/javax/validation/validation-api/1.1.0.Final/validation-api-1.1.0.Final.pom
#17 32.87 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/javax/validation/validation-api/1.1.0.Final/validation-api-1.1.0.Final.pom (7.9 kB at 17 kB/s)
#17 32.88 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/sun/jersey/jersey-client/1.17.1/jersey-client-1.17.1.pom
#17 33.34 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/sun/jersey/jersey-client/1.17.1/jersey-client-1.17.1.pom (6.7 kB at 15 kB/s)
#17 33.35 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/sun/jersey/contribs/jersey-apache-client4/1.17.1/jersey-apache-client4-1.17.1.pom
#17 33.77 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/sun/jersey/contribs/jersey-apache-client4/1.17.1/jersey-apache-client4-1.17.1.pom (6.2 kB at 15 kB/s)
#17 33.77 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/sun/jersey/contribs/jersey-contribs/1.17.1/jersey-contribs-1.17.1.pom
#17 34.18 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/sun/jersey/contribs/jersey-contribs/1.17.1/jersey-contribs-1.17.1.pom (3.6 kB at 8.7 kB/s)
#17 34.19 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/httpcomponents/httpclient/4.3.5/httpclient-4.3.5.pom
#17 34.60 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/httpcomponents/httpclient/4.3.5/httpclient-4.3.5.pom (5.2 kB at 13 kB/s)
#17 34.61 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/httpcomponents/httpcomponents-client/4.3.5/httpcomponents-client-4.3.5.pom
#17 35.11 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/httpcomponents/httpcomponents-client/4.3.5/httpcomponents-client-4.3.5.pom (15 kB at 31 kB/s)
#17 35.12 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/httpcomponents/project/7/project-7.pom
#17 35.48 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/httpcomponents/project/7/project-7.pom (27 kB at 76 kB/s)
#17 35.49 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/httpcomponents/httpcore/4.3.2/httpcore-4.3.2.pom
#17 35.86 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/httpcomponents/httpcore/4.3.2/httpcore-4.3.2.pom (4.6 kB at 12 kB/s)
#17 35.87 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/httpcomponents/httpcomponents-core/4.3.2/httpcomponents-core-4.3.2.pom
#17 36.38 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/httpcomponents/httpcomponents-core/4.3.2/httpcomponents-core-4.3.2.pom (12 kB at 24 kB/s)
#17 36.39 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/commons-codec/commons-codec/1.6/commons-codec-1.6.pom
#17 36.83 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/commons-codec/commons-codec/1.6/commons-codec-1.6.pom (11 kB at 25 kB/s)
#17 36.84 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/commons/commons-parent/22/commons-parent-22.pom
#17 37.26 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/commons/commons-parent/22/commons-parent-22.pom (42 kB at 101 kB/s)
#17 37.27 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/slf4j/jcl-over-slf4j/1.7.7/jcl-over-slf4j-1.7.7.pom
#17 37.64 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/slf4j/jcl-over-slf4j/1.7.7/jcl-over-slf4j-1.7.7.pom (1.9 kB at 5.1 kB/s)
#17 37.64 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/sisu/siesta/siesta-jackson/1.7/siesta-jackson-1.7.pom
#17 38.02 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/sisu/siesta/siesta-jackson/1.7/siesta-jackson-1.7.pom (2.6 kB at 6.7 kB/s)
#17 38.04 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/fasterxml/jackson/core/jackson-annotations/2.3.1/jackson-annotations-2.3.1.pom
#17 38.46 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/fasterxml/jackson/core/jackson-annotations/2.3.1/jackson-annotations-2.3.1.pom (1.7 kB at 4.1 kB/s)
#17 38.46 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/fasterxml/oss-parent/11/oss-parent-11.pom
#17 38.98 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/fasterxml/oss-parent/11/oss-parent-11.pom (23 kB at 45 kB/s)
#17 38.99 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/fasterxml/jackson/core/jackson-core/2.3.1/jackson-core-2.3.1.pom
#17 39.41 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/fasterxml/jackson/core/jackson-core/2.3.1/jackson-core-2.3.1.pom (6.0 kB at 14 kB/s)
#17 39.42 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/fasterxml/oss-parent/12/oss-parent-12.pom
#17 39.86 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/fasterxml/oss-parent/12/oss-parent-12.pom (23 kB at 53 kB/s)
#17 39.87 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/fasterxml/jackson/core/jackson-databind/2.3.1/jackson-databind-2.3.1.pom
#17 40.32 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/fasterxml/jackson/core/jackson-databind/2.3.1/jackson-databind-2.3.1.pom (5.9 kB at 13 kB/s)
#17 40.32 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/fasterxml/jackson/core/jackson-annotations/2.3.0/jackson-annotations-2.3.0.pom
#17 40.78 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/fasterxml/jackson/core/jackson-annotations/2.3.0/jackson-annotations-2.3.0.pom (1.3 kB at 2.9 kB/s)
#17 40.79 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/fasterxml/jackson/jaxrs/jackson-jaxrs-json-provider/2.3.1/jackson-jaxrs-json-provider-2.3.1.pom
#17 41.22 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/fasterxml/jackson/jaxrs/jackson-jaxrs-json-provider/2.3.1/jackson-jaxrs-json-provider-2.3.1.pom (3.2 kB at 7.4 kB/s)
#17 41.23 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/fasterxml/jackson/jaxrs/jackson-jaxrs-providers/2.3.1/jackson-jaxrs-providers-2.3.1.pom
#17 41.63 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/fasterxml/jackson/jaxrs/jackson-jaxrs-providers/2.3.1/jackson-jaxrs-providers-2.3.1.pom (3.8 kB at 9.4 kB/s)
#17 41.63 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/fasterxml/jackson/jaxrs/jackson-jaxrs-base/2.3.1/jackson-jaxrs-base-2.3.1.pom
#17 42.03 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/fasterxml/jackson/jaxrs/jackson-jaxrs-base/2.3.1/jackson-jaxrs-base-2.3.1.pom (1.9 kB at 4.8 kB/s)
#17 42.04 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/fasterxml/jackson/module/jackson-module-jaxb-annotations/2.3.1/jackson-module-jaxb-annotations-2.3.1.pom
#17 42.53 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/fasterxml/jackson/module/jackson-module-jaxb-annotations/2.3.1/jackson-module-jaxb-annotations-2.3.1.pom (4.5 kB at 9.2 kB/s)
#17 42.54 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/javax/inject/javax.inject/1/javax.inject-1.pom
#17 42.99 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/javax/inject/javax.inject/1/javax.inject-1.pom (612 B at 1.4 kB/s)
#17 42.99 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/spice/zapper/spice-zapper/1.3/spice-zapper-1.3.pom
#17 43.45 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/spice/zapper/spice-zapper/1.3/spice-zapper-1.3.pom (4.6 kB at 10 kB/s)
#17 43.46 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/fusesource/hawtbuf/hawtbuf-proto/1.9/hawtbuf-proto-1.9.pom
#17 43.89 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/fusesource/hawtbuf/hawtbuf-proto/1.9/hawtbuf-proto-1.9.pom (4.5 kB at 10 kB/s)
#17 43.90 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/fusesource/hawtbuf/hawtbuf-project/1.9/hawtbuf-project-1.9.pom
#17 44.31 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/fusesource/hawtbuf/hawtbuf-project/1.9/hawtbuf-project-1.9.pom (11 kB at 26 kB/s)
#17 44.31 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/fusesource/fusesource-pom/1.9/fusesource-pom-1.9.pom
#17 44.71 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/fusesource/fusesource-pom/1.9/fusesource-pom-1.9.pom (15 kB at 37 kB/s)
#17 44.72 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/fusesource/hawtbuf/hawtbuf/1.9/hawtbuf-1.9.pom
#17 45.10 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/fusesource/hawtbuf/hawtbuf/1.9/hawtbuf-1.9.pom (2.6 kB at 6.9 kB/s)
#17 45.12 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/codehaus/plexus/plexus-interpolation/1.15/plexus-interpolation-1.15.pom
#17 45.49 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/codehaus/plexus/plexus-interpolation/1.15/plexus-interpolation-1.15.pom (1.0 kB at 2.8 kB/s)
#17 45.49 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/codehaus/plexus/plexus/3.0.1/plexus-3.0.1.pom
#17 45.96 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/codehaus/plexus/plexus/3.0.1/plexus-3.0.1.pom (19 kB at 39 kB/s)
#17 45.97 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/aether/aether-api/1.13.1/aether-api-1.13.1.pom
#17 46.38 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/aether/aether-api/1.13.1/aether-api-1.13.1.pom (1.4 kB at 3.5 kB/s)
#17 46.38 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/aether/aether/1.13.1/aether-1.13.1.pom
#17 46.75 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/aether/aether/1.13.1/aether-1.13.1.pom (10 kB at 28 kB/s)
#17 46.76 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/ch/qos/logback/logback-core/1.1.2/logback-core-1.1.2.pom
#17 47.17 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/ch/qos/logback/logback-core/1.1.2/logback-core-1.1.2.pom (5.0 kB at 12 kB/s)
#17 47.18 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/ch/qos/logback/logback-parent/1.1.2/logback-parent-1.1.2.pom
#17 47.59 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/ch/qos/logback/logback-parent/1.1.2/logback-parent-1.1.2.pom (15 kB at 36 kB/s)
#17 47.60 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/ch/qos/logback/logback-classic/1.1.2/logback-classic-1.1.2.pom
#17 48.03 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/ch/qos/logback/logback-classic/1.1.2/logback-classic-1.1.2.pom (12 kB at 27 kB/s)
#17 48.07 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/plugins/nexus-staging-maven-plugin/1.6.8/nexus-staging-maven-plugin-1.6.8.jar
#17 48.08 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/plexus/plexus-cipher/1.7/plexus-cipher-1.7.jar
#17 48.08 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/plexus/plexus-sec-dispatcher/1.4/plexus-sec-dispatcher-1.4.jar
#17 48.08 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/nexus/maven/nexus-common/1.6.8/nexus-common-1.6.8.jar
#17 48.08 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/google/guava/guava/14.0.1/guava-14.0.1.jar
#17 48.60 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/nexus/maven/nexus-common/1.6.8/nexus-common-1.6.8.jar (16 kB at 31 kB/s)
#17 48.60 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/nexus/nexus-client-core/2.14.3-02/nexus-client-core-2.14.3-02.jar
#17 48.62 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/plexus/plexus-cipher/1.7/plexus-cipher-1.7.jar (13 kB at 25 kB/s)
#17 48.62 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/nexus/plugins/nexus-restlet1x-model/2.14.3-02/nexus-restlet1x-model-2.14.3-02.jar
#17 48.68 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/plexus/plexus-sec-dispatcher/1.4/plexus-sec-dispatcher-1.4.jar (28 kB at 46 kB/s)
#17 48.68 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/maven-model/3.0.4/maven-model-3.0.4.jar
#17 48.77 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/plugins/nexus-staging-maven-plugin/1.6.8/nexus-staging-maven-plugin-1.6.8.jar (251 kB at 360 kB/s)
#17 48.77 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/slf4j/slf4j-api/1.7.7/slf4j-api-1.7.7.jar
#17 48.82 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/google/guava/guava/14.0.1/guava-14.0.1.jar (2.2 MB at 2.9 MB/s)
#17 48.82 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/google/code/findbugs/jsr305/2.0.1/jsr305-2.0.1.jar
#17 49.18 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/slf4j/slf4j-api/1.7.7/slf4j-api-1.7.7.jar (29 kB at 27 kB/s)
#17 49.18 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/intellij/annotations/9.0.4/annotations-9.0.4.jar
#17 49.22 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/nexus/nexus-client-core/2.14.3-02/nexus-client-core-2.14.3-02.jar (216 kB at 188 kB/s)
#17 49.22 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/commons-io/commons-io/2.4/commons-io-2.4.jar
#17 49.24 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/google/code/findbugs/jsr305/2.0.1/jsr305-2.0.1.jar (32 kB at 27 kB/s)
#17 49.24 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/thoughtworks/xstream/xstream/1.4.7/xstream-1.4.7.jar
#17 49.26 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/nexus/plugins/nexus-restlet1x-model/2.14.3-02/nexus-restlet1x-model-2.14.3-02.jar (163 kB at 138 kB/s)
#17 49.26 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/xmlpull/xmlpull/1.1.3.1/xmlpull-1.1.3.1.jar
#17 49.41 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/maven-model/3.0.4/maven-model-3.0.4.jar (164 kB at 123 kB/s)
#17 49.41 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/xpp3/xpp3_min/1.1.4c/xpp3_min-1.1.4c.jar
#17 49.54 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/intellij/annotations/9.0.4/annotations-9.0.4.jar (10.0 kB at 6.8 kB/s)
#17 49.54 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/joda-time/joda-time/2.2/joda-time-2.2.jar
#17 49.64 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/commons-io/commons-io/2.4/commons-io-2.4.jar (185 kB at 118 kB/s)
#17 49.64 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/commons-beanutils/commons-beanutils-core/1.8.3/commons-beanutils-core-1.8.3.jar
#17 49.67 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/xmlpull/xmlpull/1.1.3.1/xmlpull-1.1.3.1.jar (7.2 kB at 4.5 kB/s)
#17 49.67 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/sisu/siesta/siesta-client/1.7/siesta-client-1.7.jar
#17 49.83 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/xpp3/xpp3_min/1.1.4c/xpp3_min-1.1.4c.jar (25 kB at 14 kB/s)
#17 49.83 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/sisu/siesta/siesta-common/1.7/siesta-common-1.7.jar
#17 50.02 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/joda-time/joda-time/2.2/joda-time-2.2.jar (574 kB at 295 kB/s)
#17 50.02 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/javax/ws/rs/jsr311-api/1.1.1/jsr311-api-1.1.1.jar
#17 50.04 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/commons-beanutils/commons-beanutils-core/1.8.3/commons-beanutils-core-1.8.3.jar (207 kB at 105 kB/s)
#17 50.04 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/sun/jersey/jersey-core/1.17.1/jersey-core-1.17.1.jar
#17 50.08 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/sisu/siesta/siesta-client/1.7/siesta-client-1.7.jar (20 kB at 10 kB/s)
#17 50.08 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/javax/validation/validation-api/1.1.0.Final/validation-api-1.1.0.Final.jar
#17 50.18 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/thoughtworks/xstream/xstream/1.4.7/xstream-1.4.7.jar (532 kB at 253 kB/s)
#17 50.18 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/sun/jersey/jersey-client/1.17.1/jersey-client-1.17.1.jar
#17 50.21 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/sisu/siesta/siesta-common/1.7/siesta-common-1.7.jar (11 kB at 5.0 kB/s)
#17 50.22 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/sun/jersey/contribs/jersey-apache-client4/1.17.1/jersey-apache-client4-1.17.1.jar
#17 50.42 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/javax/ws/rs/jsr311-api/1.1.1/jsr311-api-1.1.1.jar (46 kB at 20 kB/s)
#17 50.42 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/sisu/siesta/siesta-jackson/1.7/siesta-jackson-1.7.jar
#17 50.49 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/sun/jersey/jersey-core/1.17.1/jersey-core-1.17.1.jar (467 kB at 193 kB/s)
#17 50.49 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/fasterxml/jackson/core/jackson-annotations/2.3.1/jackson-annotations-2.3.1.jar
#17 50.52 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/javax/validation/validation-api/1.1.0.Final/validation-api-1.1.0.Final.jar (64 kB at 26 kB/s)
#17 50.52 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/fasterxml/jackson/core/jackson-core/2.3.1/jackson-core-2.3.1.jar
#17 50.62 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/sun/jersey/contribs/jersey-apache-client4/1.17.1/jersey-apache-client4-1.17.1.jar (16 kB at 6.2 kB/s)
#17 50.62 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/fasterxml/jackson/core/jackson-databind/2.3.1/jackson-databind-2.3.1.jar
#17 50.85 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/sisu/siesta/siesta-jackson/1.7/siesta-jackson-1.7.jar (6.3 kB at 2.3 kB/s)
#17 50.85 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/fasterxml/jackson/jaxrs/jackson-jaxrs-json-provider/2.3.1/jackson-jaxrs-json-provider-2.3.1.jar
#17 50.91 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/sun/jersey/jersey-client/1.17.1/jersey-client-1.17.1.jar (131 kB at 46 kB/s)
#17 50.91 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/fasterxml/jackson/jaxrs/jackson-jaxrs-base/2.3.1/jackson-jaxrs-base-2.3.1.jar
#17 50.92 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/fasterxml/jackson/core/jackson-annotations/2.3.1/jackson-annotations-2.3.1.jar (37 kB at 13 kB/s)
#17 50.92 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/fasterxml/jackson/module/jackson-module-jaxb-annotations/2.3.1/jackson-module-jaxb-annotations-2.3.1.jar
#17 50.93 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/fasterxml/jackson/core/jackson-core/2.3.1/jackson-core-2.3.1.jar (198 kB at 69 kB/s)
#17 50.93 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/httpcomponents/httpclient/4.3.5/httpclient-4.3.5.jar
#17 51.11 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/fasterxml/jackson/core/jackson-databind/2.3.1/jackson-databind-2.3.1.jar (914 kB at 302 kB/s)
#17 51.11 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/commons-codec/commons-codec/1.6/commons-codec-1.6.jar
#17 51.26 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/fasterxml/jackson/jaxrs/jackson-jaxrs-json-provider/2.3.1/jackson-jaxrs-json-provider-2.3.1.jar (16 kB at 5.2 kB/s)
#17 51.26 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/httpcomponents/httpcore/4.3.2/httpcore-4.3.2.jar
#17 51.32 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/fasterxml/jackson/jaxrs/jackson-jaxrs-base/2.3.1/jackson-jaxrs-base-2.3.1.jar (28 kB at 8.5 kB/s)
#17 51.32 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/slf4j/jcl-over-slf4j/1.7.7/jcl-over-slf4j-1.7.7.jar
#17 51.35 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/httpcomponents/httpclient/4.3.5/httpclient-4.3.5.jar (591 kB at 180 kB/s)
#17 51.36 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/javax/inject/javax.inject/1/javax.inject-1.jar
#17 51.37 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/fasterxml/jackson/module/jackson-module-jaxb-annotations/2.3.1/jackson-module-jaxb-annotations-2.3.1.jar (32 kB at 9.6 kB/s)
#17 51.38 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/spice/zapper/spice-zapper/1.3/spice-zapper-1.3.jar
#17 51.52 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/commons-codec/commons-codec/1.6/commons-codec-1.6.jar (233 kB at 68 kB/s)
#17 51.52 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/fusesource/hawtbuf/hawtbuf-proto/1.9/hawtbuf-proto-1.9.jar
#17 51.72 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/slf4j/jcl-over-slf4j/1.7.7/jcl-over-slf4j-1.7.7.jar (17 kB at 4.5 kB/s)
#17 51.72 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/fusesource/hawtbuf/hawtbuf/1.9/hawtbuf-1.9.jar
#17 51.73 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/httpcomponents/httpcore/4.3.2/httpcore-4.3.2.jar (282 kB at 77 kB/s)
#17 51.73 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/codehaus/plexus/plexus-utils/3.0.8/plexus-utils-3.0.8.jar
#17 51.74 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/javax/inject/javax.inject/1/javax.inject-1.jar (2.5 kB at 682 B/s)
#17 51.74 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/codehaus/plexus/plexus-interpolation/1.15/plexus-interpolation-1.15.jar
#17 51.91 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/fusesource/hawtbuf/hawtbuf-proto/1.9/hawtbuf-proto-1.9.jar (27 kB at 7.1 kB/s)
#17 51.91 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/aether/aether-api/1.13.1/aether-api-1.13.1.jar
#17 51.92 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/spice/zapper/spice-zapper/1.3/spice-zapper-1.3.jar (121 kB at 31 kB/s)
#17 51.92 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/ch/qos/logback/logback-core/1.1.2/logback-core-1.1.2.jar
#17 52.08 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/codehaus/plexus/plexus-utils/3.0.8/plexus-utils-3.0.8.jar (232 kB at 58 kB/s)
#17 52.09 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/ch/qos/logback/logback-classic/1.1.2/logback-classic-1.1.2.jar
#17 52.11 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/fusesource/hawtbuf/hawtbuf/1.9/hawtbuf-1.9.jar (50 kB at 12 kB/s)
#17 52.16 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/codehaus/plexus/plexus-interpolation/1.15/plexus-interpolation-1.15.jar (60 kB at 15 kB/s)
#17 52.49 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/ch/qos/logback/logback-classic/1.1.2/logback-classic-1.1.2.jar (271 kB at 61 kB/s)
#17 52.49 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/ch/qos/logback/logback-core/1.1.2/logback-core-1.1.2.jar (428 kB at 97 kB/s)
#17 52.57 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/aether/aether-api/1.13.1/aether-api-1.13.1.jar (90 kB at 20 kB/s)
#17 52.82 [INFO] Inspecting build with total of 1 modules...
#17 52.82 [INFO] Installing Nexus Staging features:
#17 52.82 [INFO]   ... total of 1 executions of maven-deploy-plugin replaced with nexus-staging-maven-plugin
#17 52.84 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/plugins/maven-checkstyle-plugin/3.0.0/maven-checkstyle-plugin-3.0.0.pom
#17 53.22 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/plugins/maven-checkstyle-plugin/3.0.0/maven-checkstyle-plugin-3.0.0.pom (15 kB at 39 kB/s)
#17 53.22 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/plugins/maven-plugins/30/maven-plugins-30.pom
#17 53.59 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/plugins/maven-plugins/30/maven-plugins-30.pom (10 kB at 28 kB/s)
#17 53.59 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/maven-parent/30/maven-parent-30.pom
#17 53.99 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/maven-parent/30/maven-parent-30.pom (41 kB at 106 kB/s)
#17 53.99 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/apache/18/apache-18.pom
#17 54.38 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/apache/18/apache-18.pom (16 kB at 41 kB/s)
#17 54.39 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/plugins/maven-checkstyle-plugin/3.0.0/maven-checkstyle-plugin-3.0.0.jar
#17 54.77 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/plugins/maven-checkstyle-plugin/3.0.0/maven-checkstyle-plugin-3.0.0.jar (108 kB at 279 kB/s)
#17 54.79 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/plugins/maven-shade-plugin/1.5/maven-shade-plugin-1.5.pom
#17 55.22 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/plugins/maven-shade-plugin/1.5/maven-shade-plugin-1.5.pom (5.9 kB at 14 kB/s)
#17 55.23 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/plugins/maven-shade-plugin/1.5/maven-shade-plugin-1.5.jar
#17 55.63 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/plugins/maven-shade-plugin/1.5/maven-shade-plugin-1.5.jar (83 kB at 207 kB/s)
#17 55.64 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/plugins/maven-source-plugin/3.0.1/maven-source-plugin-3.0.1.pom
#17 56.07 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/plugins/maven-source-plugin/3.0.1/maven-source-plugin-3.0.1.pom (5.7 kB at 13 kB/s)
#17 56.08 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/plugins/maven-source-plugin/3.0.1/maven-source-plugin-3.0.1.jar
#17 56.42 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/plugins/maven-source-plugin/3.0.1/maven-source-plugin-3.0.1.jar (31 kB at 94 kB/s)
#17 56.43 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/plugins/maven-javadoc-plugin/3.0.1/maven-javadoc-plugin-3.0.1.pom
#17 56.79 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/plugins/maven-javadoc-plugin/3.0.1/maven-javadoc-plugin-3.0.1.pom (19 kB at 52 kB/s)
#17 56.80 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/plugins/maven-plugins/31/maven-plugins-31.pom
#17 57.18 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/plugins/maven-plugins/31/maven-plugins-31.pom (10 kB at 28 kB/s)
#17 57.19 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/maven-parent/31/maven-parent-31.pom
#17 57.57 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/maven-parent/31/maven-parent-31.pom (43 kB at 114 kB/s)
#17 57.57 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/apache/19/apache-19.pom
#17 57.95 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/apache/19/apache-19.pom (15 kB at 41 kB/s)
#17 57.96 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/plugins/maven-javadoc-plugin/3.0.1/maven-javadoc-plugin-3.0.1.jar
#17 58.38 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/plugins/maven-javadoc-plugin/3.0.1/maven-javadoc-plugin-3.0.1.jar (421 kB at 1.0 MB/s)
#17 58.42 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/net/alchim31/maven/scala-maven-plugin/4.2.0/scala-maven-plugin-4.2.0.pom
#17 58.77 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/net/alchim31/maven/scala-maven-plugin/4.2.0/scala-maven-plugin-4.2.0.pom (24 kB at 70 kB/s)
#17 58.78 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/net/alchim31/maven/scala-maven-plugin/4.2.0/scala-maven-plugin-4.2.0.jar
#17 59.13 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/net/alchim31/maven/scala-maven-plugin/4.2.0/scala-maven-plugin-4.2.0.jar (119 kB at 338 kB/s)
#17 59.16 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/plugins/maven-resources-plugin/2.6/maven-resources-plugin-2.6.pom
#17 59.56 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/plugins/maven-resources-plugin/2.6/maven-resources-plugin-2.6.pom (8.1 kB at 20 kB/s)
#17 59.58 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/plugins/maven-resources-plugin/2.6/maven-resources-plugin-2.6.jar
#17 59.94 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/plugins/maven-resources-plugin/2.6/maven-resources-plugin-2.6.jar (30 kB at 82 kB/s)
#17 59.95 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/plugins/maven-jar-plugin/2.4/maven-jar-plugin-2.4.pom
#17 60.40 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/plugins/maven-jar-plugin/2.4/maven-jar-plugin-2.4.pom (5.8 kB at 13 kB/s)
#17 60.41 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/plugins/maven-jar-plugin/2.4/maven-jar-plugin-2.4.jar
#17 60.82 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/plugins/maven-jar-plugin/2.4/maven-jar-plugin-2.4.jar (34 kB at 82 kB/s)
#17 60.83 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/plugins/maven-compiler-plugin/3.1/maven-compiler-plugin-3.1.pom
#17 61.21 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/plugins/maven-compiler-plugin/3.1/maven-compiler-plugin-3.1.pom (10 kB at 27 kB/s)
#17 61.22 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/plugins/maven-compiler-plugin/3.1/maven-compiler-plugin-3.1.jar
#17 61.60 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/plugins/maven-compiler-plugin/3.1/maven-compiler-plugin-3.1.jar (43 kB at 113 kB/s)
#17 61.61 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/plugins/maven-surefire-plugin/2.12.4/maven-surefire-plugin-2.12.4.pom
#17 62.00 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/plugins/maven-surefire-plugin/2.12.4/maven-surefire-plugin-2.12.4.pom (10 kB at 27 kB/s)
#17 62.00 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/surefire/surefire/2.12.4/surefire-2.12.4.pom
#17 62.49 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/surefire/surefire/2.12.4/surefire-2.12.4.pom (14 kB at 28 kB/s)
#17 62.50 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/plugins/maven-surefire-plugin/2.12.4/maven-surefire-plugin-2.12.4.jar
#17 62.96 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/plugins/maven-surefire-plugin/2.12.4/maven-surefire-plugin-2.12.4.jar (30 kB at 66 kB/s)
#17 63.02 [INFO]
#17 63.02 [INFO] ---------------------< org.mlflow:mlflow-scoring >----------------------
#17 63.02 [INFO] Building MLflow scoring server 2.12.2
#17 63.02 [INFO] --------------------------------[ jar ]---------------------------------
#17 63.03 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/junit/junit/4.12/junit-4.12.pom
#17 63.44 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/junit/junit/4.12/junit-4.12.pom (24 kB at 58 kB/s)
#17 63.45 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/hamcrest/hamcrest-core/1.3/hamcrest-core-1.3.pom
#17 63.82 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/hamcrest/hamcrest-core/1.3/hamcrest-core-1.3.pom (766 B at 2.1 kB/s)
#17 63.82 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/hamcrest/hamcrest-parent/1.3/hamcrest-parent-1.3.pom
#17 64.15 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/hamcrest/hamcrest-parent/1.3/hamcrest-parent-1.3.pom (2.0 kB at 5.9 kB/s)
#17 64.16 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/fasterxml/jackson/dataformat/jackson-dataformat-yaml/2.10.0/jackson-dataformat-yaml-2.10.0.pom
#17 64.55 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/fasterxml/jackson/dataformat/jackson-dataformat-yaml/2.10.0/jackson-dataformat-yaml-2.10.0.pom (2.3 kB at 6.0 kB/s)
#17 64.55 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/fasterxml/jackson/dataformat/jackson-dataformats-text/2.10.0/jackson-dataformats-text-2.10.0.pom
#17 64.95 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/fasterxml/jackson/dataformat/jackson-dataformats-text/2.10.0/jackson-dataformats-text-2.10.0.pom (3.0 kB at 7.5 kB/s)
#17 64.96 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/fasterxml/jackson/jackson-base/2.10.0/jackson-base-2.10.0.pom
#17 65.30 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/fasterxml/jackson/jackson-base/2.10.0/jackson-base-2.10.0.pom (6.6 kB at 19 kB/s)
#17 65.31 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/fasterxml/jackson/jackson-bom/2.10.0/jackson-bom-2.10.0.pom
#17 65.73 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/fasterxml/jackson/jackson-bom/2.10.0/jackson-bom-2.10.0.pom (13 kB at 30 kB/s)
#17 65.73 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/fasterxml/jackson/jackson-parent/2.10/jackson-parent-2.10.pom
#17 66.24 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/fasterxml/jackson/jackson-parent/2.10/jackson-parent-2.10.pom (8.3 kB at 16 kB/s)
#17 66.24 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/fasterxml/oss-parent/38/oss-parent-38.pom
#17 66.60 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/fasterxml/oss-parent/38/oss-parent-38.pom (23 kB at 64 kB/s)
#17 66.61 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/yaml/snakeyaml/1.24/snakeyaml-1.24.pom
#17 67.04 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/yaml/snakeyaml/1.24/snakeyaml-1.24.pom (38 kB at 88 kB/s)
#17 67.05 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/fasterxml/jackson/core/jackson-core/2.10.0/jackson-core-2.10.0.pom
#17 67.47 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/fasterxml/jackson/core/jackson-core/2.10.0/jackson-core-2.10.0.pom (4.6 kB at 11 kB/s)
#17 67.48 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/fasterxml/jackson/core/jackson-databind/2.10.0/jackson-databind-2.10.0.pom
#17 67.90 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/fasterxml/jackson/core/jackson-databind/2.10.0/jackson-databind-2.10.0.pom (7.1 kB at 17 kB/s)
#17 67.91 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/fasterxml/jackson/core/jackson-annotations/2.10.0/jackson-annotations-2.10.0.pom
#17 68.34 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/fasterxml/jackson/core/jackson-annotations/2.10.0/jackson-annotations-2.10.0.pom (2.7 kB at 6.4 kB/s)
#17 68.35 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/ml/combust/mleap/mleap-spark_2.11/0.10.3/mleap-spark_2.11-0.10.3.pom
#17 68.73 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/ml/combust/mleap/mleap-spark_2.11/0.10.3/mleap-spark_2.11-0.10.3.pom (3.5 kB at 9.1 kB/s)
#17 68.73 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/scala-lang/scala-library/2.11.8/scala-library-2.11.8.pom
#17 69.16 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/scala-lang/scala-library/2.11.8/scala-library-2.11.8.pom (1.4 kB at 3.3 kB/s)
#17 69.17 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/ml/combust/mleap/mleap-spark-base_2.11/0.10.3/mleap-spark-base_2.11-0.10.3.pom
#17 69.59 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/ml/combust/mleap/mleap-spark-base_2.11/0.10.3/mleap-spark-base_2.11-0.10.3.pom (3.5 kB at 8.4 kB/s)
#17 69.59 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/ml/combust/mleap/mleap-runtime_2.11/0.10.3/mleap-runtime_2.11-0.10.3.pom
#17 70.01 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/ml/combust/mleap/mleap-runtime_2.11/0.10.3/mleap-runtime_2.11-0.10.3.pom (2.7 kB at 6.5 kB/s)
#17 70.02 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/ml/combust/mleap/mleap-core_2.11/0.10.3/mleap-core_2.11-0.10.3.pom
#17 70.44 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/ml/combust/mleap/mleap-core_2.11/0.10.3/mleap-core_2.11-0.10.3.pom (3.1 kB at 7.3 kB/s)
#17 70.45 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/ml/combust/mleap/mleap-base_2.11/0.10.3/mleap-base_2.11-0.10.3.pom
#17 70.86 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/ml/combust/mleap/mleap-base_2.11/0.10.3/mleap-base_2.11-0.10.3.pom (2.0 kB at 4.8 kB/s)
#17 70.86 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/ml/combust/mleap/mleap-tensor_2.11/0.10.3/mleap-tensor_2.11-0.10.3.pom
#17 71.21 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/ml/combust/mleap/mleap-tensor_2.11/0.10.3/mleap-tensor_2.11-0.10.3.pom (2.5 kB at 7.2 kB/s)
#17 71.22 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/io/spray/spray-json_2.11/1.3.2/spray-json_2.11-1.3.2.pom
#17 71.70 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/io/spray/spray-json_2.11/1.3.2/spray-json_2.11-1.3.2.pom (2.2 kB at 4.5 kB/s)
#17 71.70 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/scala-lang/scala-library/2.11.6/scala-library-2.11.6.pom
#17 72.14 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/scala-lang/scala-library/2.11.6/scala-library-2.11.6.pom (1.4 kB at 3.3 kB/s)
#17 72.14 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/spark/spark-mllib-local_2.11/2.3.0/spark-mllib-local_2.11-2.3.0.pom
#17 72.51 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/spark/spark-mllib-local_2.11/2.3.0/spark-mllib-local_2.11-2.3.0.pom (4.6 kB at 12 kB/s)
#17 72.52 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/spark/spark-parent_2.11/2.3.0/spark-parent_2.11-2.3.0.pom
#17 72.93 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/spark/spark-parent_2.11/2.3.0/spark-parent_2.11-2.3.0.pom (102 kB at 244 kB/s)
#17 72.95 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/scalanlp/breeze_2.11/0.13.2/breeze_2.11-0.13.2.pom
#17 73.34 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/scalanlp/breeze_2.11/0.13.2/breeze_2.11-0.13.2.pom (4.5 kB at 11 kB/s)
#17 73.35 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/scalanlp/breeze-macros_2.11/0.13.2/breeze-macros_2.11-0.13.2.pom
#17 73.77 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/scalanlp/breeze-macros_2.11/0.13.2/breeze-macros_2.11-0.13.2.pom (2.2 kB at 5.1 kB/s)
#17 73.78 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/scala-lang/scala-reflect/2.11.8/scala-reflect-2.11.8.pom
#17 74.12 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/scala-lang/scala-reflect/2.11.8/scala-reflect-2.11.8.pom (1.6 kB at 4.7 kB/s)
#17 74.12 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/github/fommil/netlib/core/1.1.2/core-1.1.2.pom
#17 74.55 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/github/fommil/netlib/core/1.1.2/core-1.1.2.pom (5.4 kB at 13 kB/s)
#17 74.55 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/github/fommil/netlib/parent/1.1/parent-1.1.pom
#17 74.95 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/github/fommil/netlib/parent/1.1/parent-1.1.pom (9.5 kB at 24 kB/s)
#17 74.96 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/net/sourceforge/f2j/arpack_combined_all/0.1/arpack_combined_all-0.1.pom
#17 75.37 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/net/sourceforge/f2j/arpack_combined_all/0.1/arpack_combined_all-0.1.pom (1.3 kB at 3.2 kB/s)
#17 75.38 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/net/sf/opencsv/opencsv/2.3/opencsv-2.3.pom
#17 75.82 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/net/sf/opencsv/opencsv/2.3/opencsv-2.3.pom (5.4 kB at 12 kB/s)
#17 75.82 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/github/rwl/jtransforms/2.4.0/jtransforms-2.4.0.pom
#17 76.22 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/github/rwl/jtransforms/2.4.0/jtransforms-2.4.0.pom (2.8 kB at 7.0 kB/s)
#17 76.23 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/spire-math/spire_2.11/0.13.0/spire_2.11-0.13.0.pom
#17 76.62 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/spire-math/spire_2.11/0.13.0/spire_2.11-0.13.0.pom (2.1 kB at 5.4 kB/s)
#17 76.62 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/spire-math/spire-macros_2.11/0.13.0/spire-macros_2.11-0.13.0.pom
#17 77.03 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/spire-math/spire-macros_2.11/0.13.0/spire-macros_2.11-0.13.0.pom (2.6 kB at 6.3 kB/s)
#17 77.03 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/typelevel/machinist_2.11/0.6.1/machinist_2.11-0.6.1.pom
#17 77.50 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/typelevel/machinist_2.11/0.6.1/machinist_2.11-0.6.1.pom (1.8 kB at 3.8 kB/s)
#17 77.51 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/chuusai/shapeless_2.11/2.3.2/shapeless_2.11-2.3.2.pom
#17 77.89 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/chuusai/shapeless_2.11/2.3.2/shapeless_2.11-2.3.2.pom (2.3 kB at 6.0 kB/s)
#17 77.90 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/typelevel/macro-compat_2.11/1.1.1/macro-compat_2.11-1.1.1.pom
#17 78.72 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/typelevel/macro-compat_2.11/1.1.1/macro-compat_2.11-1.1.1.pom (1.9 kB at 2.3 kB/s)
#17 78.72 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/scala-lang/scala-library/2.11.7/scala-library-2.11.7.pom
#17 79.07 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/scala-lang/scala-library/2.11.7/scala-library-2.11.7.pom (1.4 kB at 4.1 kB/s)
#17 79.07 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/slf4j/slf4j-api/1.7.25/slf4j-api-1.7.25.pom
#17 79.48 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/slf4j/slf4j-api/1.7.25/slf4j-api-1.7.25.pom (3.8 kB at 9.3 kB/s)
#17 79.49 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/slf4j/slf4j-parent/1.7.25/slf4j-parent-1.7.25.pom
#17 79.89 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/slf4j/slf4j-parent/1.7.25/slf4j-parent-1.7.25.pom (14 kB at 34 kB/s)
#17 79.90 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/commons/commons-math3/3.4.1/commons-math3-3.4.1.pom
#17 80.32 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/commons/commons-math3/3.4.1/commons-math3-3.4.1.pom (27 kB at 65 kB/s)
#17 80.33 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/commons/commons-parent/34/commons-parent-34.pom
#17 80.72 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/commons/commons-parent/34/commons-parent-34.pom (56 kB at 140 kB/s)
#17 80.74 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/spark/spark-tags_2.11/2.3.0/spark-tags_2.11-2.3.0.pom
#17 81.15 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/spark/spark-tags_2.11/2.3.0/spark-tags_2.11-2.3.0.pom (2.6 kB at 6.5 kB/s)
#17 81.16 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/spark-project/spark/unused/1.0.0/unused-1.0.0.pom
#17 81.52 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/spark-project/spark/unused/1.0.0/unused-1.0.0.pom (2.4 kB at 6.4 kB/s)
#17 81.53 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/oss/oss-parent/9/oss-parent-9.pom
#17 81.89 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/oss/oss-parent/9/oss-parent-9.pom (6.6 kB at 18 kB/s)
#17 81.90 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/ml/combust/bundle/bundle-ml_2.11/0.10.3/bundle-ml_2.11-0.10.3.pom
#17 82.29 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/ml/combust/bundle/bundle-ml_2.11/0.10.3/bundle-ml_2.11-0.10.3.pom (3.2 kB at 8.3 kB/s)
#17 82.30 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/trueaccord/scalapb/scalapb-runtime_2.11/0.6.0/scalapb-runtime_2.11-0.6.0.pom
#17 82.71 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/trueaccord/scalapb/scalapb-runtime_2.11/0.6.0/scalapb-runtime_2.11-0.6.0.pom (2.6 kB at 6.4 kB/s)
#17 82.71 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/scala-lang/scala-library/2.11.11/scala-library-2.11.11.pom
#17 83.13 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/scala-lang/scala-library/2.11.11/scala-library-2.11.11.pom (1.4 kB at 3.4 kB/s)
#17 83.14 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/trueaccord/lenses/lenses_2.11/0.4.12/lenses_2.11-0.4.12.pom
#17 83.52 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/trueaccord/lenses/lenses_2.11/0.4.12/lenses_2.11-0.4.12.pom (1.9 kB at 4.9 kB/s)
#17 83.53 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/lihaoyi/fastparse_2.11/0.4.2/fastparse_2.11-0.4.2.pom
#17 83.99 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/lihaoyi/fastparse_2.11/0.4.2/fastparse_2.11-0.4.2.pom (2.6 kB at 5.6 kB/s)
#17 83.99 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/lihaoyi/fastparse-utils_2.11/0.4.2/fastparse-utils_2.11-0.4.2.pom
#17 84.38 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/lihaoyi/fastparse-utils_2.11/0.4.2/fastparse-utils_2.11-0.4.2.pom (2.4 kB at 6.2 kB/s)
#17 84.38 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/lihaoyi/sourcecode_2.11/0.1.3/sourcecode_2.11-0.1.3.pom
#17 84.81 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/lihaoyi/sourcecode_2.11/0.1.3/sourcecode_2.11-0.1.3.pom (1.8 kB at 4.2 kB/s)
#17 84.82 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/google/protobuf/protobuf-java/3.19.4/protobuf-java-3.19.4.pom
#17 84.99 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/com/google/protobuf/protobuf-java/3.19.4/protobuf-java-3.19.4.pom
#17 85.03 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/com/google/protobuf/protobuf-java/3.19.4/protobuf-java-3.19.4.pom (5.4 kB at 155 kB/s)
#17 85.03 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/google/protobuf/protobuf-parent/3.19.4/protobuf-parent-3.19.4.pom
#17 85.22 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/com/google/protobuf/protobuf-parent/3.19.4/protobuf-parent-3.19.4.pom
#17 85.26 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/com/google/protobuf/protobuf-parent/3.19.4/protobuf-parent-3.19.4.pom (8.7 kB at 211 kB/s)
#17 85.26 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/google/protobuf/protobuf-bom/3.19.4/protobuf-bom-3.19.4.pom
#17 85.44 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/com/google/protobuf/protobuf-bom/3.19.4/protobuf-bom-3.19.4.pom
#17 85.47 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/com/google/protobuf/protobuf-bom/3.19.4/protobuf-bom-3.19.4.pom (3.5 kB at 107 kB/s)
#17 85.48 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/jsuereth/scala-arm_2.11/2.0/scala-arm_2.11-2.0.pom
#17 85.96 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/jsuereth/scala-arm_2.11/2.0/scala-arm_2.11-2.0.pom (2.3 kB at 4.7 kB/s)
#17 85.97 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/typesafe/config/1.3.0/config-1.3.0.pom
#17 86.39 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/typesafe/config/1.3.0/config-1.3.0.pom (1.9 kB at 4.4 kB/s)
#17 86.40 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/plugins/maven-surefire-plugin/2.22.0/maven-surefire-plugin-2.22.0.pom
#17 86.88 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/plugins/maven-surefire-plugin/2.22.0/maven-surefire-plugin-2.22.0.pom (5.0 kB at 11 kB/s)
#17 86.88 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/surefire/surefire/2.22.0/surefire-2.22.0.pom
#17 87.31 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/surefire/surefire/2.22.0/surefire-2.22.0.pom (27 kB at 65 kB/s)
#17 87.32 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/surefire/maven-surefire-common/2.22.0/maven-surefire-common-2.22.0.pom
#17 87.70 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/surefire/maven-surefire-common/2.22.0/maven-surefire-common-2.22.0.pom (11 kB at 30 kB/s)
#17 87.71 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/maven-plugin-api/2.2.1/maven-plugin-api-2.2.1.pom
#17 88.12 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/maven-plugin-api/2.2.1/maven-plugin-api-2.2.1.pom (1.5 kB at 3.6 kB/s)
#17 88.12 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/maven/2.2.1/maven-2.2.1.pom
#17 88.54 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/maven/2.2.1/maven-2.2.1.pom (22 kB at 53 kB/s)
#17 88.55 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/plugin-tools/maven-plugin-annotations/3.5/maven-plugin-annotations-3.5.pom
#17 88.92 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/plugin-tools/maven-plugin-annotations/3.5/maven-plugin-annotations-3.5.pom (1.7 kB at 4.5 kB/s)
#17 88.92 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/plugin-tools/maven-plugin-tools/3.5/maven-plugin-tools-3.5.pom
#17 89.33 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/plugin-tools/maven-plugin-tools/3.5/maven-plugin-tools-3.5.pom (16 kB at 40 kB/s)
#17 89.34 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/maven-artifact/3.0/maven-artifact-3.0.pom
#17 89.70 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/maven-artifact/3.0/maven-artifact-3.0.pom (1.9 kB at 5.4 kB/s)
#17 89.70 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/maven/3.0/maven-3.0.pom
#17 90.06 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/maven/3.0/maven-3.0.pom (22 kB at 61 kB/s)
#17 90.07 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/codehaus/plexus/plexus-utils/2.0.4/plexus-utils-2.0.4.pom
#17 90.46 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/codehaus/plexus/plexus-utils/2.0.4/plexus-utils-2.0.4.pom (3.3 kB at 8.4 kB/s)
#17 90.47 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/codehaus/plexus/plexus/2.0.6/plexus-2.0.6.pom
#17 90.88 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/codehaus/plexus/plexus/2.0.6/plexus-2.0.6.pom (17 kB at 40 kB/s)
#17 90.89 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/surefire/surefire-api/2.22.0/surefire-api-2.22.0.pom
#17 91.30 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/surefire/surefire-api/2.22.0/surefire-api-2.22.0.pom (3.5 kB at 8.6 kB/s)
#17 91.31 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/surefire/surefire-logger-api/2.22.0/surefire-logger-api-2.22.0.pom
#17 91.72 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/surefire/surefire-logger-api/2.22.0/surefire-logger-api-2.22.0.pom (2.0 kB at 4.8 kB/s)
#17 91.72 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/surefire/surefire-booter/2.22.0/surefire-booter-2.22.0.pom
#17 92.17 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/surefire/surefire-booter/2.22.0/surefire-booter-2.22.0.pom (7.4 kB at 17 kB/s)
#17 92.18 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/maven-artifact/2.2.1/maven-artifact-2.2.1.pom
#17 92.57 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/maven-artifact/2.2.1/maven-artifact-2.2.1.pom (1.6 kB at 4.0 kB/s)
#17 92.58 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/maven-plugin-descriptor/2.2.1/maven-plugin-descriptor-2.2.1.pom
#17 93.02 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/maven-plugin-descriptor/2.2.1/maven-plugin-descriptor-2.2.1.pom (2.1 kB at 4.7 kB/s)
#17 93.04 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/maven-project/2.2.1/maven-project-2.2.1.pom
#17 93.49 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/maven-project/2.2.1/maven-project-2.2.1.pom (2.8 kB at 6.1 kB/s)
#17 93.49 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/maven-settings/2.2.1/maven-settings-2.2.1.pom
#17 93.90 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/maven-settings/2.2.1/maven-settings-2.2.1.pom (2.2 kB at 5.3 kB/s)
#17 93.91 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/maven-model/2.2.1/maven-model-2.2.1.pom
#17 94.36 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/maven-model/2.2.1/maven-model-2.2.1.pom (3.2 kB at 7.3 kB/s)
#17 94.37 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/maven-profile/2.2.1/maven-profile-2.2.1.pom
#17 94.88 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/maven-profile/2.2.1/maven-profile-2.2.1.pom (2.2 kB at 4.2 kB/s)
#17 94.89 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/maven-artifact-manager/2.2.1/maven-artifact-manager-2.2.1.pom
#17 95.44 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/maven-artifact-manager/2.2.1/maven-artifact-manager-2.2.1.pom (3.1 kB at 5.6 kB/s)
#17 95.45 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/maven-repository-metadata/2.2.1/maven-repository-metadata-2.2.1.pom
#17 95.88 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/maven-repository-metadata/2.2.1/maven-repository-metadata-2.2.1.pom (1.9 kB at 4.4 kB/s)
#17 95.89 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/wagon/wagon-provider-api/1.0-beta-6/wagon-provider-api-1.0-beta-6.pom
#17 96.29 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/wagon/wagon-provider-api/1.0-beta-6/wagon-provider-api-1.0-beta-6.pom (1.8 kB at 4.4 kB/s)
#17 96.29 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/wagon/wagon/1.0-beta-6/wagon-1.0-beta-6.pom
#17 96.69 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/wagon/wagon/1.0-beta-6/wagon-1.0-beta-6.pom (12 kB at 31 kB/s)
#17 96.70 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/codehaus/plexus/plexus-utils/1.4.2/plexus-utils-1.4.2.pom
#17 97.09 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/codehaus/plexus/plexus-utils/1.4.2/plexus-utils-1.4.2.pom (2.0 kB at 5.0 kB/s)
#17 97.10 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/maven-plugin-registry/2.2.1/maven-plugin-registry-2.2.1.pom
#17 97.46 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/maven-plugin-registry/2.2.1/maven-plugin-registry-2.2.1.pom (1.9 kB at 5.3 kB/s)
#17 97.47 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/maven-core/2.2.1/maven-core-2.2.1.pom
#17 97.81 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/maven-core/2.2.1/maven-core-2.2.1.pom (12 kB at 34 kB/s)
#17 97.82 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/maven-plugin-parameter-documenter/2.2.1/maven-plugin-parameter-documenter-2.2.1.pom
#17 98.23 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/maven-plugin-parameter-documenter/2.2.1/maven-plugin-parameter-documenter-2.2.1.pom (2.0 kB at 4.7 kB/s)
#17 98.24 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/wagon/wagon-http/1.0-beta-6/wagon-http-1.0-beta-6.pom
#17 98.61 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/wagon/wagon-http/1.0-beta-6/wagon-http-1.0-beta-6.pom (3.3 kB at 9.0 kB/s)
#17 98.62 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/wagon/wagon-providers/1.0-beta-6/wagon-providers-1.0-beta-6.pom
#17 99.00 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/wagon/wagon-providers/1.0-beta-6/wagon-providers-1.0-beta-6.pom (2.1 kB at 5.6 kB/s)
#17 99.00 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/wagon/wagon-http-shared/1.0-beta-6/wagon-http-shared-1.0-beta-6.pom
#17 99.45 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/wagon/wagon-http-shared/1.0-beta-6/wagon-http-shared-1.0-beta-6.pom (2.1 kB at 4.7 kB/s)
#17 99.45 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/nekohtml/xercesMinimal/1.9.6.2/xercesMinimal-1.9.6.2.pom
#17 99.83 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/nekohtml/xercesMinimal/1.9.6.2/xercesMinimal-1.9.6.2.pom (390 B at 1.0 kB/s)
#17 99.84 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/nekohtml/nekohtml/1.9.6.2/nekohtml-1.9.6.2.pom
#17 100.3 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/nekohtml/nekohtml/1.9.6.2/nekohtml-1.9.6.2.pom (704 B at 1.6 kB/s)
#17 100.3 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/commons-httpclient/commons-httpclient/3.1/commons-httpclient-3.1.pom
#17 100.7 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/commons-httpclient/commons-httpclient/3.1/commons-httpclient-3.1.pom (7.8 kB at 20 kB/s)
#17 100.7 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/commons-codec/commons-codec/1.2/commons-codec-1.2.pom
#17 101.0 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/commons-codec/commons-codec/1.2/commons-codec-1.2.pom (3.8 kB at 9.8 kB/s)
#17 101.1 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/wagon/wagon-webdav-jackrabbit/1.0-beta-6/wagon-webdav-jackrabbit-1.0-beta-6.pom
#17 101.5 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/wagon/wagon-webdav-jackrabbit/1.0-beta-6/wagon-webdav-jackrabbit-1.0-beta-6.pom (3.5 kB at 8.4 kB/s)
#17 101.5 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/jackrabbit/jackrabbit-webdav/1.5.0/jackrabbit-webdav-1.5.0.pom
#17 101.9 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/jackrabbit/jackrabbit-webdav/1.5.0/jackrabbit-webdav-1.5.0.pom (3.3 kB at 8.2 kB/s)
#17 101.9 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/jackrabbit/jackrabbit-parent/1.5.0/jackrabbit-parent-1.5.0.pom
#17 102.3 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/jackrabbit/jackrabbit-parent/1.5.0/jackrabbit-parent-1.5.0.pom (25 kB at 59 kB/s)
#17 102.3 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/jackrabbit/jackrabbit-jcr-commons/1.5.0/jackrabbit-jcr-commons-1.5.0.pom
#17 102.7 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/jackrabbit/jackrabbit-jcr-commons/1.5.0/jackrabbit-jcr-commons-1.5.0.pom (3.0 kB at 7.2 kB/s)
#17 102.7 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/commons-httpclient/commons-httpclient/3.0/commons-httpclient-3.0.pom
#17 103.2 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/commons-httpclient/commons-httpclient/3.0/commons-httpclient-3.0.pom (8.0 kB at 19 kB/s)
#17 103.2 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/slf4j/slf4j-jdk14/1.7.25/slf4j-jdk14-1.7.25.pom
#17 103.6 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/slf4j/slf4j-jdk14/1.7.25/slf4j-jdk14-1.7.25.pom (1.0 kB at 2.7 kB/s)
#17 103.6 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/reporting/maven-reporting-api/2.2.1/maven-reporting-api-2.2.1.pom
#17 104.0 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/reporting/maven-reporting-api/2.2.1/maven-reporting-api-2.2.1.pom (1.9 kB at 4.5 kB/s)
#17 104.0 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/reporting/maven-reporting/2.2.1/maven-reporting-2.2.1.pom
#17 104.4 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/reporting/maven-reporting/2.2.1/maven-reporting-2.2.1.pom (1.4 kB at 3.4 kB/s)
#17 104.4 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/doxia/doxia-logging-api/1.1/doxia-logging-api-1.1.pom
#17 104.8 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/doxia/doxia-logging-api/1.1/doxia-logging-api-1.1.pom (1.6 kB at 3.8 kB/s)
#17 104.8 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/doxia/doxia/1.1/doxia-1.1.pom
#17 105.2 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/doxia/doxia/1.1/doxia-1.1.pom (15 kB at 42 kB/s)
#17 105.2 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/maven-error-diagnostics/2.2.1/maven-error-diagnostics-2.2.1.pom
#17 105.6 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/maven-error-diagnostics/2.2.1/maven-error-diagnostics-2.2.1.pom (1.7 kB at 4.0 kB/s)
#17 105.6 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/maven-monitor/2.2.1/maven-monitor-2.2.1.pom
#17 106.0 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/maven-monitor/2.2.1/maven-monitor-2.2.1.pom (1.3 kB at 3.1 kB/s)
#17 106.1 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/plexus/plexus-sec-dispatcher/1.3/plexus-sec-dispatcher-1.3.pom
#17 106.5 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/plexus/plexus-sec-dispatcher/1.3/plexus-sec-dispatcher-1.3.pom (3.0 kB at 7.0 kB/s)
#17 106.5 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/codehaus/plexus/plexus-utils/1.5.5/plexus-utils-1.5.5.pom
#17 106.9 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/codehaus/plexus/plexus-utils/1.5.5/plexus-utils-1.5.5.pom (5.1 kB at 14 kB/s)
#17 106.9 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/plexus/plexus-cipher/1.4/plexus-cipher-1.4.pom
#17 107.2 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/plexus/plexus-cipher/1.4/plexus-cipher-1.4.pom (2.1 kB at 5.6 kB/s)
#17 107.3 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/maven-toolchain/2.2.1/maven-toolchain-2.2.1.pom
#17 107.7 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/maven-toolchain/2.2.1/maven-toolchain-2.2.1.pom (3.3 kB at 7.9 kB/s)
#17 107.7 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/wagon/wagon-file/1.0-beta-6/wagon-file-1.0-beta-6.pom
#17 108.1 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/wagon/wagon-file/1.0-beta-6/wagon-file-1.0-beta-6.pom (1.4 kB at 3.7 kB/s)
#17 108.1 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/wagon/wagon-http-lightweight/1.0-beta-6/wagon-http-lightweight-1.0-beta-6.pom
#17 108.5 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/wagon/wagon-http-lightweight/1.0-beta-6/wagon-http-lightweight-1.0-beta-6.pom (1.8 kB at 4.3 kB/s)
#17 108.5 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/doxia/doxia-sink-api/1.1/doxia-sink-api-1.1.pom
#17 108.9 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/doxia/doxia-sink-api/1.1/doxia-sink-api-1.1.pom (2.0 kB at 5.0 kB/s)
#17 108.9 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/commons-cli/commons-cli/1.2/commons-cli-1.2.pom
#17 109.3 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/commons-cli/commons-cli/1.2/commons-cli-1.2.pom (8.0 kB at 20 kB/s)
#17 109.3 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/commons/commons-parent/11/commons-parent-11.pom
#17 109.7 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/commons/commons-parent/11/commons-parent-11.pom (25 kB at 68 kB/s)
#17 109.7 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/wagon/wagon-ssh-external/1.0-beta-6/wagon-ssh-external-1.0-beta-6.pom
#17 110.1 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/wagon/wagon-ssh-external/1.0-beta-6/wagon-ssh-external-1.0-beta-6.pom (2.6 kB at 6.5 kB/s)
#17 110.1 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/wagon/wagon-ssh-common/1.0-beta-6/wagon-ssh-common-1.0-beta-6.pom
#17 110.5 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/wagon/wagon-ssh-common/1.0-beta-6/wagon-ssh-common-1.0-beta-6.pom (1.8 kB at 4.2 kB/s)
#17 110.5 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/codehaus/plexus/plexus-interactivity-api/1.0-alpha-6/plexus-interactivity-api-1.0-alpha-6.pom
#17 110.9 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/codehaus/plexus/plexus-interactivity-api/1.0-alpha-6/plexus-interactivity-api-1.0-alpha-6.pom (726 B at 1.9 kB/s)
#17 110.9 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/codehaus/plexus/plexus-interactivity/1.0-alpha-6/plexus-interactivity-1.0-alpha-6.pom
#17 111.4 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/codehaus/plexus/plexus-interactivity/1.0-alpha-6/plexus-interactivity-1.0-alpha-6.pom (1.1 kB at 2.5 kB/s)
#17 111.4 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/codehaus/plexus/plexus-components/1.1.9/plexus-components-1.1.9.pom
#17 111.7 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/codehaus/plexus/plexus-components/1.1.9/plexus-components-1.1.9.pom (2.4 kB at 6.2 kB/s)
#17 111.8 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/codehaus/plexus/plexus-utils/1.4/plexus-utils-1.4.pom
#17 112.1 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/codehaus/plexus/plexus-utils/1.4/plexus-utils-1.4.pom (1.2 kB at 3.4 kB/s)
#17 112.1 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/wagon/wagon-ssh/1.0-beta-6/wagon-ssh-1.0-beta-6.pom
#17 112.5 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/wagon/wagon-ssh/1.0-beta-6/wagon-ssh-1.0-beta-6.pom (2.9 kB at 7.2 kB/s)
#17 112.5 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/jcraft/jsch/0.1.38/jsch-0.1.38.pom
#17 113.0 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/jcraft/jsch/0.1.38/jsch-0.1.38.pom (967 B at 2.0 kB/s)
#17 113.0 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/codehaus/plexus/plexus-java/0.9.8/plexus-java-0.9.8.pom
#17 113.4 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/codehaus/plexus/plexus-java/0.9.8/plexus-java-0.9.8.pom (2.4 kB at 5.5 kB/s)
#17 113.4 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/codehaus/plexus/plexus-languages/0.9.8/plexus-languages-0.9.8.pom
#17 113.8 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/codehaus/plexus/plexus-languages/0.9.8/plexus-languages-0.9.8.pom (2.2 kB at 5.8 kB/s)
#17 113.8 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/codehaus/plexus/plexus/4.0/plexus-4.0.pom
#17 114.2 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/codehaus/plexus/plexus/4.0/plexus-4.0.pom (22 kB at 59 kB/s)
#17 114.2 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/ow2/asm/asm/6.1.1/asm-6.1.1.pom
#17 114.6 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/ow2/asm/asm/6.1.1/asm-6.1.1.pom (2.9 kB at 7.5 kB/s)
#17 114.6 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/ow2/ow2/1.5/ow2-1.5.pom
#17 115.0 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/ow2/ow2/1.5/ow2-1.5.pom (11 kB at 30 kB/s)
#17 115.0 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/thoughtworks/qdox/qdox/2.0-M7/qdox-2.0-M7.pom
#17 115.4 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/thoughtworks/qdox/qdox/2.0-M7/qdox-2.0-M7.pom (16 kB at 41 kB/s)
#17 115.4 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/eclipse/jetty/jetty-server/9.4.11.v20180605/jetty-server-9.4.11.v20180605.pom
#17 115.8 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/eclipse/jetty/jetty-server/9.4.11.v20180605/jetty-server-9.4.11.v20180605.pom (2.6 kB at 6.1 kB/s)
#17 115.8 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/eclipse/jetty/jetty-project/9.4.11.v20180605/jetty-project-9.4.11.v20180605.pom
#17 116.2 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/eclipse/jetty/jetty-project/9.4.11.v20180605/jetty-project-9.4.11.v20180605.pom (63 kB at 157 kB/s)
#17 116.2 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/javax/servlet/javax.servlet-api/3.1.0/javax.servlet-api-3.1.0.pom
#17 116.6 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/javax/servlet/javax.servlet-api/3.1.0/javax.servlet-api-3.1.0.pom (14 kB at 33 kB/s)
#17 116.6 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/net/java/jvnet-parent/3/jvnet-parent-3.pom
#17 117.0 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/net/java/jvnet-parent/3/jvnet-parent-3.pom (4.8 kB at 12 kB/s)
#17 117.0 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/eclipse/jetty/jetty-http/9.4.11.v20180605/jetty-http-9.4.11.v20180605.pom
#17 117.4 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/eclipse/jetty/jetty-http/9.4.11.v20180605/jetty-http-9.4.11.v20180605.pom (4.7 kB at 13 kB/s)
#17 117.4 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/eclipse/jetty/jetty-util/9.4.11.v20180605/jetty-util-9.4.11.v20180605.pom
#17 117.8 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/eclipse/jetty/jetty-util/9.4.11.v20180605/jetty-util-9.4.11.v20180605.pom (5.7 kB at 15 kB/s)
#17 117.8 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/eclipse/jetty/jetty-io/9.4.11.v20180605/jetty-io-9.4.11.v20180605.pom
#17 118.2 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/eclipse/jetty/jetty-io/9.4.11.v20180605/jetty-io-9.4.11.v20180605.pom (1.3 kB at 3.4 kB/s)
#17 118.2 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/eclipse/jetty/jetty-servlet/9.4.11.v20180605/jetty-servlet-9.4.11.v20180605.pom
#17 118.7 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/eclipse/jetty/jetty-servlet/9.4.11.v20180605/jetty-servlet-9.4.11.v20180605.pom (2.1 kB at 4.3 kB/s)
#17 118.7 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/eclipse/jetty/jetty-security/9.4.11.v20180605/jetty-security-9.4.11.v20180605.pom
#17 119.1 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/eclipse/jetty/jetty-security/9.4.11.v20180605/jetty-security-9.4.11.v20180605.pom (1.3 kB at 3.7 kB/s)
#17 119.1 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/httpcomponents/httpclient/4.5.6/httpclient-4.5.6.pom
#17 119.6 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/httpcomponents/httpclient/4.5.6/httpclient-4.5.6.pom (6.6 kB at 13 kB/s)
#17 119.6 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/httpcomponents/httpcomponents-client/4.5.6/httpcomponents-client-4.5.6.pom
#17 120.0 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/httpcomponents/httpcomponents-client/4.5.6/httpcomponents-client-4.5.6.pom (15 kB at 41 kB/s)
#17 120.0 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/httpcomponents/httpcomponents-parent/10/httpcomponents-parent-10.pom
#17 120.4 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/httpcomponents/httpcomponents-parent/10/httpcomponents-parent-10.pom (34 kB at 76 kB/s)
#17 120.4 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/httpcomponents/httpcore/4.4.10/httpcore-4.4.10.pom
#17 120.8 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/httpcomponents/httpcore/4.4.10/httpcore-4.4.10.pom (5.4 kB at 15 kB/s)
#17 120.8 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/httpcomponents/httpcomponents-core/4.4.10/httpcomponents-core-4.4.10.pom
#17 121.2 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/httpcomponents/httpcomponents-core/4.4.10/httpcomponents-core-4.4.10.pom (13 kB at 38 kB/s)
#17 121.2 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/commons-logging/commons-logging/1.2/commons-logging-1.2.pom
#17 121.6 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/commons-logging/commons-logging/1.2/commons-logging-1.2.pom (19 kB at 47 kB/s)
#17 121.6 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/commons-codec/commons-codec/1.10/commons-codec-1.10.pom
#17 122.1 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/commons-codec/commons-codec/1.10/commons-codec-1.10.pom (12 kB at 25 kB/s)
#17 122.1 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/commons/commons-parent/35/commons-parent-35.pom
#17 122.5 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/commons/commons-parent/35/commons-parent-35.pom (58 kB at 136 kB/s)
#17 122.5 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/apache/15/apache-15.pom
#17 122.9 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/apache/15/apache-15.pom (15 kB at 37 kB/s)
#17 122.9 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/commons-io/commons-io/2.5/commons-io-2.5.pom
#17 123.3 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/commons-io/commons-io/2.5/commons-io-2.5.pom (13 kB at 34 kB/s)
#17 123.3 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/commons/commons-parent/39/commons-parent-39.pom
#17 123.7 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/commons/commons-parent/39/commons-parent-39.pom (62 kB at 159 kB/s)
#17 123.7 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/apache/16/apache-16.pom
#17 124.1 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/apache/16/apache-16.pom (15 kB at 39 kB/s)
#17 124.1 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/junit/junit/4.12/junit-4.12.jar
#17 124.1 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/fasterxml/jackson/dataformat/jackson-dataformat-yaml/2.10.0/jackson-dataformat-yaml-2.10.0.jar
#17 124.1 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/hamcrest/hamcrest-core/1.3/hamcrest-core-1.3.jar
#17 124.1 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/fasterxml/jackson/core/jackson-core/2.10.0/jackson-core-2.10.0.jar
#17 124.1 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/yaml/snakeyaml/1.24/snakeyaml-1.24.jar
#17 124.5 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/junit/junit/4.12/junit-4.12.jar (315 kB at 791 kB/s)
#17 124.5 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/fasterxml/jackson/core/jackson-databind/2.10.0/jackson-databind-2.10.0.jar
#17 124.6 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/fasterxml/jackson/dataformat/jackson-dataformat-yaml/2.10.0/jackson-dataformat-yaml-2.10.0.jar (47 kB at 104 kB/s)
#17 124.6 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/fasterxml/jackson/core/jackson-annotations/2.10.0/jackson-annotations-2.10.0.jar
#17 124.6 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/hamcrest/hamcrest-core/1.3/hamcrest-core-1.3.jar (45 kB at 91 kB/s)
#17 124.6 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/ml/combust/mleap/mleap-spark_2.11/0.10.3/mleap-spark_2.11-0.10.3.jar
#17 124.7 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/yaml/snakeyaml/1.24/snakeyaml-1.24.jar (303 kB at 575 kB/s)
#17 124.7 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/scala-lang/scala-library/2.11.8/scala-library-2.11.8.jar
#17 124.7 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/fasterxml/jackson/core/jackson-core/2.10.0/jackson-core-2.10.0.jar (349 kB at 606 kB/s)
#17 124.7 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/ml/combust/mleap/mleap-spark-base_2.11/0.10.3/mleap-spark-base_2.11-0.10.3.jar
#17 125.0 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/fasterxml/jackson/core/jackson-annotations/2.10.0/jackson-annotations-2.10.0.jar (68 kB at 80 kB/s)
#17 125.0 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/ml/combust/mleap/mleap-runtime_2.11/0.10.3/mleap-runtime_2.11-0.10.3.jar
#17 125.0 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/fasterxml/jackson/core/jackson-databind/2.10.0/jackson-databind-2.10.0.jar (1.4 MB at 1.6 MB/s)
#17 125.0 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/ml/combust/mleap/mleap-core_2.11/0.10.3/mleap-core_2.11-0.10.3.jar
#17 125.1 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/ml/combust/mleap/mleap-spark-base_2.11/0.10.3/mleap-spark-base_2.11-0.10.3.jar (140 kB at 146 kB/s)
#17 125.1 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/ml/combust/mleap/mleap-base_2.11/0.10.3/mleap-base_2.11-0.10.3.jar
#17 125.1 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/ml/combust/mleap/mleap-spark_2.11/0.10.3/mleap-spark_2.11-0.10.3.jar (393 kB at 405 kB/s)
#17 125.1 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/ml/combust/mleap/mleap-tensor_2.11/0.10.3/mleap-tensor_2.11-0.10.3.jar
#17 125.3 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/scala-lang/scala-library/2.11.8/scala-library-2.11.8.jar (5.7 MB at 4.9 MB/s)
#17 125.3 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/spark/spark-mllib-local_2.11/2.3.0/spark-mllib-local_2.11-2.3.0.jar
#17 125.5 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/ml/combust/mleap/mleap-core_2.11/0.10.3/mleap-core_2.11-0.10.3.jar (986 kB at 734 kB/s)
#17 125.5 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/scalanlp/breeze_2.11/0.13.2/breeze_2.11-0.13.2.jar
#17 125.5 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/ml/combust/mleap/mleap-base_2.11/0.10.3/mleap-base_2.11-0.10.3.jar (17 kB at 13 kB/s)
#17 125.5 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/scalanlp/breeze-macros_2.11/0.13.2/breeze-macros_2.11-0.13.2.jar
#17 125.5 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/ml/combust/mleap/mleap-tensor_2.11/0.10.3/mleap-tensor_2.11-0.10.3.jar (39 kB at 28 kB/s)
#17 125.6 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/github/fommil/netlib/core/1.1.2/core-1.1.2.jar
#17 125.6 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/ml/combust/mleap/mleap-runtime_2.11/0.10.3/mleap-runtime_2.11-0.10.3.jar (1.4 MB at 924 kB/s)
#17 125.6 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/net/sourceforge/f2j/arpack_combined_all/0.1/arpack_combined_all-0.1.jar
#17 125.7 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/spark/spark-mllib-local_2.11/2.3.0/spark-mllib-local_2.11-2.3.0.jar (184 kB at 115 kB/s)
#17 125.7 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/net/sf/opencsv/opencsv/2.3/opencsv-2.3.jar
#17 125.9 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/scalanlp/breeze-macros_2.11/0.13.2/breeze-macros_2.11-0.13.2.jar (188 kB at 106 kB/s)
#17 125.9 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/spire-math/spire_2.11/0.13.0/spire_2.11-0.13.0.jar
#17 126.0 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/github/fommil/netlib/core/1.1.2/core-1.1.2.jar (164 kB at 90 kB/s)
#17 126.0 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/spire-math/spire-macros_2.11/0.13.0/spire-macros_2.11-0.13.0.jar
#17 126.1 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/net/sourceforge/f2j/arpack_combined_all/0.1/arpack_combined_all-0.1.jar (1.2 MB at 611 kB/s)
#17 126.1 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/typelevel/machinist_2.11/0.6.1/machinist_2.11-0.6.1.jar
#17 126.2 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/net/sf/opencsv/opencsv/2.3/opencsv-2.3.jar (20 kB at 9.7 kB/s)
#17 126.2 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/chuusai/shapeless_2.11/2.3.2/shapeless_2.11-2.3.2.jar
#17 126.4 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/spire-math/spire-macros_2.11/0.13.0/spire-macros_2.11-0.13.0.jar (87 kB at 39 kB/s)
#17 126.4 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/typelevel/macro-compat_2.11/1.1.1/macro-compat_2.11-1.1.1.jar
#17 126.5 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/typelevel/machinist_2.11/0.6.1/machinist_2.11-0.6.1.jar (35 kB at 15 kB/s)
#17 126.5 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/commons/commons-math3/3.4.1/commons-math3-3.4.1.jar
#17 126.7 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/chuusai/shapeless_2.11/2.3.2/shapeless_2.11-2.3.2.jar (3.5 MB at 1.4 MB/s)
#17 126.7 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/spark/spark-tags_2.11/2.3.0/spark-tags_2.11-2.3.0.jar
#17 126.8 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/typelevel/macro-compat_2.11/1.1.1/macro-compat_2.11-1.1.1.jar (3.1 kB at 1.2 kB/s)
#17 126.8 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/spark-project/spark/unused/1.0.0/unused-1.0.0.jar
#17 126.9 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/spire-math/spire_2.11/0.13.0/spire_2.11-0.13.0.jar (10 MB at 3.7 MB/s)
#17 126.9 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/github/rwl/jtransforms/2.4.0/jtransforms-2.4.0.jar
#17 127.0 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/commons/commons-math3/3.4.1/commons-math3-3.4.1.jar (2.0 MB at 701 kB/s)
#17 127.0 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/ml/combust/bundle/bundle-ml_2.11/0.10.3/bundle-ml_2.11-0.10.3.jar
#17 127.1 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/spark/spark-tags_2.11/2.3.0/spark-tags_2.11-2.3.0.jar (15 kB at 5.2 kB/s)
#17 127.1 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/trueaccord/scalapb/scalapb-runtime_2.11/0.6.0/scalapb-runtime_2.11-0.6.0.jar
#17 127.2 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/spark-project/spark/unused/1.0.0/unused-1.0.0.jar (2.8 kB at 903 B/s)
#17 127.2 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/trueaccord/lenses/lenses_2.11/0.4.12/lenses_2.11-0.4.12.jar
#17 127.3 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/github/rwl/jtransforms/2.4.0/jtransforms-2.4.0.jar (765 kB at 239 kB/s)
#17 127.3 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/lihaoyi/fastparse_2.11/0.4.2/fastparse_2.11-0.4.2.jar
#17 127.4 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/scalanlp/breeze_2.11/0.13.2/breeze_2.11-0.13.2.jar (15 MB at 4.6 MB/s)
#17 127.4 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/lihaoyi/fastparse-utils_2.11/0.4.2/fastparse-utils_2.11-0.4.2.jar
#17 127.5 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/ml/combust/bundle/bundle-ml_2.11/0.10.3/bundle-ml_2.11-0.10.3.jar (1.4 MB at 407 kB/s)
#17 127.5 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/lihaoyi/sourcecode_2.11/0.1.3/sourcecode_2.11-0.1.3.jar
#17 127.6 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/trueaccord/lenses/lenses_2.11/0.4.12/lenses_2.11-0.4.12.jar (57 kB at 16 kB/s)
#17 127.6 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/google/protobuf/protobuf-java/3.19.4/protobuf-java-3.19.4.jar
#17 127.6 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/trueaccord/scalapb/scalapb-runtime_2.11/0.6.0/scalapb-runtime_2.11-0.6.0.jar (3.7 MB at 1.1 MB/s)
#17 127.6 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/jsuereth/scala-arm_2.11/2.0/scala-arm_2.11-2.0.jar
#17 127.8 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/typesafe/config/1.3.0/config-1.3.0.jar
#17 127.8 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/lihaoyi/fastparse-utils_2.11/0.4.2/fastparse-utils_2.11-0.4.2.jar (78 kB at 21 kB/s)
#17 127.8 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/io/spray/spray-json_2.11/1.3.2/spray-json_2.11-1.3.2.jar
#17 127.8 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/lihaoyi/fastparse_2.11/0.4.2/fastparse_2.11-0.4.2.jar (336 kB at 92 kB/s)
#17 127.8 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/scala-lang/scala-reflect/2.11.8/scala-reflect-2.11.8.jar
#17 128.0 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/lihaoyi/sourcecode_2.11/0.1.3/sourcecode_2.11-0.1.3.jar (117 kB at 30 kB/s)
#17 128.0 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/plugins/maven-surefire-plugin/2.22.0/maven-surefire-plugin-2.22.0.jar
#17 128.0 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/jsuereth/scala-arm_2.11/2.0/scala-arm_2.11-2.0.jar (110 kB at 28 kB/s)
#17 128.0 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/surefire/maven-surefire-common/2.22.0/maven-surefire-common-2.22.0.jar
#17 128.2 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/io/spray/spray-json_2.11/1.3.2/spray-json_2.11-1.3.2.jar (286 kB at 71 kB/s)
#17 128.2 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/maven-plugin-api/2.2.1/maven-plugin-api-2.2.1.jar
#17 128.3 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/typesafe/config/1.3.0/config-1.3.0.jar (278 kB at 67 kB/s)
#17 128.3 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/plugin-tools/maven-plugin-annotations/3.5/maven-plugin-annotations-3.5.jar
#17 128.3 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/scala-lang/scala-reflect/2.11.8/scala-reflect-2.11.8.jar (4.6 MB at 1.1 MB/s)
#17 128.3 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/surefire/surefire-api/2.22.0/surefire-api-2.22.0.jar
#17 128.4 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/plugins/maven-surefire-plugin/2.22.0/maven-surefire-plugin-2.22.0.jar (41 kB at 9.6 kB/s)
#17 128.4 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/surefire/surefire-logger-api/2.22.0/surefire-logger-api-2.22.0.jar
#17 128.5 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/surefire/maven-surefire-common/2.22.0/maven-surefire-common-2.22.0.jar (524 kB at 121 kB/s)
#17 128.5 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/surefire/surefire-booter/2.22.0/surefire-booter-2.22.0.jar
#17 128.5 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/maven-plugin-api/2.2.1/maven-plugin-api-2.2.1.jar (12 kB at 2.8 kB/s)
#17 128.5 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/maven-artifact/2.2.1/maven-artifact-2.2.1.jar
#17 128.7 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/plugin-tools/maven-plugin-annotations/3.5/maven-plugin-annotations-3.5.jar (14 kB at 3.1 kB/s)
#17 128.7 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/codehaus/plexus/plexus-utils/1.5.15/plexus-utils-1.5.15.jar
#17 128.7 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/surefire/surefire-api/2.22.0/surefire-api-2.22.0.jar (186 kB at 41 kB/s)
#17 128.7 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/maven-plugin-descriptor/2.2.1/maven-plugin-descriptor-2.2.1.jar
#17 128.8 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/surefire/surefire-logger-api/2.22.0/surefire-logger-api-2.22.0.jar (13 kB at 2.9 kB/s)
#17 128.8 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/maven-project/2.2.1/maven-project-2.2.1.jar
#17 128.9 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/surefire/surefire-booter/2.22.0/surefire-booter-2.22.0.jar (273 kB at 58 kB/s)
#17 128.9 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/maven-settings/2.2.1/maven-settings-2.2.1.jar
#17 128.9 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/maven-artifact/2.2.1/maven-artifact-2.2.1.jar (80 kB at 17 kB/s)
#17 128.9 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/maven-profile/2.2.1/maven-profile-2.2.1.jar
#17 129.1 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/maven-plugin-descriptor/2.2.1/maven-plugin-descriptor-2.2.1.jar (39 kB at 7.9 kB/s)
#17 129.1 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/maven-artifact-manager/2.2.1/maven-artifact-manager-2.2.1.jar
#17 129.2 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/codehaus/plexus/plexus-utils/1.5.15/plexus-utils-1.5.15.jar (228 kB at 45 kB/s)
#17 129.2 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/backport-util-concurrent/backport-util-concurrent/3.1/backport-util-concurrent-3.1.jar
#17 129.3 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/maven-profile/2.2.1/maven-profile-2.2.1.jar (35 kB at 6.9 kB/s)
#17 129.3 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/maven-plugin-registry/2.2.1/maven-plugin-registry-2.2.1.jar
#17 129.3 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/maven-settings/2.2.1/maven-settings-2.2.1.jar (49 kB at 9.6 kB/s)
#17 129.3 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/codehaus/plexus/plexus-interpolation/1.11/plexus-interpolation-1.11.jar
#17 129.3 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/maven-project/2.2.1/maven-project-2.2.1.jar (156 kB at 30 kB/s)
#17 129.3 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/maven-model/2.2.1/maven-model-2.2.1.jar
#17 129.5 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/maven-artifact-manager/2.2.1/maven-artifact-manager-2.2.1.jar (68 kB at 13 kB/s)
#17 129.5 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/maven-core/2.2.1/maven-core-2.2.1.jar
#17 129.6 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/backport-util-concurrent/backport-util-concurrent/3.1/backport-util-concurrent-3.1.jar (332 kB at 61 kB/s)
#17 129.6 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/maven-plugin-parameter-documenter/2.2.1/maven-plugin-parameter-documenter-2.2.1.jar
#17 129.7 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/maven-plugin-registry/2.2.1/maven-plugin-registry-2.2.1.jar (30 kB at 5.4 kB/s)
#17 129.7 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/wagon/wagon-http/1.0-beta-6/wagon-http-1.0-beta-6.jar
#17 129.7 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/maven-model/2.2.1/maven-model-2.2.1.jar (88 kB at 16 kB/s)
#17 129.7 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/wagon/wagon-http-shared/1.0-beta-6/wagon-http-shared-1.0-beta-6.jar
#17 129.7 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/codehaus/plexus/plexus-interpolation/1.11/plexus-interpolation-1.11.jar (51 kB at 9.2 kB/s)
#17 129.7 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/nekohtml/xercesMinimal/1.9.6.2/xercesMinimal-1.9.6.2.jar
#17 129.8 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/maven-core/2.2.1/maven-core-2.2.1.jar (178 kB at 31 kB/s)
#17 129.8 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/nekohtml/nekohtml/1.9.6.2/nekohtml-1.9.6.2.jar
#17 130.0 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/wagon/wagon-http/1.0-beta-6/wagon-http-1.0-beta-6.jar (11 kB at 1.8 kB/s)
#17 130.0 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/commons-httpclient/commons-httpclient/3.1/commons-httpclient-3.1.jar
#17 130.1 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/maven-plugin-parameter-documenter/2.2.1/maven-plugin-parameter-documenter-2.2.1.jar (22 kB at 3.8 kB/s)
#17 130.1 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/wagon/wagon-webdav-jackrabbit/1.0-beta-6/wagon-webdav-jackrabbit-1.0-beta-6.jar
#17 130.1 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/wagon/wagon-http-shared/1.0-beta-6/wagon-http-shared-1.0-beta-6.jar (25 kB at 4.3 kB/s)
#17 130.1 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/jackrabbit/jackrabbit-webdav/1.5.0/jackrabbit-webdav-1.5.0.jar
#17 130.2 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/nekohtml/xercesMinimal/1.9.6.2/xercesMinimal-1.9.6.2.jar (40 kB at 6.6 kB/s)
#17 130.2 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/jackrabbit/jackrabbit-jcr-commons/1.5.0/jackrabbit-jcr-commons-1.5.0.jar
#17 130.3 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/nekohtml/nekohtml/1.9.6.2/nekohtml-1.9.6.2.jar (112 kB at 18 kB/s)
#17 130.3 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/reporting/maven-reporting-api/2.2.1/maven-reporting-api-2.2.1.jar
#17 130.4 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/commons-httpclient/commons-httpclient/3.1/commons-httpclient-3.1.jar (305 kB at 49 kB/s)
#17 130.4 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/doxia/doxia-logging-api/1.1/doxia-logging-api-1.1.jar
#17 130.5 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/wagon/wagon-webdav-jackrabbit/1.0-beta-6/wagon-webdav-jackrabbit-1.0-beta-6.jar (18 kB at 2.9 kB/s)
#17 130.5 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/wagon/wagon-provider-api/1.0-beta-6/wagon-provider-api-1.0-beta-6.jar
#17 130.5 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/jackrabbit/jackrabbit-webdav/1.5.0/jackrabbit-webdav-1.5.0.jar (303 kB at 48 kB/s)
#17 130.5 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/maven-repository-metadata/2.2.1/maven-repository-metadata-2.2.1.jar
#17 130.6 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/jackrabbit/jackrabbit-jcr-commons/1.5.0/jackrabbit-jcr-commons-1.5.0.jar (203 kB at 32 kB/s)
#17 130.6 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/maven-error-diagnostics/2.2.1/maven-error-diagnostics-2.2.1.jar
#17 130.7 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/reporting/maven-reporting-api/2.2.1/maven-reporting-api-2.2.1.jar (9.8 kB at 1.5 kB/s)
#17 130.7 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/maven-monitor/2.2.1/maven-monitor-2.2.1.jar
#17 130.8 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/doxia/doxia-logging-api/1.1/doxia-logging-api-1.1.jar (11 kB at 1.7 kB/s)
#17 130.8 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/plexus/plexus-sec-dispatcher/1.3/plexus-sec-dispatcher-1.3.jar
#17 130.9 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/wagon/wagon-provider-api/1.0-beta-6/wagon-provider-api-1.0-beta-6.jar (53 kB at 7.9 kB/s)
#17 130.9 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/plexus/plexus-cipher/1.4/plexus-cipher-1.4.jar
#17 130.9 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/maven-error-diagnostics/2.2.1/maven-error-diagnostics-2.2.1.jar (13 kB at 1.9 kB/s)
#17 130.9 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/maven-toolchain/2.2.1/maven-toolchain-2.2.1.jar
#17 131.0 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/maven-repository-metadata/2.2.1/maven-repository-metadata-2.2.1.jar (26 kB at 3.7 kB/s)
#17 131.0 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/codehaus/plexus/plexus-java/0.9.8/plexus-java-0.9.8.jar
#17 131.2 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/maven-monitor/2.2.1/maven-monitor-2.2.1.jar (10 kB at 1.5 kB/s)
#17 131.2 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/ow2/asm/asm/6.1.1/asm-6.1.1.jar
#17 131.2 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/plexus/plexus-cipher/1.4/plexus-cipher-1.4.jar (13 kB at 1.9 kB/s)
#17 131.2 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/thoughtworks/qdox/qdox/2.0-M7/qdox-2.0-M7.jar
#17 131.3 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/maven/maven-toolchain/2.2.1/maven-toolchain-2.2.1.jar (38 kB at 5.3 kB/s)
#17 131.3 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/eclipse/jetty/jetty-server/9.4.11.v20180605/jetty-server-9.4.11.v20180605.jar
#17 131.3 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/sonatype/plexus/plexus-sec-dispatcher/1.3/plexus-sec-dispatcher-1.3.jar (29 kB at 4.0 kB/s)
#17 131.4 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/javax/servlet/javax.servlet-api/3.1.0/javax.servlet-api-3.1.0.jar
#17 131.5 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/codehaus/plexus/plexus-java/0.9.8/plexus-java-0.9.8.jar (35 kB at 4.8 kB/s)
#17 131.5 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/eclipse/jetty/jetty-http/9.4.11.v20180605/jetty-http-9.4.11.v20180605.jar
#17 131.9 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/javax/servlet/javax.servlet-api/3.1.0/javax.servlet-api-3.1.0.jar (96 kB at 12 kB/s)
#17 131.9 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/eclipse/jetty/jetty-io/9.4.11.v20180605/jetty-io-9.4.11.v20180605.jar
#17 131.9 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/com/thoughtworks/qdox/qdox/2.0-M7/qdox-2.0-M7.jar (315 kB at 40 kB/s)
#17 131.9 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/eclipse/jetty/jetty-servlet/9.4.11.v20180605/jetty-servlet-9.4.11.v20180605.jar
#17 132.0 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/ow2/asm/asm/6.1.1/asm-6.1.1.jar (108 kB at 14 kB/s)
#17 132.0 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/eclipse/jetty/jetty-security/9.4.11.v20180605/jetty-security-9.4.11.v20180605.jar
#17 132.0 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/eclipse/jetty/jetty-server/9.4.11.v20180605/jetty-server-9.4.11.v20180605.jar (592 kB at 75 kB/s)
#17 132.0 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/eclipse/jetty/jetty-util/9.4.11.v20180605/jetty-util-9.4.11.v20180605.jar
#17 132.0 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/eclipse/jetty/jetty-http/9.4.11.v20180605/jetty-http-9.4.11.v20180605.jar (196 kB at 25 kB/s)
#17 132.0 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/slf4j/slf4j-jdk14/1.7.25/slf4j-jdk14-1.7.25.jar
#17 132.4 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/slf4j/slf4j-jdk14/1.7.25/slf4j-jdk14-1.7.25.jar (8.5 kB at 1.0 kB/s)
#17 132.4 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/slf4j/slf4j-api/1.7.25/slf4j-api-1.7.25.jar
#17 132.5 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/eclipse/jetty/jetty-security/9.4.11.v20180605/jetty-security-9.4.11.v20180605.jar (93 kB at 11 kB/s)
#17 132.5 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/httpcomponents/httpclient/4.5.6/httpclient-4.5.6.jar
#17 132.7 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/eclipse/jetty/jetty-servlet/9.4.11.v20180605/jetty-servlet-9.4.11.v20180605.jar (111 kB at 13 kB/s)
#17 132.7 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/httpcomponents/httpcore/4.4.10/httpcore-4.4.10.jar
#17 132.7 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/eclipse/jetty/jetty-io/9.4.11.v20180605/jetty-io-9.4.11.v20180605.jar (135 kB at 16 kB/s)
#17 132.8 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/commons-logging/commons-logging/1.2/commons-logging-1.2.jar
#17 133.0 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/eclipse/jetty/jetty-util/9.4.11.v20180605/jetty-util-9.4.11.v20180605.jar (499 kB at 57 kB/s)
#17 133.0 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/commons-codec/commons-codec/1.10/commons-codec-1.10.jar
#17 133.0 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/slf4j/slf4j-api/1.7.25/slf4j-api-1.7.25.jar (41 kB at 4.6 kB/s)
#17 133.0 [INFO] Downloading from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/commons-io/commons-io/2.5/commons-io-2.5.jar
#17 133.1 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/commons-logging/commons-logging/1.2/commons-logging-1.2.jar (62 kB at 6.9 kB/s)
#17 133.3 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/httpcomponents/httpcore/4.4.10/httpcore-4.4.10.jar (326 kB at 36 kB/s)
#17 133.4 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/org/apache/httpcomponents/httpclient/4.5.6/httpclient-4.5.6.jar (767 kB at 83 kB/s)
#17 133.8 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/commons-io/commons-io/2.5/commons-io-2.5.jar (209 kB at 22 kB/s)
#17 133.8 [INFO] Downloaded from google-maven-central: https://maven-central.storage-download.googleapis.com/repos/central/data/commons-codec/commons-codec/1.10/commons-codec-1.10.jar (284 kB at 29 kB/s)
#17 133.9 [INFO] Downloading from central: https://repo.maven.apache.org/maven2/com/google/protobuf/protobuf-java/3.19.4/protobuf-java-3.19.4.jar
#17 134.0 [INFO] Downloaded from central: https://repo.maven.apache.org/maven2/com/google/protobuf/protobuf-java/3.19.4/protobuf-java-3.19.4.jar (1.7 MB at 13 MB/s)
#17 134.1 [INFO]
#17 134.1 [INFO] --- maven-dependency-plugin:2.8:copy-dependencies (default-cli) @ mlflow-scoring ---
#17 134.4 [INFO] Copying wagon-provider-api-1.0-beta-6.jar to /opt/java/jars/wagon-provider-api-1.0-beta-6.jar
#17 134.4 [INFO] Copying mleap-tensor_2.11-0.10.3.jar to /opt/java/jars/mleap-tensor_2.11-0.10.3.jar
#17 134.4 [INFO] Copying maven-error-diagnostics-2.2.1.jar to /opt/java/jars/maven-error-diagnostics-2.2.1.jar
#17 134.4 [INFO] Copying breeze_2.11-0.13.2.jar to /opt/java/jars/breeze_2.11-0.13.2.jar
#17 134.5 [INFO] Copying plexus-java-0.9.8.jar to /opt/java/jars/plexus-java-0.9.8.jar
#17 134.5 [INFO] Copying scala-library-2.11.8.jar to /opt/java/jars/scala-library-2.11.8.jar
#17 134.5 [INFO] Copying maven-settings-2.2.1.jar to /opt/java/jars/maven-settings-2.2.1.jar
#17 134.5 [INFO] Copying sourcecode_2.11-0.1.3.jar to /opt/java/jars/sourcecode_2.11-0.1.3.jar
#17 134.5 [INFO] Copying httpcore-4.4.10.jar to /opt/java/jars/httpcore-4.4.10.jar
#17 134.5 [INFO] Copying maven-surefire-common-2.22.0.jar to /opt/java/jars/maven-surefire-common-2.22.0.jar
#17 134.5 [INFO] Copying macro-compat_2.11-1.1.1.jar to /opt/java/jars/macro-compat_2.11-1.1.1.jar
#17 134.5 [INFO] Copying bundle-ml_2.11-0.10.3.jar to /opt/java/jars/bundle-ml_2.11-0.10.3.jar
#17 134.5 [INFO] Copying fastparse-utils_2.11-0.4.2.jar to /opt/java/jars/fastparse-utils_2.11-0.4.2.jar
#17 134.5 [INFO] Copying maven-artifact-manager-2.2.1.jar to /opt/java/jars/maven-artifact-manager-2.2.1.jar
#17 134.5 [INFO] Copying jackrabbit-webdav-1.5.0.jar to /opt/java/jars/jackrabbit-webdav-1.5.0.jar
#17 134.5 [INFO] Copying lenses_2.11-0.4.12.jar to /opt/java/jars/lenses_2.11-0.4.12.jar
#17 134.5 [INFO] Copying asm-6.1.1.jar to /opt/java/jars/asm-6.1.1.jar
#17 134.5 [INFO] Copying jackson-annotations-2.10.0.jar to /opt/java/jars/jackson-annotations-2.10.0.jar
#17 134.5 [INFO] Copying snakeyaml-1.24.jar to /opt/java/jars/snakeyaml-1.24.jar
#17 134.5 [INFO] Copying machinist_2.11-0.6.1.jar to /opt/java/jars/machinist_2.11-0.6.1.jar
#17 134.5 [INFO] Copying maven-toolchain-2.2.1.jar to /opt/java/jars/maven-toolchain-2.2.1.jar
#17 134.5 [INFO] Copying spire_2.11-0.13.0.jar to /opt/java/jars/spire_2.11-0.13.0.jar
#17 134.5 [INFO] Copying maven-plugin-api-2.2.1.jar to /opt/java/jars/maven-plugin-api-2.2.1.jar
#17 134.5 [INFO] Copying maven-project-2.2.1.jar to /opt/java/jars/maven-project-2.2.1.jar
#17 134.5 [INFO] Copying qdox-2.0-M7.jar to /opt/java/jars/qdox-2.0-M7.jar
#17 134.5 [INFO] Copying jetty-server-9.4.11.v20180605.jar to /opt/java/jars/jetty-server-9.4.11.v20180605.jar
#17 134.5 [INFO] Copying jackson-core-2.10.0.jar to /opt/java/jars/jackson-core-2.10.0.jar
#17 134.5 [INFO] Copying core-1.1.2.jar to /opt/java/jars/core-1.1.2.jar
#17 134.5 [INFO] Copying jetty-security-9.4.11.v20180605.jar to /opt/java/jars/jetty-security-9.4.11.v20180605.jar
#17 134.5 [INFO] Copying config-1.3.0.jar to /opt/java/jars/config-1.3.0.jar
#17 134.5 [INFO] Copying maven-monitor-2.2.1.jar to /opt/java/jars/maven-monitor-2.2.1.jar
#17 134.5 [INFO] Copying wagon-http-1.0-beta-6.jar to /opt/java/jars/wagon-http-1.0-beta-6.jar
#17 134.5 [INFO] Copying mleap-core_2.11-0.10.3.jar to /opt/java/jars/mleap-core_2.11-0.10.3.jar
#17 134.5 [INFO] Copying hamcrest-core-1.3.jar to /opt/java/jars/hamcrest-core-1.3.jar
#17 134.5 [INFO] Copying breeze-macros_2.11-0.13.2.jar to /opt/java/jars/breeze-macros_2.11-0.13.2.jar
#17 134.5 [INFO] Copying xercesMinimal-1.9.6.2.jar to /opt/java/jars/xercesMinimal-1.9.6.2.jar
#17 134.5 [INFO] Copying jetty-http-9.4.11.v20180605.jar to /opt/java/jars/jetty-http-9.4.11.v20180605.jar
#17 134.5 [INFO] Copying fastparse_2.11-0.4.2.jar to /opt/java/jars/fastparse_2.11-0.4.2.jar
#17 134.5 [INFO] Copying protobuf-java-3.19.4.jar to /opt/java/jars/protobuf-java-3.19.4.jar
#17 134.5 [INFO] Copying scala-arm_2.11-2.0.jar to /opt/java/jars/scala-arm_2.11-2.0.jar
#17 134.5 [INFO] Copying arpack_combined_all-0.1.jar to /opt/java/jars/arpack_combined_all-0.1.jar
#17 134.5 [INFO] Copying shapeless_2.11-2.3.2.jar to /opt/java/jars/shapeless_2.11-2.3.2.jar
#17 134.5 [INFO] Copying mleap-runtime_2.11-0.10.3.jar to /opt/java/jars/mleap-runtime_2.11-0.10.3.jar
#17 134.5 [INFO] Copying scala-reflect-2.11.8.jar to /opt/java/jars/scala-reflect-2.11.8.jar
#17 134.5 [INFO] Copying commons-logging-1.2.jar to /opt/java/jars/commons-logging-1.2.jar
#17 134.5 [INFO] Copying jetty-util-9.4.11.v20180605.jar to /opt/java/jars/jetty-util-9.4.11.v20180605.jar
#17 134.5 [INFO] Copying jackson-databind-2.10.0.jar to /opt/java/jars/jackson-databind-2.10.0.jar
#17 134.5 [INFO] Copying mleap-spark_2.11-0.10.3.jar to /opt/java/jars/mleap-spark_2.11-0.10.3.jar
#17 134.5 [INFO] Copying plexus-utils-1.5.15.jar to /opt/java/jars/plexus-utils-1.5.15.jar
#17 134.5 [INFO] Copying spire-macros_2.11-0.13.0.jar to /opt/java/jars/spire-macros_2.11-0.13.0.jar
#17 134.5 [INFO] Copying plexus-interpolation-1.11.jar to /opt/java/jars/plexus-interpolation-1.11.jar
#17 134.5 [INFO] Copying plexus-cipher-1.4.jar to /opt/java/jars/plexus-cipher-1.4.jar
#17 134.5 [INFO] Copying spark-mllib-local_2.11-2.3.0.jar to /opt/java/jars/spark-mllib-local_2.11-2.3.0.jar
#17 134.5 [INFO] Copying jtransforms-2.4.0.jar to /opt/java/jars/jtransforms-2.4.0.jar
#17 134.5 [INFO] Copying plexus-container-default-1.0-alpha-9-stable-1.jar to /opt/java/jars/plexus-container-default-1.0-alpha-9-stable-1.jar
#17 134.5 [INFO] Copying spray-json_2.11-1.3.2.jar to /opt/java/jars/spray-json_2.11-1.3.2.jar
#17 134.5 [INFO] Copying jetty-servlet-9.4.11.v20180605.jar to /opt/java/jars/jetty-servlet-9.4.11.v20180605.jar
#17 134.5 [INFO] Copying commons-io-2.5.jar to /opt/java/jars/commons-io-2.5.jar
#17 134.5 [INFO] Copying surefire-logger-api-2.22.0.jar to /opt/java/jars/surefire-logger-api-2.22.0.jar
#17 134.5 [INFO] Copying mleap-base_2.11-0.10.3.jar to /opt/java/jars/mleap-base_2.11-0.10.3.jar
#17 134.5 [INFO] Copying surefire-booter-2.22.0.jar to /opt/java/jars/surefire-booter-2.22.0.jar
#17 134.5 [INFO] Copying opencsv-2.3.jar to /opt/java/jars/opencsv-2.3.jar
#17 134.5 [INFO] Copying spark-tags_2.11-2.3.0.jar to /opt/java/jars/spark-tags_2.11-2.3.0.jar
#17 134.5 [INFO] Copying jackrabbit-jcr-commons-1.5.0.jar to /opt/java/jars/jackrabbit-jcr-commons-1.5.0.jar
#17 134.5 [INFO] Copying classworlds-1.1.jar to /opt/java/jars/classworlds-1.1.jar
#17 134.5 [INFO] Copying nekohtml-1.9.6.2.jar to /opt/java/jars/nekohtml-1.9.6.2.jar
#17 134.5 [INFO] Copying maven-core-2.2.1.jar to /opt/java/jars/maven-core-2.2.1.jar
#17 134.5 [INFO] Copying maven-surefire-plugin-2.22.0.jar to /opt/java/jars/maven-surefire-plugin-2.22.0.jar
#17 134.5 [INFO] Copying mleap-spark-base_2.11-0.10.3.jar to /opt/java/jars/mleap-spark-base_2.11-0.10.3.jar
#17 134.5 [INFO] Copying maven-repository-metadata-2.2.1.jar to /opt/java/jars/maven-repository-metadata-2.2.1.jar
#17 134.5 [INFO] Copying wagon-webdav-jackrabbit-1.0-beta-6.jar to /opt/java/jars/wagon-webdav-jackrabbit-1.0-beta-6.jar
#17 134.5 [INFO] Copying maven-artifact-2.2.1.jar to /opt/java/jars/maven-artifact-2.2.1.jar
#17 134.5 [INFO] Copying commons-codec-1.10.jar to /opt/java/jars/commons-codec-1.10.jar
#17 134.5 [INFO] Copying maven-plugin-descriptor-2.2.1.jar to /opt/java/jars/maven-plugin-descriptor-2.2.1.jar
#17 134.5 [INFO] Copying unused-1.0.0.jar to /opt/java/jars/unused-1.0.0.jar
#17 134.5 [INFO] Copying scalapb-runtime_2.11-0.6.0.jar to /opt/java/jars/scalapb-runtime_2.11-0.6.0.jar
#17 134.5 [INFO] Copying maven-profile-2.2.1.jar to /opt/java/jars/maven-profile-2.2.1.jar
#17 134.5 [INFO] Copying plexus-sec-dispatcher-1.3.jar to /opt/java/jars/plexus-sec-dispatcher-1.3.jar
#17 134.5 [INFO] Copying maven-plugin-annotations-3.5.jar to /opt/java/jars/maven-plugin-annotations-3.5.jar
#17 134.5 [INFO] Copying maven-reporting-api-2.2.1.jar to /opt/java/jars/maven-reporting-api-2.2.1.jar
#17 134.5 [INFO] Copying javax.servlet-api-3.1.0.jar to /opt/java/jars/javax.servlet-api-3.1.0.jar
#17 134.5 [INFO] Copying maven-plugin-parameter-documenter-2.2.1.jar to /opt/java/jars/maven-plugin-parameter-documenter-2.2.1.jar
#17 134.5 [INFO] Copying commons-math3-3.4.1.jar to /opt/java/jars/commons-math3-3.4.1.jar
#17 134.5 [INFO] Copying junit-4.12.jar to /opt/java/jars/junit-4.12.jar
#17 134.5 [INFO] Copying slf4j-jdk14-1.7.25.jar to /opt/java/jars/slf4j-jdk14-1.7.25.jar
#17 134.5 [INFO] Copying slf4j-api-1.7.25.jar to /opt/java/jars/slf4j-api-1.7.25.jar
#17 134.5 [INFO] Copying jetty-io-9.4.11.v20180605.jar to /opt/java/jars/jetty-io-9.4.11.v20180605.jar
#17 134.5 [INFO] Copying httpclient-4.5.6.jar to /opt/java/jars/httpclient-4.5.6.jar
#17 134.5 [INFO] Copying maven-model-2.2.1.jar to /opt/java/jars/maven-model-2.2.1.jar
#17 134.5 [INFO] Copying jackson-dataformat-yaml-2.10.0.jar to /opt/java/jars/jackson-dataformat-yaml-2.10.0.jar
#17 134.5 [INFO] Copying doxia-logging-api-1.1.jar to /opt/java/jars/doxia-logging-api-1.1.jar
#17 134.5 [INFO] Copying surefire-api-2.22.0.jar to /opt/java/jars/surefire-api-2.22.0.jar
#17 134.5 [INFO] Copying maven-plugin-registry-2.2.1.jar to /opt/java/jars/maven-plugin-registry-2.2.1.jar
#17 134.5 [INFO] Copying backport-util-concurrent-3.1.jar to /opt/java/jars/backport-util-concurrent-3.1.jar
#17 134.5 [INFO] Copying wagon-http-shared-1.0-beta-6.jar to /opt/java/jars/wagon-http-shared-1.0-beta-6.jar
#17 134.5 [INFO] Copying commons-httpclient-3.1.jar to /opt/java/jars/commons-httpclient-3.1.jar
#17 134.5 [INFO] ------------------------------------------------------------------------
#17 134.5 [INFO] BUILD SUCCESS
#17 134.5 [INFO] ------------------------------------------------------------------------
#17 134.5 [INFO] Total time:  02:12 min
#17 134.5 [INFO] Finished at: 2024-05-16T20:27:50Z
#17 134.5 [INFO] ------------------------------------------------------------------------
#17 DONE 134.8s

#18 [14/17] COPY model_dir/success6g_model /opt/ml/model
#18 DONE 0.1s

#19 [15/17] RUN python -c "from mlflow.models import container as C; C._install_pyfunc_deps('/opt/ml/model', install_mlflow=False, enable_mlserver=True, env_manager='virtualenv');"
#19 1.285 2024/05/16 20:27:52 INFO mlflow.models.container: creating and activating custom environment
#19 1.289 2024/05/16 20:27:52 INFO mlflow.utils.virtualenv: Installing python 3.10.11 if it does not exist
#19 1.673 Downloading Python-3.10.11.tar.xz...
#19 1.673 -> https://www.python.org/ftp/python/3.10.11/Python-3.10.11.tar.xz
#19 4.792 Installing Python-3.10.11...
#19 108.0 Installed Python-3.10.11 to /root/.pyenv/versions/3.10.11
#19 108.4 2024/05/16 20:29:39 INFO mlflow.utils.virtualenv: Creating a new environment in /root/.mlflow/envs/mlflow-bb8a0827694c8de04b7469b4f93834b709416c0b with /root/.pyenv/versions/3.10.11/bin/python
#19 109.3 created virtual environment CPython3.10.11.final.0-64 in 719ms
#19 109.3   creator CPython3Posix(dest=/root/.mlflow/envs/mlflow-bb8a0827694c8de04b7469b4f93834b709416c0b, clear=False, no_vcs_ignore=False, global=False)
#19 109.3   seeder FromAppData(download=False, pip=bundle, setuptools=bundle, wheel=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
#19 109.3     added seed packages: pip==24.0, setuptools==69.5.1, wheel==0.43.0
#19 109.3   activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator
#19 109.4 2024/05/16 20:29:40 INFO mlflow.utils.virtualenv: Installing dependencies
#19 168.4 WARNING: There was an error checking the latest version of pip.
#19 169.3 Collecting mlserver!=1.3.1,<1.4.0,>=1.2.0
#19 169.4   Downloading mlserver-1.3.5-py3-none-any.whl (113 kB)
#19 169.4      ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.2/113.2 kB 3.4 MB/s eta 0:00:00
#19 169.6 Collecting mlserver-mlflow!=1.3.1,<1.4.0,>=1.2.0
#19 169.6   Downloading mlserver_mlflow-1.3.5-py3-none-any.whl (10 kB)
#19 169.7 Requirement already satisfied: click in /root/.mlflow/envs/mlflow-bb8a0827694c8de04b7469b4f93834b709416c0b/lib/python3.10/site-packages (from mlserver!=1.3.1,<1.4.0,>=1.2.0) (8.1.3)
#19 169.8 Collecting fastapi!=0.89.0,<=0.89.1,>=0.88.0 (from mlserver!=1.3.1,<1.4.0,>=1.2.0)
#19 169.8   Downloading fastapi-0.89.1-py3-none-any.whl (55 kB)
#19 169.8      ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 7.4 MB/s eta 0:00:00
#19 169.9 Collecting python-dotenv (from mlserver!=1.3.1,<1.4.0,>=1.2.0)
#19 169.9   Downloading python_dotenv-1.0.1-py3-none-any.whl (19 kB)
#19 171.0 Collecting grpcio (from mlserver!=1.3.1,<1.4.0,>=1.2.0)
#19 171.0   Downloading grpcio-1.63.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (5.6 MB)
#19 171.3      ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 18.5 MB/s eta 0:00:00
#19 171.4 Collecting importlib-resources (from mlserver!=1.3.1,<1.4.0,>=1.2.0)
#19 171.4   Downloading importlib_resources-6.4.0-py3-none-any.whl (38 kB)
#19 171.5 Requirement already satisfied: numpy in /root/.mlflow/envs/mlflow-bb8a0827694c8de04b7469b4f93834b709416c0b/lib/python3.10/site-packages (from mlserver!=1.3.1,<1.4.0,>=1.2.0) (1.24.3)
#19 171.5 Requirement already satisfied: pandas in /root/.mlflow/envs/mlflow-bb8a0827694c8de04b7469b4f93834b709416c0b/lib/python3.10/site-packages (from mlserver!=1.3.1,<1.4.0,>=1.2.0) (2.2.2)
#19 171.5 Requirement already satisfied: protobuf in /root/.mlflow/envs/mlflow-bb8a0827694c8de04b7469b4f93834b709416c0b/lib/python3.10/site-packages (from mlserver!=1.3.1,<1.4.0,>=1.2.0) (4.25.3)
#19 171.5 Collecting uvicorn (from mlserver!=1.3.1,<1.4.0,>=1.2.0)
#19 171.5   Downloading uvicorn-0.29.0-py3-none-any.whl (60 kB)
#19 171.6      ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 8.7 MB/s eta 0:00:00
#19 171.7 Collecting starlette-exporter (from mlserver!=1.3.1,<1.4.0,>=1.2.0)
#19 171.7   Downloading starlette_exporter-0.21.0-py3-none-any.whl (14 kB)
#19 171.9 Collecting py-grpc-prometheus (from mlserver!=1.3.1,<1.4.0,>=1.2.0)
#19 171.9   Downloading py_grpc_prometheus-0.8.0-py3-none-any.whl (12 kB)
#19 172.1 Collecting aiokafka (from mlserver!=1.3.1,<1.4.0,>=1.2.0)
#19 172.1   Downloading aiokafka-0.10.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.1 MB)
#19 172.2      ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.6 MB/s eta 0:00:00
#19 172.3 Collecting tritonclient[http]>=2.24 (from mlserver!=1.3.1,<1.4.0,>=1.2.0)
#19 172.3   Downloading tritonclient-2.45.0-py3-none-manylinux1_x86_64.whl (12.6 MB)
#19 172.9      ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 19.7 MB/s eta 0:00:00
#19 173.0 Collecting aiofiles (from mlserver!=1.3.1,<1.4.0,>=1.2.0)
#19 173.0   Downloading aiofiles-23.2.1-py3-none-any.whl (15 kB)
#19 173.5 Collecting orjson (from mlserver!=1.3.1,<1.4.0,>=1.2.0)
#19 173.5   Downloading orjson-3.10.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (142 kB)
#19 173.6      ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 19.4 MB/s eta 0:00:00
#19 173.7 Collecting uvloop (from mlserver!=1.3.1,<1.4.0,>=1.2.0)
#19 173.7   Downloading uvloop-0.19.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.4 MB)
#19 173.8      ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 19.8 MB/s eta 0:00:00
#19 173.9 Requirement already satisfied: mlflow in /root/.mlflow/envs/mlflow-bb8a0827694c8de04b7469b4f93834b709416c0b/lib/python3.10/site-packages (from mlserver-mlflow!=1.3.1,<1.4.0,>=1.2.0) (2.12.2)
#19 174.2 Collecting pydantic!=1.7,!=1.7.1,!=1.7.2,!=1.7.3,!=1.8,!=1.8.1,<2.0.0,>=1.6.2 (from fastapi!=0.89.0,<=0.89.1,>=0.88.0->mlserver!=1.3.1,<1.4.0,>=1.2.0)
#19 174.2   Downloading pydantic-1.10.15-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.1 MB)
#19 174.4      ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 19.8 MB/s eta 0:00:00
#19 174.5 Collecting starlette==0.22.0 (from fastapi!=0.89.0,<=0.89.1,>=0.88.0->mlserver!=1.3.1,<1.4.0,>=1.2.0)
#19 174.5   Downloading starlette-0.22.0-py3-none-any.whl (64 kB)
#19 174.5      ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.3/64.3 kB 10.3 MB/s eta 0:00:00
#19 174.6 Collecting anyio<5,>=3.4.0 (from starlette==0.22.0->fastapi!=0.89.0,<=0.89.1,>=0.88.0->mlserver!=1.3.1,<1.4.0,>=1.2.0)
#19 174.6   Downloading anyio-4.3.0-py3-none-any.whl (85 kB)
#19 174.7      ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 12.6 MB/s eta 0:00:00
#19 175.0 Collecting python-rapidjson>=0.9.1 (from tritonclient[http]>=2.24->mlserver!=1.3.1,<1.4.0,>=1.2.0)
#19 175.0   Downloading python_rapidjson-1.16-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.7 MB)
#19 175.1      ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 19.5 MB/s eta 0:00:00
#19 175.1 Requirement already satisfied: urllib3>=2.0.7 in /root/.mlflow/envs/mlflow-bb8a0827694c8de04b7469b4f93834b709416c0b/lib/python3.10/site-packages (from tritonclient[http]>=2.24->mlserver!=1.3.1,<1.4.0,>=1.2.0) (2.2.1)
#19 175.7 Collecting aiohttp<4.0.0,>=3.8.1 (from tritonclient[http]>=2.24->mlserver!=1.3.1,<1.4.0,>=1.2.0)
#19 175.8   Downloading aiohttp-3.9.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.2 MB)
#19 175.8      ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 19.7 MB/s eta 0:00:00
#19 176.1 Collecting geventhttpclient<=2.0.2,>=1.4.4 (from tritonclient[http]>=2.24->mlserver!=1.3.1,<1.4.0,>=1.2.0)
#19 176.2   Downloading geventhttpclient-2.0.2-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (100 kB)
#19 176.2      ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 kB 10.5 MB/s eta 0:00:00
#19 176.3 Requirement already satisfied: async-timeout in /root/.mlflow/envs/mlflow-bb8a0827694c8de04b7469b4f93834b709416c0b/lib/python3.10/site-packages (from aiokafka->mlserver!=1.3.1,<1.4.0,>=1.2.0) (4.0.3)
#19 176.3 Requirement already satisfied: packaging in /root/.mlflow/envs/mlflow-bb8a0827694c8de04b7469b4f93834b709416c0b/lib/python3.10/site-packages (from aiokafka->mlserver!=1.3.1,<1.4.0,>=1.2.0) (23.1)
#19 176.4 Requirement already satisfied: Flask<4 in /root/.mlflow/envs/mlflow-bb8a0827694c8de04b7469b4f93834b709416c0b/lib/python3.10/site-packages (from mlflow->mlserver-mlflow!=1.3.1,<1.4.0,>=1.2.0) (3.0.3)
#19 176.4 Requirement already satisfied: alembic!=1.10.0,<2 in /root/.mlflow/envs/mlflow-bb8a0827694c8de04b7469b4f93834b709416c0b/lib/python3.10/site-packages (from mlflow->mlserver-mlflow!=1.3.1,<1.4.0,>=1.2.0) (1.13.1)
#19 176.4 Requirement already satisfied: cloudpickle<4 in /root/.mlflow/envs/mlflow-bb8a0827694c8de04b7469b4f93834b709416c0b/lib/python3.10/site-packages (from mlflow->mlserver-mlflow!=1.3.1,<1.4.0,>=1.2.0) (2.2.1)
#19 176.4 Requirement already satisfied: docker<8,>=4.0.0 in /root/.mlflow/envs/mlflow-bb8a0827694c8de04b7469b4f93834b709416c0b/lib/python3.10/site-packages (from mlflow->mlserver-mlflow!=1.3.1,<1.4.0,>=1.2.0) (7.0.0)
#19 176.4 Requirement already satisfied: entrypoints<1 in /root/.mlflow/envs/mlflow-bb8a0827694c8de04b7469b4f93834b709416c0b/lib/python3.10/site-packages (from mlflow->mlserver-mlflow!=1.3.1,<1.4.0,>=1.2.0) (0.4)
#19 176.4 Requirement already satisfied: gitpython<4,>=3.1.9 in /root/.mlflow/envs/mlflow-bb8a0827694c8de04b7469b4f93834b709416c0b/lib/python3.10/site-packages (from mlflow->mlserver-mlflow!=1.3.1,<1.4.0,>=1.2.0) (3.1.43)
#19 176.4 Requirement already satisfied: graphene<4 in /root/.mlflow/envs/mlflow-bb8a0827694c8de04b7469b4f93834b709416c0b/lib/python3.10/site-packages (from mlflow->mlserver-mlflow!=1.3.1,<1.4.0,>=1.2.0) (3.3)
#19 176.4 Requirement already satisfied: importlib-metadata!=4.7.0,<8,>=3.7.0 in /root/.mlflow/envs/mlflow-bb8a0827694c8de04b7469b4f93834b709416c0b/lib/python3.10/site-packages (from mlflow->mlserver-mlflow!=1.3.1,<1.4.0,>=1.2.0) (6.6.0)
#19 176.4 Requirement already satisfied: markdown<4,>=3.3 in /root/.mlflow/envs/mlflow-bb8a0827694c8de04b7469b4f93834b709416c0b/lib/python3.10/site-packages (from mlflow->mlserver-mlflow!=1.3.1,<1.4.0,>=1.2.0) (3.6)
#19 176.4 Requirement already satisfied: matplotlib<4 in /root/.mlflow/envs/mlflow-bb8a0827694c8de04b7469b4f93834b709416c0b/lib/python3.10/site-packages (from mlflow->mlserver-mlflow!=1.3.1,<1.4.0,>=1.2.0) (3.7.1)
#19 176.4 Requirement already satisfied: pyarrow<16,>=4.0.0 in /root/.mlflow/envs/mlflow-bb8a0827694c8de04b7469b4f93834b709416c0b/lib/python3.10/site-packages (from mlflow->mlserver-mlflow!=1.3.1,<1.4.0,>=1.2.0) (12.0.0)
#19 176.4 Requirement already satisfied: pytz<2025 in /root/.mlflow/envs/mlflow-bb8a0827694c8de04b7469b4f93834b709416c0b/lib/python3.10/site-packages (from mlflow->mlserver-mlflow!=1.3.1,<1.4.0,>=1.2.0) (2024.1)
#19 176.4 Requirement already satisfied: pyyaml<7,>=5.1 in /root/.mlflow/envs/mlflow-bb8a0827694c8de04b7469b4f93834b709416c0b/lib/python3.10/site-packages (from mlflow->mlserver-mlflow!=1.3.1,<1.4.0,>=1.2.0) (6.0)
#19 176.4 Requirement already satisfied: querystring-parser<2 in /root/.mlflow/envs/mlflow-bb8a0827694c8de04b7469b4f93834b709416c0b/lib/python3.10/site-packages (from mlflow->mlserver-mlflow!=1.3.1,<1.4.0,>=1.2.0) (1.2.4)
#19 176.4 Requirement already satisfied: requests<3,>=2.17.3 in /root/.mlflow/envs/mlflow-bb8a0827694c8de04b7469b4f93834b709416c0b/lib/python3.10/site-packages (from mlflow->mlserver-mlflow!=1.3.1,<1.4.0,>=1.2.0) (2.31.0)
#19 176.4 Requirement already satisfied: scikit-learn<2 in /root/.mlflow/envs/mlflow-bb8a0827694c8de04b7469b4f93834b709416c0b/lib/python3.10/site-packages (from mlflow->mlserver-mlflow!=1.3.1,<1.4.0,>=1.2.0) (1.2.2)
#19 176.4 Requirement already satisfied: scipy<2 in /root/.mlflow/envs/mlflow-bb8a0827694c8de04b7469b4f93834b709416c0b/lib/python3.10/site-packages (from mlflow->mlserver-mlflow!=1.3.1,<1.4.0,>=1.2.0) (1.10.1)
#19 176.4 Requirement already satisfied: sqlalchemy<3,>=1.4.0 in /root/.mlflow/envs/mlflow-bb8a0827694c8de04b7469b4f93834b709416c0b/lib/python3.10/site-packages (from mlflow->mlserver-mlflow!=1.3.1,<1.4.0,>=1.2.0) (2.0.30)
#19 176.4 Requirement already satisfied: sqlparse<1,>=0.4.0 in /root/.mlflow/envs/mlflow-bb8a0827694c8de04b7469b4f93834b709416c0b/lib/python3.10/site-packages (from mlflow->mlserver-mlflow!=1.3.1,<1.4.0,>=1.2.0) (0.5.0)
#19 176.4 Requirement already satisfied: Jinja2<4,>=2.11 in /root/.mlflow/envs/mlflow-bb8a0827694c8de04b7469b4f93834b709416c0b/lib/python3.10/site-packages (from mlflow->mlserver-mlflow!=1.3.1,<1.4.0,>=1.2.0) (3.1.2)
#19 176.4 Requirement already satisfied: gunicorn<23 in /root/.mlflow/envs/mlflow-bb8a0827694c8de04b7469b4f93834b709416c0b/lib/python3.10/site-packages (from mlflow->mlserver-mlflow!=1.3.1,<1.4.0,>=1.2.0) (22.0.0)
#19 176.5 Requirement already satisfied: python-dateutil>=2.8.2 in /root/.mlflow/envs/mlflow-bb8a0827694c8de04b7469b4f93834b709416c0b/lib/python3.10/site-packages (from pandas->mlserver!=1.3.1,<1.4.0,>=1.2.0) (2.9.0.post0)
#19 176.5 Requirement already satisfied: tzdata>=2022.7 in /root/.mlflow/envs/mlflow-bb8a0827694c8de04b7469b4f93834b709416c0b/lib/python3.10/site-packages (from pandas->mlserver!=1.3.1,<1.4.0,>=1.2.0) (2024.1)
#19 176.5 Requirement already satisfied: setuptools>=39.0.1 in /root/.mlflow/envs/mlflow-bb8a0827694c8de04b7469b4f93834b709416c0b/lib/python3.10/site-packages (from py-grpc-prometheus->mlserver!=1.3.1,<1.4.0,>=1.2.0) (67.7.2)
#19 176.6 Collecting prometheus-client>=0.3.0 (from py-grpc-prometheus->mlserver!=1.3.1,<1.4.0,>=1.2.0)
#19 176.6   Downloading prometheus_client-0.20.0-py3-none-any.whl (54 kB)
#19 176.6      ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 8.5 MB/s eta 0:00:00
#19 176.6 INFO: pip is looking at multiple versions of starlette-exporter to determine which version is compatible with other requirements. This could take a while.
#19 176.6 Collecting starlette-exporter (from mlserver!=1.3.1,<1.4.0,>=1.2.0)
#19 176.7   Downloading starlette_exporter-0.20.0-py3-none-any.whl (14 kB)
#19 176.7   Downloading starlette_exporter-0.19.0-py3-none-any.whl (14 kB)
#19 176.7   Downloading starlette_exporter-0.18.0-py3-none-any.whl (14 kB)
#19 176.8   Downloading starlette_exporter-0.17.1-py3-none-any.whl (14 kB)
#19 176.8 Collecting h11>=0.8 (from uvicorn->mlserver!=1.3.1,<1.4.0,>=1.2.0)
#19 176.8   Downloading h11-0.14.0-py3-none-any.whl (58 kB)
#19 176.9      ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.0 MB/s eta 0:00:00
#19 176.9 Requirement already satisfied: typing-extensions>=4.0 in /root/.mlflow/envs/mlflow-bb8a0827694c8de04b7469b4f93834b709416c0b/lib/python3.10/site-packages (from uvicorn->mlserver!=1.3.1,<1.4.0,>=1.2.0) (4.11.0)
#19 176.9 Collecting aiosignal>=1.1.2 (from aiohttp<4.0.0,>=3.8.1->tritonclient[http]>=2.24->mlserver!=1.3.1,<1.4.0,>=1.2.0)
#19 176.9   Downloading aiosignal-1.3.1-py3-none-any.whl (7.6 kB)
#19 177.0 Collecting attrs>=17.3.0 (from aiohttp<4.0.0,>=3.8.1->tritonclient[http]>=2.24->mlserver!=1.3.1,<1.4.0,>=1.2.0)
#19 177.0   Downloading attrs-23.2.0-py3-none-any.whl (60 kB)
#19 177.0      ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 8.7 MB/s eta 0:00:00
#19 177.2 Collecting frozenlist>=1.1.1 (from aiohttp<4.0.0,>=3.8.1->tritonclient[http]>=2.24->mlserver!=1.3.1,<1.4.0,>=1.2.0)
#19 177.2   Downloading frozenlist-1.4.1-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (239 kB)
#19 177.2      ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.5/239.5 kB 29.8 MB/s eta 0:00:00
#19 177.6 Collecting multidict<7.0,>=4.5 (from aiohttp<4.0.0,>=3.8.1->tritonclient[http]>=2.24->mlserver!=1.3.1,<1.4.0,>=1.2.0)
#19 177.7   Downloading multidict-6.0.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (124 kB)
#19 177.7      ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.3/124.3 kB 17.1 MB/s eta 0:00:00
#19 178.0 Collecting yarl<2.0,>=1.0 (from aiohttp<4.0.0,>=3.8.1->tritonclient[http]>=2.24->mlserver!=1.3.1,<1.4.0,>=1.2.0)
#19 178.0   Downloading yarl-1.9.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (301 kB)
#19 178.0      ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.6/301.6 kB 16.3 MB/s eta 0:00:00
#19 178.0 Requirement already satisfied: Mako in /root/.mlflow/envs/mlflow-bb8a0827694c8de04b7469b4f93834b709416c0b/lib/python3.10/site-packages (from alembic!=1.10.0,<2->mlflow->mlserver-mlflow!=1.3.1,<1.4.0,>=1.2.0) (1.3.5)
#19 178.1 Requirement already satisfied: Werkzeug>=3.0.0 in /root/.mlflow/envs/mlflow-bb8a0827694c8de04b7469b4f93834b709416c0b/lib/python3.10/site-packages (from Flask<4->mlflow->mlserver-mlflow!=1.3.1,<1.4.0,>=1.2.0) (3.0.3)
#19 178.1 Requirement already satisfied: itsdangerous>=2.1.2 in /root/.mlflow/envs/mlflow-bb8a0827694c8de04b7469b4f93834b709416c0b/lib/python3.10/site-packages (from Flask<4->mlflow->mlserver-mlflow!=1.3.1,<1.4.0,>=1.2.0) (2.2.0)
#19 178.1 Requirement already satisfied: blinker>=1.6.2 in /root/.mlflow/envs/mlflow-bb8a0827694c8de04b7469b4f93834b709416c0b/lib/python3.10/site-packages (from Flask<4->mlflow->mlserver-mlflow!=1.3.1,<1.4.0,>=1.2.0) (1.8.2)
#19 178.4 Collecting gevent>=0.13 (from geventhttpclient<=2.0.2,>=1.4.4->tritonclient[http]>=2.24->mlserver!=1.3.1,<1.4.0,>=1.2.0)
#19 178.4   Downloading gevent-24.2.1-cp310-cp310-manylinux_2_28_x86_64.whl (6.5 MB)
#19 178.7      ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 19.9 MB/s eta 0:00:00
#19 178.8 Requirement already satisfied: certifi in /root/.mlflow/envs/mlflow-bb8a0827694c8de04b7469b4f93834b709416c0b/lib/python3.10/site-packages (from geventhttpclient<=2.0.2,>=1.4.4->tritonclient[http]>=2.24->mlserver!=1.3.1,<1.4.0,>=1.2.0) (2024.2.2)
#19 178.8 Requirement already satisfied: six in /root/.mlflow/envs/mlflow-bb8a0827694c8de04b7469b4f93834b709416c0b/lib/python3.10/site-packages (from geventhttpclient<=2.0.2,>=1.4.4->tritonclient[http]>=2.24->mlserver!=1.3.1,<1.4.0,>=1.2.0) (1.16.0)
#19 178.8 Collecting brotli (from geventhttpclient<=2.0.2,>=1.4.4->tritonclient[http]>=2.24->mlserver!=1.3.1,<1.4.0,>=1.2.0)
#19 178.9   Downloading Brotli-1.1.0-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (3.0 MB)
#19 179.0      ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 20.2 MB/s eta 0:00:00
#19 179.1 Requirement already satisfied: gitdb<5,>=4.0.1 in /root/.mlflow/envs/mlflow-bb8a0827694c8de04b7469b4f93834b709416c0b/lib/python3.10/site-packages (from gitpython<4,>=3.1.9->mlflow->mlserver-mlflow!=1.3.1,<1.4.0,>=1.2.0) (4.0.11)
#19 179.1 Requirement already satisfied: graphql-core<3.3,>=3.1 in /root/.mlflow/envs/mlflow-bb8a0827694c8de04b7469b4f93834b709416c0b/lib/python3.10/site-packages (from graphene<4->mlflow->mlserver-mlflow!=1.3.1,<1.4.0,>=1.2.0) (3.2.3)
#19 179.1 Requirement already satisfied: graphql-relay<3.3,>=3.1 in /root/.mlflow/envs/mlflow-bb8a0827694c8de04b7469b4f93834b709416c0b/lib/python3.10/site-packages (from graphene<4->mlflow->mlserver-mlflow!=1.3.1,<1.4.0,>=1.2.0) (3.2.0)
#19 179.1 Requirement already satisfied: aniso8601<10,>=8 in /root/.mlflow/envs/mlflow-bb8a0827694c8de04b7469b4f93834b709416c0b/lib/python3.10/site-packages (from graphene<4->mlflow->mlserver-mlflow!=1.3.1,<1.4.0,>=1.2.0) (9.0.1)
#19 179.1 Requirement already satisfied: zipp>=0.5 in /root/.mlflow/envs/mlflow-bb8a0827694c8de04b7469b4f93834b709416c0b/lib/python3.10/site-packages (from importlib-metadata!=4.7.0,<8,>=3.7.0->mlflow->mlserver-mlflow!=1.3.1,<1.4.0,>=1.2.0) (3.18.2)
#19 179.2 Requirement already satisfied: MarkupSafe>=2.0 in /root/.mlflow/envs/mlflow-bb8a0827694c8de04b7469b4f93834b709416c0b/lib/python3.10/site-packages (from Jinja2<4,>=2.11->mlflow->mlserver-mlflow!=1.3.1,<1.4.0,>=1.2.0) (2.1.5)
#19 179.2 Requirement already satisfied: contourpy>=1.0.1 in /root/.mlflow/envs/mlflow-bb8a0827694c8de04b7469b4f93834b709416c0b/lib/python3.10/site-packages (from matplotlib<4->mlflow->mlserver-mlflow!=1.3.1,<1.4.0,>=1.2.0) (1.2.1)
#19 179.2 Requirement already satisfied: cycler>=0.10 in /root/.mlflow/envs/mlflow-bb8a0827694c8de04b7469b4f93834b709416c0b/lib/python3.10/site-packages (from matplotlib<4->mlflow->mlserver-mlflow!=1.3.1,<1.4.0,>=1.2.0) (0.12.1)
#19 179.2 Requirement already satisfied: fonttools>=4.22.0 in /root/.mlflow/envs/mlflow-bb8a0827694c8de04b7469b4f93834b709416c0b/lib/python3.10/site-packages (from matplotlib<4->mlflow->mlserver-mlflow!=1.3.1,<1.4.0,>=1.2.0) (4.51.0)
#19 179.2 Requirement already satisfied: kiwisolver>=1.0.1 in /root/.mlflow/envs/mlflow-bb8a0827694c8de04b7469b4f93834b709416c0b/lib/python3.10/site-packages (from matplotlib<4->mlflow->mlserver-mlflow!=1.3.1,<1.4.0,>=1.2.0) (1.4.5)
#19 179.2 Requirement already satisfied: pillow>=6.2.0 in /root/.mlflow/envs/mlflow-bb8a0827694c8de04b7469b4f93834b709416c0b/lib/python3.10/site-packages (from matplotlib<4->mlflow->mlserver-mlflow!=1.3.1,<1.4.0,>=1.2.0) (10.3.0)
#19 179.2 Requirement already satisfied: pyparsing>=2.3.1 in /root/.mlflow/envs/mlflow-bb8a0827694c8de04b7469b4f93834b709416c0b/lib/python3.10/site-packages (from matplotlib<4->mlflow->mlserver-mlflow!=1.3.1,<1.4.0,>=1.2.0) (3.1.2)
#19 179.3 Requirement already satisfied: charset-normalizer<4,>=2 in /root/.mlflow/envs/mlflow-bb8a0827694c8de04b7469b4f93834b709416c0b/lib/python3.10/site-packages (from requests<3,>=2.17.3->mlflow->mlserver-mlflow!=1.3.1,<1.4.0,>=1.2.0) (3.3.2)
#19 179.3 Requirement already satisfied: idna<4,>=2.5 in /root/.mlflow/envs/mlflow-bb8a0827694c8de04b7469b4f93834b709416c0b/lib/python3.10/site-packages (from requests<3,>=2.17.3->mlflow->mlserver-mlflow!=1.3.1,<1.4.0,>=1.2.0) (3.7)
#19 179.3 Requirement already satisfied: joblib>=1.1.1 in /root/.mlflow/envs/mlflow-bb8a0827694c8de04b7469b4f93834b709416c0b/lib/python3.10/site-packages (from scikit-learn<2->mlflow->mlserver-mlflow!=1.3.1,<1.4.0,>=1.2.0) (1.4.2)
#19 179.3 Requirement already satisfied: threadpoolctl>=2.0.0 in /root/.mlflow/envs/mlflow-bb8a0827694c8de04b7469b4f93834b709416c0b/lib/python3.10/site-packages (from scikit-learn<2->mlflow->mlserver-mlflow!=1.3.1,<1.4.0,>=1.2.0) (3.5.0)
#19 179.4 Requirement already satisfied: greenlet!=0.4.17 in /root/.mlflow/envs/mlflow-bb8a0827694c8de04b7469b4f93834b709416c0b/lib/python3.10/site-packages (from sqlalchemy<3,>=1.4.0->mlflow->mlserver-mlflow!=1.3.1,<1.4.0,>=1.2.0) (3.0.3)
#19 179.6 Collecting sniffio>=1.1 (from anyio<5,>=3.4.0->starlette==0.22.0->fastapi!=0.89.0,<=0.89.1,>=0.88.0->mlserver!=1.3.1,<1.4.0,>=1.2.0)
#19 179.6   Downloading sniffio-1.3.1-py3-none-any.whl (10 kB)
#19 179.7 Collecting exceptiongroup>=1.0.2 (from anyio<5,>=3.4.0->starlette==0.22.0->fastapi!=0.89.0,<=0.89.1,>=0.88.0->mlserver!=1.3.1,<1.4.0,>=1.2.0)
#19 179.7   Downloading exceptiongroup-1.2.1-py3-none-any.whl (16 kB)
#19 179.9 Collecting zope.event (from gevent>=0.13->geventhttpclient<=2.0.2,>=1.4.4->tritonclient[http]>=2.24->mlserver!=1.3.1,<1.4.0,>=1.2.0)
#19 180.0   Downloading zope.event-5.0-py3-none-any.whl (6.8 kB)
#19 180.2 Collecting zope.interface (from gevent>=0.13->geventhttpclient<=2.0.2,>=1.4.4->tritonclient[http]>=2.24->mlserver!=1.3.1,<1.4.0,>=1.2.0)
#19 180.2   Downloading zope.interface-6.4-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (247 kB)
#19 180.2      ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.5/247.5 kB 31.3 MB/s eta 0:00:00
#19 180.2 Requirement already satisfied: smmap<6,>=3.0.1 in /root/.mlflow/envs/mlflow-bb8a0827694c8de04b7469b4f93834b709416c0b/lib/python3.10/site-packages (from gitdb<5,>=4.0.1->gitpython<4,>=3.1.9->mlflow->mlserver-mlflow!=1.3.1,<1.4.0,>=1.2.0) (5.0.1)
#19 181.2 Installing collected packages: brotli, zope.interface, zope.event, uvloop, sniffio, python-rapidjson, python-dotenv, pydantic, prometheus-client, orjson, multidict, importlib-resources, h11, grpcio, frozenlist, exceptiongroup, attrs, aiokafka, aiofiles, yarl, uvicorn, tritonclient, py-grpc-prometheus, gevent, anyio, aiosignal, starlette, geventhttpclient, aiohttp, starlette-exporter, fastapi, mlserver, mlserver-mlflow
#19 184.4 Successfully installed aiofiles-23.2.1 aiohttp-3.9.5 aiokafka-0.10.0 aiosignal-1.3.1 anyio-4.3.0 attrs-23.2.0 brotli-1.1.0 exceptiongroup-1.2.1 fastapi-0.89.1 frozenlist-1.4.1 gevent-24.2.1 geventhttpclient-2.0.2 grpcio-1.63.0 h11-0.14.0 importlib-resources-6.4.0 mlserver-1.3.5 mlserver-mlflow-1.3.5 multidict-6.0.5 orjson-3.10.3 prometheus-client-0.20.0 py-grpc-prometheus-0.8.0 pydantic-1.10.15 python-dotenv-1.0.1 python-rapidjson-1.16 sniffio-1.3.1 starlette-0.22.0 starlette-exporter-0.17.1 tritonclient-2.45.0 uvicorn-0.29.0 uvloop-0.19.0 yarl-1.9.4 zope.event-5.0 zope.interface-6.4
#19 184.4 WARNING: There was an error checking the latest version of pip.
#19 DONE 186.7s

#20 [16/17] RUN chmod o+rwX /opt/mlflow/
#20 DONE 0.4s

#21 [17/17] RUN rm -rf /var/lib/apt/lists/*
#21 DONE 0.6s

#22 exporting to image
#22 exporting layers
#22 exporting layers 12.0s done
#22 writing image sha256:29875270957bd037ee2b833cf9201a001d353e4fef34b1d8c4923b278ef8ac02 done
#22 naming to docker.io/5uperpalo/mlflow-success6g done
#22 DONE 12.1s
```
</details>